<a href="https://colab.research.google.com/github/novice108/ITT2025/blob/main/2025_15_05_PID_Consence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.patches import Circle
from IPython.display import HTML
import random
import math
from typing import List, Dict, Tuple
import time
from dataclasses import dataclass, field
from enum import Enum

# Константи
DEFAULT_WIDTH = 800
DEFAULT_HEIGHT = 600
DRONE_COLOR = 'blue'
COLLISION_COLOR = 'red'
TRAJECTORY_COLOR = 'lightgray'
VELOCITY_ARROW_COLOR = 'red'
TEXT_COLOR = 'black'
DRONE_RADIUS = 5
ARROW_SCALE = 10
FPS = 60

class DroneState(Enum):
    NORMAL = 1
    COLLISION = 2

@dataclass
class Drone:
    position: np.ndarray
    velocity: np.ndarray
    state: DroneState = DroneState.NORMAL
    neighbors: List['Drone'] = field(default_factory=list)
    collision_count: int = 0

@dataclass
class SimulationParameters:
    num_drones: int = 20
    circle_radius: float = 250
    circle_center: Tuple[float, float] = (DEFAULT_WIDTH//2, DEFAULT_HEIGHT//2)
    desired_distance: float = 40
    min_distance: float = 20
    max_speed: float = 3
    time_step: float = 1.0
    separation_weight: float = 1.5
    alignment_weight: float = 1.0
    cohesion_weight: float = 1.0
    circle_weight: float = 0.5
    perception_radius: float = 100
    show_velocity: bool = False
    show_metrics: bool = True
    simulation_speed: float = 1.0

class DroneSimulation:
    def __init__(self, params: SimulationParameters):
        self.params = params
        self.fig, self.ax = plt.subplots(figsize=(10, 8))
        self.ax.set_xlim(0, DEFAULT_WIDTH)
        self.ax.set_ylim(0, DEFAULT_HEIGHT)
        self.ax.set_aspect('equal')
        self.ax.grid(True, linestyle='--', alpha=0.5)
        self.ax.set_title('Симуляція руху зграї дронів по колу')

        self.drones: List[Drone] = []
        self.total_collisions = 0
        self.avg_distance = 0
        self.simulation_time = 0
        self.frame_count = 0
        self.metrics_history = []
        self.animation = None

        # Елементи візуалізації
        self.trajectory_circle = Circle(
            self.params.circle_center, self.params.circle_radius,
            fill=False, color=TRAJECTORY_COLOR, linestyle='--'
        )
        self.ax.add_patch(self.trajectory_circle)

        self.drone_plots = []
        self.velocity_arrows = []
        self.metrics_text = None

        self.initialize_drones()
        self.setup_visuals()

    def initialize_drones(self):
        """Ініціалізація дронів на випадкових позиціях всередині кола"""
        self.drones = []
        center_x, center_y = self.params.circle_center

        for _ in range(self.params.num_drones):
            # Випадковий кут і відстань від центру
            angle = random.uniform(0, 2 * math.pi)
            distance = random.uniform(0, self.params.circle_radius * 0.8)

            x = center_x + distance * math.cos(angle)
            y = center_y + distance * math.sin(angle)

            # Випадкова початкова швидкість
            speed = random.uniform(0.5, self.params.max_speed)
            vx = random.uniform(-1, 1)
            vy = random.uniform(-1, 1)
            velocity = np.array([vx, vy], dtype=np.float64)
            if np.linalg.norm(velocity) > 0:
                velocity = velocity / np.linalg.norm(velocity) * speed

            self.drones.append(Drone(
                position=np.array([x, y], dtype=np.float64),
                velocity=velocity
            ))

    def setup_visuals(self):
        """Налаштування графічних елементів"""
        # Очистити попередні елементи
        for arrow in self.velocity_arrows:
            arrow.remove()
        self.velocity_arrows.clear()

        for plot in self.drone_plots:
            plot.remove()
        self.drone_plots.clear()

        # Створити нові елементи
        for drone in self.drones:
            color = DRONE_COLOR if drone.state == DroneState.NORMAL else COLLISION_COLOR
            drone_plot = self.ax.plot(
                drone.position[0], drone.position[1],
                'o', color=color, markersize=DRONE_RADIUS
            )[0]
            self.drone_plots.append(drone_plot)

            if self.params.show_velocity:
                arrow = self.ax.arrow(
                    drone.position[0], drone.position[1],
                    drone.velocity[0] * ARROW_SCALE, drone.velocity[1] * ARROW_SCALE,
                    color=VELOCITY_ARROW_COLOR, width=1, head_width=5
                )
                self.velocity_arrows.append(arrow)

        # Текст метрик
        if self.params.show_metrics:
            if self.metrics_text is not None:
                self.metrics_text.remove()

            metrics_str = self.get_metrics_string()
            self.metrics_text = self.ax.text(
                0.02, 0.95, metrics_str,
                transform=self.ax.transAxes,
                fontsize=10, color=TEXT_COLOR,
                bbox=dict(facecolor='white', alpha=0.7, edgecolor='gray')
            )

    def get_metrics_string(self):
        """Формування рядка з метриками"""
        return (
            f"Час: {self.simulation_time:.1f} с\n"
            f"Дронів: {self.params.num_drones}\n"
            f"Зіткнень: {self.total_collisions}\n"
            f"Середня відстань: {self.avg_distance:.1f}\n"
            f"Бажана відстань: {self.params.desired_distance:.1f}\n"
            f"Швидкість: x{self.params.simulation_speed:.1f}"
        )

    def find_neighbors(self, drone: Drone) -> List[Drone]:
        """Знаходження сусідів для даного дрона"""
        neighbors = []
        for other in self.drones:
            if other is not drone:
                distance = np.linalg.norm(drone.position - other.position)
                if distance < self.params.perception_radius:
                    neighbors.append(other)
        return neighbors

    def apply_separation(self, drone: Drone) -> np.ndarray:
        """Правило розділення: уникання зіткнень з сусідами"""
        steering = np.zeros(2)
        total = 0

        for other in drone.neighbors:
            distance = np.linalg.norm(drone.position - other.position)
            if distance < self.params.min_distance:
                diff = drone.position - other.position
                if distance > 0:
                    diff = diff / distance  # Нормалізація
                steering += diff
                total += 1

        if total > 0:
            steering = steering / total
            if np.linalg.norm(steering) > 0:
                steering = steering / np.linalg.norm(steering) * self.params.max_speed
                steering = steering - drone.velocity
                if np.linalg.norm(steering) > 0:
                    steering = steering / np.linalg.norm(steering)

        return steering * self.params.separation_weight

    def apply_alignment(self, drone: Drone) -> np.ndarray:
        """Правило вирівнювання: узгодження швидкості з сусідами"""
        steering = np.zeros(2)
        total = 0

        for other in drone.neighbors:
            steering += other.velocity
            total += 1

        if total > 0:
            steering = steering / total
            if np.linalg.norm(steering) > 0:
                steering = steering / np.linalg.norm(steering) * self.params.max_speed
            steering = steering - drone.velocity
            if np.linalg.norm(steering) > 0:
                steering = steering / np.linalg.norm(steering)

        return steering * self.params.alignment_weight

    def apply_cohesion(self, drone: Drone) -> np.ndarray:
        """Правило згуртованості: рух до центру маси сусідів"""
        steering = np.zeros(2)
        total = 0

        for other in drone.neighbors:
            steering += other.position
            total += 1

        if total > 0:
            steering = steering / total  # Центр маси
            steering = steering - drone.position  # Вектор до центру
            if np.linalg.norm(steering) > 0:
                steering = steering / np.linalg.norm(steering) * self.params.max_speed
            steering = steering - drone.velocity
            if np.linalg.norm(steering) > 0:
                steering = steering / np.linalg.norm(steering)

        return steering * self.params.cohesion_weight

    def apply_circle_movement(self, drone: Drone) -> np.ndarray:
        """Додаткове правило для руху по колу"""
        center_x, center_y = self.params.circle_center
        center = np.array([center_x, center_y])

        # Вектор від центру до дрона
        to_center = center - drone.position
        distance_to_center = np.linalg.norm(to_center)

        if distance_to_center > 0:
            # Нормалізований вектор до центру
            to_center_normalized = to_center / distance_to_center

            # Бажаний вектор швидкості - дотичний до кола
            desired_velocity = np.array([-to_center_normalized[1], to_center_normalized[0]])
            desired_velocity = desired_velocity * self.params.max_speed

            # Корекція для підтримки бажаного радіусу
            radius_error = distance_to_center - self.params.circle_radius
            correction_factor = 0.05 * radius_error

            # Комбінуємо дотичний рух з корекцією радіусу
            steering = desired_velocity - drone.velocity + to_center_normalized * correction_factor

            if np.linalg.norm(steering) > 0:
                steering = steering / np.linalg.norm(steering)

            return steering * self.params.circle_weight
        return np.zeros(2)

    def update_drone(self, drone: Drone):
        """Оновлення стану одного дрона"""
        # Знаходження сусідів
        drone.neighbors = self.find_neighbors(drone)

        # Обчислення векторів правил Бойдза
        separation = self.apply_separation(drone)
        alignment = self.apply_alignment(drone)
        cohesion = self.apply_cohesion(drone)
        circle_move = self.apply_circle_movement(drone)

        # Комбінування правил
        acceleration = separation + alignment + cohesion + circle_move

        # Оновлення швидкості
        drone.velocity += acceleration

        # Обмеження швидкості
        speed = np.linalg.norm(drone.velocity)
        if speed > self.params.max_speed:
            drone.velocity = (drone.velocity / speed) * self.params.max_speed

        # Оновлення позиції
        drone.position += drone.velocity * self.params.time_step * self.params.simulation_speed

        # Перевірка на зіткнення
        drone.state = DroneState.NORMAL
        for other in self.drones:
            if other is not drone:
                distance = np.linalg.norm(drone.position - other.position)
                if distance < self.params.min_distance * 0.8:
                    drone.state = DroneState.COLLISION
                    drone.collision_count += 1
                    self.total_collisions += 1
                    break

    def update_metrics(self):
        """Оновлення метрик симуляції"""
        total_distance = 0
        valid_pairs = 0

        for i, drone in enumerate(self.drones):
            min_dist = float('inf')
            for j, other in enumerate(self.drones):
                if i != j:
                    distance = np.linalg.norm(drone.position - other.position)
                    total_distance += distance
                    valid_pairs += 1
                    if distance < min_dist:
                        min_dist = distance

            # Для кожного дрона можна також зберігати мінімальну відстань

        if valid_pairs > 0:
            self.avg_distance = total_distance / valid_pairs

        # Збереження метрик для історії
        self.metrics_history.append({
            'time': self.simulation_time,
            'avg_distance': self.avg_distance,
            'collisions': self.total_collisions,
            'separation': self.params.separation_weight,
            'alignment': self.params.alignment_weight,
            'cohesion': self.params.cohesion_weight
        })

        # Обмеження розміру історії
        if len(self.metrics_history) > 1000:
            self.metrics_history.pop(0)

    def update(self, frame):
        """Оновлення стану симуляції для анімації"""
        for drone in self.drones:
            self.update_drone(drone)

        self.update_metrics()
        self.simulation_time += self.params.time_step * self.params.simulation_speed
        self.frame_count += 1

        # Оновлення візуалізації
        self.update_visuals()

        return self.drone_plots + self.velocity_arrows + [self.metrics_text]

    def update_visuals(self):
        """Оновлення графічних елементів"""
        for i, drone in enumerate(self.drones):
            color = DRONE_COLOR if drone.state == DroneState.NORMAL else COLLISION_COLOR
            self.drone_plots[i].set_data([drone.position[0]], [drone.position[1]])
            self.drone_plots[i].set_color(color)

            if self.params.show_velocity and i < len(self.velocity_arrows):
                self.velocity_arrows[i].remove()
                arrow = self.ax.arrow(
                    drone.position[0], drone.position[1],
                    drone.velocity[0] * ARROW_SCALE, drone.velocity[1] * ARROW_SCALE,
                    color=VELOCITY_ARROW_COLOR, width=1, head_width=5
                )
                self.velocity_arrows[i] = arrow

        if self.params.show_metrics and self.metrics_text is not None:
            self.metrics_text.set_text(self.get_metrics_string())

    def run_simulation(self):
        """Запуск симуляції з анімацією"""
        self.animation = animation.FuncAnimation(
            self.fig, self.update,
            frames=200, interval=1000/FPS, blit=True
        )

        plt.close()  # Запобігаємо подвійному відображенню

        return HTML(self.animation.to_jshtml())

    def reset_simulation(self):
        """Перезапуск симуляції"""
        self.initialize_drones()
        self.total_collisions = 0
        self.avg_distance = 0
        self.simulation_time = 0
        self.frame_count = 0
        self.metrics_history = []
        self.setup_visuals()

# Параметри за замовчуванням
params = SimulationParameters(
    num_drones=30,
    circle_radius=200,
    desired_distance=40,
    min_distance=20,
    max_speed=2.5,
    separation_weight=1.8,
    alignment_weight=1.2,
    cohesion_weight=1.0,
    circle_weight=0.7,
    perception_radius=80,
    show_velocity=True,
    show_metrics=True
)

# Створення та запуск симуляції
simulation = DroneSimulation(params)
simulation.run_simulation()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.patches import Circle
from IPython.display import HTML
import random
import math
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass, field
from enum import Enum

import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 500

# Константи
DEFAULT_WIDTH = 800
DEFAULT_HEIGHT = 600
DRONE_COLOR = 'blue'
COLLISION_COLOR = 'red'
TRAJECTORY_COLOR = 'lightgray'
VELOCITY_ARROW_COLOR = 'red'
TEXT_COLOR = 'black'
DRONE_RADIUS = 5
ARROW_SCALE = 10
FPS = 60

class DroneState(Enum):
    NORMAL = 1
    COLLISION = 2

@dataclass
class PIDController:
    Kp: float = 0.5
    Ki: float = 0.1
    Kd: float = 0.2
    target: float = 2.0
    integral: float = 0.0
    prev_error: float = 0.0
    dt: float = 1.0

    def compute(self, current_value: float) -> float:
        """Обчислення вихідного сигналу PID-контролера"""
        error = self.target - current_value
        self.integral += error * self.dt
        derivative = (error - self.prev_error) / self.dt
        output = self.Kp * error + self.Ki * self.integral + self.Kd * derivative
        self.prev_error = error
        return output

    def update_params(self, Kp: float, Ki: float, Kd: float):
        """Оновлення параметрів PID"""
        self.Kp = Kp
        self.Ki = Ki
        self.Kd = Kd

@dataclass
class Drone:
    position: np.ndarray
    velocity: np.ndarray
    state: DroneState = DroneState.NORMAL
    neighbors: List['Drone'] = field(default_factory=list)
    collision_count: int = 0
    pid: PIDController = field(default_factory=PIDController)
    communication_radius: float = 150
    pid_params_history: List[Tuple[float, float, float]] = field(default_factory=list)

@dataclass
class SimulationParameters:
    num_drones: int = 20
    circle_radius: float = 250
    circle_center: Tuple[float, float] = (DEFAULT_WIDTH//2, DEFAULT_HEIGHT//2)
    desired_distance: float = 40
    min_distance: float = 20
    max_speed: float = 3
    time_step: float = 1.0
    separation_weight: float = 1.5
    alignment_weight: float = 1.0
    cohesion_weight: float = 1.0
    circle_weight: float = 0.5
    perception_radius: float = 100
    show_velocity: bool = False
    show_metrics: bool = True
    show_pid_graphs: bool = True
    simulation_speed: float = 1.0
    # Параметри консенсусу PID
    alpha_p: float = 0.1  # Швидкість консенсусу для Kp
    alpha_i: float = 0.1  # Швидкість консенсусу для Ki
    alpha_d: float = 0.1  # Швидкість консенсусу для Kd
    beta_p: float = 0.01  # Коефіцієнт адаптації для Kp
    beta_i: float = 0.01  # Коефіцієнт адаптації для Ki
    beta_d: float = 0.01  # Коефіцієнт адаптації для Kd

class DroneSimulation:
    def __init__(self, params: SimulationParameters):
        self.params = params
        self.fig = plt.figure(figsize=(14, 8))

        # Основний графік для симуляції
        self.ax = self.fig.add_subplot(121)
        self.ax.set_xlim(0, DEFAULT_WIDTH)
        self.ax.set_ylim(0, DEFAULT_HEIGHT)
        self.ax.set_aspect('equal')
        self.ax.grid(True, linestyle='--', alpha=0.5)
        self.ax.set_title('Симуляція руху зграї дронів')

        # Графіки для PID параметрів
        self.pid_ax = self.fig.add_subplot(322)
        self.pid_ax.set_title('Середні значення PID параметрів')
        self.pid_ax.grid(True)

        self.error_ax = self.fig.add_subplot(324)
        self.error_ax.set_title('Похибка швидкості')
        self.error_ax.grid(True)

        self.collision_ax = self.fig.add_subplot(326)
        self.collision_ax.set_title('Кількість зіткнень')
        self.collision_ax.grid(True)

        self.drones: List[Drone] = []
        self.total_collisions = 0
        self.avg_distance = 0
        self.simulation_time = 0
        self.frame_count = 0
        self.metrics_history = []
        self.avg_pid_history = {'Kp': [], 'Ki': [], 'Kd': []}
        self.avg_velocity_error = []
        self.collision_history = []
        self.animation = None

        # Елементи візуалізації
        self.trajectory_circle = Circle(
            self.params.circle_center, self.params.circle_radius,
            fill=False, color=TRAJECTORY_COLOR, linestyle='--'
        )
        self.ax.add_patch(self.trajectory_circle)

        self.drone_plots = []
        self.velocity_arrows = []
        self.metrics_text = None
        self.pid_lines = {'Kp': None, 'Ki': None, 'Kd': None}
        self.error_line = None
        self.collision_line = None

        self.initialize_drones()
        self.setup_visuals()

    def initialize_drones(self):
        """Ініціалізація дронів з випадковими PID-параметрами"""
        self.drones = []
        center_x, center_y = self.params.circle_center

        for _ in range(self.params.num_drones):
            # Випадковий кут і відстань від центру
            angle = random.uniform(0, 2 * math.pi)
            distance = random.uniform(0, self.params.circle_radius * 0.8)

            x = center_x + distance * math.cos(angle)
            y = center_y + distance * math.sin(angle)

            # Випадкова початкова швидкість
            speed = random.uniform(0.5, self.params.max_speed)
            vx = random.uniform(-1, 1)
            vy = random.uniform(-1, 1)
            velocity = np.array([vx, vy], dtype=np.float64)
            if np.linalg.norm(velocity) > 0:
                velocity = velocity / np.linalg.norm(velocity) * speed

            # Випадкові початкові PID-параметри
            Kp = random.uniform(0.1, 1.0)
            Ki = random.uniform(0.01, 0.2)
            Kd = random.uniform(0.05, 0.3)

            self.drones.append(Drone(
                position=np.array([x, y], dtype=np.float64),
                velocity=velocity,
                pid=PIDController(Kp=Kp, Ki=Ki, Kd=Kd, dt=self.params.time_step)
            ))

    def setup_visuals(self):
        """Налаштування графічних елементів"""
        # Очистити попередні елементи
        for arrow in self.velocity_arrows:
            arrow.remove()
        self.velocity_arrows.clear()

        for plot in self.drone_plots:
            plot.remove()
        self.drone_plots.clear()

        # Створити нові елементи
        for drone in self.drones:
            color = DRONE_COLOR if drone.state == DroneState.NORMAL else COLLISION_COLOR
            drone_plot = self.ax.plot(
                drone.position[0], drone.position[1],
                'o', color=color, markersize=DRONE_RADIUS
            )[0]
            self.drone_plots.append(drone_plot)

            if self.params.show_velocity:
                arrow = self.ax.arrow(
                    drone.position[0], drone.position[1],
                    drone.velocity[0] * ARROW_SCALE, drone.velocity[1] * ARROW_SCALE,
                    color=VELOCITY_ARROW_COLOR, width=1, head_width=5
                )
                self.velocity_arrows.append(arrow)

        # Текст метрик
        if self.params.show_metrics:
            if self.metrics_text is not None:
                self.metrics_text.remove()

            metrics_str = self.get_metrics_string()
            self.metrics_text = self.ax.text(
                0.02, 0.95, metrics_str,
                transform=self.ax.transAxes,
                fontsize=10, color=TEXT_COLOR,
                bbox=dict(facecolor='white', alpha=0.7, edgecolor='gray')
            )

        # Ініціалізація графіків PID
        if self.params.show_pid_graphs:
            time_points = [self.simulation_time]
            self.pid_lines['Kp'], = self.pid_ax.plot(time_points, [self.get_avg_pid_param('Kp')], 'r-', label='Kp')
            self.pid_lines['Ki'], = self.pid_ax.plot(time_points, [self.get_avg_pid_param('Ki')], 'g-', label='Ki')
            self.pid_lines['Kd'], = self.pid_ax.plot(time_points, [self.get_avg_pid_param('Kd')], 'b-', label='Kd')
            self.pid_ax.legend()

            self.error_line, = self.error_ax.plot(time_points, [0], 'm-', label='Похибка')
            self.error_ax.legend()

            self.collision_line, = self.collision_ax.plot(time_points, [0], 'k-', label='Зіткнення')
            self.collision_ax.legend()

    def get_metrics_string(self):
        """Формування рядка з метриками"""
        return (
            f"Час: {self.simulation_time:.1f} с\n"
            f"Дронів: {self.params.num_drones}\n"
            f"Зіткнень: {self.total_collisions}\n"
            f"Середня відстань: {self.avg_distance:.1f}\n"
            f"Бажана відстань: {self.params.desired_distance:.1f}\n"
            f"Kp: {self.get_avg_pid_param('Kp'):.3f}\n"
            f"Ki: {self.get_avg_pid_param('Ki'):.3f}\n"
            f"Kd: {self.get_avg_pid_param('Kd'):.3f}"
        )

    def get_avg_pid_param(self, param: str) -> float:
        """Отримання середнього значення PID-параметра по зграї"""
        if not self.drones:
            return 0.0
        if param == 'Kp':
            return sum(d.pid.Kp for d in self.drones) / len(self.drones)
        elif param == 'Ki':
            return sum(d.pid.Ki for d in self.drones) / len(self.drones)
        elif param == 'Kd':
            return sum(d.pid.Kd for d in self.drones) / len(self.drones)
        return 0.0

    def find_neighbors(self, drone: Drone) -> List[Drone]:
        """Знаходження сусідів для даного дрона"""
        neighbors = []
        for other in self.drones:
            if other is not drone:
                distance = np.linalg.norm(drone.position - other.position)
                if distance < drone.communication_radius:
                    neighbors.append(other)
        return neighbors

    def apply_separation(self, drone: Drone) -> np.ndarray:
        """Правило розділення: уникання зіткнень з сусідами"""
        steering = np.zeros(2)
        total = 0

        for other in drone.neighbors:
            distance = np.linalg.norm(drone.position - other.position)
            if distance < self.params.min_distance:
                diff = drone.position - other.position
                if distance > 0:
                    diff = diff / distance  # Нормалізація
                steering += diff
                total += 1

        if total > 0:
            steering = steering / total
            if np.linalg.norm(steering) > 0:
                steering = steering / np.linalg.norm(steering) * self.params.max_speed
                steering = steering - drone.velocity
                if np.linalg.norm(steering) > 0:
                    steering = steering / np.linalg.norm(steering)

        return steering * self.params.separation_weight

    def apply_alignment(self, drone: Drone) -> np.ndarray:
        """Правило вирівнювання: узгодження швидкості з сусідами"""
        steering = np.zeros(2)
        total = 0

        for other in drone.neighbors:
            steering += other.velocity
            total += 1

        if total > 0:
            steering = steering / total
            if np.linalg.norm(steering) > 0:
                steering = steering / np.linalg.norm(steering) * self.params.max_speed
            steering = steering - drone.velocity
            if np.linalg.norm(steering) > 0:
                steering = steering / np.linalg.norm(steering)

        return steering * self.params.alignment_weight

    def apply_cohesion(self, drone: Drone) -> np.ndarray:
        """Правило згуртованості: рух до центру маси сусідів"""
        steering = np.zeros(2)
        total = 0

        for other in drone.neighbors:
            steering += other.position
            total += 1

        if total > 0:
            steering = steering / total  # Центр маси
            steering = steering - drone.position  # Вектор до центру
            if np.linalg.norm(steering) > 0:
                steering = steering / np.linalg.norm(steering) * self.params.max_speed
            steering = steering - drone.velocity
            if np.linalg.norm(steering) > 0:
                steering = steering / np.linalg.norm(steering)

        return steering * self.params.cohesion_weight

    def apply_circle_movement(self, drone: Drone) -> np.ndarray:
        """Додаткове правило для руху по колу"""
        center_x, center_y = self.params.circle_center
        center = np.array([center_x, center_y])

        # Вектор від центру до дрона
        to_center = center - drone.position
        distance_to_center = np.linalg.norm(to_center)

        if distance_to_center > 0:
            # Нормалізований вектор до центру
            to_center_normalized = to_center / distance_to_center

            # Бажаний вектор швидкості - дотичний до кола
            desired_velocity = np.array([-to_center_normalized[1], to_center_normalized[0]])
            desired_velocity = desired_velocity * self.params.max_speed

            # Корекція для підтримки бажаного радіусу
            radius_error = distance_to_center - self.params.circle_radius
            correction_factor = 0.05 * radius_error

            # Комбінуємо дотичний рух з корекцією радіусу
            steering = desired_velocity - drone.velocity + to_center_normalized * correction_factor

            if np.linalg.norm(steering) > 0:
                steering = steering / np.linalg.norm(steering)

            return steering * self.params.circle_weight
        return np.zeros(2)

    def update_pid_consensus(self, drone: Drone):
        """Алгоритм консенсусу для PID-параметрів"""
        if not drone.neighbors:
            return

        # Середнє значення параметрів сусідів
        avg_Kp = sum(n.pid.Kp for n in drone.neighbors) / len(drone.neighbors)
        avg_Ki = sum(n.pid.Ki for n in drone.neighbors) / len(drone.neighbors)
        avg_Kd = sum(n.pid.Kd for n in drone.neighbors) / len(drone.neighbors)

        # Похибка швидкості для локальної адаптації
        speed_error = abs(np.linalg.norm(drone.velocity) - drone.pid.target)

        # Оновлення параметрів за алгоритмом консенсусу
        new_Kp = drone.pid.Kp + self.params.alpha_p * (avg_Kp - drone.pid.Kp) + self.params.beta_p * speed_error
        new_Ki = drone.pid.Ki + self.params.alpha_i * (avg_Ki - drone.pid.Ki) + self.params.beta_i * speed_error
        new_Kd = drone.pid.Kd + self.params.alpha_d * (avg_Kd - drone.pid.Kd) + self.params.beta_d * speed_error

        # Обмеження значень параметрів
        new_Kp = max(0.01, min(2.0, new_Kp))
        new_Ki = max(0.001, min(0.5, new_Ki))
        new_Kd = max(0.01, min(1.0, new_Kd))

        # Оновлення PID-контролера
        drone.pid.update_params(new_Kp, new_Ki, new_Kd)

        # Збереження історії параметрів
        drone.pid_params_history.append((new_Kp, new_Ki, new_Kd))

    def update_drone(self, drone: Drone):
        """Оновлення стану одного дрона"""
        # Знаходження сусідів
        drone.neighbors = self.find_neighbors(drone)

        # Оновлення PID-параметрів за алгоритмом консенсусу
        self.update_pid_consensus(drone)

        # Обчислення векторів правил Бойдза
        separation = self.apply_separation(drone)
        alignment = self.apply_alignment(drone)
        cohesion = self.apply_cohesion(drone)
        circle_move = self.apply_circle_movement(drone)

        # Комбінування правил
        acceleration = separation + alignment + cohesion + circle_move

        # Обчислення PID-корекції швидкості
        current_speed = np.linalg.norm(drone.velocity)
        pid_correction = drone.pid.compute(current_speed)

        # Застосування PID-корекції до прискорення
        if current_speed > 0:
            velocity_direction = drone.velocity / current_speed
            acceleration += velocity_direction * pid_correction

        # Оновлення швидкості
        drone.velocity += acceleration

        # Обмеження швидкості
        speed = np.linalg.norm(drone.velocity)
        if speed > self.params.max_speed:
            drone.velocity = (drone.velocity / speed) * self.params.max_speed

        # Оновлення позиції
        drone.position += drone.velocity * self.params.time_step * self.params.simulation_speed

        # Перевірка на зіткнення
        drone.state = DroneState.NORMAL
        for other in self.drones:
            if other is not drone:
                distance = np.linalg.norm(drone.position - other.position)
                if distance < self.params.min_distance * 0.8:
                    drone.state = DroneState.COLLISION
                    drone.collision_count += 1
                    self.total_collisions += 1
                    break

    def update_metrics(self):
        """Оновлення метрик симуляції"""
        total_distance = 0
        valid_pairs = 0
        total_speed_error = 0

        for i, drone in enumerate(self.drones):
            min_dist = float('inf')
            for j, other in enumerate(self.drones):
                if i != j:
                    distance = np.linalg.norm(drone.position - other.position)
                    total_distance += distance
                    valid_pairs += 1
                    if distance < min_dist:
                        min_dist = distance

            # Обчислення похибки швидкості
            total_speed_error += abs(np.linalg.norm(drone.velocity) - drone.pid.target)

        if valid_pairs > 0:
            self.avg_distance = total_distance / valid_pairs
            avg_speed_error = total_speed_error / len(self.drones)
        else:
            avg_speed_error = 0

        # Збереження метрик для історії
        self.metrics_history.append({
            'time': self.simulation_time,
            'avg_distance': self.avg_distance,
            'collisions': self.total_collisions,
            'avg_Kp': self.get_avg_pid_param('Kp'),
            'avg_Ki': self.get_avg_pid_param('Ki'),
            'avg_Kd': self.get_avg_pid_param('Kd'),
            'speed_error': avg_speed_error
        })

        # Оновлення історії PID параметрів
        self.avg_pid_history['Kp'].append(self.get_avg_pid_param('Kp'))
        self.avg_pid_history['Ki'].append(self.get_avg_pid_param('Ki'))
        self.avg_pid_history['Kd'].append(self.get_avg_pid_param('Kd'))
        self.avg_velocity_error.append(avg_speed_error)
        self.collision_history.append(self.total_collisions)

    def update_visuals(self):
        """Оновлення графічних елементів"""
        for i, drone in enumerate(self.drones):
            color = DRONE_COLOR if drone.state == DroneState.NORMAL else COLLISION_COLOR
            self.drone_plots[i].set_data([drone.position[0]], [drone.position[1]])
            self.drone_plots[i].set_color(color)

            if self.params.show_velocity and i < len(self.velocity_arrows):
                self.velocity_arrows[i].remove()
                arrow = self.ax.arrow(
                    drone.position[0], drone.position[1],
                    drone.velocity[0] * ARROW_SCALE, drone.velocity[1] * ARROW_SCALE,
                    color=VELOCITY_ARROW_COLOR, width=1, head_width=5
                )
                self.velocity_arrows[i] = arrow

        if self.params.show_metrics and self.metrics_text is not None:
            self.metrics_text.set_text(self.get_metrics_string())

        # Оновлення графіків PID
        if self.params.show_pid_graphs and len(self.metrics_history) > 0:
            times = [m['time'] for m in self.metrics_history]

            self.pid_lines['Kp'].set_data(times, self.avg_pid_history['Kp'])
            self.pid_lines['Ki'].set_data(times, self.avg_pid_history['Ki'])
            self.pid_lines['Kd'].set_data(times, self.avg_pid_history['Kd'])
            self.pid_ax.relim()
            self.pid_ax.autoscale_view()

            self.error_line.set_data(times, self.avg_velocity_error)
            self.error_ax.relim()
            self.error_ax.autoscale_view()

            self.collision_line.set_data(times, self.collision_history)
            self.collision_ax.relim()
            self.collision_ax.autoscale_view()

    def update(self, frame):
        """Оновлення стану симуляції для анімації"""
        for drone in self.drones:
            self.update_drone(drone)

        self.update_metrics()
        self.simulation_time += self.params.time_step * self.params.simulation_speed
        self.frame_count += 1

        # Оновлення візуалізації
        self.update_visuals()

        # Повертаємо всі елементи, які потрібно оновити
        return self.drone_plots + self.velocity_arrows + [self.metrics_text] + \
               list(self.pid_lines.values()) + [self.error_line, self.collision_line]

    def run_simulation(self):
        """Запуск симуляції з анімацією"""
        self.animation = animation.FuncAnimation(
            self.fig, self.update,
            frames=200, interval=1000/FPS, blit=True
        )

        plt.close()  # Запобігаємо подвійному відображенню

        return HTML(self.animation.to_jshtml())

# Параметри за замовчуванням
params = SimulationParameters(
    num_drones=20,
    circle_radius=200,
    desired_distance=40,
    min_distance=20,
    max_speed=2.5,
    separation_weight=1.5,
    alignment_weight=1.0,
    cohesion_weight=1.0,
    circle_weight=0.7,
    perception_radius=100,
    show_velocity=True,
    show_metrics=True,
    show_pid_graphs=True,
    # Параметри консенсусу PID
    alpha_p=0.1,
    alpha_i=0.1,
    alpha_d=0.1,
    beta_p=0.01,
    beta_i=0.01,
    beta_d=0.01
)

# Створення та запуск симуляції
simulation = DroneSimulation(params)
simulation.run_simulation()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.patches import Circle
from IPython.display import HTML
import random
import math
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass, field
from enum import Enum

import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 500

# Константи
DEFAULT_WIDTH = 800
DEFAULT_HEIGHT = 600
DRONE_COLOR = 'blue'
NEAR_COLLISION_COLOR = 'orange'
COLLISION_COLOR = 'red'
TRAJECTORY_COLOR = 'lightgray'
VELOCITY_ARROW_COLOR = 'red'
TEXT_COLOR = 'black'
DRONE_RADIUS = 5
ARROW_SCALE = 10
FPS = 60
NEAR_COLLISION_DISTANCE_FACTOR = 1.2  # Відстань для "майже зіткнення"

class DroneState(Enum):
    NORMAL = 1
    NEAR_COLLISION = 2  # Новий стан для "майже зіткнень"
    COLLISION = 3

@dataclass
class PIDController:
    Kp: float = 0.5
    Ki: float = 0.1
    Kd: float = 0.2
    target: float = 2.0
    integral: float = 0.0
    prev_error: float = 0.0
    dt: float = 1.0

    def compute(self, current_value: float) -> float:
        """Обчислення вихідного сигналу PID-контролера"""
        error = self.target - current_value
        self.integral += error * self.dt
        derivative = (error - self.prev_error) / self.dt
        output = self.Kp * error + self.Ki * self.integral + self.Kd * derivative
        self.prev_error = error
        return output

    def update_params(self, Kp: float, Ki: float, Kd: float):
        """Оновлення параметрів PID"""
        self.Kp = Kp
        self.Ki = Ki
        self.Kd = Kd

@dataclass
class Drone:
    position: np.ndarray
    velocity: np.ndarray
    state: DroneState = DroneState.NORMAL
    neighbors: List['Drone'] = field(default_factory=list)
    collision_count: int = 0
    near_collision_count: int = 0  # Лічильник "майже зіткнень"
    pid: PIDController = field(default_factory=PIDController)
    communication_radius: float = 150
    pid_params_history: List[Tuple[float, float, float]] = field(default_factory=list)
    collision_risk: float = 0.0  # Оцінка ризику зіткнення (0-1)

@dataclass
class SimulationParameters:
    num_drones: int = 20
    circle_radius: float = 250
    circle_center: Tuple[float, float] = (DEFAULT_WIDTH//2, DEFAULT_HEIGHT//2)
    desired_distance: float = 40
    min_distance: float = 20
    max_speed: float = 3
    max_acceleration: float = 0.5  # Максимальне прискорення
    time_step: float = 1.0
    # Параметри модифікованого алгоритму Бойдза
    separation_weight: float = 1.5
    alignment_weight: float = 1.0
    cohesion_weight: float = 1.0
    circle_weight: float = 0.5
    perception_radius: float = 100
    # Параметри для уникнення зіткнень
    collision_avoidance_radius: float = 50  # Радіус для активного уникнення
    avoidance_strength: float = 2.0  # Сила уникнення
    predictive_avoidance: bool = True  # Використовувати прогнозування зіткнень
    # Візуалізація
    show_velocity: bool = False
    show_metrics: bool = True
    show_pid_graphs: bool = True
    show_collision_zones: bool = True  # Показувати зони ризику зіткнення
    simulation_speed: float = 1.0
    # Параметри консенсусу PID
    alpha_p: float = 0.1
    alpha_i: float = 0.1
    alpha_d: float = 0.1
    beta_p: float = 0.01
    beta_i: float = 0.01
    beta_d: float = 0.01
    # Параметри для оптимізації консенсусу з урахуванням зіткнень
    risk_aware_consensus: bool = True  # Враховувати ризик зіткнення
    min_consensus_speed: float = 0.01  # Мінімальна швидкість консенсусу
    success_weight: float = 0.5  # Вага успішності сусідів

class DroneSimulation:
    def __init__(self, params: SimulationParameters):
        self.params = params
        self.fig = plt.figure(figsize=(14, 8))

        # Основний графік для симуляції
        self.ax = self.fig.add_subplot(121)
        self.ax.set_xlim(0, DEFAULT_WIDTH)
        self.ax.set_ylim(0, DEFAULT_HEIGHT)
        self.ax.set_aspect('equal')
        self.ax.grid(True, linestyle='--', alpha=0.5)
        self.ax.set_title('Симуляція руху зграї дронів з уникненням зіткнень')

        # Графіки для метрик
        self.pid_ax = self.fig.add_subplot(322)
        self.pid_ax.set_title('Середні значення PID параметрів')
        self.pid_ax.grid(True)

        self.error_ax = self.fig.add_subplot(324)
        self.error_ax.set_title('Похибка швидкості та ризик зіткнення')
        self.error_ax.grid(True)

        self.collision_ax = self.fig.add_subplot(326)
        self.collision_ax.set_title('Зіткнення та майже зіткнення')
        self.collision_ax.grid(True)

        self.drones: List[Drone] = []
        self.total_collisions = 0
        self.total_near_collisions = 0
        self.min_distance = float('inf')
        self.avg_distance = 0
        self.simulation_time = 0
        self.frame_count = 0
        self.metrics_history = []
        self.avg_pid_history = {'Kp': [], 'Ki': [], 'Kd': []}
        self.avg_velocity_error = []
        self.avg_collision_risk = []
        self.collision_history = []
        self.near_collision_history = []
        self.animation = None

        # Елементи візуалізації
        self.trajectory_circle = Circle(
            self.params.circle_center, self.params.circle_radius,
            fill=False, color=TRAJECTORY_COLOR, linestyle='--'
        )
        self.ax.add_patch(self.trajectory_circle)

        self.drone_plots = []
        self.velocity_arrows = []
        self.collision_zones = []  # Кола для відображення зон ризику
        self.metrics_text = None
        self.pid_lines = {'Kp': None, 'Ki': None, 'Kd': None}
        self.error_line = None
        self.risk_line = None
        self.collision_line = None
        self.near_collision_line = None

        self.initialize_drones()
        self.setup_visuals()

    def initialize_drones(self):
        """Ініціалізація дронів з випадковими PID-параметрами"""
        self.drones = []
        center_x, center_y = self.params.circle_center

        for _ in range(self.params.num_drones):
            # Випадковий кут і відстань від центру
            angle = random.uniform(0, 2 * math.pi)
            distance = random.uniform(0, self.params.circle_radius * 0.8)

            x = center_x + distance * math.cos(angle)
            y = center_y + distance * math.sin(angle)

            # Випадкова початкова швидкість
            speed = random.uniform(0.5, self.params.max_speed)
            vx = random.uniform(-1, 1)
            vy = random.uniform(-1, 1)
            velocity = np.array([vx, vy], dtype=np.float64)
            if np.linalg.norm(velocity) > 0:
                velocity = velocity / np.linalg.norm(velocity) * speed

            # Випадкові початкові PID-параметри
            Kp = random.uniform(0.1, 1.0)
            Ki = random.uniform(0.01, 0.2)
            Kd = random.uniform(0.05, 0.3)

            self.drones.append(Drone(
                position=np.array([x, y], dtype=np.float64),
                velocity=velocity,
                pid=PIDController(Kp=Kp, Ki=Ki, Kd=Kd, dt=self.params.time_step)
            ))

    def setup_visuals(self):
        """Налаштування графічних елементів"""
        # Очистити попередні елементи
        for arrow in self.velocity_arrows:
            arrow.remove()
        self.velocity_arrows.clear()

        for plot in self.drone_plots:
            plot.remove()
        self.drone_plots.clear()

        for zone in self.collision_zones:
            zone.remove()
        self.collision_zones.clear()

        # Створити нові елементи
        for drone in self.drones:
            color = DRONE_COLOR
            if drone.state == DroneState.NEAR_COLLISION:
                color = NEAR_COLLISION_COLOR
            elif drone.state == DroneState.COLLISION:
                color = COLLISION_COLOR

            drone_plot = self.ax.plot(
                drone.position[0], drone.position[1],
                'o', color=color, markersize=DRONE_RADIUS
            )[0]
            self.drone_plots.append(drone_plot)

            if self.params.show_velocity:
                arrow = self.ax.arrow(
                    drone.position[0], drone.position[1],
                    drone.velocity[0] * ARROW_SCALE, drone.velocity[1] * ARROW_SCALE,
                    color=VELOCITY_ARROW_COLOR, width=1, head_width=5
                )
                self.velocity_arrows.append(arrow)

            if self.params.show_collision_zones:
                zone = Circle(
                    (drone.position[0], drone.position[1]),
                    self.params.min_distance * NEAR_COLLISION_DISTANCE_FACTOR,
                    fill=False, color='orange', alpha=0.3, linestyle='--'
                )
                self.ax.add_patch(zone)
                self.collision_zones.append(zone)

        # Текст метрик
        if self.params.show_metrics:
            if self.metrics_text is not None:
                self.metrics_text.remove()

            metrics_str = self.get_metrics_string()
            self.metrics_text = self.ax.text(
                0.02, 0.95, metrics_str,
                transform=self.ax.transAxes,
                fontsize=10, color=TEXT_COLOR,
                bbox=dict(facecolor='white', alpha=0.7, edgecolor='gray')
            )

        # Ініціалізація графіків
        if self.params.show_pid_graphs:
            time_points = [self.simulation_time]

            self.pid_lines['Kp'], = self.pid_ax.plot(time_points, [self.get_avg_pid_param('Kp')], 'r-', label='Kp')
            self.pid_lines['Ki'], = self.pid_ax.plot(time_points, [self.get_avg_pid_param('Ki')], 'g-', label='Ki')
            self.pid_lines['Kd'], = self.pid_ax.plot(time_points, [self.get_avg_pid_param('Kd')], 'b-', label='Kd')
            self.pid_ax.legend()

            self.error_line, = self.error_ax.plot(time_points, [0], 'm-', label='Похибка швидкості')
            self.risk_line, = self.error_ax.plot(time_points, [0], 'c-', label='Ризик зіткнення')
            self.error_ax.legend()

            self.collision_line, = self.collision_ax.plot(time_points, [0], 'r-', label='Зіткнення')
            self.near_collision_line, = self.collision_ax.plot(time_points, [0], 'y-', label='Майже зіткнення')
            self.collision_ax.legend()

    def get_metrics_string(self):
        """Формування рядка з метриками"""
        return (
            f"Час: {self.simulation_time:.1f} с\n"
            f"Дронів: {self.params.num_drones}\n"
            f"Зіткнень: {self.total_collisions}\n"
            f"Майже зіткнень: {self.total_near_collisions}\n"
            f"Мін. відстань: {self.min_distance:.1f}\n"
            f"Середня відстань: {self.avg_distance:.1f}\n"
            f"Kp: {self.get_avg_pid_param('Kp'):.3f}\n"
            f"Ki: {self.get_avg_pid_param('Ki'):.3f}\n"
            f"Kd: {self.get_avg_pid_param('Kd'):.3f}"
        )

    def get_avg_pid_param(self, param: str) -> float:
        """Отримання середнього значення PID-параметра по зграї"""
        if not self.drones:
            return 0.0
        if param == 'Kp':
            return sum(d.pid.Kp for d in self.drones) / len(self.drones)
        elif param == 'Ki':
            return sum(d.pid.Ki for d in self.drones) / len(self.drones)
        elif param == 'Kd':
            return sum(d.pid.Kd for d in self.drones) / len(self.drones)
        return 0.0

    def find_neighbors(self, drone: Drone) -> List[Drone]:
        """Знаходження сусідів для даного дрона"""
        neighbors = []
        for other in self.drones:
            if other is not drone:
                distance = np.linalg.norm(drone.position - other.position)
                if distance < drone.communication_radius:
                    neighbors.append(other)
        return neighbors

    def calculate_collision_risk(self, drone: Drone) -> float:
        """Розрахунок ризику зіткнення для дрона"""
        min_risk_distance = self.params.min_distance * NEAR_COLLISION_DISTANCE_FACTOR
        risk = 0.0

        for other in drone.neighbors:
            distance = np.linalg.norm(drone.position - other.position)
            if distance < min_risk_distance:
                # Розрахунок відносної швидкості
                relative_velocity = drone.velocity - other.velocity
                relative_speed = np.linalg.norm(relative_velocity)

                # Час до потенційного зіткнення (якщо рухаються один на одного)
                time_to_collision = distance / relative_speed if relative_speed > 0 else float('inf')

                # Ризик зіткнення (більше для менших відстаней і часу)
                current_risk = (min_risk_distance - distance) / min_risk_distance
                if time_to_collision < 5:  # 5 кроків симуляції
                    current_risk += (5 - time_to_collision) / 5

                risk = max(risk, min(current_risk, 1.0))  # Обмеження ризику до 1.0

        return risk

    def apply_enhanced_separation(self, drone: Drone) -> np.ndarray:
        """Посилене правило розділення з уникненням зіткнень"""
        steering = np.zeros(2)
        total = 0
        min_distance = float('inf')

        for other in drone.neighbors:
            distance = np.linalg.norm(drone.position - other.position)
            min_distance = min(min_distance, distance)

            if distance < self.params.collision_avoidance_radius:
                diff = drone.position - other.position
                if distance > 0:
                    diff = diff / distance  # Нормалізація

                # Нелінійне збільшення сили на близьких відстанях
                avoidance_factor = 1.0
                if distance < self.params.min_distance * 2:
                    avoidance_factor = (self.params.min_distance * 2 - distance) / self.params.min_distance

                # Врахування відносної швидкості для прогнозування
                if self.params.predictive_avoidance:
                    relative_velocity = drone.velocity - other.velocity
                    if np.linalg.norm(relative_velocity) > 0:
                        # Додавання компоненти, що враховує напрямок відносної швидкості
                        predictive_factor = np.dot(diff, relative_velocity) / (distance * np.linalg.norm(relative_velocity))
                        avoidance_factor *= (1 + max(0, predictive_factor))

                steering += diff * avoidance_factor
                total += 1

        if total > 0:
            steering = steering / total
            if np.linalg.norm(steering) > 0:
                steering = steering / np.linalg.norm(steering) * self.params.max_speed
                steering = steering - drone.velocity
                if np.linalg.norm(steering) > 0:
                    steering = steering / np.linalg.norm(steering)

        # Збереження ризику зіткнення для дрона
        drone.collision_risk = 1.0 - min(1.0, min_distance / (self.params.min_distance * NEAR_COLLISION_DISTANCE_FACTOR))

        return steering * self.params.separation_weight * (1 + drone.collision_risk * self.params.avoidance_strength)

    def apply_alignment(self, drone: Drone) -> np.ndarray:
        """Правило вирівнювання з урахуванням ризику зіткнення"""
        steering = np.zeros(2)
        total = 0

        for other in drone.neighbors:
            # Зменшення впливу дронів з високим ризиком зіткнення
            weight = 1.0 - other.collision_risk if self.params.risk_aware_consensus else 1.0
            steering += other.velocity * weight
            total += weight

        if total > 0:
            steering = steering / total
            if np.linalg.norm(steering) > 0:
                steering = steering / np.linalg.norm(steering) * self.params.max_speed
            steering = steering - drone.velocity
            if np.linalg.norm(steering) > 0:
                steering = steering / np.linalg.norm(steering)

        return steering * self.params.alignment_weight * (1 - drone.collision_risk * 0.5)

    def apply_cohesion(self, drone: Drone) -> np.ndarray:
        """Правило згуртованості з урахуванням ризику зіткнення"""
        steering = np.zeros(2)
        total = 0

        for other in drone.neighbors:
            # Зменшення впливу дронів з високим ризиком зіткнення
            weight = 1.0 - other.collision_risk if self.params.risk_aware_consensus else 1.0
            steering += other.position * weight
            total += weight

        if total > 0:
            steering = steering / total  # Центр маси
            steering = steering - drone.position  # Вектор до центру
            if np.linalg.norm(steering) > 0:
                steering = steering / np.linalg.norm(steering) * self.params.max_speed
            steering = steering - drone.velocity
            if np.linalg.norm(steering) > 0:
                steering = steering / np.linalg.norm(steering)

        return steering * self.params.cohesion_weight * (1 - drone.collision_risk * 0.3)

    def apply_circle_movement(self, drone: Drone) -> np.ndarray:
        """Додаткове правило для руху по колу з урахуванням ризику"""
        center_x, center_y = self.params.circle_center
        center = np.array([center_x, center_y])

        to_center = center - drone.position
        distance_to_center = np.linalg.norm(to_center)

        if distance_to_center > 0:
            to_center_normalized = to_center / distance_to_center
            desired_velocity = np.array([-to_center_normalized[1], to_center_normalized[0]])
            desired_velocity = desired_velocity * self.params.max_speed

            radius_error = distance_to_center - self.params.circle_radius
            correction_factor = 0.05 * radius_error

            steering = desired_velocity - drone.velocity + to_center_normalized * correction_factor

            if np.linalg.norm(steering) > 0:
                steering = steering / np.linalg.norm(steering)

            # Зменшення впливу руху по колу при високому ризику зіткнення
            return steering * self.params.circle_weight * (1 - drone.collision_risk * 0.7)
        return np.zeros(2)

    def update_pid_consensus(self, drone: Drone):
        """Оптимізований алгоритм консенсусу для PID-параметрів з уникненням зіткнень"""
        if not drone.neighbors:
            return

        # Розрахунок успішності сусідів (менше зіткнень = більш успішний)
        neighbor_success = []
        for n in drone.neighbors:
            success = 1.0 - min(1.0, n.collision_count / (self.simulation_time + 1))
            neighbor_success.append(success)

        total_success = sum(neighbor_success)
        if total_success == 0:
            return

        # Середньозважене значення параметрів сусідів
        avg_Kp = sum(n.pid.Kp * s for n, s in zip(drone.neighbors, neighbor_success)) / total_success
        avg_Ki = sum(n.pid.Ki * s for n, s in zip(drone.neighbors, neighbor_success)) / total_success
        avg_Kd = sum(n.pid.Kd * s for n, s in zip(drone.neighbors, neighbor_success)) / total_success

        # Адаптація швидкості консенсусу залежно від ризику зіткнення
        alpha_p = max(self.params.min_consensus_speed,
                     self.params.alpha_p * (1 - drone.collision_risk * 0.7))
        alpha_i = max(self.params.min_consensus_speed,
                     self.params.alpha_i * (1 - drone.collision_risk * 0.7))
        alpha_d = max(self.params.min_consensus_speed,
                     self.params.alpha_d * (1 - drone.collision_risk * 0.7))

        # Похибка швидкості для локальної адаптації
        speed_error = abs(np.linalg.norm(drone.velocity) - drone.pid.target)

        # Штрафний член для параметрів при високому ризику зіткнення
        penalty = drone.collision_risk * 0.5

        # Оновлення параметрів за оптимізованим алгоритмом консенсусу
        new_Kp = drone.pid.Kp + alpha_p * (avg_Kp - drone.pid.Kp) + \
                self.params.beta_p * speed_error + penalty
        new_Ki = drone.pid.Ki + alpha_i * (avg_Ki - drone.pid.Ki) + \
                self.params.beta_i * speed_error - penalty * 0.5
        new_Kd = drone.pid.Kd + alpha_d * (avg_Kd - drone.pid.Kd) + \
                self.params.beta_d * speed_error + penalty * 0.3

        # Обмеження значень параметрів
        new_Kp = max(0.01, min(2.0, new_Kp))
        new_Ki = max(0.001, min(0.5, new_Ki))
        new_Kd = max(0.01, min(1.0, new_Kd))

        # Оновлення PID-контролера
        drone.pid.update_params(new_Kp, new_Ki, new_Kd)

        # Збереження історії параметрів
        drone.pid_params_history.append((new_Kp, new_Ki, new_Kd))

    def update_drone(self, drone: Drone):
        """Оновлення стану одного дрона з розширеним виявленням зіткнень"""
        # Знаходження сусідів
        drone.neighbors = self.find_neighbors(drone)

        # Оновлення PID-параметрів за оптимізованим алгоритмом консенсусу
        self.update_pid_consensus(drone)

        # Обчислення векторів правил Бойдза з уникненням зіткнень
        separation = self.apply_enhanced_separation(drone)
        alignment = self.apply_alignment(drone)
        cohesion = self.apply_cohesion(drone)
        circle_move = self.apply_circle_movement(drone)

        # Комбінування правил
        acceleration = separation + alignment + cohesion + circle_move

        # Обмеження прискорення
        accel_norm = np.linalg.norm(acceleration)
        if accel_norm > self.params.max_acceleration:
            acceleration = acceleration / accel_norm * self.params.max_acceleration

        # Обчислення PID-корекції швидкості
        current_speed = np.linalg.norm(drone.velocity)
        pid_correction = drone.pid.compute(current_speed)

        # Застосування PID-корекції до прискорення
        if current_speed > 0:
            velocity_direction = drone.velocity / current_speed
            acceleration += velocity_direction * pid_correction

        # Оновлення швидкості
        drone.velocity += acceleration * self.params.time_step

        # Обмеження швидкості
        speed = np.linalg.norm(drone.velocity)
        if speed > self.params.max_speed:
            drone.velocity = (drone.velocity / speed) * self.params.max_speed

        # Оновлення позиції
        drone.position += drone.velocity * self.params.time_step * self.params.simulation_speed

        # Виявлення зіткнень та "майже зіткнень"
        drone.state = DroneState.NORMAL
        min_distance = float('inf')

        for other in self.drones:
            if other is not drone:
                distance = np.linalg.norm(drone.position - other.position)
                min_distance = min(min_distance, distance)

                if distance < self.params.min_distance * 0.8:
                    drone.state = DroneState.COLLISION
                    drone.collision_count += 1
                    self.total_collisions += 1
                    break
                elif distance < self.params.min_distance * NEAR_COLLISION_DISTANCE_FACTOR:
                    drone.state = DroneState.NEAR_COLLISION
                    drone.near_collision_count += 1
                    self.total_near_collisions += 1

        # Оновлення мінімальної відстані
        if min_distance < self.min_distance:
            self.min_distance = min_distance

    def update_metrics(self):
        """Оновлення метрик симуляції"""
        total_distance = 0
        valid_pairs = 0
        total_speed_error = 0
        total_collision_risk = 0

        for i, drone in enumerate(self.drones):
            min_dist = float('inf')
            for j, other in enumerate(self.drones):
                if i != j:
                    distance = np.linalg.norm(drone.position - other.position)
                    total_distance += distance
                    valid_pairs += 1
                    if distance < min_dist:
                        min_dist = distance

            # Обчислення похибки швидкості
            total_speed_error += abs(np.linalg.norm(drone.velocity) - drone.pid.target)

            # Сумування ризику зіткнення
            total_collision_risk += drone.collision_risk

        if valid_pairs > 0:
            self.avg_distance = total_distance / valid_pairs
            avg_speed_error = total_speed_error / len(self.drones)
            avg_collision_risk = total_collision_risk / len(self.drones)
        else:
            avg_speed_error = 0
            avg_collision_risk = 0

        # Збереження метрик для історії
        self.metrics_history.append({
            'time': self.simulation_time,
            'avg_distance': self.avg_distance,
            'min_distance': self.min_distance,
            'collisions': self.total_collisions,
            'near_collisions': self.total_near_collisions,
            'avg_Kp': self.get_avg_pid_param('Kp'),
            'avg_Ki': self.get_avg_pid_param('Ki'),
            'avg_Kd': self.get_avg_pid_param('Kd'),
            'speed_error': avg_speed_error,
            'collision_risk': avg_collision_risk
        })

        # Оновлення історії PID параметрів
        self.avg_pid_history['Kp'].append(self.get_avg_pid_param('Kp'))
        self.avg_pid_history['Ki'].append(self.get_avg_pid_param('Ki'))
        self.avg_pid_history['Kd'].append(self.get_avg_pid_param('Kd'))
        self.avg_velocity_error.append(avg_speed_error)
        self.avg_collision_risk.append(avg_collision_risk)
        self.collision_history.append(self.total_collisions)
        self.near_collision_history.append(self.total_near_collisions)

    def update_visuals(self):
        """Оновлення графічних елементів"""
        for i, drone in enumerate(self.drones):
            color = DRONE_COLOR
            if drone.state == DroneState.NEAR_COLLISION:
                color = NEAR_COLLISION_COLOR
            elif drone.state == DroneState.COLLISION:
                color = COLLISION_COLOR

            self.drone_plots[i].set_data([drone.position[0]], [drone.position[1]])
            self.drone_plots[i].set_color(color)

            if self.params.show_velocity and i < len(self.velocity_arrows):
                self.velocity_arrows[i].remove()
                arrow = self.ax.arrow(
                    drone.position[0], drone.position[1],
                    drone.velocity[0] * ARROW_SCALE, drone.velocity[1] * ARROW_SCALE,
                    color=VELOCITY_ARROW_COLOR, width=1, head_width=5
                )
                self.velocity_arrows[i] = arrow

            if self.params.show_collision_zones and i < len(self.collision_zones):
                self.collision_zones[i].center = (drone.position[0], drone.position[1])
                self.collision_zones[i].set_radius(self.params.min_distance * NEAR_COLLISION_DISTANCE_FACTOR)

        if self.params.show_metrics and self.metrics_text is not None:
            self.metrics_text.set_text(self.get_metrics_string())

        # Оновлення графіків
        if self.params.show_pid_graphs and len(self.metrics_history) > 0:
            times = [m['time'] for m in self.metrics_history]

            self.pid_lines['Kp'].set_data(times, self.avg_pid_history['Kp'])
            self.pid_lines['Ki'].set_data(times, self.avg_pid_history['Ki'])
            self.pid_lines['Kd'].set_data(times, self.avg_pid_history['Kd'])
            self.pid_ax.relim()
            self.pid_ax.autoscale_view()

            self.error_line.set_data(times, self.avg_velocity_error)
            self.risk_line.set_data(times, self.avg_collision_risk)
            self.error_ax.relim()
            self.error_ax.autoscale_view()

            self.collision_line.set_data(times, self.collision_history)
            self.near_collision_line.set_data(times, self.near_collision_history)
            self.collision_ax.relim()
            self.collision_ax.autoscale_view()

    def update(self, frame):
        """Оновлення стану симуляції для анімації"""
        # Скидання мінімальної відстані для нового кадру
        self.min_distance = float('inf')

        for drone in self.drones:
            self.update_drone(drone)

        self.update_metrics()
        self.simulation_time += self.params.time_step * self.params.simulation_speed
        self.frame_count += 1

        # Оновлення візуалізації
        self.update_visuals()

        # Повертаємо всі елементи, які потрібно оновити
        return self.drone_plots + self.velocity_arrows + self.collision_zones + \
               [self.metrics_text] + list(self.pid_lines.values()) + \
               [self.error_line, self.risk_line, self.collision_line, self.near_collision_line]

    def run_simulation(self):
        """Запуск симуляції з анімацією"""
        self.animation = animation.FuncAnimation(
            self.fig, self.update,
            frames=200, interval=1000/FPS, blit=True
        )

        plt.close()  # Запобігаємо подвійному відображенню

        return HTML(self.animation.to_jshtml())

# Параметри симуляції з акцентом на уникнення зіткнень
params = SimulationParameters(
    num_drones=25,
    circle_radius=200,
    desired_distance=35,
    min_distance=15,
    max_speed=2.5,
    max_acceleration=0.3,
    # Параметри алгоритму Бойдза
    separation_weight=2.0,  # Збільшена вага розділення
    alignment_weight=0.8,
    cohesion_weight=0.8,
    circle_weight=0.5,
    perception_radius=80,
    # Параметри уникнення зіткнень
    collision_avoidance_radius=60,
    avoidance_strength=2.5,
    predictive_avoidance=True,
    # Візуалізація
    show_velocity=True,
    show_metrics=True,
    show_pid_graphs=True,
    show_collision_zones=True,
    simulation_speed=1.0,
    # Параметри консенсусу PID
    alpha_p=0.15,
    alpha_i=0.15,
    alpha_d=0.15,
    beta_p=0.02,
    beta_i=0.02,
    beta_d=0.02,
    # Оптимізація консенсусу для уникнення зіткнень
    risk_aware_consensus=True,
    min_consensus_speed=0.02,
    success_weight=0.7
)

# Створення та запуск симуляції
simulation = DroneSimulation(params)
simulation.run_simulation()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.patches import Circle
from IPython.display import HTML
import random
import math
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass, field
from enum import Enum
from collections import deque

import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 500

# Константи
DEFAULT_WIDTH = 800
DEFAULT_HEIGHT = 600
DRONE_COLOR = 'blue'
NEAR_COLLISION_COLOR = 'orange'
COLLISION_COLOR = 'red'
TRAJECTORY_COLOR = 'lightgray'
VELOCITY_ARROW_COLOR = 'red'
TEXT_COLOR = 'black'
DRONE_RADIUS = 5
ARROW_SCALE = 10
FPS = 60
HISTORY_LENGTH = 50

class DroneState(Enum):
    NORMAL = 1
    NEAR_COLLISION = 2
    COLLISION = 3
    AVOIDING = 4

@dataclass
class PIDController:
    Kp: float = 0.5
    Ki: float = 0.1
    Kd: float = 0.2
    target: float = 2.0
    integral: float = 0.0
    prev_error: float = 0.0
    dt: float = 1.0

    def compute(self, current_value: float) -> float:
        """Обчислення вихідного сигналу PID-контролера"""
        error = self.target - current_value
        self.integral += error * self.dt
        derivative = (error - self.prev_error) / self.dt
        output = self.Kp * error + self.Ki * self.integral + self.Kd * derivative
        self.prev_error = error
        return output

    def update_params(self, Kp: float, Ki: float, Kd: float):
        """Оновлення параметрів PID"""
        self.Kp = Kp
        self.Ki = Ki
        self.Kd = Kd

@dataclass
class Drone:
    id: int
    position: np.ndarray  # [x, y, z]
    velocity: np.ndarray  # [vx, vy, vz]
    state: DroneState = DroneState.NORMAL
    neighbors: List['Drone'] = field(default_factory=list)
    collision_count: int = 0
    near_collision_count: int = 0
    pid: PIDController = field(default_factory=PIDController)
    communication_radius: float = 150
    pid_params_history: deque = field(default_factory=lambda: deque(maxlen=HISTORY_LENGTH))
    collision_risk: float = 0.0
    z_target: float = 0.0
    avoidance_direction: float = 0.0  # +1 або -1 для напрямку уникнення
    intention: np.ndarray = field(default_factory=lambda: np.zeros(3))  # Намір руху
    performance_score: float = 1.0  # Оцінка ефективності (менше зіткнень = вище оцінка)

@dataclass
class SimulationParameters:
    # Основні параметри
    num_drones: int = 20
    circle_radius: float = 250
    circle_center: Tuple[float, float] = (DEFAULT_WIDTH//2, DEFAULT_HEIGHT//2)
    desired_distance: float = 40
    min_distance: float = 20
    max_speed: float = 3
    max_acceleration: float = 0.5
    time_step: float = 1.0

    # Параметри алгоритму Бойдза
    boids_weights: Dict[str, float] = field(default_factory=lambda: {
        'separation': 1.5,
        'alignment': 1.0,
        'cohesion': 1.0,
        'circle': 0.5
    })
    boids_radii: Dict[str, float] = field(default_factory=lambda: {
        'separation': 50,
        'alignment': 100,
        'cohesion': 100
    })

    # Параметри уникнення зіткнень
    z_avoidance_distance: float = 30
    z_displacement: float = 50
    z_return_speed: float = 0.5
    avoidance_strength: float = 2.0
    predictive_avoidance: bool = True
    avoidance_decay: float = 0.95  # Швидкість зменшення ефекту уникнення

    # Параметри консенсусу PID
    local_consensus_weight: float = 0.7
    global_consensus_weight: float = 0.3
    global_consensus_interval: int = 10
    adaptation_rate: float = 0.01
    min_Kp: float = 0.1
    max_Kp: float = 2.0
    min_Ki: float = 0.01
    max_Ki: float = 0.5
    min_Kd: float = 0.05
    max_Kd: float = 1.0

    # Візуалізація
    show_velocity: bool = False
    show_metrics: bool = True
    show_pid_graphs: bool = True
    show_collision_zones: bool = True
    show_z_coordinate: bool = True
    simulation_speed: float = 1.0

class DroneSimulation:
    def __init__(self, params: SimulationParameters):
        self.params = params
        self.fig = plt.figure(figsize=(16, 8))

        # Основний графік для симуляції
        self.ax = self.fig.add_subplot(121)
        self.ax.set_xlim(0, DEFAULT_WIDTH)
        self.ax.set_ylim(0, DEFAULT_HEIGHT)
        self.ax.set_aspect('equal')
        self.ax.grid(True, linestyle='--', alpha=0.5)
        self.ax.set_title('Симуляція руху зграї дронів з 3D уникненням зіткнень')

        # Графіки для метрик
        self.pid_ax = self.fig.add_subplot(322)
        self.pid_ax.set_title('Середні значення PID параметрів')
        self.pid_ax.grid(True)

        self.error_ax = self.fig.add_subplot(324)
        self.error_ax.set_title('Похибка швидкості та ризик зіткнення')
        self.error_ax.grid(True)

        self.collision_ax = self.fig.add_subplot(326)
        self.collision_ax.set_title('Зіткнення та майже зіткнень')
        self.collision_ax.grid(True)

        self.drones: List[Drone] = []
        self.total_collisions = 0
        self.total_near_collisions = 0
        self.min_distance = float('inf')
        self.avg_distance = 0
        self.simulation_time = 0
        self.frame_count = 0
        self.global_consensus_counter = 0
        self.metrics_history = []
        self.avg_pid_history = {'Kp': [], 'Ki': [], 'Kd': []}
        self.avg_velocity_error = []
        self.avg_collision_risk = []
        self.collision_history = []
        self.near_collision_history = []
        self.z_usage_history = []
        self.animation = None

        # Елементи візуалізації
        self.trajectory_circle = Circle(
            self.params.circle_center, self.params.circle_radius,
            fill=False, color=TRAJECTORY_COLOR, linestyle='--'
        )
        self.ax.add_patch(self.trajectory_circle)

        self.drone_plots = []
        self.velocity_arrows = []
        self.collision_zones = []
        self.z_indicator = []
        self.metrics_text = None
        self.pid_lines = {'Kp': None, 'Ki': None, 'Kd': None}
        self.error_line = None
        self.risk_line = None
        self.collision_line = None
        self.near_collision_line = None
        self.z_usage_line = None

        self.initialize_drones()
        self.setup_visuals()

    def initialize_drones(self):
        """Ініціалізація дронів з випадковими параметрами"""
        center_x, center_y = self.params.circle_center

        for i in range(self.params.num_drones):
            # Випадковий кут і відстань від центру
            angle = random.uniform(0, 2 * math.pi)
            distance = random.uniform(0, self.params.circle_radius * 0.8)

            x = center_x + distance * math.cos(angle)
            y = center_y + distance * math.sin(angle)
            z = 0  # Початкова Z-координата

            # Випадкова початкова швидкість
            speed = random.uniform(0.5, self.params.max_speed)
            vx = random.uniform(-1, 1)
            vy = random.uniform(-1, 1)
            vz = 0  # Початкова швидкість по Z
            velocity = np.array([vx, vy, vz], dtype=np.float64)
            if np.linalg.norm(velocity) > 0:
                velocity = velocity / np.linalg.norm(velocity) * speed

            # Випадкові початкові PID-параметри
            Kp = random.uniform(self.params.min_Kp, self.params.max_Kp)
            Ki = random.uniform(self.params.min_Ki, self.params.max_Ki)
            Kd = random.uniform(self.params.min_Kd, self.params.max_Kd)

            self.drones.append(Drone(
                id=i,
                position=np.array([x, y, z], dtype=np.float64),
                velocity=velocity,
                pid=PIDController(Kp=Kp, Ki=Ki, Kd=Kd, dt=self.params.time_step)
            ))

    def setup_visuals(self):
        """Налаштування графічних елементів"""
        # Очистити попередні елементи
        for arrow in self.velocity_arrows:
            arrow.remove()
        self.velocity_arrows.clear()

        for plot in self.drone_plots:
            plot.remove()
        self.drone_plots.clear()

        for zone in self.collision_zones:
            zone.remove()
        self.collision_zones.clear()

        for indicator in self.z_indicator:
            indicator.remove()
        self.z_indicator.clear()

        # Створити нові елементи
        for drone in self.drones:
            color = DRONE_COLOR
            if drone.state == DroneState.NEAR_COLLISION:
                color = NEAR_COLLISION_COLOR
            elif drone.state == DroneState.COLLISION:
                color = COLLISION_COLOR
            elif drone.state == DroneState.AVOIDING:
                color = 'purple'

            drone_plot = self.ax.plot(
                drone.position[0], drone.position[1],
                'o', color=color, markersize=DRONE_RADIUS
            )[0]
            self.drone_plots.append(drone_plot)

            if self.params.show_velocity:
                arrow = self.ax.arrow(
                    drone.position[0], drone.position[1],
                    drone.velocity[0] * ARROW_SCALE, drone.velocity[1] * ARROW_SCALE,
                    color=VELOCITY_ARROW_COLOR, width=1, head_width=5
                )
                self.velocity_arrows.append(arrow)

            if self.params.show_collision_zones:
                zone = Circle(
                    (drone.position[0], drone.position[1]),
                    self.params.z_avoidance_distance,
                    fill=False, color='orange', alpha=0.2, linestyle='--'
                )
                self.ax.add_patch(zone)
                self.collision_zones.append(zone)

            if self.params.show_z_coordinate:
                z_text = self.ax.text(
                    drone.position[0], drone.position[1],
                    f"{drone.position[2]:.0f}",
                    fontsize=8, color='black', ha='center', va='center'
                )
                self.z_indicator.append(z_text)

        # Текст метрик
        if self.params.show_metrics:
            if self.metrics_text is not None:
                self.metrics_text.remove()

            metrics_str = self.get_metrics_string()
            self.metrics_text = self.ax.text(
                0.02, 0.95, metrics_str,
                transform=self.ax.transAxes,
                fontsize=10, color=TEXT_COLOR,
                bbox=dict(facecolor='white', alpha=0.7, edgecolor='gray')
            )

        # Ініціалізація графіків
        if self.params.show_pid_graphs:
            time_points = [self.simulation_time]

            self.pid_lines['Kp'], = self.pid_ax.plot(time_points, [self.get_avg_pid_param('Kp')], 'r-', label='Kp')
            self.pid_lines['Ki'], = self.pid_ax.plot(time_points, [self.get_avg_pid_param('Ki')], 'g-', label='Ki')
            self.pid_lines['Kd'], = self.pid_ax.plot(time_points, [self.get_avg_pid_param('Kd')], 'b-', label='Kd')
            self.pid_ax.legend()

            self.error_line, = self.error_ax.plot(time_points, [0], 'm-', label='Похибка швидкості')
            self.risk_line, = self.error_ax.plot(time_points, [0], 'c-', label='Ризик зіткнення')
            self.error_ax.legend()

            self.collision_line, = self.collision_ax.plot(time_points, [0], 'r-', label='Зіткнення')
            self.near_collision_line, = self.collision_ax.plot(time_points, [0], 'y-', label='Майже зіткнення')
            self.z_usage_line, = self.collision_ax.plot(time_points, [0], 'b-', label='Використання Z')
            self.collision_ax.legend()

    def get_metrics_string(self):
        """Формування рядка з метриками"""
        return (
            f"Час: {self.simulation_time:.1f} с\n"
            f"Дронів: {self.params.num_drones}\n"
            f"Зіткнень: {self.total_collisions}\n"
            f"Майже зіткнень: {self.total_near_collisions}\n"
            f"Мін. відстань: {self.min_distance:.1f}\n"
            f"Середня відстань: {self.avg_distance:.1f}\n"
            f"Kp: {self.get_avg_pid_param('Kp'):.3f}\n"
            f"Ki: {self.get_avg_pid_param('Ki'):.3f}\n"
            f"Kd: {self.get_avg_pid_param('Kd'):.3f}"
        )

    def get_avg_pid_param(self, param: str) -> float:
        """Отримання середнього значення PID-параметра по зграї"""
        if not self.drones:
            return 0.0
        if param == 'Kp':
            return sum(d.pid.Kp for d in self.drones) / len(self.drones)
        elif param == 'Ki':
            return sum(d.pid.Ki for d in self.drones) / len(self.drones)
        elif param == 'Kd':
            return sum(d.pid.Kd for d in self.drones) / len(self.drones)
        return 0.0

    def find_neighbors(self, drone: Drone) -> List[Drone]:
        """Знаходження сусідів для даного дрона"""
        neighbors = []
        for other in self.drones:
            if other is not drone:
                distance = np.linalg.norm(drone.position[:2] - other.position[:2])  # Тільки XY-площина
                if distance < drone.communication_radius:
                    neighbors.append(other)
        return neighbors

    def calculate_collision_risk(self, drone: Drone) -> float:
        """Розрахунок ризику зіткнення для дрона"""
        min_risk_distance = self.params.z_avoidance_distance
        risk = 0.0

        for other in drone.neighbors:
            distance = np.linalg.norm(drone.position - other.position)  # 3D відстань
            if distance < min_risk_distance:
                # Розрахунок відносної швидкості
                relative_velocity = drone.velocity - other.velocity
                relative_speed = np.linalg.norm(relative_velocity)

                # Час до потенційного зіткнення
                time_to_collision = distance / relative_speed if relative_speed > 0 else float('inf')

                # Ризик зіткнення
                current_risk = (min_risk_distance - distance) / min_risk_distance
                if time_to_collision < 5:  # 5 кроків симуляції
                    current_risk += (5 - time_to_collision) / 5

                risk = max(risk, min(current_risk, 1.0))

        return risk

    def apply_enhanced_separation(self, drone: Drone) -> np.ndarray:
        """Посилене правило розділення з уникненням зіткнень"""
        steering = np.zeros(3)  # Тепер 3D вектор
        total = 0
        min_distance = float('inf')

        for other in drone.neighbors:
            distance = np.linalg.norm(drone.position - other.position)
            min_distance = min(min_distance, distance)

            if distance < self.params.boids_radii['separation']:
                diff = drone.position - other.position
                if distance > 0:
                    diff = diff / distance  # Нормалізація

                # Нелінійне збільшення сили на близьких відстанях
                avoidance_factor = 1.0
                if distance < self.params.min_distance * 2:
                    avoidance_factor = (self.params.min_distance * 2 - distance) / self.params.min_distance

                # Врахування відносної швидкості для прогнозування
                if self.params.predictive_avoidance:
                    relative_velocity = drone.velocity - other.velocity
                    if np.linalg.norm(relative_velocity) > 0:
                        predictive_factor = np.dot(diff, relative_velocity) / (distance * np.linalg.norm(relative_velocity))
                        avoidance_factor *= (1 + max(0, predictive_factor))

                steering += diff * avoidance_factor
                total += 1

        if total > 0:
            steering = steering / total
            if np.linalg.norm(steering) > 0:
                steering = steering / np.linalg.norm(steering) * self.params.max_speed
                steering = steering - drone.velocity
                if np.linalg.norm(steering) > 0:
                    steering = steering / np.linalg.norm(steering)

        # Збереження ризику зіткнення для дрона
        drone.collision_risk = 1.0 - min(1.0, min_distance / self.params.z_avoidance_distance)

        return steering * self.params.boids_weights['separation'] * (1 + drone.collision_risk * self.params.avoidance_strength)

    def apply_alignment(self, drone: Drone) -> np.ndarray:
        """Правило вирівнювання з урахуванням ризику зіткнення"""
        steering = np.zeros(3)
        total = 0

        for other in drone.neighbors:
            # Зменшення впливу дронів з високим ризиком зіткнення
            weight = 1.0 - other.collision_risk
            steering += other.velocity * weight
            total += weight

        if total > 0:
            steering = steering / total
            if np.linalg.norm(steering) > 0:
                steering = steering / np.linalg.norm(steering) * self.params.max_speed
            steering = steering - drone.velocity
            if np.linalg.norm(steering) > 0:
                steering = steering / np.linalg.norm(steering)

        return steering * self.params.boids_weights['alignment'] * (1 - drone.collision_risk * 0.5)

    def apply_cohesion(self, drone: Drone) -> np.ndarray:
        """Правило згуртованості з урахуванням ризику зіткнення"""
        steering = np.zeros(3)
        total = 0

        for other in drone.neighbors:
            # Зменшення впливу дронів з високим ризиком зіткнення
            weight = 1.0 - other.collision_risk
            steering += other.position * weight
            total += weight

        if total > 0:
            steering = steering / total  # Центр маси
            steering = steering - drone.position  # Вектор до центру
            if np.linalg.norm(steering) > 0:
                steering = steering / np.linalg.norm(steering) * self.params.max_speed
            steering = steering - drone.velocity
            if np.linalg.norm(steering) > 0:
                steering = steering / np.linalg.norm(steering)

        return steering * self.params.boids_weights['cohesion'] * (1 - drone.collision_risk * 0.3)

    def apply_circle_movement(self, drone: Drone) -> np.ndarray:
        """Додаткове правило для руху по колу"""
        center_x, center_y = self.params.circle_center
        center = np.array([center_x, center_y, 0])

        to_center = center - drone.position
        distance_to_center = np.linalg.norm(to_center[:2])  # Тільки XY-відстань

        if distance_to_center > 0:
            to_center_normalized = to_center / np.linalg.norm(to_center)
            desired_velocity = np.array([-to_center_normalized[1], to_center_normalized[0], 0])
            desired_velocity = desired_velocity * self.params.max_speed

            radius_error = distance_to_center - self.params.circle_radius
            correction_factor = 0.05 * radius_error

            steering = desired_velocity - drone.velocity + to_center_normalized * correction_factor

            if np.linalg.norm(steering) > 0:
                steering = steering / np.linalg.norm(steering)

            # Зменшення впливу руху по колу при високому ризику зіткнення
            return steering * self.params.boids_weights['circle'] * (1 - drone.collision_risk * 0.7)
        return np.zeros(3)

    def avoid_collision_3d(self, drone: Drone):
        """Механізм уникнення зіткнень з використанням Z-координати"""
        if drone.state == DroneState.AVOIDING:
            # Плавне повернення до z=0
            if abs(drone.position[2]) > 0.1:
                drone.velocity[2] = -np.sign(drone.position[2]) * self.params.z_return_speed
            else:
                drone.position[2] = 0
                drone.velocity[2] = 0
                drone.state = DroneState.NORMAL
            return

        # Перевірка на необхідність уникнення
        for other in drone.neighbors:
            distance_xy = np.linalg.norm(drone.position[:2] - other.position[:2])
            if distance_xy < self.params.z_avoidance_distance:
                # Визначення напрямку уникнення
                if drone.avoidance_direction == 0:
                    drone.avoidance_direction = 1 if random.random() > 0.5 else -1

                drone.state = DroneState.AVOIDING
                drone.z_target = drone.avoidance_direction * self.params.z_displacement
                drone.velocity[2] = drone.avoidance_direction * self.params.z_return_speed * 2
                break

    def update_pid_consensus(self, drone: Drone):
        """Оптимізований алгоритм консенсусу для PID-параметрів"""
        if not drone.neighbors:
            return

        # Локальний консенсус
        neighbor_Kp = [n.pid.Kp for n in drone.neighbors]
        neighbor_Ki = [n.pid.Ki for n in drone.neighbors]
        neighbor_Kd = [n.pid.Kd for n in drone.neighbors]
        neighbor_performance = [n.performance_score for n in drone.neighbors]

        # Зважене середнє з урахуванням продуктивності сусідів
        total_performance = sum(neighbor_performance)
        if total_performance > 0:
            avg_Kp = sum(k * s for k, s in zip(neighbor_Kp, neighbor_performance)) / total_performance
            avg_Ki = sum(k * s for k, s in zip(neighbor_Ki, neighbor_performance)) / total_performance
            avg_Kd = sum(k * s for k, s in zip(neighbor_Kd, neighbor_performance)) / total_performance
        else:
            avg_Kp = sum(neighbor_Kp) / len(neighbor_Kp)
            avg_Ki = sum(neighbor_Ki) / len(neighbor_Ki)
            avg_Kd = sum(neighbor_Kd) / len(neighbor_Kd)

        # Глобальний консенсус (періодично)
        if self.global_consensus_counter % self.params.global_consensus_interval == 0:
            global_avg_Kp = self.get_avg_pid_param('Kp')
            global_avg_Ki = self.get_avg_pid_param('Ki')
            global_avg_Kd = self.get_avg_pid_param('Kd')

            # Комбінування локального та глобального консенсусу
            new_Kp = (self.params.local_consensus_weight * avg_Kp +
                     self.params.global_consensus_weight * global_avg_Kp)
            new_Ki = (self.params.local_consensus_weight * avg_Ki +
                     self.params.global_consensus_weight * global_avg_Ki)
            new_Kd = (self.params.local_consensus_weight * avg_Kd +
                     self.params.global_consensus_weight * global_avg_Kd)
        else:
            new_Kp = avg_Kp
            new_Ki = avg_Ki
            new_Kd = avg_Kd

        # Локальна адаптація на основі власного досвіду
        speed_error = abs(np.linalg.norm(drone.velocity) - drone.pid.target)

        # Адаптація Kp: збільшуємо при великій похибці
        delta_Kp = self.params.adaptation_rate * speed_error
        # Адаптація Ki: зменшуємо при осциляціях
        delta_Ki = -self.params.adaptation_rate * 0.1 * abs(drone.pid.integral)
        # Адаптація Kd: збільшуємо при наближенні до сусідів
        delta_Kd = self.params.adaptation_rate * drone.collision_risk

        new_Kp += delta_Kp
        new_Ki += delta_Ki
        new_Kd += delta_Kd

        # Обмеження значень параметрів
        new_Kp = max(self.params.min_Kp, min(self.params.max_Kp, new_Kp))
        new_Ki = max(self.params.min_Ki, min(self.params.max_Ki, new_Ki))
        new_Kd = max(self.params.min_Kd, min(self.params.max_Kd, new_Kd))

        # Оновлення PID-контролера
        drone.pid.update_params(new_Kp, new_Ki, new_Kd)

        # Збереження історії параметрів
        drone.pid_params_history.append((new_Kp, new_Ki, new_Kd))

    def update_drone(self, drone: Drone):
        """Оновлення стану одного дрона"""
        # Знаходження сусідів
        drone.neighbors = self.find_neighbors(drone)

        # Оновлення PID-параметрів за алгоритмом консенсусу
        self.update_pid_consensus(drone)

        # Обчислення векторів правил Бойдза з уникненням зіткнень
        separation = self.apply_enhanced_separation(drone)
        alignment = self.apply_alignment(drone)
        cohesion = self.apply_cohesion(drone)
        circle_move = self.apply_circle_movement(drone)

        # Комбінування правил
        acceleration = separation + alignment + cohesion + circle_move

        # Обмеження прискорення
        accel_norm = np.linalg.norm(acceleration)
        if accel_norm > self.params.max_acceleration:
            acceleration = acceleration / accel_norm * self.params.max_acceleration

        # Обчислення PID-корекції швидкості (тільки для XY-площини)
        current_speed = np.linalg.norm(drone.velocity[:2])
        pid_correction = drone.pid.compute(current_speed)

        # Застосування PID-корекції до прискорення (тільки XY)
        if current_speed > 0:
            velocity_direction = drone.velocity[:2] / current_speed
            acceleration[:2] += velocity_direction * pid_correction

        # Оновлення швидкості
        drone.velocity += acceleration * self.params.time_step

        # Обмеження швидкості
        speed = np.linalg.norm(drone.velocity)
        if speed > self.params.max_speed:
            drone.velocity = (drone.velocity / speed) * self.params.max_speed

        # Механізм уникнення зіткнень з використанням Z-координати
        self.avoid_collision_3d(drone)

        # Оновлення позиції
        drone.position += drone.velocity * self.params.time_step * self.params.simulation_speed

        # Виявлення зіткнень та "майже зіткнень"
        drone.state = DroneState.NORMAL
        min_distance = float('inf')

        for other in self.drones:
            if other is not drone:
                distance = np.linalg.norm(drone.position - other.position)
                min_distance = min(min_distance, distance)

                if distance < self.params.min_distance * 0.8:
                    drone.state = DroneState.COLLISION
                    drone.collision_count += 1
                    self.total_collisions += 1
                    drone.performance_score *= 0.9  # Зменшення оцінки продуктивності
                    break
                elif distance < self.params.z_avoidance_distance:
                    drone.state = DroneState.NEAR_COLLISION
                    drone.near_collision_count += 1
                    self.total_near_collisions += 1
                    drone.performance_score *= 0.95  # Незначне зменшення оцінки

        # Плавне збільшення оцінки продуктивності, якщо немає зіткнень
        if drone.state == DroneState.NORMAL:
            drone.performance_score = min(1.0, drone.performance_score * 1.01)

        # Оновлення мінімальної відстані
        if min_distance < self.min_distance:
            self.min_distance = min_distance

    def update_metrics(self):
        """Оновлення метрик симуляції"""
        total_distance = 0
        valid_pairs = 0
        total_speed_error = 0
        total_collision_risk = 0
        z_usage_count = 0

        for i, drone in enumerate(self.drones):
            min_dist = float('inf')
            for j, other in enumerate(self.drones):
                if i != j:
                    distance = np.linalg.norm(drone.position[:2] - other.position[:2])  # Тільки XY
                    total_distance += distance
                    valid_pairs += 1
                    if distance < min_dist:
                        min_dist = distance

            # Обчислення похибки швидкості (тільки XY)
            total_speed_error += abs(np.linalg.norm(drone.velocity[:2]) - drone.pid.target)

            # Сумування ризику зіткнення
            total_collision_risk += drone.collision_risk

            # Підрахунок дронів, що використовують Z-координату
            if abs(drone.position[2]) > 1:
                z_usage_count += 1

        if valid_pairs > 0:
            self.avg_distance = total_distance / valid_pairs
            avg_speed_error = total_speed_error / len(self.drones)
            avg_collision_risk = total_collision_risk / len(self.drones)
            z_usage_percent = z_usage_count / len(self.drones) * 100
        else:
            avg_speed_error = 0
            avg_collision_risk = 0
            z_usage_percent = 0

        # Збереження метрик для історії
        self.metrics_history.append({
            'time': self.simulation_time,
            'avg_distance': self.avg_distance,
            'min_distance': self.min_distance,
            'collisions': self.total_collisions,
            'near_collisions': self.total_near_collisions,
            'avg_Kp': self.get_avg_pid_param('Kp'),
            'avg_Ki': self.get_avg_pid_param('Ki'),
            'avg_Kd': self.get_avg_pid_param('Kd'),
            'speed_error': avg_speed_error,
            'collision_risk': avg_collision_risk,
            'z_usage': z_usage_percent
        })

        # Оновлення історії PID параметрів
        self.avg_pid_history['Kp'].append(self.get_avg_pid_param('Kp'))
        self.avg_pid_history['Ki'].append(self.get_avg_pid_param('Ki'))
        self.avg_pid_history['Kd'].append(self.get_avg_pid_param('Kd'))
        self.avg_velocity_error.append(avg_speed_error)
        self.avg_collision_risk.append(avg_collision_risk)
        self.collision_history.append(self.total_collisions)
        self.near_collision_history.append(self.total_near_collisions)
        self.z_usage_history.append(z_usage_percent)

        # Оновлення лічильника глобального консенсусу
        self.global_consensus_counter += 1

    def update_visuals(self):
        """Оновлення графічних елементів"""
        for i, drone in enumerate(self.drones):
            color = DRONE_COLOR
            if drone.state == DroneState.NEAR_COLLISION:
                color = NEAR_COLLISION_COLOR
            elif drone.state == DroneState.COLLISION:
                color = COLLISION_COLOR
            elif drone.state == DroneState.AVOIDING:
                color = 'purple'

            self.drone_plots[i].set_data([drone.position[0]], [drone.position[1]])
            self.drone_plots[i].set_color(color)

            if self.params.show_velocity and i < len(self.velocity_arrows):
                self.velocity_arrows[i].remove()
                arrow = self.ax.arrow(
                    drone.position[0], drone.position[1],
                    drone.velocity[0] * ARROW_SCALE, drone.velocity[1] * ARROW_SCALE,
                    color=VELOCITY_ARROW_COLOR, width=1, head_width=5
                )
                self.velocity_arrows[i] = arrow

            if self.params.show_collision_zones and i < len(self.collision_zones):
                self.collision_zones[i].center = (drone.position[0], drone.position[1])
                self.collision_zones[i].set_radius(self.params.z_avoidance_distance)

            if self.params.show_z_coordinate and i < len(self.z_indicator):
                self.z_indicator[i].set_position((drone.position[0], drone.position[1]))
                self.z_indicator[i].set_text(f"{drone.position[2]:.0f}")

        if self.params.show_metrics and self.metrics_text is not None:
            self.metrics_text.set_text(self.get_metrics_string())

        # Оновлення графіків
        if self.params.show_pid_graphs and len(self.metrics_history) > 0:
            times = [m['time'] for m in self.metrics_history]

            self.pid_lines['Kp'].set_data(times, self.avg_pid_history['Kp'])
            self.pid_lines['Ki'].set_data(times, self.avg_pid_history['Ki'])
            self.pid_lines['Kd'].set_data(times, self.avg_pid_history['Kd'])
            self.pid_ax.relim()
            self.pid_ax.autoscale_view()

            self.error_line.set_data(times, self.avg_velocity_error)
            self.risk_line.set_data(times, self.avg_collision_risk)
            self.error_ax.relim()
            self.error_ax.autoscale_view()

            self.collision_line.set_data(times, self.collision_history)
            self.near_collision_line.set_data(times, self.near_collision_history)
            self.z_usage_line.set_data(times, self.z_usage_history)
            self.collision_ax.relim()
            self.collision_ax.autoscale_view()

    def update(self, frame):
        """Оновлення стану симуляції для анімації"""
        # Скидання мінімальної відстані для нового кадру
        self.min_distance = float('inf')

        for drone in self.drones:
            self.update_drone(drone)

        self.update_metrics()
        self.simulation_time += self.params.time_step * self.params.simulation_speed
        self.frame_count += 1

        # Оновлення візуалізації
        self.update_visuals()

        # Повертаємо всі елементи, які потрібно оновити
        return self.drone_plots + self.velocity_arrows + self.collision_zones + self.z_indicator + \
               [self.metrics_text] + list(self.pid_lines.values()) + \
               [self.error_line, self.risk_line, self.collision_line, self.near_collision_line, self.z_usage_line]

    def run_simulation(self):
        """Запуск симуляції з анімацією"""
        self.animation = animation.FuncAnimation(
            self.fig, self.update,
            frames=200, interval=1000/FPS, blit=True
        )

        plt.close()  # Запобігаємо подвійному відображенню

        return HTML(self.animation.to_jshtml())

# Параметри симуляції
params = SimulationParameters(
    num_drones=25,
    circle_radius=200,
    desired_distance=35,
    min_distance=15,
    max_speed=2.5,
    max_acceleration=0.3,

    # Параметри алгоритму Бойдза
    boids_weights={
        'separation': 2.0,
        'alignment': 0.8,
        'cohesion': 0.8,
        'circle': 0.5
    },
    boids_radii={
        'separation': 60,
        'alignment': 100,
        'cohesion': 100
    },

    # Параметри уникнення зіткнень
    z_avoidance_distance=30,
    z_displacement=50,
    z_return_speed=0.5,
    avoidance_strength=2.5,
    predictive_avoidance=True,
    avoidance_decay=0.95,

    # Параметри консенсусу PID
    local_consensus_weight=0.7,
    global_consensus_weight=0.3,
    global_consensus_interval=10,
    adaptation_rate=0.01,
    min_Kp=0.1,
    max_Kp=2.0,
    min_Ki=0.01,
    max_Ki=0.5,
    min_Kd=0.05,
    max_Kd=1.0,

    # Візуалізація
    show_velocity=True,
    show_metrics=True,
    show_pid_graphs=True,
    show_collision_zones=True,
    show_z_coordinate=True,
    simulation_speed=1.0
)

# Створення та запуск симуляції
simulation = DroneSimulation(params)
simulation.run_simulation()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.patches import Circle
from IPython.display import HTML
import random
import math
from dataclasses import dataclass, field
from enum import Enum, auto
from typing import List, Dict, Tuple, Optional
from collections import deque

# Константи з технічного завдання
class BaseParams:
    SIM_AREA = (800, 600)  # Ширина, висота (px)
    Z_RANGE = (-100, 100)  # Мінімальна та максимальна висота (м)
    FPS = 60               # Кадрів на секунду
    PREDICTION_HORIZON = 10  # Кроки прогнозування
    BACKGROUND_COLOR = (240/255, 240/255, 240/255)
    TRAJECTORY_COLOR = (200/255, 200/255, 200/255)

class DroneSpecs:
    MAX_SPEED_XY = 5.0     # м/с
    MAX_SPEED_Z = 3.0      # м/с
    MAX_ACCEL = 2.5        # м/с²
    COMM_RANGE = 150       # м
    MIN_SEPARATION = 2.0   # м
    DRONE_RADIUS = 5       # px
    SAFETY_FACTOR = 1.5    # Запас безпеки

class SafetyZones:
    WARNING = 3 * DroneSpecs.MIN_SEPARATION  # Жовта зона
    DANGER = 2 * DroneSpecs.MIN_SEPARATION   # Оранжева зона
    CRITICAL = 1.5 * DroneSpecs.MIN_SEPARATION  # Червона зона

# Стани дронів
class DroneState(Enum):
    NORMAL = auto()        # Звичайний політ
    WARNING = auto()       # Попередження
    DANGER = auto()        # Загроза
    CRITICAL = auto()      # Критичний стан
    AVOIDING = auto()      # Виконує маневр уникнення

# Кольори для візуалізації
STATE_COLORS = {
    DroneState.NORMAL: 'green',
    DroneState.WARNING: 'yellow',
    DroneState.DANGER: 'orange',
    DroneState.CRITICAL: 'red',
    DroneState.AVOIDING: 'blue'
}

@dataclass
class PIDController:
    """Адаптивний PID-контролер для керування швидкістю"""
    Kp: float = 0.8
    Ki: float = 0.1
    Kd: float = 0.3
    target: float = 3.0
    integral: float = 0.0
    prev_error: float = 0.0
    dt: float = 1.0/BaseParams.FPS

    def compute(self, current: float) -> float:
        error = self.target - current
        self.integral += error * self.dt
        derivative = (error - self.prev_error) / self.dt
        output = self.Kp*error + self.Ki*self.integral + self.Kd*derivative
        self.prev_error = error
        return output

    def adapt_for_safety(self, risk_level: float):
        """Адаптація параметрів залежно від рівня ризику"""
        self.Kp *= (1 + 0.5 * risk_level)
        self.Kd *= (1 + 0.8 * risk_level)
        self.Ki *= max(0.5, (1 - 0.3 * risk_level))  # Не даємо Ki стати дуже малим

@dataclass
class Drone:
    """Клас, що представляє окремий дрон у зграї"""
    id: int
    position: np.ndarray  # [x, y, z]
    velocity: np.ndarray  # [vx, vy, vz]
    state: DroneState = DroneState.NORMAL
    neighbors: List['Drone'] = field(default_factory=list)
    pid: PIDController = field(default_factory=PIDController)
    avoidance_direction: float = 0.0  # Напрямок уникнення (-1 або +1)
    risk_level: float = 0.0  # Рівень ризику 0-1
    trajectory_history: deque = field(default_factory=lambda: deque(maxlen=50))
    communication_range: float = DroneSpecs.COMM_RANGE

    def __post_init__(self):
        self.trajectory_history.append(self.position.copy())

    def update_state_based_on_risk(self):
        """Оновлення стану на основі поточного рівня ризику"""
        if self.risk_level > 0.8:
            self.state = DroneState.CRITICAL
        elif self.risk_level > 0.5:
            self.state = DroneState.DANGER
        elif self.risk_level > 0.2:
            self.state = DroneState.WARNING
        else:
            self.state = DroneState.NORMAL

    def predict_trajectory(self, steps: int) -> List[np.ndarray]:
        """Прогнозування траєкторії на задану кількість кроків"""
        trajectory = []
        pos = self.position.copy()
        vel = self.velocity.copy()

        for _ in range(steps):
            pos += vel * self.pid.dt
            trajectory.append(pos.copy())

            # Просте прогнозування з постійною швидкістю
            # (можна ускладнити з урахуванням прискорення)
        return trajectory

    def avoid_collision(self, neighbors: List['Drone']):
        """Виконання маневру уникнення"""
        if self.state not in [DroneState.DANGER, DroneState.CRITICAL]:
            return

        self.state = DroneState.AVOIDING

        # Якщо напрямок ще не вибрано - вибираємо випадково
        if self.avoidance_direction == 0:
            self.avoidance_direction = 1 if random.random() > 0.5 else -1

        # Обчислюємо силу уникнення
        avoidance_force = np.zeros(3)
        for neighbor in neighbors:
            dist = np.linalg.norm(self.position - neighbor.position)
            if dist < SafetyZones.CRITICAL:
                dir_vec = (self.position - neighbor.position)
                if np.linalg.norm(dir_vec) > 0:
                    dir_vec = dir_vec / np.linalg.norm(dir_vec)
                avoidance_force += dir_vec * (1 - dist/SafetyZones.CRITICAL)

        # Додаємо Z-компоненту
        if np.linalg.norm(avoidance_force) > 0:
            avoidance_force = avoidance_force / np.linalg.norm(avoidance_force)
            avoidance_force[2] = self.avoidance_direction * 0.5  # Плавний набір висоти

        return avoidance_force * DroneSpecs.MAX_ACCEL * 2  # Підсилюємо прискорення уникнення

class SwarmSimulation:
    """Основний клас симулятора зграї дронів"""

    def __init__(self, num_drones: int = 15):
        self.num_drones = num_drones
        self.drones: List[Drone] = []
        self.fig, self.ax = plt.subplots(figsize=(12, 8))
        self.animation = None
        self.simulation_time = 0
        self.collisions_count = 0
        self.near_misses_count = 0
        self.setup_environment()
        self.initialize_drones()

    def setup_environment(self):
        """Налаштування графічного середовища"""
        self.ax.set_xlim(0, BaseParams.SIM_AREA[0])
        self.ax.set_ylim(0, BaseParams.SIM_AREA[1])
        self.ax.set_aspect('equal')
        self.ax.grid(True, linestyle='--', alpha=0.5)
        self.ax.set_title('SwarmSafe 3D - Симулятор зграї дронів')

        # Додаємо коло траєкторії
        self.trajectory_circle = Circle(
            (BaseParams.SIM_AREA[0]/2, BaseParams.SIM_AREA[1]/2),
            200, fill=False, color=BaseParams.TRAJECTORY_COLOR, linestyle='--'
        )
        self.ax.add_patch(self.trajectory_circle)

        # Елементи візуалізації
        self.drone_plots = []
        self.z_indicator = []
        self.risk_circles = []
        self.info_text = self.ax.text(
            0.02, 0.95, '', transform=self.ax.transAxes,
            bbox=dict(facecolor='white', alpha=0.7)
        )

    def initialize_drones(self):
        """Ініціалізація дронів у випадкових позиціях"""
        center_x, center_y = BaseParams.SIM_AREA[0]/2, BaseParams.SIM_AREA[1]/2

        for i in range(self.num_drones):
            # Випадкове положення навколо кола
            angle = random.uniform(0, 2*math.pi)
            radius = random.uniform(50, 200)
            x = center_x + radius * math.cos(angle)
            y = center_y + radius * math.sin(angle)
            z = random.uniform(-20, 20)

            # Випадкова початкова швидкість (напрямлена по дотичній до кола)
            speed = random.uniform(1, DroneSpecs.MAX_SPEED_XY)
            vx = -speed * math.sin(angle)
            vy = speed * math.cos(angle)
            vz = 0

            self.drones.append(Drone(
                id=i,
                position=np.array([x, y, z]),
                velocity=np.array([vx, vy, vz])
            ))

            # Додаємо графічні елементи
            self.drone_plots.append(self.ax.plot([], [], 'o', markersize=DroneSpecs.DRONE_RADIUS)[0])
            self.z_indicator.append(self.ax.text(0, 0, '', fontsize=8, ha='center', va='center'))
            self.risk_circles.append(Circle((0,0), SafetyZones.WARNING, fill=False, alpha=0.1))
            self.ax.add_patch(self.risk_circles[-1])

    def find_neighbors(self, drone: Drone) -> List[Drone]:
        """Знаходження сусідніх дронів у межах радіусу зв'язку"""
        return [d for d in self.drones
                if d.id != drone.id
                and np.linalg.norm(drone.position[:2] - d.position[:2]) < drone.communication_range]

    def calculate_risk_level(self, drone: Drone) -> float:
        """Розрахунок рівня ризику зіткнення"""
        if not drone.neighbors:
            return 0.0

        min_dist = min(np.linalg.norm(drone.position - n.position) for n in drone.neighbors)

        if min_dist < SafetyZones.CRITICAL:
            return 1.0
        elif min_dist < SafetyZones.DANGER:
            return 0.7
        elif min_dist < SafetyZones.WARNING:
            return 0.3
        return 0.0

    def apply_boids_rules(self, drone: Drone) -> np.ndarray:
        """Застосування правил Бойдза (розділення, вирівнювання, згуртування)"""
        separation = np.zeros(3)
        alignment = np.zeros(3)
        cohesion = np.zeros(3)
        center = np.zeros(3)
        total = len(drone.neighbors)

        if total == 0:
            return np.zeros(3)

        for neighbor in drone.neighbors:
            diff = drone.position - neighbor.position
            dist = np.linalg.norm(diff)

            # Правило розділення
            if dist < SafetyZones.WARNING:
                separation += diff / (dist + 0.1)  # Додаємо 0.1, щоб уникнути ділення на 0

            # Правило вирівнювання
            alignment += neighbor.velocity

            # Для правила згуртування
            center += neighbor.position

        # Нормалізація векторів
        if np.linalg.norm(separation) > 0:
            separation = (separation / np.linalg.norm(separation)) * DroneSpecs.MAX_SPEED_XY

        alignment = (alignment / total - drone.velocity) * 0.5

        center = center / total
        cohesion = (center - drone.position) * 0.05

        # Комбінуємо правила з вагами
        return separation * 1.5 + alignment * 1.0 + cohesion * 1.0

    def update_drone(self, drone: Drone):
        """Оновлення стану одного дрона"""
        # Оновлюємо список сусідів
        drone.neighbors = self.find_neighbors(drone)

        # Розраховуємо рівень ризику та оновлюємо стан
        drone.risk_level = self.calculate_risk_level(drone)
        drone.update_state_based_on_risk()

        # Адаптуємо PID-контролер
        drone.pid.adapt_for_safety(drone.risk_level)

        # Обчислюємо сили від правил Бойдза
        boids_force = self.apply_boids_rules(drone)

        # Обчислюємо силу уникнення (якщо потрібно)
        avoidance_force = np.zeros(3)
        if drone.state in [DroneState.DANGER, DroneState.CRITICAL]:
            avoidance_force = drone.avoid_collision(drone.neighbors)

        # Комбінуємо сили з пріоритетом уникнення
        if np.linalg.norm(avoidance_force) > 0:
            total_force = avoidance_force * 2.0 + boids_force * 0.5
        else:
            total_force = boids_force

        # Обмежуємо прискорення
        accel_norm = np.linalg.norm(total_force)
        if accel_norm > DroneSpecs.MAX_ACCEL:
            total_force = total_force / accel_norm * DroneSpecs.MAX_ACCEL

        # Оновлюємо швидкість
        drone.velocity += total_force * (1.0/BaseParams.FPS)

        # Обмежуємо швидкість
        speed_xy = np.linalg.norm(drone.velocity[:2])
        if speed_xy > DroneSpecs.MAX_SPEED_XY:
            drone.velocity[:2] = drone.velocity[:2] / speed_xy * DroneSpecs.MAX_SPEED_XY

        speed_z = abs(drone.velocity[2])
        if speed_z > DroneSpecs.MAX_SPEED_Z:
            drone.velocity[2] = drone.velocity[2] / speed_z * DroneSpecs.MAX_SPEED_Z

        # Оновлюємо позицію
        drone.position += drone.velocity * (1.0/BaseParams.FPS)

        # Застосовуємо граничні умови
        self.apply_boundary_conditions(drone)

        # Додаємо позицію до історії траєкторії
        drone.trajectory_history.append(drone.position.copy())

        # Перевіряємо на зіткнення
        self.check_collisions(drone)

    def apply_boundary_conditions(self, drone: Drone):
        """Обробка виходу за межі симуляції"""
        # Обмеження XY
        if drone.position[0] < 0:
            drone.position[0] = 0
            drone.velocity[0] *= -0.5
        elif drone.position[0] > BaseParams.SIM_AREA[0]:
            drone.position[0] = BaseParams.SIM_AREA[0]
            drone.velocity[0] *= -0.5

        if drone.position[1] < 0:
            drone.position[1] = 0
            drone.velocity[1] *= -0.5
        elif drone.position[1] > BaseParams.SIM_AREA[1]:
            drone.position[1] = BaseParams.SIM_AREA[1]
            drone.velocity[1] *= -0.5

        # Обмеження Z
        if drone.position[2] < BaseParams.Z_RANGE[0]:
            drone.position[2] = BaseParams.Z_RANGE[0]
            drone.velocity[2] = 0
        elif drone.position[2] > BaseParams.Z_RANGE[1]:
            drone.position[2] = BaseParams.Z_RANGE[1]
            drone.velocity[2] = 0

        # Плавне повернення до z=0, якщо немає загрози
        if drone.state == DroneState.NORMAL and abs(drone.position[2]) > 1:
            drone.velocity[2] += -0.1 * drone.position[2]

    def check_collisions(self, drone: Drone):
        """Перевірка зіткнень та майже зіткнень"""
        for neighbor in drone.neighbors:
            dist = np.linalg.norm(drone.position - neighbor.position)
            if dist < DroneSpecs.MIN_SEPARATION * DroneSpecs.SAFETY_FACTOR:
                self.near_misses_count += 1
            if dist < DroneSpecs.MIN_SEPARATION:
                self.collisions_count += 1
                # Відштовхуємо дрони при зіткненні
                direction = (drone.position - neighbor.position)
                if np.linalg.norm(direction) > 0:
                    direction = direction / np.linalg.norm(direction)
                drone.velocity += direction * DroneSpecs.MAX_ACCEL * 0.5

    def update_visuals(self):
        """Оновлення графічних елементів"""
        for i, drone in enumerate(self.drones):
            # Оновлюємо положення маркера
            self.drone_plots[i].set_data([drone.position[0]], [drone.position[1]])
            self.drone_plots[i].set_color(STATE_COLORS[drone.state])

            # Оновлюємо індикатор висоти
            self.z_indicator[i].set_position((drone.position[0], drone.position[1]))
            self.z_indicator[i].set_text(f'{drone.position[2]:.0f}')

            # Оновлюємо коло ризику
            self.risk_circles[i].center = (drone.position[0], drone.position[1])
            if drone.state == DroneState.CRITICAL:
                self.risk_circles[i].set_radius(SafetyZones.CRITICAL)
                self.risk_circles[i].set_color('red')
            elif drone.state == DroneState.DANGER:
                self.risk_circles[i].set_radius(SafetyZones.DANGER)
                self.risk_circles[i].set_color('orange')
            elif drone.state == DroneState.WARNING:
                self.risk_circles[i].set_radius(SafetyZones.WARNING)
                self.risk_circles[i].set_color('yellow')
            else:
                self.risk_circles[i].set_radius(0)  # Приховуємо

            # Прозорість залежить від рівня ризику
            self.risk_circles[i].set_alpha(drone.risk_level * 0.3)

        # Оновлюємо інформаційний текст
        info_text = (
            f'Час: {self.simulation_time:.1f} с\n'
            f'Дронів: {self.num_drones}\n'
            f'Зіткнень: {self.collisions_count}\n'
            f'Майже зіткнень: {self.near_misses_count}\n'
            f'Ризик (середній): {sum(d.risk_level for d in self.drones)/len(self.drones):.2f}'
        )
        self.info_text.set_text(info_text)

    def update(self, frame):
        """Оновлення симуляції для анімації"""
        self.simulation_time += 1.0/BaseParams.FPS

        # Оновлюємо всіх дронів
        for drone in self.drones:
            self.update_drone(drone)

        # Оновлюємо візуалізацію
        self.update_visuals()

        # Повертаємо всі елементи, які потрібно оновити
        return self.drone_plots + self.z_indicator + self.risk_circles + [self.info_text]

    def run_simulation(self):
        """Запуск симуляції"""
        self.animation = animation.FuncAnimation(
            self.fig, self.update,
            frames=200, interval=1000/BaseParams.FPS, blit=True
        )
        plt.close()
        return HTML(self.animation.to_jshtml())

# Запуск симуляції
sim = SwarmSimulation(num_drones=15)
sim.run_simulation()

In [ ]:
# ТЕХНІЧНЕ ЗАВДАННЯ
# на розробку симуляції руху зграї дронів з проактивним 3D уникненням зіткнень та адаптивним керуванням

import numpy as np
import matplotlib.pyplot as plt
import time
from typing import Tuple, List, Dict, Callable

class BaseParams:
    # Просторові параметри
    SIM_AREA: Tuple[int, int] = (800, 600)  # Розміри 2D полотна (px)
    Z_RANGE: Tuple[int, int] = (-100, 100)  # Діапазон висот (м)

    # Часові параметри
    FPS: int = 60                     # Частота симуляції
    PREDICTION_HORIZON: int = 10      # Кроки прогнозування

class DroneSpecs:
    MAX_SPEED_XY: float = 5.0         # м/с (горизонталь)
    MAX_SPEED_Z: float = 3.0          # м/с (вертикаль)
    MAX_ACCEL: float = 2.5            # м/с²
    COMM_RANGE: float = 150.0         # Радіус зв'язку (м)
    MIN_SEPARATION: float = 2.0       # Габаритний радіус (м)

class Drone:
    def __init__(self, id: int, initial_position: np.ndarray, initial_velocity: np.ndarray, pid_params: Dict[str, float]):
        self.id = id
        self.position = np.array(initial_position, dtype=float)
        self.velocity = np.array(initial_velocity, dtype=float)
        self.acceleration = np.zeros(3, dtype=float)
        self.pid_params = pid_params
        self.target_velocity_xy = np.zeros(2, dtype=float)
        self.z_step = 5.0  # Величина кроку по Z для уникнення
        self.is_avoiding_z = False
        self.avoidance_direction_z = 0
        self.prediction_history: List[np.ndarray] = [self.position.copy()]

    def predict_trajectory(self, horizon: int, dt: float) -> List[np.ndarray]:
        trajectory = [self.position.copy()]
        current_position = self.position.copy()
        current_velocity = self.velocity.copy()
        for _ in range(horizon):
            current_velocity += self.acceleration * dt
            current_velocity[:2] = np.clip(current_velocity[:2], -DroneSpecs.MAX_SPEED_XY, DroneSpecs.MAX_SPEED_XY)
            current_velocity[2] = np.clip(current_velocity[2], -DroneSpecs.MAX_SPEED_Z, DroneSpecs.MAX_SPEED_Z)
            current_position += current_velocity * dt
            trajectory.append(current_position.copy())
        return trajectory

    def generate_avoidance_options(self) -> List[Dict[str, any]]:
        return [
            {'dz': DroneSpecs.MAX_SPEED_Z * self.get_sign(), 'priority': 0},
            {'brake_xy': True, 'priority': 1}
        ]

    def get_sign(self):
        return 1 if np.random.rand() > 0.5 else -1

    def update_pid_params(self, risk_level: float):
        if risk_level > 0.7:
            self.pid_params['Kp'] *= 1.2
            self.pid_params['Kd'] *= 1.5
            self.pid_params['Ki'] *= 0.8
        else:
            # Поступова оптимізація до базових значень (потрібна більш детальна реалізація)
            pass

    def apply_boids_rules(self, neighbors: List['Drone'], weights: Dict[str, float], radii: Dict[str, float]):
        separation = np.zeros(2)
        alignment = np.zeros(2)
        cohesion = np.zeros(2)
        local_neighbors_sep = [n for n in neighbors if np.linalg.norm(self.position[:2] - n.position[:2]) < radii['separation']]
        local_neighbors_ali = [n for n in neighbors if np.linalg.norm(self.position[:2] - n.position[:2]) < radii['alignment']]
        local_neighbors_coh = [n for n in neighbors if np.linalg.norm(self.position[:2] - n.position[:2]) < radii['cohesion']]

        if local_neighbors_sep:
            for n in local_neighbors_sep:
                diff = self.position[:2] - n.position[:2]
                separation += diff / (np.linalg.norm(diff)**2 + 1e-6)
            separation /= len(local_neighbors_sep)

        if local_neighbors_ali:
            alignment = np.mean([n.velocity[:2] for n in local_neighbors_ali], axis=0)

        if local_neighbors_coh:
            center_of_mass = np.mean([n.position[:2] for n in local_neighbors_coh], axis=0)
            cohesion = center_of_mass - self.position[:2]

        self.target_velocity_xy = weights['separation'] * separation + weights['alignment'] * alignment + weights['cohesion'] * cohesion

    def update(self, dt: float):
        # PID контролер (спрощена реалізація для XY)
        error = self.target_velocity_xy - self.velocity[:2]
        self.acceleration[:2] = (self.pid_params['Kp'] * error +
                                   self.pid_params['Ki'] * np.cumsum(error * dt)[-1] +
                                   self.pid_params['Kd'] * (error - (self.target_velocity_xy - self.velocity[:2])) / dt) # Спрощена похідна

        # Управління по Z (просте уникнення)
        if self.is_avoiding_z:
            self.acceleration[2] = DroneSpecs.MAX_ACCEL * self.avoidance_direction_z
            if (self.avoidance_direction_z > 0 and self.position[2] >= BaseParams.Z_RANGE[1]) or \
               (self.avoidance_direction_z < 0 and self.position[2] <= BaseParams.Z_RANGE[0]):
                self.is_avoiding_z = False
                self.acceleration[2] = -np.sign(self.position[2]) * DroneSpecs.MAX_ACCEL # Початок повернення

        self.acceleration = np.clip(self.acceleration, -DroneSpecs.MAX_ACCEL, DroneSpecs.MAX_ACCEL)
        self.velocity += self.acceleration * dt
        self.velocity[:2] = np.clip(self.velocity[:2], -DroneSpecs.MAX_SPEED_XY, DroneSpecs.MAX_SPEED_XY)
        self.velocity[2] = np.clip(self.velocity[2], -DroneSpecs.MAX_SPEED_Z, DroneSpecs.MAX_SPEED_Z)
        self.position += self.velocity * dt
        self.position[:2] = np.clip(self.position[:2], 0, BaseParams.SIM_AREA)
        self.position[2] = np.clip(self.position[2], BaseParams.Z_RANGE[0], BaseParams.Z_RANGE[1])
        self.prediction_history.append(self.position.copy())
        if len(self.prediction_history) > BaseParams.PREDICTION_HORIZON + 1:
            self.prediction_history.pop(0)

class Swarm:
    def __init__(self, num_drones: int, initial_positions: List[np.ndarray], initial_velocities: List[np.ndarray], pid_initial_params: List[Dict[str, float]]):
        self.drones = [Drone(i, initial_positions[i], initial_velocities[i], pid_initial_params[i]) for i in range(num_drones)]
        self.time = 0.0
        self.metrics = {
            'collision_avoided': 0,
            'near_misses': 0,
            'avg_reaction_time': 0.0,
            'consensus_success_rate': 0.0,
            '3d_maneuver_efficiency': 0.0
        }
        self.collision_history = []

    def get_neighbors(self, drone: Drone) -> List[Drone]:
        neighbors = []
        for other in self.drones:
            if other != drone and np.linalg.norm(drone.position[:2] - other.position[:2]) < DroneSpecs.COMM_RANGE:
                neighbors.append(other)
        return neighbors

    def check_collisions(self):
        collisions = []
        for i in range(len(self.drones)):
            for j in range(i + 1, len(self.drones)):
                distance = np.linalg.norm(self.drones[i].position - self.drones[j].position)
                if distance < DroneSpecs.MIN_SEPARATION * 1.5:
                    self.collision_history.append((self.time, self.drones[i].id, self.drones[j].id, distance))
                    collisions.append((self.drones[i], self.drones[j]))
                elif distance < DroneSpecs.MIN_SEPARATION * 2:
                    self.metrics['near_misses'] += 1
        return collisions

    def run_step(self, dt: float, boids_weights: Dict[str, float], boids_radii: Dict[str, float]):
        for drone in self.drones:
            neighbors = self.get_neighbors(drone)
            drone.apply_boids_rules(neighbors, boids_weights, boids_radii)
            # Проактивне уникнення (спрощена логіка)
            for other in neighbors:
                predicted_self = drone.predict_trajectory(BaseParams.PREDICTION_HORIZON, dt)
                predicted_other = other.predict_trajectory(BaseParams.PREDICTION_HORIZON, dt)
                for i in range(BaseParams.PREDICTION_HORIZON):
                    if np.linalg.norm(predicted_self[min(i, len(predicted_self)-1)][:2] - predicted_other[min(i, len(predicted_other)-1)][:2]) < DroneSpecs.MIN_SEPARATION * 1.8:
                        drone.is_avoiding_z = True
                        drone.avoidance_direction_z = drone.get_sign()
                        break
            drone.update_pid_params(0.0) # Потрібна реалізація оцінки ризику
            drone.update(dt)
        self.check_collisions()
        self.time += dt

# --- Параметри симуляції ---
num_drones = 20
initial_positions = [np.random.rand(2) * BaseParams.SIM_AREA for _ in range(num_drones)]
initial_velocities = [np.random.rand(2) * 2 - 1 for _ in range(num_drones)]
initial_z = np.random.rand(num_drones) * (BaseParams.Z_RANGE[1] - BaseParams.Z_RANGE[0]) + BaseParams.Z_RANGE[0]
initial_positions = [np.array([p[0], p[1], initial_z[i]]) for i, p in enumerate(initial_positions)]
initial_velocities = [np.array([v[0], v[1], 0.0]) for v in initial_velocities]
pid_initial_params = [{'Kp': 1.0, 'Ki': 0.01, 'Kd': 0.1} for _ in range(num_drones)]
boids_weights = {'separation': 0.5, 'alignment': 0.3, 'cohesion': 0.2}
boids_radii = {'separation': 30, 'alignment': 60, 'cohesion': 90}
dt = 1 / BaseParams.FPS
simulation_duration = 60  # секунд

# --- Запуск симуляції ---
swarm = Swarm(num_drones, initial_positions, initial_velocities, pid_initial_params)

plt.style.use('seaborn-v0_8-darkgrid')
fig, ax = plt.subplots(figsize=(10, 8))

for step in range(int(simulation_duration * BaseParams.FPS)):
    ax.clear()
    ax.set_xlim(0, BaseParams.SIM_AREA[0])
    ax.set_ylim(0, BaseParams.SIM_AREA[1])
    ax.set_title(f"Симуляція зграї дронів (час: {swarm.time:.2f} с)")

    for drone in swarm.drones:
        ax.scatter(drone.position[0], drone.position[1], s=50, c='green' if not drone.is_avoiding_z else 'blue')
        # Відображення прогнозованої траєкторії (опціонально)
        # trajectory = drone.predict_trajectory(BaseParams.PREDICTION_HORIZON, dt)
        # ax.plot([t[0] for t in trajectory], [t[1] for t in trajectory], '--', linewidth=0.5, alpha=0.5)

    collisions = swarm.check_collisions()
    if collisions:
        print(f"Зіткнення на час {swarm.time:.2f} с")
        for d1, d2 in collisions:
            ax.scatter(d1.position[0], d1.position[1], s=100, c='red', marker='x')
            ax.scatter(d2.position[0], d2.position[1], s=100, c='red', marker='x')

    swarm.run_step(dt, boids_weights, boids_radii)
    plt.pause(0.001)

print("Симуляція завершена")
print("Історія зіткнень:", swarm.collision_history)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.patches import Circle
from IPython.display import HTML
import random
import math
from dataclasses import dataclass, field
from enum import Enum, auto
from typing import List, Dict, Tuple, Optional
from collections import deque

import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 1000

# Константи з технічного завдання
class BaseParams:
    SIM_AREA = (800, 600)  # Ширина, висота симуляційного простору у пікселях.
    Z_RANGE = (-100, 100)  # Мінімальна та максимальна висота польоту дронів у метрах.
    FPS = 60               # Кількість кадрів симуляції на секунду.
    PREDICTION_HORIZON = 10  # Кількість кроків для прогнозування траєкторії дрона.
    BACKGROUND_COLOR = (240/255, 240/255, 240/255) # Колір фону візуалізації (RGB).
    TRAJECTORY_COLOR = (200/255, 200/255, 200/255) # Колір відображення історії траєкторії (RGB).

class DroneSpecs:
    MAX_SPEED_XY = 5.0     # Максимальна горизонтальна швидкість дрона у м/с.
    MAX_SPEED_Z = 3.0      # Максимальна вертикальна швидкість дрона у м/с.
    MAX_ACCEL = 2.5        # Максимальне прискорення дрона у м/с².
    COMM_RANGE = 150.0     # Радіус зв'язку між дронами у метрах.
    MIN_SEPARATION = 2.0   # Мінімальна безпечна відстань між центрами дронів у метрах.
    DRONE_RADIUS = 5       # Радіус дрона для візуалізації у пікселях.
    SAFETY_FACTOR = 1.5    # Коефіцієнт запасу безпеки при перевірці зіткнень.

class SafetyZones:
    WARNING = 3 * DroneSpecs.MIN_SEPARATION  # Жовта зона
    DANGER = 2 * DroneSpecs.MIN_SEPARATION   # Оранжева зона
    CRITICAL = 1.5 * DroneSpecs.MIN_SEPARATION  # Червона зона

# Стани дронів
class DroneState(Enum):
    NORMAL = auto()        # Звичайний політ
    WARNING = auto()       # Попередження
    DANGER = auto()        # Загроза
    CRITICAL = auto()      # Критичний стан
    AVOIDING = auto()      # Виконує маневр уникнення

# Кольори для візуалізації
STATE_COLORS = {
    DroneState.NORMAL: 'green',
    DroneState.WARNING: 'yellow',
    DroneState.DANGER: 'orange',
    DroneState.CRITICAL: 'red',
    DroneState.AVOIDING: 'blue'
}

@dataclass
class PIDController:
    """Адаптивний PID-контролер для керування швидкістю"""
    Kp: float = 0.8
    Ki: float = 0.1
    Kd: float = 0.3
    target: float = 3.0
    integral: float = 0.0
    prev_error: float = 0.0
    dt: float = 1.0/BaseParams.FPS

    def compute(self, current: float) -> float:
        error = self.target - current
        self.integral += error * self.dt
        derivative = (error - self.prev_error) / self.dt
        output = self.Kp*error + self.Ki*self.integral + self.Kd*derivative
        self.prev_error = error
        return output

    def adapt_for_safety(self, risk_level: float):
        """Адаптація параметрів залежно від рівня ризику"""
        self.Kp *= (1 + 0.5 * risk_level)
        self.Kd *= (1 + 0.8 * risk_level)
        self.Ki *= max(0.5, (1 - 0.3 * risk_level))  # Забезпечуємо мінімальне значення Ki

@dataclass
class Drone:
    """Клас, що представляє окремий дрон у зграї"""
    id: int
    position: np.ndarray  # [x, y, z]
    velocity: np.ndarray  # [vx, vy, vz]
    state: DroneState = DroneState.NORMAL
    neighbors: List['Drone'] = field(default_factory=list)
    pid: PIDController = field(default_factory=PIDController)
    avoidance_direction: float = 0.0  # Напрямок уникнення (-1 або +1)
    risk_level: float = 0.0  # Рівень ризику 0-1
    trajectory_history: deque = field(default_factory=lambda: deque(maxlen=50))
    communication_range: float = DroneSpecs.COMM_RANGE

    def __post_init__(self):
        self.trajectory_history.append(self.position.copy())

    def update_state_based_on_risk(self):
        """Оновлення стану на основі поточного рівня ризику"""
        if self.risk_level > 0.8:
            self.state = DroneState.CRITICAL
        elif self.risk_level > 0.5:
            self.state = DroneState.DANGER
        elif self.risk_level > 0.2:
            self.state = DroneState.WARNING
        else:
            self.state = DroneState.NORMAL

    def predict_trajectory(self, steps: int) -> List[np.ndarray]:
        """Прогнозування траєкторії на задану кількість кроків"""
        trajectory = []
        pos = self.position.copy()
        vel = self.velocity.copy()

        for _ in range(steps):
            pos += vel * self.pid.dt
            trajectory.append(pos.copy())

            # Просте прогнозування з постійною швидкістю
            # (можна ускладнити з урахуванням прискорення)
        return trajectory

    def avoid_collision(self, neighbors: List['Drone']):
        """Виконання маневру уникнення"""
        if self.state not in [DroneState.DANGER, DroneState.CRITICAL]:
            return

        self.state = DroneState.AVOIDING

        # Якщо напрямок ще не вибрано - вибираємо випадково
        if self.avoidance_direction == 0:
            self.avoidance_direction = 1 if random.random() > 0.5 else -1

        # Обчислюємо силу уникнення
        avoidance_force = np.zeros(3)
        for neighbor in neighbors:
            dist = np.linalg.norm(self.position - neighbor.position)
            if dist < SafetyZones.CRITICAL:
                dir_vec = (self.position - neighbor.position)
                if np.linalg.norm(dir_vec) > 0:
                    dir_vec = dir_vec / np.linalg.norm(dir_vec)
                avoidance_force += dir_vec * (1 - dist/SafetyZones.CRITICAL)

        # Додаємо Z-компоненту
        if np.linalg.norm(avoidance_force) > 0:
            avoidance_force = avoidance_force / np.linalg.norm(avoidance_force)
            avoidance_force[2] = self.avoidance_direction * 0.5  # Плавний набір висоти

        return avoidance_force * DroneSpecs.MAX_ACCEL * 2  # Підсилюємо прискорення уникнення

    def generate_avoidance_options(self):
        """Генерація варіантів маневру уникнення"""
        return [
            {'dz': DroneSpecs.MAX_SPEED_Z * self.get_sign(), 'priority': 0},  # Зміна висоти
            {'brake_xy': True, 'priority': 1}                                  # Гальмування в XY площині
        ]

    def get_sign(self):
        """Повертає випадковий знак (+1 або -1)"""
        return 1 if random.random() > 0.5 else -1

class SwarmSimulation:
    """Основний клас симулятора зграї дронів"""

    def __init__(self, num_drones: int = 15):
        self.num_drones = num_drones
        self.drones: List[Drone] = []
        self.fig, self.ax = plt.subplots(figsize=(12, 8))
        self.animation = None
        self.simulation_time = 0
        self.collisions_count = 0
        self.near_misses_count = 0
        self.setup_environment()
        self.initialize_drones()

    def setup_environment(self):
        """Налаштування графічного середовища"""
        self.ax.set_xlim(0, BaseParams.SIM_AREA[0])
        self.ax.set_ylim(0, BaseParams.SIM_AREA[1])
        self.ax.set_aspect('equal')
        self.ax.grid(True, linestyle='--', alpha=0.5)
        self.ax.set_title('SwarmSafe 3D - Симулятор зграї дронів')

        # Додаємо коло траєкторії
        self.trajectory_circle = Circle(
            (BaseParams.SIM_AREA[0]/2, BaseParams.SIM_AREA[1]/2),
            200, fill=False, color=BaseParams.TRAJECTORY_COLOR, linestyle='--'
        )
        self.ax.add_patch(self.trajectory_circle)

        # Елементи візуалізації
        self.drone_plots = []
        self.z_indicator = []
        self.risk_circles = []
        self.predicted_trajectories = []  # Додано для візуалізації прогнозованих траєкторій
        self.info_text = self.ax.text(
            0.02, 0.95, '', transform=self.ax.transAxes,
            bbox=dict(facecolor='white', alpha=0.7)
        )

    def initialize_drones(self):
        """Ініціалізація дронів у випадкових позиціях"""
        center_x, center_y = BaseParams.SIM_AREA[0]/2, BaseParams.SIM_AREA[1]/2

        for i in range(self.num_drones):
            # Випадкове положення навколо кола
            angle = random.uniform(0, 2*math.pi)
            radius = random.uniform(50, 200)
            x = center_x + radius * math.cos(angle)
            y = center_y + radius * math.sin(angle)
            z = random.uniform(-20, 20)

            # Випадкова початкова швидкість (напрямлена по дотичній до кола)
            speed = random.uniform(1, DroneSpecs.MAX_SPEED_XY)
            vx = -speed * math.sin(angle)
            vy = speed * math.cos(angle)
            vz = 0

            self.drones.append(Drone(
                id=i,
                position=np.array([x, y, z]),
                velocity=np.array([vx, vy, vz])
            ))

            # Додаємо графічні елементи
            self.drone_plots.append(self.ax.plot([], [], 'o', markersize=DroneSpecs.DRONE_RADIUS)[0])
            self.z_indicator.append(self.ax.text(0, 0, '', fontsize=8, ha='center', va='center'))
            self.risk_circles.append(Circle((0,0), SafetyZones.WARNING, fill=False, alpha=0.1))
            self.ax.add_patch(self.risk_circles[-1])
            self.predicted_trajectories.append(self.ax.plot([], [], '--', color='gray', alpha=0.5)[0])  # Додано для візуалізації прогнозованих траєкторій

    def find_neighbors(self, drone: Drone) -> List[Drone]:
        """Знаходження сусідніх дронів у межах радіусу зв'язку"""
        return [d for d in self.drones
                if d.id != drone.id
                and np.linalg.norm(drone.position[:2] - d.position[:2]) < drone.communication_range]

    def calculate_risk_level(self, drone: Drone) -> float:
        """Розрахунок рівня ризику зіткнення"""
        if not drone.neighbors:
            return 0.0

        min_dist = min(np.linalg.norm(drone.position - n.position) for n in drone.neighbors)

        if min_dist < SafetyZones.CRITICAL:
            return 1.0
        elif min_dist < SafetyZones.DANGER:
            return 0.7
        elif min_dist < SafetyZones.WARNING:
            return 0.3
        return 0.0

    def apply_boids_rules(self, drone: Drone) -> np.ndarray:
        """Застосування правил Бойдза (розділення, вирівнювання, згуртування)"""
        separation = np.zeros(3)
        alignment = np.zeros(3)
        cohesion = np.zeros(3)
        center = np.zeros(3)
        total = len(drone.neighbors)

        if total == 0:
            return np.zeros(3)

        for neighbor in drone.neighbors:
            diff = drone.position - neighbor.position
            dist = np.linalg.norm(diff)

            # Правило розділення
            if dist < SafetyZones.WARNING:
                separation += diff / (dist + 0.1)  # Додаємо 0.1, щоб уникнути ділення на 0

            # Правило вирівнювання
            alignment += neighbor.velocity

            # Для правила згуртування
            center += neighbor.position

        # Нормалізація векторів
        if np.linalg.norm(separation) > 0:
            separation = (separation / np.linalg.norm(separation)) * DroneSpecs.MAX_SPEED_XY

        alignment = (alignment / total - drone.velocity) * 0.5

        center = center / total
        cohesion = (center - drone.position) * 0.05

        # Комбінуємо правила з вагами
        return separation * 1.5 + alignment * 1.0 + cohesion * 1.0

    def update_drone(self, drone: Drone):
        """Оновлення стану одного дрона"""
        # Оновлюємо список сусідів
        drone.neighbors = self.find_neighbors(drone)

        # Розраховуємо рівень ризику та оновлюємо стан
        drone.risk_level = self.calculate_risk_level(drone)
        drone.update_state_based_on_risk()

        # Адаптуємо PID-контролер
        drone.pid.adapt_for_safety(drone.risk_level)

        # Обчислюємо сили від правил Бойдза
        boids_force = self.apply_boids_rules(drone)

        # Обчислюємо силу уникнення (якщо потрібно)
        avoidance_force = np.zeros(3)
        if drone.state in [DroneState.DANGER, DroneState.CRITICAL]:
            avoidance_force = drone.avoid_collision(drone.neighbors)

        # Комбінуємо сили з пріоритетом уникнення
        if np.linalg.norm(avoidance_force) > 0:
            total_force = avoidance_force * 2.0 + boids_force * 0.5
        else:
            total_force = boids_force

        # Обмежуємо прискорення
        accel_norm = np.linalg.norm(total_force)
        if accel_norm > DroneSpecs.MAX_ACCEL:
            total_force = total_force / accel_norm * DroneSpecs.MAX_ACCEL

        # Оновлюємо швидкість
        drone.velocity += total_force * (1.0/BaseParams.FPS)

        # Обмежуємо швидкість
        speed_xy = np.linalg.norm(drone.velocity[:2])
        if speed_xy > DroneSpecs.MAX_SPEED_XY:
            drone.velocity[:2] = drone.velocity[:2] / speed_xy * DroneSpecs.MAX_SPEED_XY

        speed_z = abs(drone.velocity[2])
        if speed_z > DroneSpecs.MAX_SPEED_Z:
            drone.velocity[2] = drone.velocity[2] / speed_z * DroneSpecs.MAX_SPEED_Z

        # Оновлюємо позицію
        drone.position += drone.velocity * (1.0/BaseParams.FPS)

        # Застосовуємо граничні умови
        self.apply_boundary_conditions(drone)

        # Додаємо позицію до історії траєкторії
        drone.trajectory_history.append(drone.position.copy())

        # Перевіряємо на зіткнення
        self.check_collisions(drone)

    def apply_boundary_conditions(self, drone: Drone):
        """Обробка виходу за межі симуляції"""
        # Обмеження XY
        if drone.position[0] < 0:
            drone.position[0] = 0
            drone.velocity[0] *= -0.5
        elif drone.position[0] > BaseParams.SIM_AREA[0]:
            drone.position[0] = BaseParams.SIM_AREA[0]
            drone.velocity[0] *= -0.5

        if drone.position[1] < 0:
            drone.position[1] = 0
            drone.velocity[1] *= -0.5
        elif drone.position[1] > BaseParams.SIM_AREA[1]:
            drone.position[1] = BaseParams.SIM_AREA[1]
            drone.velocity[1] *= -0.5

        # Обмеження Z
        if drone.position[2] < BaseParams.Z_RANGE[0]:
            drone.position[2] = BaseParams.Z_RANGE[0]
            drone.velocity[2] = 0
        elif drone.position[2] > BaseParams.Z_RANGE[1]:
            drone.position[2] = BaseParams.Z_RANGE[1]
            drone.velocity[2] = 0

        # Плавне повернення до z=0, якщо немає загрози
        if drone.state == DroneState.NORMAL and abs(drone.position[2]) > 1:
            drone.velocity[2] += -0.1 * drone.position[2]

    def check_collisions(self, drone: Drone):
        """Перевірка зіткнень та майже зіткнень"""
        for neighbor in drone.neighbors:
            dist = np.linalg.norm(drone.position - neighbor.position)
            if dist < DroneSpecs.MIN_SEPARATION * DroneSpecs.SAFETY_FACTOR:
                self.near_misses_count += 1
            if dist < DroneSpecs.MIN_SEPARATION:
                self.collisions_count += 1
                # Відштовхуємо дрони при зіткненні
                direction = (drone.position - neighbor.position)
                if np.linalg.norm(direction) > 0:
                    direction = direction / np.linalg.norm(direction)
                drone.velocity += direction * DroneSpecs.MAX_ACCEL * 0.5

    def update_visuals(self):
        """Оновлення графічних елементів"""
        for i, drone in enumerate(self.drones):
            # Оновлюємо положення маркера
            self.drone_plots[i].set_data([drone.position[0]], [drone.position[1]])
            self.drone_plots[i].set_color(STATE_COLORS[drone.state])

            # Оновлюємо індикатор висоти
            self.z_indicator[i].set_position((drone.position[0], drone.position[1]))
            self.z_indicator[i].set_text(f'{drone.position[2]:.0f}')

            # Оновлюємо коло ризику
            self.risk_circles[i].center = (drone.position[0], drone.position[1])
            if drone.state == DroneState.CRITICAL:
                self.risk_circles[i].set_radius(SafetyZones.CRITICAL)
                self.risk_circles[i].set_color('red')
            elif drone.state == DroneState.DANGER:
                self.risk_circles[i].set_radius(SafetyZones.DANGER)
                self.risk_circles[i].set_color('orange')
            elif drone.state == DroneState.WARNING:
                self.risk_circles[i].set_radius(SafetyZones.WARNING)
                self.risk_circles[i].set_color('yellow')
            else:
                self.risk_circles[i].set_radius(0)  # Приховуємо

            # Прозорість залежить від рівня ризику
            self.risk_circles[i].set_alpha(drone.risk_level * 0.3)

            # Оновлюємо прогнозовану траєкторію
            predicted_positions = drone.predict_trajectory(BaseParams.PREDICTION_HORIZON)
            x_coords = [pos[0] for pos in predicted_positions]
            y_coords = [pos[1] for pos in predicted_positions]
            self.predicted_trajectories[i].set_data(x_coords, y_coords)

        # Оновлюємо інформаційний текст
        info_text = (
            f'Час: {self.simulation_time:.1f} с\n'
            f'Дронів: {self.num_drones}\n'
            f'Зіткнень: {self.collisions_count}\n'
            f'Майже зіткнень: {self.near_misses_count}\n'
            f'Ризик (середній): {sum(d.risk_level for d in self.drones)/len(self.drones):.2f}'
        )
        self.info_text.set_text(info_text)

    def update(self, frame):
        """Оновлення симуляції для анімації"""
        self.simulation_time += 1.0/BaseParams.FPS

        # Оновлюємо всіх дронів
        for drone in self.drones:
            self.update_drone(drone)

        # Оновлюємо візуалізацію
        self.update_visuals()

        # Повертаємо всі елементи, які потрібно оновити
        return self.drone_plots + self.z_indicator + self.risk_circles + self.predicted_trajectories + [self.info_text]

    def run_simulation(self):
        """Запуск симуляції"""
        self.animation = animation.FuncAnimation(
            self.fig, self.update,
            frames=600, interval=1000/BaseParams.FPS, blit=True
        )
        plt.close()
        return HTML(self.animation.to_jshtml())

# Запуск симуляції
sim = SwarmSimulation(num_drones=15)
sim.run_simulation()

In [ ]:
# Install required packages
!pip install cvxpy matplotlib numpy

import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML

# Set up matplotlib for Colab
%matplotlib inline
plt.ioff()  # Turn off interactive mode to prevent double displays

class Drone:
    def __init__(self, id, initial_pos, target_pos, max_speed=1.0, max_accel=0.3, safe_dist=2.0):
        self.id = id
        self.pos = np.array(initial_pos, dtype=float)
        self.vel = np.zeros(3)
        self.target = np.array(target_pos, dtype=float)
        self.max_speed = max_speed
        self.max_accel = max_accel
        self.safe_dist = safe_dist

def cbf_constraints(drones, dt=0.1):
    constraints = []
    for i in range(len(drones)):
        for j in range(i+1, len(drones)):
            pos_diff = drones[i].pos - drones[j].pos
            vel_diff = drones[i].vel - drones[j].vel
            h = cp.sum_squares(pos_diff) - drones[i].safe_dist**2
            h_dot = 2 * pos_diff @ vel_diff
            alpha = 1.0
            constraints.append(h_dot + alpha * h >= 0)
    return constraints

def update_drones(drones, dt=0.1):
    for drone in drones:
        u = cp.Variable(3)
        objective = cp.Minimize(cp.sum_squares(drone.pos + dt*drone.vel + 0.5*dt**2*u - drone.target))

        constraints = [
            cp.norm(u, 'inf') <= drone.max_accel,
            cp.norm(drone.vel + dt*u, 'inf') <= drone.max_speed
        ]

        # Add only valid CVXPY constraints
        for con in cbf_constraints(drones, dt):
            if isinstance(con, cp.constraints.nonpos.Inequality):
                constraints.append(con)

        prob = cp.Problem(objective, constraints)
        try:
            prob.solve(solver=cp.ECOS)
            if prob.status == cp.OPTIMAL:
                drone.vel += dt*u.value
                drone.pos += dt*drone.vel + 0.5*dt**2*u.value
        except:
            # If optimization fails, continue with current velocity
            drone.pos += dt*drone.vel

# Initialize drones
drones = [
    Drone(0, [0, 0, 0], [10, 10, 5]),
    Drone(1, [10, 0, 5], [0, 10, 5]),
    Drone(2, [5, 10, 3], [5, 0, 7])
]

# Set up figure
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
ax.set_xlim(0, 10)
ax.set_ylim(0, 10)
ax.set_zlim(0, 10)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('3D Drone Swarm Simulation with Collision Avoidance')

# Initialize plot elements
points = [ax.plot([], [], [], 'o', markersize=8, label=f'Drone {i}')[0] for i in range(len(drones))]
traces = [ax.plot([], [], [], '-', alpha=0.5)[0] for _ in drones]
paths = [[] for _ in drones]

def init():
    for point, trace in zip(points, traces):
        point.set_data([], [])
        point.set_3d_properties([])
        trace.set_data([], [])
        trace.set_3d_properties([])
    return points + traces

def animate(frame):
    update_drones(drones)
    for i, (drone, point, trace, path) in enumerate(zip(drones, points, traces, paths)):
        x, y, z = drone.pos
        point.set_data([x], [y])
        point.set_3d_properties([z])
        path.append(drone.pos.copy())
        if len(path) > 1:
            trace.set_data([p[0] for p in path], [p[1] for p in path])
            trace.set_3d_properties([p[2] for p in path])
    ax.view_init(elev=20, azim=frame % 360)
    return points + traces

# Create and display animation
ani = FuncAnimation(fig, animate, frames=50, init_func=init, blit=True, interval=100)
plt.legend()

# Properly display the animation
from IPython.display import display
plt.close(fig)  # Close the initial figure to prevent duplicate display
display(HTML(ani.to_jshtml()))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.patches import Circle, Polygon, FancyBboxPatch
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection, Line3DCollection, pathpatch_2d_to_3d
from IPython.display import HTML
import random
import math
from dataclasses import dataclass, field
from enum import Enum, auto
from typing import List, Dict, Tuple, Optional
from collections import deque
from shapely.geometry import Point, Polygon as ShapelyPolygon
from scipy.spatial import cKDTree

import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 1000

# Константи з технічного завдання
class BaseParams:
    SIM_AREA = (800, 600)  # Ширина, висота симуляційного простору у пікселях.
    Z_RANGE = (-100, 100)  # Мінімальна та максимальна висота польоту дронів у метрах.
    FPS = 60               # Кількість кадрів симуляції на секунду.
    PREDICTION_HORIZON = 10  # Кількість кроків для прогнозування траєкторії дрона.
    BACKGROUND_COLOR = (240/255, 240/255, 240/255) # Колір фону візуалізації (RGB).
    TRAJECTORY_COLOR = (200/255, 200/255, 200/255) # Колір відображення історії траєкторії (RGB).

class DroneSpecs:
    MAX_SPEED_XY = 5.0     # Максимальна горизонтальна швидкість дрона у м/с.
    MAX_SPEED_Z = 3.0      # Максимальна вертикальна швидкість дрона у м/с.
    MAX_ACCEL = 2.5        # Максимальне прискорення дрона у м/с².
    COMM_RANGE = 150.0     # Радіус зв'язку між дронами у метрах.
    MIN_SEPARATION = 2.0   # Мінімальна безпечна відстань між центрами дронів у метрах.
    DRONE_RADIUS = 5       # Радіус дрона для візуалізації у пікселях.
    SAFETY_FACTOR = 1.5    # Коефіцієнт запасу безпеки при перевірці зіткнень.

class SafetyZones:
    WARNING = 3 * DroneSpecs.MIN_SEPARATION  # Жовта зона
    DANGER = 2 * DroneSpecs.MIN_SEPARATION   # Оранжева зона
    CRITICAL = 1.5 * DroneSpecs.MIN_SEPARATION  # Червона зона

# Стани дронів
class DroneState(Enum):
    NORMAL = auto()        # Звичайний політ
    WARNING = auto()       # Попередження
    DANGER = auto()        # Загроза
    CRITICAL = auto()      # Критичний стан
    AVOIDING = auto()      # Виконує маневр уникнення

# Кольори для візуалізації
STATE_COLORS = {
    DroneState.NORMAL: 'green',
    DroneState.WARNING: 'yellow',
    DroneState.DANGER: 'orange',
    DroneState.CRITICAL: 'red',
    DroneState.AVOIDING: 'blue'
}

@dataclass
class PIDController:
    """Адаптивний PID-контролер для керування швидкістю"""
    Kp: float = 0.8
    Ki: float = 0.1
    Kd: float = 0.3
    target: float = 3.0
    integral: float = 0.0
    prev_error: float = 0.0
    dt: float = 1.0/BaseParams.FPS

    def compute(self, current: float) -> float:
        error = self.target - current
        self.integral += error * self.dt
        derivative = (error - self.prev_error) / self.dt
        output = self.Kp*error + self.Ki*self.integral + self.Kd*derivative
        self.prev_error = error
        return output

    def adapt_for_safety(self, risk_level: float):
        """Адаптація параметрів залежно від рівня ризику"""
        self.Kp *= (1 + 0.5 * risk_level)
        self.Kd *= (1 + 0.8 * risk_level)
        self.Ki *= max(0.5, (1 - 0.3 * risk_level))  # Забезпечуємо мінімальне значення Ki

@dataclass
class Drone:
    """Клас, що представляє окремий дрон у зграї"""
    id: int
    position: np.ndarray  # [x, y, z]
    velocity: np.ndarray  # [vx, vy, vz]
    state: DroneState = DroneState.NORMAL
    neighbors: List['Drone'] = field(default_factory=list)
    pid: PIDController = field(default_factory=PIDController)
    avoidance_direction: float = 0.0  # Напрямок уникнення (-1 або +1)
    risk_level: float = 0.0  # Рівень ризику 0-1
    trajectory_history: deque = field(default_factory=lambda: deque(maxlen=50))
    communication_range: float = DroneSpecs.COMM_RANGE

    def __post_init__(self):
        self.trajectory_history.append(self.position.copy())

    def update_state_based_on_risk(self):
        """Оновлення стану на основі поточного рівня ризику"""
        if self.risk_level > 0.8:
            self.state = DroneState.CRITICAL
        elif self.risk_level > 0.5:
            self.state = DroneState.DANGER
        elif self.risk_level > 0.2:
            self.state = DroneState.WARNING
        else:
            self.state = DroneState.NORMAL

    def predict_trajectory(self, steps: int) -> List[np.ndarray]:
        """Прогнозування траєкторії на задану кількість кроків"""
        trajectory = []
        pos = self.position.copy()
        vel = self.velocity.copy()

        for _ in range(steps):
            pos += vel * self.pid.dt
            trajectory.append(pos.copy())

        return trajectory

    def avoid_collision(self, neighbors: List['Drone']):
        """Виконання маневру уникнення"""
        if self.state not in [DroneState.DANGER, DroneState.CRITICAL]:
            return

        self.state = DroneState.AVOIDING

        # Якщо напрямок ще не вибрано - вибираємо випадково
        if self.avoidance_direction == 0:
            self.avoidance_direction = 1 if random.random() > 0.5 else -1

        # Обчислюємо силу уникнення
        avoidance_force = np.zeros(3)
        for neighbor in neighbors:
            dist = np.linalg.norm(self.position - neighbor.position)
            if dist < SafetyZones.CRITICAL:
                dir_vec = (self.position - neighbor.position)
                if np.linalg.norm(dir_vec) > 0:
                    dir_vec = dir_vec / np.linalg.norm(dir_vec)
                avoidance_force += dir_vec * (1 - dist/SafetyZones.CRITICAL)

        # Додаємо Z-компоненту
        if np.linalg.norm(avoidance_force) > 0:
            avoidance_force = avoidance_force / np.linalg.norm(avoidance_force)
            avoidance_force[2] = self.avoidance_direction * 0.5  # Плавний набір висоти

        return avoidance_force * DroneSpecs.MAX_ACCEL * 2  # Підсилюємо прискорення уникнення

    def generate_avoidance_options(self):
        """Генерація варіантів маневру уникнення"""
        return [
            {'dz': DroneSpecs.MAX_SPEED_Z * self.get_sign(), 'priority': 0},  # Зміна висоти
            {'brake_xy': True, 'priority': 1}                                  # Гальмування в XY площині
        ]

    def get_sign(self):
        """Повертає випадковий знак (+1 або -1)"""
        return 1 if random.random() > 0.5 else -1

class SwarmSimulation:
    """Основний клас симулятора зграї дронів"""

    def __init__(self, num_drones: int = 15):
        self.num_drones = num_drones
        self.drones: List[Drone] = []
        self.fig, self.ax = plt.subplots(figsize=(12, 8), subplot_kw={'projection': '3d'})
        self.animation = None
        self.simulation_time = 0
        self.collisions_count = 0
        self.near_misses_count = 0
        self.setup_environment()
        self.initialize_drones()

    def setup_environment(self):
        """Налаштування графічного середовища"""
        self.ax.set_xlim(0, BaseParams.SIM_AREA[0])
        self.ax.set_ylim(0, BaseParams.SIM_AREA[1])
        self.ax.set_zlim(BaseParams.Z_RANGE[0], BaseParams.Z_RANGE[1])
        self.ax.set_xlabel('X')
        self.ax.set_ylabel('Y')
        self.ax.set_zlabel('Z')
        self.ax.set_title('SwarmSafe 3D - Симулятор зграї дронів')

        # Додаємо коло траєкторії
        self.trajectory_circle = Circle(
            (BaseParams.SIM_AREA[0]/2, BaseParams.SIM_AREA[1]/2),
            200, fill=False, color=BaseParams.TRAJECTORY_COLOR, linestyle='--'
        )
        self.ax.add_patch(self.trajectory_circle)
        pathpatch_2d_to_3d(self.trajectory_circle, z=0, zdir="z")

        # Елементи візуалізації
        self.drone_plots = []
        self.z_indicator = []
        self.risk_circles = []
        self.predicted_trajectories = []  # Додано для візуалізації прогнозованих траєкторій
        self.info_text = self.ax.text2D(
            0.02, 0.95, '', transform=self.ax.transAxes,
            bbox=dict(facecolor='white', alpha=0.7)
        )

    def initialize_drones(self):
        """Ініціалізація дронів у випадкових позиціях"""
        center_x, center_y = BaseParams.SIM_AREA[0]/2, BaseParams.SIM_AREA[1]/2

        for i in range(self.num_drones):
            # Випадкове положення навколо кола
            angle = random.uniform(0, 2*math.pi)
            radius = random.uniform(50, 200)
            x = center_x + radius * math.cos(angle)
            y = center_y + radius * math.sin(angle)
            z = random.uniform(BaseParams.Z_RANGE[0], BaseParams.Z_RANGE[1])

            # Випадкова початкова швидкість (напрямлена по дотичній до кола)
            speed = random.uniform(1, DroneSpecs.MAX_SPEED_XY)
            vx = -speed * math.sin(angle)
            vy = speed * math.cos(angle)
            vz = 0

            self.drones.append(Drone(
                id=i,
                position=np.array([x, y, z]),
                velocity=np.array([vx, vy, vz])
            ))

            # Додаємо графічні елементи
            self.drone_plots.append(self.ax.plot([], [], [], 'o', markersize=DroneSpecs.DRONE_RADIUS)[0])
            self.z_indicator.append(self.ax.text(0, 0, 0, '', fontsize=8, ha='center', va='center'))
            self.risk_circles.append(FancyBboxPatch(
                (0, 0), SafetyZones.WARNING, SafetyZones.WARNING,
                fill=False, alpha=0.1, edgecolor='yellow'
            ))
            self.ax.add_patch(self.risk_circles[-1])
            pathpatch_2d_to_3d(self.risk_circles[-1], z=0, zdir="z")
            self.predicted_trajectories.append(self.ax.plot([], [], [], '--', color='gray', alpha=0.5)[0])  # Додано для візуалізації прогнозованих траєкторій

    def find_neighbors(self, drone: Drone) -> List[Drone]:
        """Знаходження сусідніх дронів у межах радіусу зв'язку"""
        return [d for d in self.drones
                if d.id != drone.id
                and np.linalg.norm(drone.position[:2] - d.position[:2]) < drone.communication_range]

    def calculate_risk_level(self, drone: Drone) -> float:
        """Розрахунок рівня ризику зіткнення"""
        if not drone.neighbors:
            return 0.0

        min_dist = min(np.linalg.norm(drone.position - n.position) for n in drone.neighbors)

        if min_dist < SafetyZones.CRITICAL:
            return 1.0
        elif min_dist < SafetyZones.DANGER:
            return 0.7
        elif min_dist < SafetyZones.WARNING:
            return 0.3
        return 0.0

    def apply_boids_rules(self, drone: Drone) -> np.ndarray:
        """Застосування правил Бойдза (розділення, вирівнювання, згуртування)"""
        separation = np.zeros(3)
        alignment = np.zeros(3)
        cohesion = np.zeros(3)
        center = np.zeros(3)
        total = len(drone.neighbors)

        if total == 0:
            return np.zeros(3)

        for neighbor in drone.neighbors:
            diff = drone.position - neighbor.position
            dist = np.linalg.norm(diff)

            # Правило розділення
            if dist < SafetyZones.WARNING:
                separation += diff / (dist + 0.1)  # Додаємо 0.1, щоб уникнути ділення на 0

            # Правило вирівнювання
            alignment += neighbor.velocity

            # Для правила згуртування
            center += neighbor.position

        # Нормалізація векторів
        if np.linalg.norm(separation) > 0:
            separation = (separation / np.linalg.norm(separation)) * DroneSpecs.MAX_SPEED_XY

        alignment = (alignment / total - drone.velocity) * 0.5

        center = center / total
        cohesion = (center - drone.position) * 0.05

        # Комбінуємо правила з вагами
        return separation * 1.5 + alignment * 1.0 + cohesion * 1.0

    def update_drone(self, drone: Drone):
        """Оновлення стану одного дрона"""
        # Оновлюємо список сусідів
        drone.neighbors = self.find_neighbors(drone)

        # Розраховуємо рівень ризику та оновлюємо стан
        drone.risk_level = self.calculate_risk_level(drone)
        drone.update_state_based_on_risk()

        # Адаптуємо PID-контролер
        drone.pid.adapt_for_safety(drone.risk_level)

        # Обчислюємо сили від правил Бойдза
        boids_force = self.apply_boids_rules(drone)

        # Обчислюємо силу уникнення (якщо потрібно)
        avoidance_force = np.zeros(3)
        if drone.state in [DroneState.DANGER, DroneState.CRITICAL]:
            avoidance_force = drone.avoid_collision(drone.neighbors)

        # Комбінуємо сили з пріоритетом уникнення
        if np.linalg.norm(avoidance_force) > 0:
            total_force = avoidance_force * 2.0 + boids_force * 0.5
        else:
            total_force = boids_force

        # Обмежуємо прискорення
        accel_norm = np.linalg.norm(total_force)
        if accel_norm > DroneSpecs.MAX_ACCEL:
            total_force = total_force / accel_norm * DroneSpecs.MAX_ACCEL

        # Оновлюємо швидкість
        drone.velocity += total_force * (1.0/BaseParams.FPS)

        # Обмежуємо швидкість
        speed_xy = np.linalg.norm(drone.velocity[:2])
        if speed_xy > DroneSpecs.MAX_SPEED_XY:
            drone.velocity[:2] = drone.velocity[:2] / speed_xy * DroneSpecs.MAX_SPEED_XY

        speed_z = abs(drone.velocity[2])
        if speed_z > DroneSpecs.MAX_SPEED_Z:
            drone.velocity[2] = drone.velocity[2] / speed_z * DroneSpecs.MAX_SPEED_Z

        # Оновлюємо позицію
        drone.position += drone.velocity * (1.0/BaseParams.FPS)

        # Застосовуємо граничні умови
        self.apply_boundary_conditions(drone)

        # Додаємо позицію до історії траєкторії
        drone.trajectory_history.append(drone.position.copy())

        # Перевіряємо на зіткнення
        self.check_collisions(drone)

    def apply_boundary_conditions(self, drone: Drone):
        """Обробка виходу за межі симуляції"""
        # Обмеження XY
        if drone.position[0] < 0:
            drone.position[0] = 0
            drone.velocity[0] *= -0.5
        elif drone.position[0] > BaseParams.SIM_AREA[0]:
            drone.position[0] = BaseParams.SIM_AREA[0]
            drone.velocity[0] *= -0.5

        if drone.position[1] < 0:
            drone.position[1] = 0
            drone.velocity[1] *= -0.5
        elif drone.position[1] > BaseParams.SIM_AREA[1]:
            drone.position[1] = BaseParams.SIM_AREA[1]
            drone.velocity[1] *= -0.5

        # Обмеження Z
        if drone.position[2] < BaseParams.Z_RANGE[0]:
            drone.position[2] = BaseParams.Z_RANGE[0]
            drone.velocity[2] = 0
        elif drone.position[2] > BaseParams.Z_RANGE[1]:
            drone.position[2] = BaseParams.Z_RANGE[1]
            drone.velocity[2] = 0

        # Плавне повернення до z=0, якщо немає загрози
        if drone.state == DroneState.NORMAL and abs(drone.position[2]) > 1:
            drone.velocity[2] += -0.1 * drone.position[2]

    def check_collisions(self, drone: Drone):
        """Перевірка зіткнень та майже зіткнень"""
        for neighbor in drone.neighbors:
            dist = np.linalg.norm(drone.position - neighbor.position)
            if dist < DroneSpecs.MIN_SEPARATION * DroneSpecs.SAFETY_FACTOR:
                self.near_misses_count += 1
            if dist < DroneSpecs.MIN_SEPARATION:
                self.collisions_count += 1
                # Відштовхуємо дрони при зіткненні
                direction = (drone.position - neighbor.position)
                if np.linalg.norm(direction) > 0:
                    direction = direction / np.linalg.norm(direction)
                drone.velocity += direction * DroneSpecs.MAX_ACCEL * 0.5

    def update_visuals(self):
        """Оновлення графічних елементів"""
        for i, drone in enumerate(self.drones):
            # Оновлюємо положення маркера
            self.drone_plots[i].set_data([drone.position[0]], [drone.position[1]])
            self.drone_plots[i].set_3d_properties(drone.position[2], 'z')
            self.drone_plots[i].set_color(STATE_COLORS[drone.state])

            # Оновлюємо індикатор висоти
            self.z_indicator[i].set_position((drone.position[0], drone.position[1], drone.position[2]))
            self.z_indicator[i].set_text(f'{drone.position[2]:.0f}')

            # Оновлюємо коло ризику
            self.risk_circles[i]._width = SafetyZones.CRITICAL if drone.state == DroneState.CRITICAL else SafetyZones.DANGER if drone.state == DroneState.DANGER else SafetyZones.WARNING if drone.state == DroneState.WARNING else 0
            self.risk_circles[i]._height = self.risk_circles[i]._width
            self.risk_circles[i].set_edgecolor('red' if drone.state == DroneState.CRITICAL else 'orange' if drone.state == DroneState.DANGER else 'yellow')
            self.risk_circles[i].set_alpha(drone.risk_level * 0.3)

            # Оновлюємо прогнозовану траєкторію
            predicted_positions = drone.predict_trajectory(BaseParams.PREDICTION_HORIZON)
            x_coords = [pos[0] for pos in predicted_positions]
            y_coords = [pos[1] for pos in predicted_positions]
            z_coords = [pos[2] for pos in predicted_positions]
            self.predicted_trajectories[i].set_data(x_coords, y_coords)
            self.predicted_trajectories[i].set_3d_properties(z_coords, 'z')

        # Оновлюємо інформаційний текст
        info_text = (
            f'Час: {self.simulation_time:.1f} с\n'
            f'Дронів: {self.num_drones}\n'
            f'Зіткнень: {self.collisions_count}\n'
            f'Майже зіткнень: {self.near_misses_count}\n'
            f'Ризик (середній): {sum(d.risk_level for d in self.drones)/len(self.drones):.2f}'
        )
        self.info_text.set_text(info_text)

    def update(self, frame):
        """Оновлення симуляції для анімації"""
        self.simulation_time += 1.0/BaseParams.FPS

        # Оновлюємо всіх дронів
        for drone in self.drones:
            self.update_drone(drone)

        # Оновлюємо візуалізацію
        self.update_visuals()

        # Повертаємо всі елементи, які потрібно оновити
        return self.drone_plots + self.z_indicator + self.risk_circles + self.predicted_trajectories + [self.info_text]

    def run_simulation(self):
        """Запуск симуляції"""
        self.animation = animation.FuncAnimation(
            self.fig, self.update,
            frames=1000, interval=1000/BaseParams.FPS, blit=True
        )
        plt.close()
        return HTML(self.animation.to_jshtml())

# Запуск симуляції
sim = SwarmSimulation(num_drones=100)
sim.run_simulation()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.patches import Circle, Polygon, FancyBboxPatch
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection, Line3DCollection, pathpatch_2d_to_3d
from IPython.display import HTML
import random
import math
from dataclasses import dataclass, field
from enum import Enum, auto
from typing import List, Dict, Tuple, Optional
from collections import deque
from shapely.geometry import Point, Polygon as ShapelyPolygon
from scipy.spatial import cKDTree

import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 1000

# Константи з технічного завдання
class BaseParams:
    SIM_AREA = (800, 600)  # Ширина, висота симуляційного простору у пікселях.
    Z_RANGE = (-100, 100)  # Мінімальна та максимальна висота польоту дронів у метрах.
    FPS = 60               # Кількість кадрів симуляції на секунду.
    PREDICTION_HORIZON = 10  # Кількість кроків для прогнозування траєкторії дрона.
    BACKGROUND_COLOR = (240/255, 240/255, 240/255) # Колір фону візуалізації (RGB).
    TRAJECTORY_COLOR = (200/255, 200/255, 200/255) # Колір відображення історії траєкторії (RGB).

class DroneSpecs:
    MAX_SPEED_XY = 5.0     # Максимальна горизонтальна швидкість дрона у м/с.
    MAX_SPEED_Z = 3.0      # Максимальна вертикальна швидкість дрона у м/с.
    MAX_ACCEL = 2.5        # Максимальне прискорення дрона у м/с².
    COMM_RANGE = 150.0     # Радіус зв'язку між дронами у метрах.
    MIN_SEPARATION = 2.0   # Мінімальна безпечна відстань між центрами дронів у метрах.
    DRONE_RADIUS = 5       # Радіус дрона для візуалізації у пікселях.
    SAFETY_FACTOR = 1.5    # Коефіцієнт запасу безпеки при перевірці зіткнень.

class SafetyZones:
    WARNING = 3 * DroneSpecs.MIN_SEPARATION  # Жовта зона
    DANGER = 2 * DroneSpecs.MIN_SEPARATION   # Оранжева зона
    CRITICAL = 1.5 * DroneSpecs.MIN_SEPARATION  # Червона зона

# Стани дронів
class DroneState(Enum):
    NORMAL = auto()        # Звичайний політ
    WARNING = auto()       # Попередження
    DANGER = auto()        # Загроза
    CRITICAL = auto()      # Критичний стан
    AVOIDING = auto()      # Виконує маневр уникнення

# Кольори для візуалізації
STATE_COLORS = {
    DroneState.NORMAL: 'green',
    DroneState.WARNING: 'yellow',
    DroneState.DANGER: 'orange',
    DroneState.CRITICAL: 'red',
    DroneState.AVOIDING: 'blue'
}

TARGET_ZONE_COLORS = ['blue', 'cyan', 'magenta', 'purple', 'brown']  # Кольори для цільових зон

@dataclass
class PIDController:
    """Адаптивний PID-контролер для керування швидкістю"""
    Kp: float = 0.8
    Ki: float = 0.1
    Kd: float = 0.3
    target: float = 3.0
    integral: float = 0.0
    prev_error: float = 0.0
    dt: float = 1.0/BaseParams.FPS

    def compute(self, current: float) -> float:
        error = self.target - current
        self.integral += error * self.dt
        derivative = (error - self.prev_error) / self.dt
        output = self.Kp*error + self.Ki*self.integral + self.Kd*derivative
        self.prev_error = error
        return output

    def adapt_for_safety(self, risk_level: float):
        """Адаптація параметрів залежно від рівня ризику"""
        self.Kp *= (1 + 0.5 * risk_level)
        self.Kd *= (1 + 0.8 * risk_level)
        self.Ki *= max(0.5, (1 - 0.3 * risk_level))  # Забезпечуємо мінімальне значення Ki

@dataclass
class Drone:
    """Клас, що представляє окремий дрон у зграї"""
    id: int
    position: np.ndarray  # [x, y, z]
    velocity: np.ndarray  # [vx, vy, vz]
    state: DroneState = DroneState.NORMAL
    neighbors: List['Drone'] = field(default_factory=list)
    pid: PIDController = field(default_factory=PIDController)
    avoidance_direction: float = 0.0  # Напрямок уникнення (-1 або +1)
    risk_level: float = 0.0  # Рівень ризику 0-1
    trajectory_history: deque = field(default_factory=lambda: deque(maxlen=50))
    communication_range: float = DroneSpecs.COMM_RANGE
    target_zone_id: Optional[int] = None  # ID цільової зони

    def __post_init__(self):
        self.trajectory_history.append(self.position.copy())

    def update_state_based_on_risk(self):
        """Оновлення стану на основі поточного рівня ризику"""
        if self.risk_level > 0.8:
            self.state = DroneState.CRITICAL
        elif self.risk_level > 0.5:
            self.state = DroneState.DANGER
        elif self.risk_level > 0.2:
            self.state = DroneState.WARNING
        else:
            self.state = DroneState.NORMAL

    def predict_trajectory(self, steps: int) -> List[np.ndarray]:
        """Прогнозування траєкторії на задану кількість кроків"""
        trajectory = []
        pos = self.position.copy()
        vel = self.velocity.copy()

        for _ in range(steps):
            pos += vel * self.pid.dt
            trajectory.append(pos.copy())

        return trajectory

    def avoid_collision(self, neighbors: List['Drone']):
        """Виконання маневру уникнення"""
        if self.state not in [DroneState.DANGER, DroneState.CRITICAL]:
            return

        self.state = DroneState.AVOIDING

        # Якщо напрямок ще не вибрано - вибираємо випадково
        if self.avoidance_direction == 0:
            self.avoidance_direction = 1 if random.random() > 0.5 else -1

        # Обчислюємо силу уникнення
        avoidance_force = np.zeros(3)
        for neighbor in neighbors:
            dist = np.linalg.norm(self.position - neighbor.position)
            if dist < SafetyZones.CRITICAL:
                dir_vec = (self.position - neighbor.position)
                if np.linalg.norm(dir_vec) > 0:
                    dir_vec = dir_vec / np.linalg.norm(dir_vec)
                avoidance_force += dir_vec * (1 - dist/SafetyZones.CRITICAL)

        # Додаємо Z-компоненту
        if np.linalg.norm(avoidance_force) > 0:
            avoidance_force = avoidance_force / np.linalg.norm(avoidance_force)
            avoidance_force[2] = self.avoidance_direction * 0.5  # Плавний набір висоти

        return avoidance_force * DroneSpecs.MAX_ACCEL * 2  # Підсилюємо прискорення уникнення

    def generate_avoidance_options(self):
        """Генерація варіантів маневру уникнення"""
        return [
            {'dz': DroneSpecs.MAX_SPEED_Z * self.get_sign(), 'priority': 0},  # Зміна висоти
            {'brake_xy': True, 'priority': 1}                                  # Гальмування в XY площині
        ]

    def get_sign(self):
        """Повертає випадковий знак (+1 або -1)"""
        return 1 if random.random() > 0.5 else -1

class SwarmSimulation:
    """Основний клас симулятора зграї дронів"""

    def __init__(self, num_drones: int = 15, num_target_zones: int = 5):
        self.num_drones = num_drones
        self.num_target_zones = num_target_zones
        self.drones: List[Drone] = []
        self.target_zones: List[Tuple[float, float, float, int]] = []  # (x, y, z, id)
        self.fig, self.ax = plt.subplots(figsize=(18, 12), subplot_kw={'projection': '3d'})
        self.animation = None
        self.simulation_time = 0
        self.collisions_count = 0
        self.near_misses_count = 0
        self.setup_environment()
        self.initialize_drones()
        self.initialize_target_zones()

    def setup_environment(self):
        """Налаштування графічного середовища"""
        self.ax.set_xlim(0, BaseParams.SIM_AREA[0])
        self.ax.set_ylim(0, BaseParams.SIM_AREA[1])
        self.ax.set_zlim(BaseParams.Z_RANGE[0], BaseParams.Z_RANGE[1])
        self.ax.set_xlabel('X')
        self.ax.set_ylabel('Y')
        self.ax.set_zlabel('Z')
        self.ax.set_title('SwarmSafe 3D - Симулятор зграї дронів')

        # Елементи візуалізації
        self.drone_plots = []
        self.z_indicator = []
        self.risk_circles = []
        self.predicted_trajectories = []  # Додано для візуалізації прогнозованих траєкторій
        self.target_zone_lines = []  # Додано для візуалізації околиць цільових зон
        self.info_text = self.ax.text2D(
            0.02, 0.95, '', transform=self.ax.transAxes,
            bbox=dict(facecolor='white', alpha=0.7)
        )

    def initialize_drones(self):
        """Ініціалізація дронів у випадкових позиціях"""
        center_x, center_y = BaseParams.SIM_AREA[0]/2, BaseParams.SIM_AREA[1]/2

        for i in range(self.num_drones):
            # Випадкове положення навколо кола
            angle = random.uniform(0, 2*math.pi)
            radius = random.uniform(50, 200)
            x = center_x + radius * math.cos(angle)
            y = center_y + radius * math.sin(angle)
            z = random.uniform(BaseParams.Z_RANGE[0], BaseParams.Z_RANGE[1])

            # Випадкова початкова швидкість (напрямлена по дотичній до кола)
            speed = random.uniform(1, DroneSpecs.MAX_SPEED_XY)
            vx = -speed * math.sin(angle)
            vy = speed * math.cos(angle)
            vz = 0

            self.drones.append(Drone(
                id=i,
                position=np.array([x, y, z]),
                velocity=np.array([vx, vy, vz])
            ))

            # Додаємо графічні елементи
            self.drone_plots.append(self.ax.plot([], [], [], 'o', markersize=DroneSpecs.DRONE_RADIUS)[0])
            self.z_indicator.append(self.ax.text(0, 0, 0, '', fontsize=8, ha='center', va='center'))
            self.risk_circles.append(FancyBboxPatch(
                (0, 0), SafetyZones.WARNING, SafetyZones.WARNING,
                fill=False, alpha=0.1, edgecolor='yellow'
            ))
            self.ax.add_patch(self.risk_circles[-1])
            pathpatch_2d_to_3d(self.risk_circles[-1], z=0, zdir="z")
            self.predicted_trajectories.append(self.ax.plot([], [], [], '--', color='gray', alpha=0.5)[0])  # Додано для візуалізації прогнозованих траєкторій

    def initialize_target_zones(self):
        """Ініціалізація цільових зон у випадкових позиціях"""
        for i in range(self.num_target_zones):
            x = random.uniform(0, BaseParams.SIM_AREA[0])
            y = random.uniform(0, BaseParams.SIM_AREA[1])
            z = random.uniform(BaseParams.Z_RANGE[0], BaseParams.Z_RANGE[1])
            self.target_zones.append((x, y, z, i))

            # Додаємо графічні елементи для візуалізації околиць цільових зон
            color = TARGET_ZONE_COLORS[i % len(TARGET_ZONE_COLORS)]
            self.target_zone_lines.extend([
                self.ax.plot([x-50, x+50], [y, y], [z, z], ':', color=color)[0],
                self.ax.plot([x, x], [y-50, y+50], [z, z], ':', color=color)[0],
                self.ax.plot([x, x], [y, y], [z-50, z+50], ':', color=color)[0]
            ])

    def find_neighbors(self, drone: Drone) -> List[Drone]:
        """Знаходження сусідніх дронів у межах радіусу зв'язку"""
        return [d for d in self.drones
                if d.id != drone.id
                and np.linalg.norm(drone.position[:2] - d.position[:2]) < drone.communication_range]

    def calculate_risk_level(self, drone: Drone) -> float:
        """Розрахунок рівня ризику зіткнення"""
        if not drone.neighbors:
            return 0.0

        min_dist = min(np.linalg.norm(drone.position - n.position) for n in drone.neighbors)

        if min_dist < SafetyZones.CRITICAL:
            return 1.0
        elif min_dist < SafetyZones.DANGER:
            return 0.7
        elif min_dist < SafetyZones.WARNING:
            return 0.3
        return 0.0

    def apply_boids_rules(self, drone: Drone) -> np.ndarray:
        """Застосування правил Бойдза (розділення, вирівнювання, згуртування)"""
        separation = np.zeros(3)
        alignment = np.zeros(3)
        cohesion = np.zeros(3)
        center = np.zeros(3)
        total = len(drone.neighbors)

        if total == 0:
            return np.zeros(3)

        for neighbor in drone.neighbors:
            diff = drone.position - neighbor.position
            dist = np.linalg.norm(diff)

            # Правило розділення
            if dist < SafetyZones.WARNING:
                separation += diff / (dist + 0.1)  # Додаємо 0.1, щоб уникнути ділення на 0

            # Правило вирівнювання
            alignment += neighbor.velocity

            # Для правила згуртування
            center += neighbor.position

        # Нормалізація векторів
        if np.linalg.norm(separation) > 0:
            separation = (separation / np.linalg.norm(separation)) * DroneSpecs.MAX_SPEED_XY

        alignment = (alignment / total - drone.velocity) * 0.5

        center = center / total
        cohesion = (center - drone.position) * 0.05

        # Комбінуємо правила з вагами
        return separation * 1.5 + alignment * 1.0 + cohesion * 1.0

    def update_drone(self, drone: Drone):
        """Оновлення стану одного дрона"""
        # Оновлюємо список сусідів
        drone.neighbors = self.find_neighbors(drone)

        # Розраховуємо рівень ризику та оновлюємо стан
        drone.risk_level = self.calculate_risk_level(drone)
        drone.update_state_based_on_risk()

        # Адаптуємо PID-контролер
        drone.pid.adapt_for_safety(drone.risk_level)

        # Обчислюємо сили від правил Бойдза
        boids_force = self.apply_boids_rules(drone)

        # Обчислюємо силу уникнення (якщо потрібно)
        avoidance_force = np.zeros(3)
        if drone.state in [DroneState.DANGER, DroneState.CRITICAL]:
            avoidance_force = drone.avoid_collision(drone.neighbors)

        # Комбінуємо сили з пріоритетом уникнення
        if np.linalg.norm(avoidance_force) > 0:
            total_force = avoidance_force * 2.0 + boids_force * 0.5
        else:
            total_force = boids_force

        # Обмежуємо прискорення
        accel_norm = np.linalg.norm(total_force)
        if accel_norm > DroneSpecs.MAX_ACCEL:
            total_force = total_force / accel_norm * DroneSpecs.MAX_ACCEL

        # Оновлюємо швидкість
        drone.velocity += total_force * (1.0/BaseParams.FPS)

        # Обмежуємо швидкість
        speed_xy = np.linalg.norm(drone.velocity[:2])
        if speed_xy > DroneSpecs.MAX_SPEED_XY:
            drone.velocity[:2] = drone.velocity[:2] / speed_xy * DroneSpecs.MAX_SPEED_XY

        speed_z = abs(drone.velocity[2])
        if speed_z > DroneSpecs.MAX_SPEED_Z:
            drone.velocity[2] = drone.velocity[2] / speed_z * DroneSpecs.MAX_SPEED_Z

        # Оновлюємо позицію
        drone.position += drone.velocity * (1.0/BaseParams.FPS)

        # Застосовуємо граничні умови
        self.apply_boundary_conditions(drone)

        # Додаємо позицію до історії траєкторії
        drone.trajectory_history.append(drone.position.copy())

        # Перевіряємо на зіткнення
        self.check_collisions(drone)

    def apply_boundary_conditions(self, drone: Drone):
        """Обробка виходу за межі симуляції"""
        # Обмеження XY
        if drone.position[0] < 0:
            drone.position[0] = 0
            drone.velocity[0] *= -0.5
        elif drone.position[0] > BaseParams.SIM_AREA[0]:
            drone.position[0] = BaseParams.SIM_AREA[0]
            drone.velocity[0] *= -0.5

        if drone.position[1] < 0:
            drone.position[1] = 0
            drone.velocity[1] *= -0.5
        elif drone.position[1] > BaseParams.SIM_AREA[1]:
            drone.position[1] = BaseParams.SIM_AREA[1]
            drone.velocity[1] *= -0.5

        # Обмеження Z
        if drone.position[2] < BaseParams.Z_RANGE[0]:
            drone.position[2] = BaseParams.Z_RANGE[0]
            drone.velocity[2] = 0
        elif drone.position[2] > BaseParams.Z_RANGE[1]:
            drone.position[2] = BaseParams.Z_RANGE[1]
            drone.velocity[2] = 0

        # Плавне повернення до z=0, якщо немає загрози
        if drone.state == DroneState.NORMAL and abs(drone.position[2]) > 1:
            drone.velocity[2] += -0.1 * drone.position[2]

    def check_collisions(self, drone: Drone):
        """Перевірка зіткнень та майже зіткнень"""
        for neighbor in drone.neighbors:
            dist = np.linalg.norm(drone.position - neighbor.position)
            if dist < DroneSpecs.MIN_SEPARATION * DroneSpecs.SAFETY_FACTOR:
                self.near_misses_count += 1
            if dist < DroneSpecs.MIN_SEPARATION:
                self.collisions_count += 1
                # Відштовхуємо дрони при зіткненні
                direction = (drone.position - neighbor.position)
                if np.linalg.norm(direction) > 0:
                    direction = direction / np.linalg.norm(direction)
                drone.velocity += direction * DroneSpecs.MAX_ACCEL * 0.5

    def update_visuals(self):
        """Оновлення графічних елементів"""
        for i, drone in enumerate(self.drones):
            # Оновлюємо положення маркера
            self.drone_plots[i].set_data([drone.position[0]], [drone.position[1]])
            self.drone_plots[i].set_3d_properties(drone.position[2], 'z')
            self.drone_plots[i].set_color(STATE_COLORS[drone.state])

            # Оновлюємо індикатор висоти
            self.z_indicator[i].set_position((drone.position[0], drone.position[1], drone.position[2]))
            self.z_indicator[i].set_text(f'{drone.position[2]:.0f}')

            # Оновлюємо коло ризику
            self.risk_circles[i]._width = SafetyZones.CRITICAL if drone.state == DroneState.CRITICAL else SafetyZones.DANGER if drone.state == DroneState.DANGER else SafetyZones.WARNING if drone.state == DroneState.WARNING else 0
            self.risk_circles[i]._height = self.risk_circles[i]._width
            self.risk_circles[i].set_edgecolor('red' if drone.state == DroneState.CRITICAL else 'orange' if drone.state == DroneState.DANGER else 'yellow')
            self.risk_circles[i].set_alpha(drone.risk_level * 0.3)

            # Оновлюємо прогнозовану траєкторію
            predicted_positions = drone.predict_trajectory(BaseParams.PREDICTION_HORIZON)
            x_coords = [pos[0] for pos in predicted_positions]
            y_coords = [pos[1] for pos in predicted_positions]
            z_coords = [pos[2] for pos in predicted_positions]
            self.predicted_trajectories[i].set_data(x_coords, y_coords)
            self.predicted_trajectories[i].set_3d_properties(z_coords, 'z')

        # Оновлюємо інформаційний текст
        info_text = (
            f'Час: {self.simulation_time:.1f} с\n'
            f'Дронів: {self.num_drones}\n'
            f'Зіткнень: {self.collisions_count}\n'
            f'Майже зіткнень: {self.near_misses_count}\n'
            f'Ризик (середній): {sum(d.risk_level for d in self.drones)/len(self.drones):.2f}'
        )
        self.info_text.set_text(info_text)

    def update(self, frame):
        """Оновлення симуляції для анімації"""
        self.simulation_time += 1.0/BaseParams.FPS

        # Оновлюємо всіх дронів
        for drone in self.drones:
            self.update_drone(drone)

        # Оновлюємо візуалізацію
        self.update_visuals()

        # Повертаємо всі елементи, які потрібно оновити
        return self.drone_plots + self.z_indicator + self.risk_circles + self.predicted_trajectories + self.target_zone_lines + [self.info_text]

    def run_simulation(self):
        """Запуск симуляції"""
        self.animation = animation.FuncAnimation(
            self.fig, self.update,
            frames=1000, interval=1000/BaseParams.FPS, blit=True
        )
        plt.close()
        return HTML(self.animation.to_jshtml())

# Запуск симуляції
sim = SwarmSimulation(num_drones=50, num_target_zones=2)
sim.run_simulation()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML
import random
import math
from dataclasses import dataclass, field
from enum import Enum, auto
from typing import List, Dict, Tuple, Optional
from collections import deque
from scipy.spatial import KDTree
from scipy.optimize import minimize
import cvxpy as cp

# Константи з технічного завдання
class BaseParams:
    SIM_AREA = (1000, 800)  # Ширина, висота симуляційного простору (м)
    Z_RANGE = (0, 200)      # Діапазон висот (м)
    FPS = 30                # Кадрів на секунду
    PREDICTION_HORIZON = 15 # Кроків прогнозування
    BACKGROUND_COLOR = (0.95, 0.95, 0.95)
    TRAJECTORY_COLOR = (0.8, 0.8, 0.8)

class DroneSpecs:
    MAX_SPEED_XY = 10.0     # м/с
    MAX_SPEED_Z = 5.0       # м/с
    MAX_ACCEL = 3.0         # м/с²
    COMM_RANGE = 200.0      # м
    MIN_SEPARATION = 5.0    # м
    SAFETY_FACTOR = 1.2

class SafetyZones:
    WARNING = 3 * DroneSpecs.MIN_SEPARATION
    DANGER = 2 * DroneSpecs.MIN_SEPARATION
    CRITICAL = 1.5 * DroneSpecs.MIN_SEPARATION

# Стани дронів
class DroneState(Enum):
    NORMAL = auto()
    WARNING = auto()
    DANGER = auto()
    CRITICAL = auto()
    AVOIDING = auto()

STATE_COLORS = {
    DroneState.NORMAL: 'green',
    DroneState.WARNING: 'yellow',
    DroneState.DANGER: 'orange',
    DroneState.CRITICAL: 'red',
    DroneState.AVOIDING: 'blue'
}

TARGET_ZONE_COLORS = ['blue', 'cyan', 'magenta', 'purple', 'brown']

@dataclass
class PIDController:
    """Розширений PID-контролер з адаптацією"""
    Kp: float = 1.2
    Ki: float = 0.2
    Kd: float = 0.5
    target: float = 5.0
    integral: float = 0.0
    prev_error: float = 0.0
    dt: float = 1.0/BaseParams.FPS

    def compute(self, current: float) -> float:
        error = self.target - current
        self.integral += error * self.dt
        derivative = (error - self.prev_error) / self.dt
        output = self.Kp*error + self.Ki*self.integral + self.Kd*derivative
        self.prev_error = error
        return output

    def adapt_for_safety(self, risk_level: float):
        self.Kp = min(2.0, 1.0 + risk_level)
        self.Kd = min(1.0, 0.5 + risk_level*0.8)
        self.Ki = max(0.1, 0.2 - risk_level*0.15)

@dataclass
class Drone:
    id: int
    position: np.ndarray
    velocity: np.ndarray = field(default_factory=lambda: np.zeros(3))
    target: np.ndarray = None
    state: DroneState = DroneState.NORMAL
    neighbors: List['Drone'] = field(default_factory=list)
    pid: PIDController = field(default_factory=PIDController)
    avoidance_dir: float = 0.0
    risk_level: float = 0.0
    trajectory: deque = field(default_factory=lambda: deque(maxlen=100))
    comm_range: float = DroneSpecs.COMM_RANGE
    target_zone: int = None
    collision_count: int = 0

    def __post_init__(self):
        self.trajectory.append(self.position.copy())
        if self.target is None:
            self.target = self.position.copy()

    def update_state(self):
        if self.risk_level > 0.8:
            self.state = DroneState.CRITICAL
        elif self.risk_level > 0.5:
            self.state = DroneState.DANGER
        elif self.risk_level > 0.2:
            self.state = DroneState.WARNING
        else:
            self.state = DroneState.NORMAL

    def predict_trajectory(self, steps: int) -> List[np.ndarray]:
        return [self.position + self.velocity * t * self.pid.dt
               for t in range(1, steps+1)]

    def avoid_collision(self) -> np.ndarray:
        if self.state not in [DroneState.DANGER, DroneState.CRITICAL]:
            return np.zeros(3)

        if self.avoidance_dir == 0:
            self.avoidance_dir = random.choice([-1, 1])

        avoidance_force = np.zeros(3)
        for neighbor in self.neighbors:
            dist_vec = self.position - neighbor.position
            dist = np.linalg.norm(dist_vec)
            if dist < SafetyZones.CRITICAL:
                avoidance_force += dist_vec / (dist + 0.1)

        if np.linalg.norm(avoidance_force) > 0:
            avoidance_force = avoidance_force / np.linalg.norm(avoidance_force)
            avoidance_force[2] = self.avoidance_dir * 0.7

        return avoidance_force * DroneSpecs.MAX_ACCEL * 1.5

class SwarmSimulator:
    def __init__(self, num_drones=20, num_zones=3):
        self.num_drones = num_drones
        self.num_zones = num_zones
        self.drones = []
        self.zones = []
        self.collisions = 0
        self.near_misses = 0
        self.time = 0
        self.setup_visualization()
        self.init_drones()
        self.init_target_zones()

    def setup_visualization(self):
        self.fig = plt.figure(figsize=(16, 10))
        self.ax = self.fig.add_subplot(111, projection='3d')
        self.ax.set_xlim(0, BaseParams.SIM_AREA[0])
        self.ax.set_ylim(0, BaseParams.SIM_AREA[1])
        self.ax.set_zlim(*BaseParams.Z_RANGE)
        self.ax.set_xlabel('X (m)')
        self.ax.set_ylabel('Y (m)')
        self.ax.set_zlabel('Z (m)')
        self.ax.set_title('3D Drone Swarm Simulation with Collision Avoidance')

        self.drone_plots = []
        self.traj_plots = []
        self.zone_plots = []
        self.info_text = self.ax.text2D(0.02, 0.95, '', transform=self.ax.transAxes,
                                       bbox=dict(facecolor='white', alpha=0.7))

    def init_drones(self):
        center = np.array(BaseParams.SIM_AREA)/2
        for i in range(self.num_drones):
            angle = random.uniform(0, 2*math.pi)
            radius = random.uniform(100, 300)
            pos = center + radius * np.array([math.cos(angle), math.sin(angle), 0])
            pos[2] = random.uniform(*BaseParams.Z_RANGE)

            vel = np.array([-math.sin(angle), math.cos(angle), 0]) * random.uniform(3, 8)

            drone = Drone(i, pos.copy(), vel.copy())
            self.drones.append(drone)

            self.drone_plots.append(
                self.ax.plot([], [], [], 'o', markersize=6, alpha=0.8)[0])
            self.traj_plots.append(
                self.ax.plot([], [], [], '-', linewidth=1, alpha=0.4)[0])

    def init_target_zones(self):
        for i in range(self.num_zones):
            zone_pos = np.array([
                random.uniform(200, BaseParams.SIM_AREA[0]-200),
                random.uniform(200, BaseParams.SIM_AREA[1]-200),
                random.uniform(*BaseParams.Z_RANGE)
            ])
            self.zones.append(zone_pos)

            color = TARGET_ZONE_COLORS[i % len(TARGET_ZONE_COLORS)]
            self.zone_plots.extend([
                self.ax.plot([zone_pos[0]-50, zone_pos[0]+50],
                            [zone_pos[1], zone_pos[1]],
                            [zone_pos[2], zone_pos[2]], ':', color=color)[0],
                self.ax.plot([zone_pos[0], zone_pos[0]],
                            [zone_pos[1]-50, zone_pos[1]+50],
                            [zone_pos[2], zone_pos[2]], ':', color=color)[0]
            ])

            # Призначити дронам цільові зони
            drones_per_zone = self.num_drones // self.num_zones
            for j in range(i*drones_per_zone, (i+1)*drones_per_zone):
                if j < len(self.drones):
                    self.drones[j].target = zone_pos.copy()
                    self.drones[j].target_zone = i

    def find_neighbors(self, drone):
        positions = np.array([d.position for d in self.drones if d.id != drone.id])
        tree = KDTree(positions)
        idx = tree.query_ball_point(drone.position, drone.comm_range)
        return [self.drones[i] for i in idx if i < len(self.drones)]

    def calculate_risk(self, drone):
        if not drone.neighbors:
            return 0.0

        min_dist = min(np.linalg.norm(drone.position - n.position)
                      for n in drone.neighbors)

        if min_dist < SafetyZones.CRITICAL:
            return 1.0
        elif min_dist < SafetyZones.DANGER:
            return 0.7
        elif min_dist < SafetyZones.WARNING:
            return 0.3
        return 0.0

    def apply_boids_rules(self, drone):
        separation = np.zeros(3)
        alignment = np.zeros(3)
        cohesion = np.zeros(3)
        center = np.zeros(3)

        for neighbor in drone.neighbors:
            diff = drone.position - neighbor.position
            dist = np.linalg.norm(diff)

            if dist < SafetyZones.WARNING:
                separation += diff / (dist + 0.1)

            alignment += neighbor.velocity
            center += neighbor.position

        if len(drone.neighbors) > 0:
            alignment = (alignment/len(drone.neighbors) - drone.velocity) * 0.5
            center = center / len(drone.neighbors)
            cohesion = (center - drone.position) * 0.05

        if np.linalg.norm(separation) > 0:
            separation = (separation / np.linalg.norm(separation)) * DroneSpecs.MAX_SPEED_XY

        return separation * 1.5 + alignment * 1.0 + cohesion * 1.0

    def navigate_to_target(self, drone):
        target_vec = drone.target - drone.position
        dist = np.linalg.norm(target_vec)

        if dist > 10:
            target_dir = target_vec / (dist + 0.1)
            return target_dir * min(DroneSpecs.MAX_SPEED_XY, dist * 0.3)
        return np.zeros(3)

    def update_drone(self, drone):
        drone.neighbors = self.find_neighbors(drone)
        drone.risk_level = self.calculate_risk(drone)
        drone.update_state()
        drone.pid.adapt_for_safety(drone.risk_level)

        # Комбінуємо всі сили
        boids_force = self.apply_boids_rules(drone)
        nav_force = self.navigate_to_target(drone)
        avoid_force = drone.avoid_collision()

        if np.linalg.norm(avoid_force) > 0:
            total_force = avoid_force * 2.0 + boids_force * 0.5 + nav_force * 0.3
        else:
            total_force = boids_force * 1.0 + nav_force * 1.0

        # Обмеження прискорення
        accel_norm = np.linalg.norm(total_force)
        if accel_norm > DroneSpecs.MAX_ACCEL:
            total_force = total_force / accel_norm * DroneSpecs.MAX_ACCEL

        # Оновлення швидкості та позиції
        drone.velocity += total_force * (1.0/BaseParams.FPS)

        # Обмеження швидкості
        speed_xy = np.linalg.norm(drone.velocity[:2])
        if speed_xy > DroneSpecs.MAX_SPEED_XY:
            drone.velocity[:2] = drone.velocity[:2] / speed_xy * DroneSpecs.MAX_SPEED_XY

        speed_z = abs(drone.velocity[2])
        if speed_z > DroneSpecs.MAX_SPEED_Z:
            drone.velocity[2] = drone.velocity[2] / speed_z * DroneSpecs.MAX_SPEED_Z

        drone.position += drone.velocity * (1.0/BaseParams.FPS)
        self.apply_boundary_conditions(drone)
        drone.trajectory.append(drone.position.copy())
        self.check_collisions(drone)

    def apply_boundary_conditions(self, drone):
        for i in range(3):
            if i < 2:  # X, Y
                if drone.position[i] < 0:
                    drone.position[i] = 0
                    drone.velocity[i] *= -0.5
                elif drone.position[i] > BaseParams.SIM_AREA[i]:
                    drone.position[i] = BaseParams.SIM_AREA[i]
                    drone.velocity[i] *= -0.5
            else:  # Z
                if drone.position[i] < BaseParams.Z_RANGE[0]:
                    drone.position[i] = BaseParams.Z_RANGE[0]
                    drone.velocity[i] = 0
                elif drone.position[i] > BaseParams.Z_RANGE[1]:
                    drone.position[i] = BaseParams.Z_RANGE[1]
                    drone.velocity[i] = 0

    def check_collisions(self, drone):
        for neighbor in drone.neighbors:
            dist = np.linalg.norm(drone.position - neighbor.position)
            if dist < DroneSpecs.MIN_SEPARATION * DroneSpecs.SAFETY_FACTOR:
                self.near_misses += 1
                drone.collision_count += 1

            if dist < DroneSpecs.MIN_SEPARATION:
                self.collisions += 1
                direction = (drone.position - neighbor.position)
                if np.linalg.norm(direction) > 0:
                    direction = direction / np.linalg.norm(direction)
                drone.velocity += direction * DroneSpecs.MAX_ACCEL * 0.8

    def update_visualization(self):
        for i, drone in enumerate(self.drones):
            # Оновлення позиції дрона
            self.drone_plots[i].set_data([drone.position[0]], [drone.position[1]])
            self.drone_plots[i].set_3d_properties([drone.position[2]])
            self.drone_plots[i].set_color(STATE_COLORS[drone.state])
            self.drone_plots[i].set_alpha(0.8)

            # Оновлення траєкторії
            if len(drone.trajectory) > 1:
                traj = np.array(drone.trajectory)
                self.traj_plots[i].set_data(traj[:,0], traj[:,1])
                self.traj_plots[i].set_3d_properties(traj[:,2])
                self.traj_plots[i].set_color(STATE_COLORS[drone.state])

        # Оновлення інформаційного тексту
        info = (
            f"Час: {self.time:.1f} с\n"
            f"Дронів: {self.num_drones}\n"
            f"Зіткнень: {self.collisions}\n"
            f"Майже зіткнень: {self.near_misses}\n"
            f"Середній рівень ризику: {sum(d.risk_level for d in self.drones)/len(self.drones):.2f}"
        )
        self.info_text.set_text(info)

    def update(self, frame):
        self.time += 1.0/BaseParams.FPS

        for drone in self.drones:
            self.update_drone(drone)

        self.update_visualization()

        return self.drone_plots + self.traj_plots + self.zone_plots + [self.info_text]

    def run_simulation(self):
        anim = animation.FuncAnimation(
            self.fig, self.update,
            frames=500, interval=1000/BaseParams.FPS, blit=True
        )
        plt.close()
        return HTML(anim.to_jshtml())

# Запуск симуляції
simulator = SwarmSimulator(num_drones=30, num_zones=3)
simulator.run_simulation()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML
import random
import math
from dataclasses import dataclass, field
from enum import Enum, auto
from typing import List, Dict, Tuple, Optional
from collections import deque
from scipy.spatial import KDTree
from scipy.optimize import minimize
import cvxpy as cp

# Константи з технічного завдання
class BaseParams:
    SIM_AREA = (1000, 800)  # Ширина, висота симуляційного простору (м)
    Z_RANGE = (0, 200)      # Діапазон висот (м)
    FPS = 30                # Кадрів на секунду
    PREDICTION_HORIZON = 15 # Кроків прогнозування
    BACKGROUND_COLOR = (0.95, 0.95, 0.95)
    TRAJECTORY_COLOR = (0.8, 0.8, 0.8)

class DroneSpecs:
    MAX_SPEED_XY = 10.0     # м/с
    MAX_SPEED_Z = 5.0       # м/с
    MAX_ACCEL = 3.0         # м/с²
    COMM_RANGE = 200.0      # м
    MIN_SEPARATION = 5.0    # м
    SAFETY_FACTOR = 1.2

class SafetyZones:
    WARNING = 3 * DroneSpecs.MIN_SEPARATION
    DANGER = 2 * DroneSpecs.MIN_SEPARATION
    CRITICAL = 1.5 * DroneSpecs.MIN_SEPARATION

# Стани дронів
class DroneState(Enum):
    NORMAL = auto()
    WARNING = auto()
    DANGER = auto()
    CRITICAL = auto()
    AVOIDING = auto()

STATE_COLORS = {
    DroneState.NORMAL: 'green',
    DroneState.WARNING: 'yellow',
    DroneState.DANGER: 'orange',
    DroneState.CRITICAL: 'red',
    DroneState.AVOIDING: 'blue'
}

TARGET_ZONE_COLORS = ['blue', 'cyan', 'magenta', 'purple', 'brown']

@dataclass
class PIDController:
    """Розширений PID-контролер з адаптацією"""
    Kp: float = 1.2
    Ki: float = 0.2
    Kd: float = 0.5
    target: float = 5.0
    integral: float = 0.0
    prev_error: float = 0.0
    dt: float = 1.0/BaseParams.FPS

    def compute(self, current: float) -> float:
        error = self.target - current
        self.integral += error * self.dt
        derivative = (error - self.prev_error) / self.dt
        output = self.Kp*error + self.Ki*self.integral + self.Kd*derivative
        self.prev_error = error
        return output

    def adapt_for_safety(self, risk_level: float):
        self.Kp = min(2.0, 1.0 + risk_level)
        self.Kd = min(1.0, 0.5 + risk_level*0.8)
        self.Ki = max(0.1, 0.2 - risk_level*0.15)

@dataclass
class Drone:
    id: int
    position: np.ndarray
    velocity: np.ndarray = field(default_factory=lambda: np.zeros(3))
    target: np.ndarray = None
    state: DroneState = DroneState.NORMAL
    neighbors: List['Drone'] = field(default_factory=list)
    pid: PIDController = field(default_factory=PIDController)
    avoidance_dir: float = 0.0
    risk_level: float = 0.0
    trajectory: deque = field(default_factory=lambda: deque(maxlen=100))
    comm_range: float = DroneSpecs.COMM_RANGE
    target_zone: int = None
    collision_count: int = 0

    def __post_init__(self):
        self.trajectory.append(self.position.copy())
        if self.target is None:
            self.target = self.position.copy()

    def update_state(self):
        if self.risk_level > 0.8:
            self.state = DroneState.CRITICAL
        elif self.risk_level > 0.5:
            self.state = DroneState.DANGER
        elif self.risk_level > 0.2:
            self.state = DroneState.WARNING
        else:
            self.state = DroneState.NORMAL

    def predict_trajectory(self, steps: int) -> List[np.ndarray]:
        return [self.position + self.velocity * t * self.pid.dt
                for t in range(1, steps+1)]

    def avoid_collision(self) -> np.ndarray:
        if self.state not in [DroneState.DANGER, DroneState.CRITICAL]:
            return np.zeros(3)

        if self.avoidance_dir == 0:
            self.avoidance_dir = random.choice([-1, 1])

        avoidance_force = np.zeros(3)
        for neighbor in self.neighbors:
            dist_vec = self.position - neighbor.position
            dist = np.linalg.norm(dist_vec)
            if dist < SafetyZones.CRITICAL:
                avoidance_force += dist_vec / (dist + 0.1)

        if np.linalg.norm(avoidance_force) > 0:
            avoidance_force = avoidance_force / np.linalg.norm(avoidance_force)
            avoidance_force[2] = self.avoidance_dir * 0.7

        return avoidance_force * DroneSpecs.MAX_ACCEL * 1.5

class SwarmSimulator:
    def __init__(self, num_drones=20, num_zones=3):
        self.num_drones = num_drones
        self.num_zones = num_zones
        self.drones = []
        self.zones = []
        self.collisions = 0
        self.near_misses = 0
        self.time = 0
        self.setup_visualization()
        self.init_drones()
        self.init_target_zones()

    def setup_visualization(self):
        self.fig = plt.figure(figsize=(16, 10))
        self.ax = self.fig.add_subplot(111, projection='3d')
        self.ax.set_xlim(0, BaseParams.SIM_AREA[0])
        self.ax.set_ylim(0, BaseParams.SIM_AREA[1])
        self.ax.set_zlim(*BaseParams.Z_RANGE)
        self.ax.set_xlabel('X (m)')
        self.ax.set_ylabel('Y (m)')
        self.ax.set_zlabel('Z (m)')
        self.ax.set_title('3D Drone Swarm Simulation with Collision Avoidance')

        self.drone_plots = []
        self.traj_plots = []
        self.zone_plots = []
        self.info_text = self.ax.text2D(0.02, 0.95, '', transform=self.ax.transAxes,
                                       bbox=dict(facecolor='white', alpha=0.7))

    def init_drones(self):
        # Визначаємо центр як тривимірний масив
        center = np.array([
            BaseParams.SIM_AREA[0] / 2,
            BaseParams.SIM_AREA[1] / 2,
            (BaseParams.Z_RANGE[0] + BaseParams.Z_RANGE[1]) / 2
        ])
        for i in range(self.num_drones):
            angle = random.uniform(0, 2 * math.pi)
            radius = random.uniform(100, 300)
            # Генеруємо позицію в площині XY навколо центру
            offset = radius * np.array([math.cos(angle), math.sin(angle), 0])
            pos = center + offset
            # Встановлюємо z-координату в межах Z_RANGE
            pos[2] = random.uniform(*BaseParams.Z_RANGE)

            # Генеруємо початкову швидкість
            vel = np.array([-math.sin(angle), math.cos(angle), 0]) * random.uniform(3, 8)

            drone = Drone(i, pos.copy(), vel.copy())
            self.drones.append(drone)

            self.drone_plots.append(
                self.ax.plot([], [], [], 'o', markersize=6, alpha=0.8)[0])
            self.traj_plots.append(
                self.ax.plot([], [], [], '-', linewidth=1, alpha=0.4)[0])

    def init_target_zones(self):
        for i in range(self.num_zones):
            zone_pos = np.array([
                random.uniform(200, BaseParams.SIM_AREA[0]-200),
                random.uniform(200, BaseParams.SIM_AREA[1]-200),
                random.uniform(*BaseParams.Z_RANGE)
            ])
            self.zones.append(zone_pos)

            color = TARGET_ZONE_COLORS[i % len(TARGET_ZONE_COLORS)]
            self.zone_plots.extend([
                self.ax.plot([zone_pos[0]-50, zone_pos[0]+50],
                            [zone_pos[1], zone_pos[1]],
                            [zone_pos[2], zone_pos[2]], ':', color=color)[0],
                self.ax.plot([zone_pos[0], zone_pos[0]],
                            [zone_pos[1]-50, zone_pos[1]+50],
                            [zone_pos[2], zone_pos[2]], ':', color=color)[0]
            ])

            # Призначити дронам цільові зони
            drones_per_zone = self.num_drones // self.num_zones
            for j in range(i*drones_per_zone, (i+1)*drones_per_zone):
                if j < len(self.drones):
                    self.drones[j].target = zone_pos.copy()
                    self.drones[j].target_zone = i

    def find_neighbors(self, drone):
        positions = np.array([d.position for d in self.drones if d.id != drone.id])
        tree = KDTree(positions)
        idx = tree.query_ball_point(drone.position, drone.comm_range)
        return [self.drones[i] for i in idx if i < len(self.drones)]

    def calculate_risk(self, drone):
        if not drone.neighbors:
            return 0.0

        min_dist = min(np.linalg.norm(drone.position - n.position)
                      for n in drone.neighbors)

        if min_dist < SafetyZones.CRITICAL:
            return 1.0
        elif min_dist < SafetyZones.DANGER:
            return 0.7
        elif min_dist < SafetyZones.WARNING:
            return 0.3
        return 0.0

    def apply_boids_rules(self, drone):
        separation = np.zeros(3)
        alignment = np.zeros(3)
        cohesion = np.zeros(3)
        center = np.zeros(3)

        for neighbor in drone.neighbors:
            diff = drone.position - neighbor.position
            dist = np.linalg.norm(diff)

            if dist < SafetyZones.WARNING:
                separation += diff / (dist + 0.1)

            alignment += neighbor.velocity
            center += neighbor.position

        if len(drone.neighbors) > 0:
            alignment = (alignment/len(drone.neighbors) - drone.velocity) * 0.5
            center = center / len(drone.neighbors)
            cohesion = (center - drone.position) * 0.05

        if np.linalg.norm(separation) > 0:
            separation = (separation / np.linalg.norm(separation)) * DroneSpecs.MAX_SPEED_XY

        return separation * 1.5 + alignment * 1.0 + cohesion * 1.0

    def navigate_to_target(self, drone):
        target_vec = drone.target - drone.position
        dist = np.linalg.norm(target_vec)

        if dist > 10:
            target_dir = target_vec / (dist + 0.1)
            return target_dir * min(DroneSpecs.MAX_SPEED_XY, dist * 0.3)
        return np.zeros(3)

    def update_drone(self, drone):
        drone.neighbors = self.find_neighbors(drone)
        drone.risk_level = self.calculate_risk(drone)
        drone.update_state()
        drone.pid.adapt_for_safety(drone.risk_level)

        # Комбінуємо всі сили
        boids_force = self.apply_boids_rules(drone)
        nav_force = self.navigate_to_target(drone)
        avoid_force = drone.avoid_collision()

        if np.linalg.norm(avoid_force) > 0:
            total_force = avoid_force * 2.0 + boids_force * 0.5 + nav_force * 0.3
        else:
            total_force = boids_force * 1.0 + nav_force * 1.0

        # Обмеження прискорення
        accel_norm = np.linalg.norm(total_force)
        if accel_norm > DroneSpecs.MAX_ACCEL:
            total_force = total_force / accel_norm * DroneSpecs.MAX_ACCEL

        # Оновлення швидкості та позиції
        drone.velocity += total_force * (1.0/BaseParams.FPS)

        # Обмеження швидкості
        speed_xy = np.linalg.norm(drone.velocity[:2])
        if speed_xy > DroneSpecs.MAX_SPEED_XY:
            drone.velocity[:2] = drone.velocity[:2] / speed_xy * DroneSpecs.MAX_SPEED_XY

        speed_z = abs(drone.velocity[2])
        if speed_z > DroneSpecs.MAX_SPEED_Z:
            drone.velocity[2] = drone.velocity[2] / speed_z * DroneSpecs.MAX_SPEED_Z

        drone.position += drone.velocity * (1.0/BaseParams.FPS)
        self.apply_boundary_conditions(drone)
        drone.trajectory.append(drone.position.copy())
        self.check_collisions(drone)

    def apply_boundary_conditions(self, drone):
        for i in range(3):
            if i < 2:  # X, Y
                if drone.position[i] < 0:
                    drone.position[i] = 0
                    drone.velocity[i] *= -0.5
                elif drone.position[i] > BaseParams.SIM_AREA[i]:
                    drone.position[i] = BaseParams.SIM_AREA[i]
                    drone.velocity[i] *= -0.5
            else:  # Z
                if drone.position[i] < BaseParams.Z_RANGE[0]:
                    drone.position[i] = BaseParams.Z_RANGE[0]
                    drone.velocity[i] = 0
                elif drone.position[i] > BaseParams.Z_RANGE[1]:
                    drone.position[i] = BaseParams.Z_RANGE[1]
                    drone.velocity[i] = 0

    def check_collisions(self, drone):
        for neighbor in drone.neighbors:
            dist = np.linalg.norm(drone.position - neighbor.position)
            if dist < DroneSpecs.MIN_SEPARATION * DroneSpecs.SAFETY_FACTOR:
                self.near_misses += 1
                drone.collision_count += 1

            if dist < DroneSpecs.MIN_SEPARATION:
                self.collisions += 1
                direction = (drone.position - neighbor.position)
                if np.linalg.norm(direction) > 0:
                    direction = direction / np.linalg.norm(direction)
                drone.velocity += direction * DroneSpecs.MAX_ACCEL * 0.8

    def update_visualization(self):
        for i, drone in enumerate(self.drones):
            # Оновлення позиції дрона
            self.drone_plots[i].set_data([drone.position[0]], [drone.position[1]])
            self.drone_plots[i].set_3d_properties([drone.position[2]])
            self.drone_plots[i].set_color(STATE_COLORS[drone.state])
            self.drone_plots[i].set_alpha(0.8)

            # Оновлення траєкторії
            if len(drone.trajectory) > 1:
                traj = np.array(drone.trajectory)
                self.traj_plots[i].set_data(traj[:,0], traj[:,1])
                self.traj_plots[i].set_3d_properties(traj[:,2])
                self.traj_plots[i].set_color(STATE_COLORS[drone.state])

        # Оновлення інформаційного тексту
        info = (
            f"Час: {self.time:.1f} с\n"
            f"Дронів: {self.num_drones}\n"
            f"Зіткнень: {self.collisions}\n"
            f"Майже зіткнень: {self.near_misses}\n"
            f"Середній рівень ризику: {sum(d.risk_level for d in self.drones)/len(self.drones):.2f}"
        )
        self.info_text.set_text(info)

    def update(self, frame):
        self.time += 1.0/BaseParams.FPS

        for drone in self.drones:
            self.update_drone(drone)

        self.update_visualization()

        return self.drone_plots + self.traj_plots + self.zone_plots + [self.info_text]

    def run_simulation(self):
        anim = animation.FuncAnimation(
            self.fig, self.update,
            frames=500, interval=1000/BaseParams.FPS, blit=True
        )
        plt.close()
        return HTML(anim.to_jshtml())

# Запуск симуляції
simulator = SwarmSimulator(num_drones=3, num_zones=1)
simulator.run_simulation()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML
import random
import math
from dataclasses import dataclass, field
from enum import Enum, auto
from typing import List
from collections import deque
from scipy.spatial import KDTree

import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 1000

# Константи
class BaseParams:
    SIM_AREA = (1000, 800, 200)  # Тепер це кортеж з трьох значень (width, height, depth)
    FPS = 30
    PREDICTION_HORIZON = 15

class DroneSpecs:
    MAX_SPEED_XY = 10.0
    MAX_SPEED_Z = 5.0
    MAX_ACCEL = 3.0
    COMM_RANGE = 200.0
    MIN_SEPARATION = 5.0
    SAFETY_FACTOR = 1.2

class SafetyZones:
    WARNING = 3 * DroneSpecs.MIN_SEPARATION
    DANGER = 2 * DroneSpecs.MIN_SEPARATION
    CRITICAL = 1.5 * DroneSpecs.MIN_SEPARATION

# Стани дронів
class DroneState(Enum):
    NORMAL = auto()
    WARNING = auto()
    DANGER = auto()
    CRITICAL = auto()
    AVOIDING = auto()

STATE_COLORS = {
    DroneState.NORMAL: 'green',
    DroneState.WARNING: 'yellow',
    DroneState.DANGER: 'orange',
    DroneState.CRITICAL: 'red',
    DroneState.AVOIDING: 'blue'
}

TARGET_ZONE_COLORS = ['blue', 'cyan', 'magenta', 'purple', 'brown']

@dataclass
class Drone:
    id: int
    position: np.ndarray
    velocity: np.ndarray = field(default_factory=lambda: np.zeros(3))
    target: np.ndarray = None
    state: DroneState = DroneState.NORMAL
    neighbors: List['Drone'] = field(default_factory=list)
    avoidance_dir: float = 0.0
    risk_level: float = 0.0
    trajectory: deque = field(default_factory=lambda: deque(maxlen=100))
    comm_range: float = DroneSpecs.COMM_RANGE
    target_zone: int = None
    collision_count: int = 0

    def __post_init__(self):
        self.trajectory.append(self.position.copy())
        if self.target is None:
            self.target = self.position.copy()

    def update_state(self):
        if self.risk_level > 0.8:
            self.state = DroneState.CRITICAL
        elif self.risk_level > 0.5:
            self.state = DroneState.DANGER
        elif self.risk_level > 0.2:
            self.state = DroneState.WARNING
        else:
            self.state = DroneState.NORMAL

    def predict_trajectory(self, steps: int) -> List[np.ndarray]:
        return [self.position + self.velocity * t * (1.0/BaseParams.FPS)
               for t in range(1, steps+1)]

    def avoid_collision(self) -> np.ndarray:
        if self.state not in [DroneState.DANGER, DroneState.CRITICAL]:
            return np.zeros(3)

        if self.avoidance_dir == 0:
            self.avoidance_dir = random.choice([-1, 1])

        avoidance_force = np.zeros(3)
        for neighbor in self.neighbors:
            dist_vec = self.position - neighbor.position
            dist = np.linalg.norm(dist_vec)
            if dist < SafetyZones.CRITICAL:
                avoidance_force += dist_vec / (dist + 0.1)

        if np.linalg.norm(avoidance_force) > 0:
            avoidance_force = avoidance_force / np.linalg.norm(avoidance_force)
            avoidance_force[2] = self.avoidance_dir * 0.7

        return avoidance_force * DroneSpecs.MAX_ACCEL * 1.5

class SwarmSimulator:
    def __init__(self, num_drones=20, num_zones=3):
        self.num_drones = num_drones
        self.num_zones = num_zones
        self.drones = []
        self.zones = []
        self.collisions = 0
        self.near_misses = 0
        self.time = 0
        self.setup_visualization()
        self.init_drones()
        self.init_target_zones()

    def setup_visualization(self):
        self.fig = plt.figure(figsize=(16, 10))
        self.ax = self.fig.add_subplot(111, projection='3d')
        self.ax.set_xlim(0, BaseParams.SIM_AREA[0])
        self.ax.set_ylim(0, BaseParams.SIM_AREA[1])
        self.ax.set_zlim(0, BaseParams.SIM_AREA[2])
        self.ax.set_xlabel('X (m)')
        self.ax.set_ylabel('Y (m)')
        self.ax.set_zlabel('Z (m)')
        self.ax.set_title('3D Drone Swarm Simulation with Collision Avoidance')

        self.drone_plots = []
        self.traj_plots = []
        self.zone_plots = []
        self.info_text = self.ax.text2D(0.02, 0.95, '', transform=self.ax.transAxes,
                                      bbox=dict(facecolor='white', alpha=0.7))

    def init_drones(self):
        center = np.array([BaseParams.SIM_AREA[0]/2, BaseParams.SIM_AREA[1]/2, BaseParams.SIM_AREA[2]/2])
        for i in range(self.num_drones):
            angle = random.uniform(0, 2*math.pi)
            radius = random.uniform(100, 300)
            # Генеруємо позицію у 3D просторі
            pos = center + np.array([
                radius * math.cos(angle),
                radius * math.sin(angle),
                random.uniform(0, BaseParams.SIM_AREA[2])
            ])

            vel = np.array([
                -math.sin(angle) * random.uniform(3, 8),
                math.cos(angle) * random.uniform(3, 8),
                0  # Початкова швидкість по Z
            ])

            drone = Drone(i, pos.copy(), vel.copy())
            self.drones.append(drone)

            self.drone_plots.append(
                self.ax.plot([], [], [], 'o', markersize=6, alpha=0.8)[0])
            self.traj_plots.append(
                self.ax.plot([], [], [], '-', linewidth=1, alpha=0.4)[0])

    def init_target_zones(self):
        for i in range(self.num_zones):
            zone_pos = np.array([
                random.uniform(200, BaseParams.SIM_AREA[0]-200),
                random.uniform(200, BaseParams.SIM_AREA[1]-200),
                random.uniform(50, BaseParams.SIM_AREA[2]-50)
            ])
            self.zones.append(zone_pos)

            color = TARGET_ZONE_COLORS[i % len(TARGET_ZONE_COLORS)]
            self.zone_plots.extend([
                self.ax.plot([zone_pos[0]-50, zone_pos[0]+50],
                            [zone_pos[1], zone_pos[1]],
                            [zone_pos[2], zone_pos[2]], ':', color=color)[0],
                self.ax.plot([zone_pos[0], zone_pos[0]],
                            [zone_pos[1]-50, zone_pos[1]+50],
                            [zone_pos[2], zone_pos[2]], ':', color=color)[0],
                self.ax.plot([zone_pos[0], zone_pos[0]],
                            [zone_pos[1], zone_pos[1]],
                            [zone_pos[2]-50, zone_pos[2]+50], ':', color=color)[0]
            ])

            # Призначити дронам цільові зони
            drones_per_zone = self.num_drones // self.num_zones
            for j in range(i*drones_per_zone, (i+1)*drones_per_zone):
                if j < len(self.drones):
                    self.drones[j].target = zone_pos.copy()
                    self.drones[j].target_zone = i

    def find_neighbors(self, drone):
        positions = np.array([d.position for d in self.drones if d.id != drone.id])
        tree = KDTree(positions)
        idx = tree.query_ball_point(drone.position, drone.comm_range)
        return [self.drones[i] for i in idx if i < len(self.drones)]

    def calculate_risk(self, drone):
        if not drone.neighbors:
            return 0.0

        min_dist = min(np.linalg.norm(drone.position - n.position)
                     for n in drone.neighbors)

        if min_dist < SafetyZones.CRITICAL:
            return 1.0
        elif min_dist < SafetyZones.DANGER:
            return 0.7
        elif min_dist < SafetyZones.WARNING:
            return 0.3
        return 0.0

    def apply_boids_rules(self, drone):
        separation = np.zeros(3)
        alignment = np.zeros(3)
        cohesion = np.zeros(3)
        center = np.zeros(3)

        for neighbor in drone.neighbors:
            diff = drone.position - neighbor.position
            dist = np.linalg.norm(diff)

            if dist < SafetyZones.WARNING:
                separation += diff / (dist + 0.1)

            alignment += neighbor.velocity
            center += neighbor.position

        if len(drone.neighbors) > 0:
            alignment = (alignment/len(drone.neighbors) - drone.velocity) * 0.5
            center = center / len(drone.neighbors)
            cohesion = (center - drone.position) * 0.05

        if np.linalg.norm(separation) > 0:
            separation = (separation / np.linalg.norm(separation)) * DroneSpecs.MAX_SPEED_XY

        return separation * 1.5 + alignment * 1.0 + cohesion * 1.0

    def navigate_to_target(self, drone):
        target_vec = drone.target - drone.position
        dist = np.linalg.norm(target_vec)

        if dist > 10:
            target_dir = target_vec / (dist + 0.1)
            return target_dir * min(DroneSpecs.MAX_SPEED_XY, dist * 0.3)
        return np.zeros(3)

    def update_drone(self, drone):
        drone.neighbors = self.find_neighbors(drone)
        drone.risk_level = self.calculate_risk(drone)
        drone.update_state()

        # Комбінуємо всі сили
        boids_force = self.apply_boids_rules(drone)
        nav_force = self.navigate_to_target(drone)
        avoid_force = drone.avoid_collision()

        if np.linalg.norm(avoid_force) > 0:
            total_force = avoid_force * 2.0 + boids_force * 0.5 + nav_force * 0.3
        else:
            total_force = boids_force * 1.0 + nav_force * 1.0

        # Обмеження прискорення
        accel_norm = np.linalg.norm(total_force)
        if accel_norm > DroneSpecs.MAX_ACCEL:
            total_force = total_force / accel_norm * DroneSpecs.MAX_ACCEL

        # Оновлення швидкості та позиції
        drone.velocity += total_force * (1.0/BaseParams.FPS)

        # Обмеження швидкості
        speed_xy = np.linalg.norm(drone.velocity[:2])
        if speed_xy > DroneSpecs.MAX_SPEED_XY:
            drone.velocity[:2] = drone.velocity[:2] / speed_xy * DroneSpecs.MAX_SPEED_XY

        speed_z = abs(drone.velocity[2])
        if speed_z > DroneSpecs.MAX_SPEED_Z:
            drone.velocity[2] = drone.velocity[2] / speed_z * DroneSpecs.MAX_SPEED_Z

        drone.position += drone.velocity * (1.0/BaseParams.FPS)
        self.apply_boundary_conditions(drone)
        drone.trajectory.append(drone.position.copy())
        self.check_collisions(drone)

    def apply_boundary_conditions(self, drone):
        for i in range(3):
            if drone.position[i] < 0:
                drone.position[i] = 0
                drone.velocity[i] *= -0.5
            elif drone.position[i] > BaseParams.SIM_AREA[i]:
                drone.position[i] = BaseParams.SIM_AREA[i]
                drone.velocity[i] *= -0.5

    def check_collisions(self, drone):
        for neighbor in drone.neighbors:
            dist = np.linalg.norm(drone.position - neighbor.position)
            if dist < DroneSpecs.MIN_SEPARATION * DroneSpecs.SAFETY_FACTOR:
                self.near_misses += 1
                drone.collision_count += 1

            if dist < DroneSpecs.MIN_SEPARATION:
                self.collisions += 1
                direction = (drone.position - neighbor.position)
                if np.linalg.norm(direction) > 0:
                    direction = direction / np.linalg.norm(direction)
                drone.velocity += direction * DroneSpecs.MAX_ACCEL * 0.8

    def update_visualization(self):
        for i, drone in enumerate(self.drones):
            # Оновлення позиції дрона
            self.drone_plots[i].set_data([drone.position[0]], [drone.position[1]])
            self.drone_plots[i].set_3d_properties([drone.position[2]])
            self.drone_plots[i].set_color(STATE_COLORS[drone.state])
            self.drone_plots[i].set_alpha(0.8)

            # Оновлення траєкторії
            if len(drone.trajectory) > 1:
                traj = np.array(drone.trajectory)
                self.traj_plots[i].set_data(traj[:,0], traj[:,1])
                self.traj_plots[i].set_3d_properties(traj[:,2])
                self.traj_plots[i].set_color(STATE_COLORS[drone.state])

        # Оновлення інформаційного тексту
        info = (
            f"Час: {self.time:.1f} с\n"
            f"Дронів: {self.num_drones}\n"
            f"Зіткнень: {self.collisions}\n"
            f"Майже зіткнень: {self.near_misses}\n"
            f"Середній рівень ризику: {sum(d.risk_level for d in self.drones)/len(self.drones):.2f}"
        )
        self.info_text.set_text(info)

    def update(self, frame):
        self.time += 1.0/BaseParams.FPS

        for drone in self.drones:
            self.update_drone(drone)

        self.update_visualization()

        return self.drone_plots + self.traj_plots + self.zone_plots + [self.info_text]

    def run_simulation(self):
        anim = animation.FuncAnimation(
            self.fig, self.update,
            frames=1000, interval=1000/BaseParams.FPS, blit=True
        )
        plt.close()
        return HTML(anim.to_jshtml())

# Запуск симуляції - cbr_1.py
simulator = SwarmSimulator(num_drones=3, num_zones=3)
simulator.run_simulation()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation
from IPython.display import HTML
import random
import math
from dataclasses import dataclass, field
from enum import Enum, auto
from typing import List, Dict, Tuple
from collections import deque
from scipy.spatial import KDTree
from scipy.optimize import minimize
import json
import pandas as pd
from ipywidgets import interact, FloatSlider, IntSlider

import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 1000

# Константи та параметри симуляції
class SimulationParams:
    # Просторові параметри
    AREA_SIZE = (1000, 800, 200)  # Розміри простору (ширина, довжина, висота)
    OBSTACLE_COLOR = (0.5, 0.5, 0.5, 0.5)  # Колір перешкод (RGBA)

    # Параметри агентів
    AGENT_RADIUS = 2.5  # Фізичний радіус агента (м)
    SAFETY_DISTANCE = 5.0  # Мінімальна безпечна відстань (м)
    SAFETY_FACTOR = 1.2  # Коефіцієнт безпеки для виявлення майже зіткнень
    MAX_SPEED_XY = 10.0  # Максимальна швидкість у горизонтальній площині (м/с)
    MAX_SPEED_Z = 5.0  # Максимальна швидкість по вертикалі (м/с)
    MAX_ACCEL = 3.0  # Максимальне прискорення (м/с²)
    COMM_RANGE = 150.0  # Дальність зв'язку між агентами (м)

    # Параметри симуляції
    FPS = 30  # Кадрів на секунду
    PREDICTION_HORIZON = 10  # Горизонт прогнозування (кроків)

    # Визначення зон безпеки
    WARNING_ZONE = 3 * SAFETY_DISTANCE  # Жовта зона попередження
    DANGER_ZONE = 2 * SAFETY_DISTANCE  # Оранжева зона небезпеки
    CRITICAL_ZONE = 1.5 * SAFETY_DISTANCE  # Червона критична зона

# Стани агентів
class AgentState(Enum):
    NORMAL = auto()      # Звичайний режим
    WARNING = auto()     # Попередження про можливе зіткнення
    DANGER = auto()      # Небезпечна ситуація
    CRITICAL = auto()    # Критична ситуація
    AVOIDING = auto()    # Виконує маневр уникнення

# Кольори для візуалізації станів
STATE_COLORS = {
    AgentState.NORMAL: 'green',
    AgentState.WARNING: 'yellow',
    AgentState.DANGER: 'orange',
    AgentState.CRITICAL: 'red',
    AgentState.AVOIDING: 'blue'
}

# Кольори для цільових зон
TARGET_ZONE_COLORS = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']

@dataclass
class Agent:
    """Клас, що представляє окремого агента у зграї"""
    id: int
    position: np.ndarray  # Поточна позиція [x, y, z]
    velocity: np.ndarray = field(default_factory=lambda: np.zeros(3))  # Поточна швидкість
    target: np.ndarray = None  # Цільова позиція
    state: AgentState = AgentState.NORMAL  # Поточний стан
    neighbors: List['Agent'] = field(default_factory=list)  # Сусідні агенти
    avoidance_dir: float = 0.0  # Напрямок уникнення (-1 або +1)
    risk_level: float = 0.0  # Рівень ризику (0-1)
    trajectory: deque = field(default_factory=lambda: deque(maxlen=100))  # Історія руху
    target_zone: int = None  # ID цільової зони
    collision_count: int = 0  # Лічильник зіткнень
    comm_range: float = SimulationParams.COMM_RANGE  # Дальність зв'язку

    def __post_init__(self):
        self.trajectory.append(self.position.copy())
        if self.target is None:
            self.target = self.position.copy()

    def update_state(self):
        """Оновлення стану на основі рівня ризику"""
        if self.risk_level > 0.8:
            self.state = AgentState.CRITICAL
        elif self.risk_level > 0.5:
            self.state = AgentState.DANGER
        elif self.risk_level > 0.2:
            self.state = AgentState.WARNING
        else:
            self.state = AgentState.NORMAL

    def predict_trajectory(self, steps: int) -> List[np.ndarray]:
        """Прогнозування траєкторії на задану кількість кроків"""
        return [self.position + self.velocity * t * (1.0/SimulationParams.FPS)
               for t in range(1, steps+1)]

    def avoid_collision(self) -> np.ndarray:
        """Розрахунок сили уникнення зіткнення"""
        if self.state not in [AgentState.DANGER, AgentState.CRITICAL]:
            return np.zeros(3)

        if self.avoidance_dir == 0:
            self.avoidance_dir = random.choice([-1, 1])

        avoidance_force = np.zeros(3)
        for neighbor in self.neighbors:
            dist_vec = self.position - neighbor.position
            dist = np.linalg.norm(dist_vec)
            if dist < SimulationParams.CRITICAL_ZONE:
                avoidance_force += dist_vec / (dist + 0.1)  # Додаємо 0.1, щоб уникнути ділення на 0

        if np.linalg.norm(avoidance_force) > 0:
            avoidance_force = avoidance_force / np.linalg.norm(avoidance_force)
            avoidance_force[2] = self.avoidance_dir * 0.7  # Вертикальна складова

        return avoidance_force * SimulationParams.MAX_ACCEL * 1.5

@dataclass
class TargetZone:
    """Клас цільової зони"""
    id: int
    position: np.ndarray  # Центр зони [x, y, z]
    radius: float = 50.0  # Радіус зони (м)
    priority: float = 1.0  # Пріоритет зони (1.0 - нормальний)
    capacity: int = 10  # Максимальна кількість агентів

class SwarmSimulator:
    """Основний клас симулятора"""

    def __init__(self, num_agents=30, num_zones=3):
        self.num_agents = num_agents
        self.num_zones = num_zones
        self.agents = []
        self.target_zones = []
        self.obstacles = []
        self.collisions = 0
        self.near_misses = 0
        self.sim_time = 0
        self.setup_visualization()
        self.init_agents()
        self.init_target_zones()
        self.init_obstacles()

    def setup_visualization(self):
        """Ініціалізація графічного інтерфейсу"""
        self.fig = plt.figure(figsize=(16, 10))
        self.ax = self.fig.add_subplot(111, projection='3d')
        self.ax.set_xlim(0, SimulationParams.AREA_SIZE[0])
        self.ax.set_ylim(0, SimulationParams.AREA_SIZE[1])
        self.ax.set_zlim(0, SimulationParams.AREA_SIZE[2])
        self.ax.set_xlabel('X (m)')
        self.ax.set_ylabel('Y (m)')
        self.ax.set_zlabel('Z (m)')
        self.ax.set_title('SwarmCoverSimAI - Симуляція оптимального розподілу агентів')

        # Елементи візуалізації
        self.agent_plots = []
        self.traj_plots = []
        self.zone_plots = []
        self.obstacle_plots = []
        self.info_text = self.ax.text2D(
            0.02, 0.95, '', transform=self.ax.transAxes,
            bbox=dict(facecolor='white', alpha=0.7))

    def init_agents(self):
        """Ініціалізація агентів у випадкових позиціях"""
        center = np.array(SimulationParams.AREA_SIZE) / 2

        for i in range(self.num_agents):
            # Випадкова позиція навколо центру
            angle = random.uniform(0, 2*math.pi)
            radius = random.uniform(100, 300)
            pos = center + np.array([
                radius * math.cos(angle),
                radius * math.sin(angle),
                random.uniform(0, SimulationParams.AREA_SIZE[2])
            ])

            # Випадкова початкова швидкість
            vel = np.array([
                -math.sin(angle) * random.uniform(3, 8),
                math.cos(angle) * random.uniform(3, 8),
                0  # Початкова швидкість по Z
            ])

            agent = Agent(i, pos.copy(), vel.copy())
            self.agents.append(agent)

            # Ініціалізація графічних елементів
            self.agent_plots.append(
                self.ax.plot([], [], [], 'o', markersize=6, alpha=0.8)[0])
            self.traj_plots.append(
                self.ax.plot([], [], [], '-', linewidth=1, alpha=0.4)[0])

    def init_target_zones(self):
        """Ініціалізація цільових зон"""
        for i in range(self.num_zones):
            zone_pos = np.array([
                random.uniform(200, SimulationParams.AREA_SIZE[0]-200),
                random.uniform(200, SimulationParams.AREA_SIZE[1]-200),
                random.uniform(50, SimulationParams.AREA_SIZE[2]-50)
            ])

            zone = TargetZone(
                id=i,
                position=zone_pos,
                radius=random.uniform(40, 80),
                priority=random.uniform(0.5, 2.0),
                capacity=self.num_agents // self.num_zones + random.randint(-2, 2)
            )
            self.target_zones.append(zone)

            # Призначення агентів до цільових зон
            agents_per_zone = self.num_agents // self.num_zones
            for j in range(i*agents_per_zone, min((i+1)*agents_per_zone, len(self.agents))):
                self.agents[j].target = zone_pos.copy()
                self.agents[j].target_zone = i

            # Візуалізація цільових зон
            color = TARGET_ZONE_COLORS[i % len(TARGET_ZONE_COLORS)]
            self.zone_plots.extend([
                self.ax.plot([zone_pos[0]-50, zone_pos[0]+50],
                            [zone_pos[1], zone_pos[1]],
                            [zone_pos[2], zone_pos[2]], ':', color=color)[0],
                self.ax.plot([zone_pos[0], zone_pos[0]],
                            [zone_pos[1]-50, zone_pos[1]+50],
                            [zone_pos[2], zone_pos[2]], ':', color=color)[0],
                self.ax.plot([zone_pos[0], zone_pos[0]],
                            [zone_pos[1], zone_pos[1]],
                            [zone_pos[2]-50, zone_pos[2]+50], ':', color=color)[0]
            ])

    def init_obstacles(self):
        """Ініціалізація перешкод (опціонально)"""
        # Додамо кілька випадкових перешкод
        for _ in range(5):
            obs_pos = np.array([
                random.uniform(100, SimulationParams.AREA_SIZE[0]-100),
                random.uniform(100, SimulationParams.AREA_SIZE[1]-100),
                random.uniform(0, SimulationParams.AREA_SIZE[2]/2)
            ])
            obs_size = random.uniform(30, 80)
            self.obstacles.append((obs_pos, obs_size))

            # Візуалізація перешкод
            u = np.linspace(0, 2 * np.pi, 20)
            v = np.linspace(0, np.pi, 20)
            x = obs_pos[0] + obs_size * np.outer(np.cos(u), np.sin(v))
            y = obs_pos[1] + obs_size * np.outer(np.sin(u), np.sin(v))
            z = obs_pos[2] + obs_size * np.outer(np.ones(np.size(u)), np.cos(v))

            self.obstacle_plots.append(
                self.ax.plot_surface(x, y, z, color=SimulationParams.OBSTACLE_COLOR[:3],
                                   alpha=SimulationParams.OBSTACLE_COLOR[3]))

    def find_neighbors(self, agent: Agent) -> List[Agent]:
        """Пошук сусідніх агентів у межах дальності зв'язку"""
        positions = np.array([a.position for a in self.agents if a.id != agent.id])
        if len(positions) == 0:
            return []

        tree = KDTree(positions)
        idx = tree.query_ball_point(agent.position, agent.comm_range)
        return [self.agents[i] for i in idx if i < len(self.agents)]

    def calculate_risk(self, agent: Agent) -> float:
        """Розрахунок рівня ризику зіткнення"""
        if not agent.neighbors:
            return 0.0

        min_dist = min(np.linalg.norm(agent.position - n.position)
                      for n in agent.neighbors)

        if min_dist < SimulationParams.CRITICAL_ZONE:
            return 1.0
        elif min_dist < SimulationParams.DANGER_ZONE:
            return 0.7
        elif min_dist < SimulationParams.WARNING_ZONE:
            return 0.3
        return 0.0

    def apply_flocking_rules(self, agent: Agent) -> np.ndarray:
        """Застосування правил роєвої поведінки (розділення, вирівнювання, згуртування)"""
        separation = np.zeros(3)
        alignment = np.zeros(3)
        cohesion = np.zeros(3)
        center = np.zeros(3)

        for neighbor in agent.neighbors:
            diff = agent.position - neighbor.position
            dist = np.linalg.norm(diff)

            if dist < SimulationParams.WARNING_ZONE:
                separation += diff / (dist + 0.1)

            alignment += neighbor.velocity
            center += neighbor.position

        if len(agent.neighbors) > 0:
            alignment = (alignment/len(agent.neighbors) - agent.velocity) * 0.5
            center = center / len(agent.neighbors)
            cohesion = (center - agent.position) * 0.05

        if np.linalg.norm(separation) > 0:
            separation = (separation / np.linalg.norm(separation)) * SimulationParams.MAX_SPEED_XY

        return separation * 1.5 + alignment * 1.0 + cohesion * 1.0

    def navigate_to_target(self, agent: Agent) -> np.ndarray:
        """Навігація до цільової позиції"""
        target_vec = agent.target - agent.position
        dist = np.linalg.norm(target_vec)

        if dist > 10:  # Якщо далеко від цілі
            target_dir = target_vec / (dist + 0.1)
            return target_dir * min(SimulationParams.MAX_SPEED_XY, dist * 0.3)
        return np.zeros(3)

    def check_obstacle_collision(self, position: np.ndarray) -> bool:
        """Перевірка зіткнення з перешкодами"""
        for obs_pos, obs_size in self.obstacles:
            if np.linalg.norm(position - obs_pos) < obs_size + SimulationParams.AGENT_RADIUS:
                return True
        return False

    def update_agent(self, agent: Agent):
        """Оновлення стану одного агента"""
        # Оновлення списку сусідів
        agent.neighbors = self.find_neighbors(agent)

        # Розрахунок рівня ризику та оновлення стану
        agent.risk_level = self.calculate_risk(agent)
        agent.update_state()

        # Комбінування всіх сил
        flock_force = self.apply_flocking_rules(agent)
        nav_force = self.navigate_to_target(agent)
        avoid_force = agent.avoid_collision()

        if np.linalg.norm(avoid_force) > 0:
            total_force = avoid_force * 2.0 + flock_force * 0.5 + nav_force * 0.3
        else:
            total_force = flock_force * 1.0 + nav_force * 1.0

        # Обмеження прискорення
        accel_norm = np.linalg.norm(total_force)
        if accel_norm > SimulationParams.MAX_ACCEL:
            total_force = total_force / accel_norm * SimulationParams.MAX_ACCEL

        # Оновлення швидкості
        agent.velocity += total_force * (1.0/SimulationParams.FPS)

        # Обмеження швидкості
        speed_xy = np.linalg.norm(agent.velocity[:2])
        if speed_xy > SimulationParams.MAX_SPEED_XY:
            agent.velocity[:2] = agent.velocity[:2] / speed_xy * SimulationParams.MAX_SPEED_XY

        speed_z = abs(agent.velocity[2])
        if speed_z > SimulationParams.MAX_SPEED_Z:
            agent.velocity[2] = agent.velocity[2] / speed_z * SimulationParams.MAX_SPEED_Z

        # Оновлення позиції з перевіркою на зіткнення з перешкодами
        new_position = agent.position + agent.velocity * (1.0/SimulationParams.FPS)
        if not self.check_obstacle_collision(new_position):
            agent.position = new_position
        else:
            # Відскок від перешкоди
            agent.velocity *= -0.5

        # Застосування граничних умов
        self.apply_boundary_conditions(agent)

        # Додавання позиції до історії траєкторії
        agent.trajectory.append(agent.position.copy())

        # Перевірка на зіткнення з іншими агентами
        self.check_agent_collisions(agent)

    def apply_boundary_conditions(self, agent: Agent):
        """Обробка виходу за межі симуляційного простору"""
        for i in range(3):
            if agent.position[i] < 0:
                agent.position[i] = 0
                agent.velocity[i] *= -0.5
            elif agent.position[i] > SimulationParams.AREA_SIZE[i]:
                agent.position[i] = SimulationParams.AREA_SIZE[i]
                agent.velocity[i] *= -0.5

    def check_agent_collisions(self, agent: Agent):
        """Перевірка зіткнень між агентами"""
        for neighbor in agent.neighbors:
            dist = np.linalg.norm(agent.position - neighbor.position)
            if dist < SimulationParams.SAFETY_DISTANCE * SimulationParams.SAFETY_FACTOR:
                self.near_misses += 1
                agent.collision_count += 1

            if dist < SimulationParams.SAFETY_DISTANCE:
                self.collisions += 1
                direction = (agent.position - neighbor.position)
                if np.linalg.norm(direction) > 0:
                    direction = direction / np.linalg.norm(direction)
                agent.velocity += direction * SimulationParams.MAX_ACCEL * 0.8

    def update_visualization(self):
        """Оновлення графічного відображення"""
        for i, agent in enumerate(self.agents):
            # Оновлення позиції агента
            self.agent_plots[i].set_data([agent.position[0]], [agent.position[1]])
            self.agent_plots[i].set_3d_properties([agent.position[2]])
            self.agent_plots[i].set_color(STATE_COLORS[agent.state])
            self.agent_plots[i].set_alpha(0.8)

            # Оновлення траєкторії
            if len(agent.trajectory) > 1:
                traj = np.array(agent.trajectory)
                self.traj_plots[i].set_data(traj[:,0], traj[:,1])
                self.traj_plots[i].set_3d_properties(traj[:,2])
                self.traj_plots[i].set_color(STATE_COLORS[agent.state])

        # Оновлення інформаційного тексту
        info = (
            f"Час симуляції: {self.sim_time:.1f} с\n"
            f"Кількість агентів: {self.num_agents}\n"
            f"Зіткнень: {self.collisions}\n"
            f"Майже зіткнень: {self.near_misses}\n"
            f"Середній рівень ризику: {sum(a.risk_level for a in self.agents)/len(self.agents):.2f}"
        )
        self.info_text.set_text(info)

    def update(self, frame):
        """Оновлення стану симуляції для анімації"""
        self.sim_time += 1.0/SimulationParams.FPS

        # Оновлення всіх агентів
        for agent in self.agents:
            self.update_agent(agent)

        # Оновлення візуалізації
        self.update_visualization()

        return (self.agent_plots + self.traj_plots +
                self.zone_plots + self.obstacle_plots +
                [self.info_text])

    def run_simulation(self):
        """Запуск симуляції з анімацією"""
        anim = animation.FuncAnimation(
            self.fig, self.update,
            frames=500, interval=1000/SimulationParams.FPS, blit=True
        )
        plt.close()
        return HTML(anim.to_jshtml())

    def save_configuration(self, filename: str):
        """Збереження конфігурації симуляції у файл JSON"""
        config = {
            "num_agents": self.num_agents,
            "num_zones": self.num_zones,
            "area_size": SimulationParams.AREA_SIZE,
            "agents": [
                {
                    "id": agent.id,
                    "initial_position": agent.position.tolist(),
                    "target_zone": agent.target_zone
                } for agent in self.agents
            ],
            "target_zones": [
                {
                    "id": zone.id,
                    "position": zone.position.tolist(),
                    "radius": zone.radius,
                    "priority": zone.priority,
                    "capacity": zone.capacity
                } for zone in self.target_zones
            ]
        }

        with open(filename, 'w') as f:
            json.dump(config, f, indent=4)

    def load_configuration(self, filename: str):
        """Завантаження конфігурації симуляції з файлу JSON"""
        with open(filename, 'r') as f:
            config = json.load(f)

        # Оновлення параметрів симуляції
        self.num_agents = config["num_agents"]
        self.num_zones = config["num_zones"]

        # Очищення поточних даних
        self.agents = []
        self.target_zones = []

        # Завантаження цільових зон
        for zone_data in config["target_zones"]:
            zone = TargetZone(
                id=zone_data["id"],
                position=np.array(zone_data["position"]),
                radius=zone_data["radius"],
                priority=zone_data["priority"],
                capacity=zone_data["capacity"]
            )
            self.target_zones.append(zone)

        # Завантаження агентів
        for agent_data in config["agents"]:
            agent = Agent(
                id=agent_data["id"],
                position=np.array(agent_data["initial_position"]),
                target=np.array(self.target_zones[agent_data["target_zone"]].position),
                target_zone=agent_data["target_zone"]
            )
            self.agents.append(agent)

        # Оновлення візуалізації
        self.setup_visualization()

# Функція для інтерактивного запуску симуляції
def interactive_simulation(num_agents=30, num_zones=3, max_speed=10.0, safety_dist=5.0):
    """Інтерактивний запуск симуляції з можливістю налаштування параметрів"""
    # Оновлення параметрів
    SimulationParams.MAX_SPEED_XY = max_speed
    SimulationParams.SAFETY_DISTANCE = safety_dist
    SimulationParams.COMM_RANGE = safety_dist * 3  # Дальність зв'язку пропорційна безпечній відстані

    # Створення та запуск симуляції
    simulator = SwarmSimulator(num_agents=num_agents, num_zones=num_zones)
    return simulator.run_simulation()

# Приклад використання інтерактивного інтерфейсу
interact(
    interactive_simulation,
    num_agents=IntSlider(min=10, max=100, step=5, value=10),
    num_zones=IntSlider(min=1, max=5, step=1, value=2),
    max_speed=FloatSlider(min=5.0, max=20.0, step=1.0, value=10.0),
    safety_dist=FloatSlider(min=2.0, max=10.0, step=0.5, value=5.0)
)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.patches import Polygon, Circle
from shapely.geometry import Polygon as ShapelyPolygon, Point
import random
import math
from dataclasses import dataclass, field
from typing import List, Dict, Tuple
from enum import Enum, auto

# Константи симуляції
SIM_WIDTH = 150
SIM_HEIGHT = 150
FPS = 30
DT = 1/FPS

class DroneState(Enum):
    NORMAL = auto()
    AVOIDING = auto()
    EMERGENCY = auto()

@dataclass
class DroneSpecs:
    safety_radius: float = 2.5
    max_speed: float = 10.0
    max_accel: float = 3.0
    comm_range: float = 50.0

@dataclass
class Drone:
    id: int
    position: np.ndarray
    velocity: np.ndarray = field(default_factory=lambda: np.zeros(2))
    acceleration: np.ndarray = field(default_factory=lambda: np.zeros(2))
    target: np.ndarray = field(default_factory=lambda: np.array([SIM_WIDTH/2, SIM_HEIGHT/2]))
    state: DroneState = DroneState.NORMAL
    specs: DroneSpecs = field(default_factory=DroneSpecs)
    trajectory: List[np.ndarray] = field(default_factory=list)

    def update(self, acceleration: np.ndarray):
        # Обмеження прискорення
        accel_norm = np.linalg.norm(acceleration)
        if accel_norm > self.specs.max_accel:
            acceleration = acceleration / accel_norm * self.specs.max_accel

        self.acceleration = acceleration
        self.velocity += self.acceleration * DT

        # Обмеження швидкості
        speed = np.linalg.norm(self.velocity)
        if speed > self.specs.max_speed:
            self.velocity = self.velocity / speed * self.specs.max_speed

        self.position += self.velocity * DT
        self.trajectory.append(self.position.copy())

class Obstacle:
    def __init__(self, vertices: List[Tuple[float, float]]):
        self.vertices = np.array(vertices)
        self.polygon = ShapelyPolygon(vertices)

    def contains(self, point: np.ndarray) -> bool:
        return self.polygon.contains(Point(point))

class SwarmSimulator:
    def __init__(self, num_drones: int = 20):
        self.num_drones = num_drones
        self.drones = []
        self.obstacles = []
        self.collisions = 0
        self.setup_environment()

        # Ініціалізація візуалізації
        self.fig, self.ax = plt.subplots(figsize=(10, 10))
        self.ax.set_xlim(0, SIM_WIDTH)
        self.ax.set_ylim(0, SIM_HEIGHT)
        self.ax.set_aspect('equal')
        self.ax.grid(True)

        self.drone_artists = []
        self.traj_artists = []
        self.obstacle_artists = []
        self.info_text = self.ax.text(0.02, 0.95, '', transform=self.ax.transAxes)

    def setup_environment(self):
        # Створення дронів вздовж лівої межі
        for i in range(self.num_drones):
            y_pos = random.uniform(10, SIM_HEIGHT-10)
            drone = Drone(
                id=i,
                position=np.array([10.0, y_pos]),
                target=np.array([SIM_WIDTH-10, y_pos]),
                velocity=np.array([random.uniform(3, 5), 0])
            )
            self.drones.append(drone)

        # Додавання перешкод
        self.add_obstacle([(50, 50), (70, 50), (70, 100), (50, 100)])
        self.add_obstacle([(100, 30), (120, 30), (120, 80), (100, 80)])

    def add_obstacle(self, vertices: List[Tuple[float, float]]):
        self.obstacles.append(Obstacle(vertices))

    def check_collisions(self):
        for i, drone1 in enumerate(self.drones):
            for j, drone2 in enumerate(self.drones[i+1:], i+1):
                dist = np.linalg.norm(drone1.position - drone2.position)
                if dist < drone1.specs.safety_radius + drone2.specs.safety_radius:
                    self.collisions += 1
                    # Реакція на зіткнення
                    direction = drone1.position - drone2.position
                    if np.linalg.norm(direction) > 0:
                        direction = direction / np.linalg.norm(direction)
                    drone1.velocity += direction * drone1.specs.max_accel * 0.5
                    drone2.velocity -= direction * drone2.specs.max_accel * 0.5

    def apply_cbf(self):
        for i, drone in enumerate(self.drones):
            # Пошук сусідів у межах комунікації
            neighbors = []
            for j, other in enumerate(self.drones):
                if i != j and np.linalg.norm(drone.position - other.position) < drone.specs.comm_range:
                    neighbors.append(other)

            # Обчислення сил уникнення
            avoidance_force = np.zeros(2)
            for neighbor in neighbors:
                dist_vec = drone.position - neighbor.position
                dist = np.linalg.norm(dist_vec)

                # CBF умова
                h = dist**2 - (2*drone.specs.safety_radius)**2
                if h < 0:
                    drone.state = DroneState.EMERGENCY
                    avoidance_force += dist_vec / (dist + 0.1) * 10.0
                elif h < (3*drone.specs.safety_radius)**2:
                    drone.state = DroneState.AVOIDING
                    avoidance_force += dist_vec / (dist + 0.1) * 5.0
                else:
                    drone.state = DroneState.NORMAL

            # Додавання сили руху до цілі
            target_dir = drone.target - drone.position
            target_dist = np.linalg.norm(target_dir)
            if target_dist > 0:
                target_force = target_dir / target_dist * 3.0
            else:
                target_force = np.zeros(2)

            # Комбінування сил
            total_force = target_force + avoidance_force
            drone.update(total_force)

    def update_visualization(self):
        # Очищення попередніх об'єктів
        for artist in self.drone_artists + self.traj_artists + self.obstacle_artists:
            artist.remove()
        self.drone_artists = []
        self.traj_artists = []
        self.obstacle_artists = []

        # Відображення перешкод
        for obstacle in self.obstacles:
            patch = Polygon(obstacle.vertices, closed=True,
                           facecolor='gray', alpha=0.5)
            self.ax.add_patch(patch)
            self.obstacle_artists.append(patch)

        # Відображення дронів та траєкторій
        for drone in self.drones:
            # Вибір кольору за станом
            color = {
                DroneState.NORMAL: 'green',
                DroneState.AVOIDING: 'orange',
                DroneState.EMERGENCY: 'red'
            }[drone.state]

            # Дрон
            drone_circle = Circle(drone.position, drone.specs.safety_radius,
                                facecolor=color, alpha=0.7)
            self.ax.add_patch(drone_circle)
            self.drone_artists.append(drone_circle)

            # Траєкторія
            if len(drone.trajectory) > 1:
                traj = np.array(drone.trajectory)
                line, = self.ax.plot(traj[:,0], traj[:,1], '-',
                                    color=color, alpha=0.4, linewidth=1)
                self.traj_artists.append(line)

        # Оновлення інформації
        self.info_text.set_text(f'Час: {self.time:.1f} с\nЗіткнень: {self.collisions}')

    def run_simulation(self, duration: float = 20.0):
        self.time = 0.0
        num_frames = int(duration * FPS)

        def update(frame):
            self.time += DT

            # Оновлення фізики
            self.apply_cbf()
            self.check_collisions()

            # Оновлення візуалізації
            self.update_visualization()

            return self.drone_artists + self.traj_artists + self.obstacle_artists + [self.info_text]

        anim = FuncAnimation(self.fig, update, frames=num_frames,
                            interval=1000/FPS, blit=True)
        plt.close()
        return anim

# Запуск симуляції
if __name__ == "__main__":
    simulator = SwarmSimulator(num_drones=20)
    anim = simulator.run_simulation(duration=15)

    # Для відображення в Google Colab
    from IPython.display import HTML
    HTML(anim.to_html5_video())

In [ ]:
import numpy as np #CBF !!!
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.patches import Circle
from shapely.geometry import Point
from typing import List

# Константи симуляції
SIM_WIDTH = 150
SIM_HEIGHT = 150
FPS = 30
DT = 1 / FPS

class Drone:
    def __init__(self, id: int, position: np.ndarray, target: np.ndarray, specs: dict):
        self.id = id
        self.position = position
        self.velocity = np.zeros(2)
        self.acceleration = np.zeros(2)
        self.target = target
        self.specs = specs
        self.trajectory = [position.copy()]

    def update(self, acceleration: np.ndarray):
        # Обмеження прискорення
        accel_norm = np.linalg.norm(acceleration)
        if accel_norm > self.specs['max_accel']:
            acceleration = acceleration / accel_norm * self.specs['max_accel']

        self.acceleration = acceleration
        self.velocity += self.acceleration * DT

        # Обмеження швидкості
        speed = np.linalg.norm(self.velocity)
        if speed > self.specs['max_speed']:
            self.velocity = self.velocity / speed * self.specs['max_speed']

        self.position += self.velocity * DT
        self.trajectory.append(self.position.copy())

class SwarmSimulator:
    def __init__(self, num_drones: int = 20):
        self.num_drones = num_drones
        self.drones = []
        self.collisions = 0
        self.setup_environment()

        # Ініціалізація візуалізації
        self.fig, self.ax = plt.subplots(figsize=(10, 10))
        self.ax.set_xlim(0, SIM_WIDTH)
        self.ax.set_ylim(0, SIM_HEIGHT)
        self.ax.set_aspect('equal')
        self.ax.grid(True)

        self.drone_artists = []
        self.traj_artists = []
        self.info_text = self.ax.text(0.02, 0.95, '', transform=self.ax.transAxes)

    def setup_environment(self):
        # Створення дронів вздовж лівої межі
        drone_specs = {
            'safety_radius': 2.5,
            'max_speed': 10.0,
            'max_accel': 3.0
        }

        for i in range(self.num_drones):
            y_pos = np.random.uniform(10, SIM_HEIGHT - 10)
            drone = Drone(
                id=i,
                position=np.array([10.0, y_pos]),
                target=np.array([SIM_WIDTH - 10, y_pos]),
                specs=drone_specs
            )
            self.drones.append(drone)

    def check_collisions(self):
        for i, drone1 in enumerate(self.drones):
            for j, drone2 in enumerate(self.drones[i + 1:], i + 1):
                dist = np.linalg.norm(drone1.position - drone2.position)
                if dist < drone1.specs['safety_radius'] + drone2.specs['safety_radius']:
                    self.collisions += 1
                    # Реакція на зіткнення
                    direction = drone1.position - drone2.position
                    if np.linalg.norm(direction) > 0:
                        direction = direction / np.linalg.norm(direction)
                    drone1.velocity += direction * drone1.specs['max_accel'] * 0.5
                    drone2.velocity -= direction * drone2.specs['max_accel'] * 0.5

    def apply_cbf(self):
        for drone in self.drones:
            # Обчислення сил уникнення
            avoidance_force = np.zeros(2)
            for other in self.drones:
                if drone.id != other.id:
                    dist_vec = drone.position - other.position
                    dist = np.linalg.norm(dist_vec)

                    # CBF умова
                    h = dist ** 2 - (2 * drone.specs['safety_radius']) ** 2
                    if h < 0:
                        avoidance_force += dist_vec / (dist + 0.1) * 10.0
                    elif h < (3 * drone.specs['safety_radius']) ** 2:
                        avoidance_force += dist_vec / (dist + 0.1) * 5.0

            # Додавання сили руху до цілі
            target_dir = drone.target - drone.position
            target_dist = np.linalg.norm(target_dir)
            if target_dist > 0:
                target_force = target_dir / target_dist * 3.0
            else:
                target_force = np.zeros(2)

            # Комбінування сил
            total_force = target_force + avoidance_force
            drone.update(total_force)

    def update_visualization(self):
        # Очищення попередніх об'єктів
        for artist in self.drone_artists + self.traj_artists:
            artist.remove()
        self.drone_artists = []
        self.traj_artists = []

        # Відображення дронів та траєкторій
        for drone in self.drones:
            # Дрон
            drone_circle = Circle(drone.position, drone.specs['safety_radius'], facecolor='blue', alpha=0.7)
            self.ax.add_patch(drone_circle)
            self.drone_artists.append(drone_circle)

            # Траєкторія
            if len(drone.trajectory) > 1:
                traj = np.array(drone.trajectory)
                line, = self.ax.plot(traj[:, 0], traj[:, 1], '-', color='blue', alpha=0.4, linewidth=1)
                self.traj_artists.append(line)

        # Оновлення інформації
        self.info_text.set_text(f'Час: {self.time:.1f} с\nЗіткнень: {self.collisions}')

    def run_simulation(self, duration: float = 20.0):
        self.time = 0.0
        num_frames = int(duration * FPS)

        def update(frame):
            self.time += DT

            # Оновлення фізики
            self.apply_cbf()
            self.check_collisions()

            # Оновлення візуалізації
            self.update_visualization()

            return self.drone_artists + self.traj_artists + [self.info_text]

        anim = FuncAnimation(self.fig, update, frames=num_frames, interval=1000 / FPS, blit=True)
        plt.close()
        return anim

# Запуск симуляції
if __name__ == "__main__":
    simulator = SwarmSimulator(num_drones=20)
    anim = simulator.run_simulation(duration=15)

    # Для відображення в Google Colab
    from IPython.display import HTML
    HTML(anim.to_html5_video())

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.patches import Circle
from IPython.display import HTML

import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 1000

# Константи симуляції
SIM_WIDTH = 150
SIM_HEIGHT = 150
FPS = 30
DT = 1 / FPS

class Drone:
    def __init__(self, id: int, position: np.ndarray, target: np.ndarray, specs: dict):
        self.id = id
        self.position = position
        self.velocity = np.zeros(2)
        self.acceleration = np.zeros(2)
        self.target = target
        self.specs = specs
        self.trajectory = [position.copy()]

    def update(self, acceleration: np.ndarray):
        # Обмеження прискорення
        accel_norm = np.linalg.norm(acceleration)
        if accel_norm > self.specs['max_accel']:
            acceleration = acceleration / accel_norm * self.specs['max_accel']

        self.acceleration = acceleration
        self.velocity += self.acceleration * DT

        # Обмеження швидкості
        speed = np.linalg.norm(self.velocity)
        if speed > self.specs['max_speed']:
            self.velocity = self.velocity / speed * self.specs['max_speed']

        self.position += self.velocity * DT
        self.trajectory.append(self.position.copy())

class SwarmSimulator:
    def __init__(self, num_drones: int = 20):
        self.num_drones = num_drones
        self.drones = []
        self.collisions = 0
        self.setup_environment()

        # Ініціалізація візуалізації
        self.fig, self.ax = plt.subplots(figsize=(10, 10))
        self.ax.set_xlim(0, SIM_WIDTH)
        self.ax.set_ylim(0, SIM_HEIGHT)
        self.ax.set_aspect('equal')
        self.ax.grid(True)

        self.drone_artists = []
        self.traj_artists = []
        self.info_text = self.ax.text(0.02, 0.95, '', transform=self.ax.transAxes)

    def setup_environment(self):
        # Створення дронів вздовж лівої межі
        drone_specs = {
            'safety_radius': 2.5,
            'max_speed': 10.0,
            'max_accel': 3.0
        }

        for i in range(self.num_drones):
            y_pos = np.random.uniform(10, SIM_HEIGHT - 10)
            drone = Drone(
                id=i,
                position=np.array([10.0, y_pos]),
                target=np.array([SIM_WIDTH - 10, y_pos]),
                specs=drone_specs
            )
            self.drones.append(drone)

    def check_collisions(self):
        for i, drone1 in enumerate(self.drones):
            for j, drone2 in enumerate(self.drones[i + 1:], i + 1):
                dist = np.linalg.norm(drone1.position - drone2.position)
                if dist < drone1.specs['safety_radius'] + drone2.specs['safety_radius']:
                    self.collisions += 1
                    # Реакція на зіткнення
                    direction = drone1.position - drone2.position
                    if np.linalg.norm(direction) > 0:
                        direction = direction / np.linalg.norm(direction)
                    drone1.velocity += direction * drone1.specs['max_accel'] * 0.5
                    drone2.velocity -= direction * drone2.specs['max_accel'] * 0.5

    def apply_cbf(self):
        for drone in self.drones:
            # Обчислення сил уникнення
            avoidance_force = np.zeros(2)
            for other in self.drones:
                if drone.id != other.id:
                    dist_vec = drone.position - other.position
                    dist = np.linalg.norm(dist_vec)

                    # CBF умова
                    h = dist ** 2 - (2 * drone.specs['safety_radius']) ** 2
                    if h < 0:
                        avoidance_force += dist_vec / (dist + 0.1) * 10.0
                    elif h < (3 * drone.specs['safety_radius']) ** 2:
                        avoidance_force += dist_vec / (dist + 0.1) * 5.0

            # Додавання сили руху до цілі
            target_dir = drone.target - drone.position
            target_dist = np.linalg.norm(target_dir)
            if target_dist > 0:
                target_force = target_dir / target_dist * 3.0
            else:
                target_force = np.zeros(2)

            # Комбінування сил
            total_force = target_force + avoidance_force
            drone.update(total_force)

    def update_visualization(self):
        # Очищення попередніх об'єктів
        for artist in self.drone_artists + self.traj_artists:
            artist.remove()
        self.drone_artists = []
        self.traj_artists = []

        # Відображення дронів та траєкторій
        for drone in self.drones:
            # Дрон
            drone_circle = Circle(drone.position, drone.specs['safety_radius'], facecolor='blue', alpha=0.7)
            self.ax.add_patch(drone_circle)
            self.drone_artists.append(drone_circle)

            # Траєкторія
            if len(drone.trajectory) > 1:
                traj = np.array(drone.trajectory)
                line, = self.ax.plot(traj[:, 0], traj[:, 1], '-', color='blue', alpha=0.4, linewidth=1)
                self.traj_artists.append(line)

        # Оновлення інформації
        self.info_text.set_text(f'Час: {self.time:.1f} с\nЗіткнень: {self.collisions}')

    def run_simulation(self, duration: float = 20.0):
        self.time = 0.0
        num_frames = int(duration * FPS)

        def update(frame):
            self.time += DT

            # Оновлення фізики
            self.apply_cbf()
            self.check_collisions()

            # Оновлення візуалізації
            self.update_visualization()

            return self.drone_artists + self.traj_artists + [self.info_text]

        anim = FuncAnimation(self.fig, update, frames=num_frames, interval=1000 / FPS, blit=True)
        plt.close()
        return anim

# Запуск симуляції
if __name__ == "__main__":
    simulator = SwarmSimulator(num_drones=20)
    anim = simulator.run_simulation(duration=15)

    # Для відображення в Google Colab
    HTML(anim.to_jshtml())

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.patches import Circle
from dataclasses import dataclass, field
from typing import List
from enum import Enum, auto
import random
from IPython.display import HTML

import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 1000

# Константи симуляції
SIM_WIDTH = 150
SIM_HEIGHT = 150
FPS = 30
DT = 1/FPS

@dataclass
class DroneSpecs:
    safety_radius: float = 2.5
    max_speed: float = 10.0
    max_accel: float = 3.0
    comm_range: float = 50.0

@dataclass
class Drone:
    id: int
    position: np.ndarray
    velocity: np.ndarray = field(default_factory=lambda: np.zeros(2))
    acceleration: np.ndarray = field(default_factory=lambda: np.zeros(2))
    target: np.ndarray = field(default_factory=lambda: np.array([SIM_WIDTH/2, SIM_HEIGHT/2]))
    specs: DroneSpecs = field(default_factory=DroneSpecs)
    trajectory: List[np.ndarray] = field(default_factory=list)

    def update(self, acceleration: np.ndarray):
        # Обмеження прискорення
        accel_norm = np.linalg.norm(acceleration)
        if accel_norm > self.specs.max_accel:
            acceleration = acceleration / accel_norm * self.specs.max_accel

        self.acceleration = acceleration
        self.velocity += self.acceleration * DT

        # Обмеження швидкості
        speed = np.linalg.norm(self.velocity)
        if speed > self.specs.max_speed:
            self.velocity = self.velocity / speed * self.specs.max_speed

        self.position += self.velocity * DT
        self.trajectory.append(self.position.copy())

class SwarmSimulator:
    def __init__(self, num_drones: int = 20):
        self.num_drones = num_drones
        self.drones = []
        self.collisions = 0
        self.setup_environment()

        # Ініціалізація візуалізації
        self.fig, self.ax = plt.subplots(figsize=(10, 10))
        self.ax.set_xlim(0, SIM_WIDTH)
        self.ax.set_ylim(0, SIM_HEIGHT)
        self.ax.set_aspect('equal')
        self.ax.grid(True)
        self.ax.set_title('Симуляція руху дронів з уникненням зіткнень')

        self.drone_artists = []
        self.traj_artists = []
        self.info_text = self.ax.text(0.02, 0.95, '', transform=self.ax.transAxes,
                                    bbox=dict(facecolor='white', alpha=0.7))

    def setup_environment(self):
        # Створення дронів вздовж лівої межі
        for i in range(self.num_drones):
            y_pos = random.uniform(10, SIM_HEIGHT-10)
            drone = Drone(
                id=i,
                position=np.array([10.0, y_pos]),
                target=np.array([SIM_WIDTH-10, y_pos]),
                velocity=np.array([random.uniform(3, 5), 0])
            )
            self.drones.append(drone)

    def check_collisions(self):
        for i, drone1 in enumerate(self.drones):
            for j, drone2 in enumerate(self.drones[i+1:], i+1):
                dist = np.linalg.norm(drone1.position - drone2.position)
                if dist < drone1.specs.safety_radius + drone2.specs.safety_radius:
                    self.collisions += 1
                    # Реакція на зіткнення
                    direction = drone1.position - drone2.position
                    if np.linalg.norm(direction) > 0:
                        direction = direction / np.linalg.norm(direction)
                    drone1.velocity += direction * drone1.specs.max_accel * 0.5
                    drone2.velocity -= direction * drone2.specs.max_accel * 0.5

    def apply_cbf(self):
        for i, drone in enumerate(self.drones):
            # Пошук сусідів у межах комунікації
            neighbors = []
            for j, other in enumerate(self.drones):
                if i != j and np.linalg.norm(drone.position - other.position) < drone.specs.comm_range:
                    neighbors.append(other)

            # Обчислення сил уникнення
            avoidance_force = np.zeros(2)
            for neighbor in neighbors:
                dist_vec = drone.position - neighbor.position
                dist = np.linalg.norm(dist_vec)

                # CBF умова
                h = dist**2 - (2*drone.specs.safety_radius)**2
                if h < 0:
                    avoidance_force += dist_vec / (dist + 0.1) * 10.0
                elif h < (3*drone.specs.safety_radius)**2:
                    avoidance_force += dist_vec / (dist + 0.1) * 5.0

            # Додавання сили руху до цілі
            target_dir = drone.target - drone.position
            target_dist = np.linalg.norm(target_dir)
            if target_dist > 0:
                target_force = target_dir / target_dist * 3.0
            else:
                target_force = np.zeros(2)

            # Комбінування сил
            total_force = target_force + avoidance_force
            drone.update(total_force)

    def update_visualization(self):
        # Очищення попередніх об'єктів
        for artist in self.drone_artists + self.traj_artists:
            artist.remove()
        self.drone_artists = []
        self.traj_artists = []

        # Відображення дронів та траєкторій
        for drone in self.drones:
            # Дрон (синій колір)
            drone_circle = Circle(drone.position, drone.specs.safety_radius,
                                facecolor='blue', alpha=0.7)
            self.ax.add_patch(drone_circle)
            self.drone_artists.append(drone_circle)

            # Траєкторія
            if len(drone.trajectory) > 1:
                traj = np.array(drone.trajectory)
                line, = self.ax.plot(traj[:,0], traj[:,1], '-',
                                    color='blue', alpha=0.4, linewidth=1)
                self.traj_artists.append(line)

        # Оновлення інформації
        self.info_text.set_text(f'Час: {self.time:.1f} с\nЗіткнень: {self.collisions}')

    def run_simulation(self, duration: float = 20.0):
        self.time = 0.0
        num_frames = int(duration * FPS)

        def update(frame):
            self.time += DT

            # Оновлення фізики
            self.apply_cbf()
            self.check_collisions()

            # Оновлення візуалізації
            self.update_visualization()

            return self.drone_artists + self.traj_artists + [self.info_text]

        anim = FuncAnimation(self.fig, update, frames=num_frames,
                            interval=1000/FPS, blit=True)
        #plt.close()
        return anim

# Запуск симуляції
if __name__ == "__main__":
    simulator = SwarmSimulator(num_drones=20)
    anim = simulator.run_simulation(duration=15)

    # Для відображення в Google Colab
    #HTML(anim.to_html5_video())
    # Для відображення в Google Colab
    HTML(anim.to_jshtml())

In [ ]:
# Симулятор руху дронів з уникненням зіткнень (текстова версія)
import numpy as np
from dataclasses import dataclass, field
from typing import List
import random

# Константи симуляції
SIM_WIDTH = 150
SIM_HEIGHT = 150
FPS = 30
DT = 1/FPS

@dataclass
class DroneSpecs:
    safety_radius: float = 2.5
    max_speed: float = 10.0
    max_accel: float = 3.0
    comm_range: float = 50.0

@dataclass
class Drone:
    id: int
    position: np.ndarray
    velocity: np.ndarray = field(default_factory=lambda: np.zeros(2))
    acceleration: np.ndarray = field(default_factory=lambda: np.zeros(2))
    target: np.ndarray = field(default_factory=lambda: np.array([SIM_WIDTH/2, SIM_HEIGHT/2]))
    specs: DroneSpecs = field(default_factory=DroneSpecs)
    trajectory: List[np.ndarray] = field(default_factory=list)

    def update(self, acceleration: np.ndarray):
        # Обмеження прискорення
        accel_norm = np.linalg.norm(acceleration)
        if accel_norm > self.specs.max_accel:
            acceleration = acceleration / accel_norm * self.specs.max_accel

        self.acceleration = acceleration
        self.velocity += self.acceleration * DT

        # Обмеження швидкості
        speed = np.linalg.norm(self.velocity)
        if speed > self.specs.max_speed:
            self.velocity = self.velocity / speed * self.specs.max_speed

        self.position += self.velocity * DT
        self.trajectory.append(self.position.copy())

class SwarmSimulator:
    def __init__(self, num_drones: int = 20):
        self.num_drones = num_drones
        self.drones = []
        self.collisions = 0
        self.time = 0.0
        self.setup_environment()

    def setup_environment(self):
        # Створення дронів вздовж лівої межі
        for i in range(self.num_drones):
            y_pos = random.uniform(10, SIM_HEIGHT-10)
            drone = Drone(
                id=i,
                position=np.array([10.0, y_pos]),
                target=np.array([SIM_WIDTH-10, y_pos]),
                velocity=np.array([random.uniform(3, 5), 0])
            )
            self.drones.append(drone)

    def check_collisions(self):
        for i, drone1 in enumerate(self.drones):
            for j, drone2 in enumerate(self.drones[i+1:], i+1):
                dist = np.linalg.norm(drone1.position - drone2.position)
                if dist < drone1.specs.safety_radius + drone2.specs.safety_radius:
                    self.collisions += 1
                    # Реакція на зіткнення
                    direction = drone1.position - drone2.position
                    if np.linalg.norm(direction) > 0:
                        direction = direction / np.linalg.norm(direction)
                    drone1.velocity += direction * drone1.specs.max_accel * 0.5
                    drone2.velocity -= direction * drone2.specs.max_accel * 0.5

    def apply_cbf(self):
        for i, drone in enumerate(self.drones):
            # Пошук сусідів у межах комунікації
            neighbors = []
            for j, other in enumerate(self.drones):
                if i != j and np.linalg.norm(drone.position - other.position) < drone.specs.comm_range:
                    neighbors.append(other)

            # Обчислення сил уникнення
            avoidance_force = np.zeros(2)
            for neighbor in neighbors:
                dist_vec = drone.position - neighbor.position
                dist = np.linalg.norm(dist_vec)

                # CBF умова
                h = dist**2 - (2*drone.specs.safety_radius)**2
                if h < 0:
                    avoidance_force += dist_vec / (dist + 0.1) * 10.0
                elif h < (3*drone.specs.safety_radius)**2:
                    avoidance_force += dist_vec / (dist + 0.1) * 5.0

            # Додавання сили руху до цілі
            target_dir = drone.target - drone.position
            target_dist = np.linalg.norm(target_dir)
            if target_dist > 0:
                target_force = target_dir / target_dist * 3.0
            else:
                target_force = np.zeros(2)

            # Комбінування сил
            total_force = target_force + avoidance_force
            drone.update(total_force)

    def run_simulation(self, duration: float = 20.0):
        num_steps = int(duration * FPS)

        for _ in range(num_steps):
            self.time += DT
            self.apply_cbf()
            self.check_collisions()

        # Вивід результатів
        print("=== Результати симуляції ===")
        print(f"Час симуляції: {self.time:.2f} секунд")
        print(f"Кількість дронів: {self.num_drones}")
        print(f"Загальна кількість зіткнень: {self.collisions}")
        print(f"Середня кількість зіткнень на дрон: {self.collisions/self.num_drones:.2f}")

        # Додаткові метрики
        avg_speed = np.mean([np.mean([np.linalg.norm(p) for p in drone.trajectory]) for drone in self.drones])
        max_speed = np.max([np.max([np.linalg.norm(p) for p in drone.trajectory]) for drone in self.drones])
        print(f"\nСередня швидкість дронів: {avg_speed:.2f} одиниць/сек")
        print(f"Максимальна швидкість: {max_speed:.2f} одиниць/сек")

        # Відстань до цілі в кінці симуляції
        final_distances = [np.linalg.norm(drone.position - drone.target) for drone in self.drones]
        print(f"\nСередня відстань до цілі в кінці: {np.mean(final_distances):.2f} одиниць")
        print(f"Максимальна відстань до цілі в кінці: {np.max(final_distances):.2f} одиниць")
        print(f"Мінімальна відстань до цілі в кінці: {np.min(final_distances):.2f} одиниць")

# Запуск симуляції
if __name__ == "__main__":
    simulator = SwarmSimulator(num_drones=20)
    simulator.run_simulation(duration=15)

In [ ]:
import numpy as np
from dataclasses import dataclass, field
from typing import List
import random

# Константи симуляції
SIM_WIDTH = 150
SIM_HEIGHT = 150
FPS = 30
DT = 1/FPS

@dataclass
class DroneSpecs:
    safety_radius: float = 5.0  # Збільшено радіус безпеки
    max_speed: float = 8.0      # Зменшено максимальну швидкість
    max_accel: float = 2.0      # Зменшено прискорення
    comm_range: float = 60.0    # Збільшено дальність "бачення"
    repulsion_gain: float = 15.0  # Коефіцієнт відштовхування

@dataclass
class Drone:
    id: int
    position: np.ndarray
    velocity: np.ndarray = field(default_factory=lambda: np.zeros(2))
    acceleration: np.ndarray = field(default_factory=lambda: np.zeros(2))
    target: np.ndarray = field(default_factory=lambda: np.array([SIM_WIDTH/2, SIM_HEIGHT/2]))
    specs: DroneSpecs = field(default_factory=DroneSpecs)
    trajectory: List[np.ndarray] = field(default_factory=list)

    def update(self, acceleration: np.ndarray):
        # Обмеження прискорення
        accel_norm = np.linalg.norm(acceleration)
        if accel_norm > self.specs.max_accel:
            acceleration = acceleration / accel_norm * self.specs.max_accel

        self.acceleration = acceleration
        self.velocity += self.acceleration * DT

        # Обмеження швидкості
        speed = np.linalg.norm(self.velocity)
        if speed > self.specs.max_speed:
            self.velocity = self.velocity / speed * self.specs.max_speed

        self.position += self.velocity * DT
        self.trajectory.append(self.position.copy())

class SwarmSimulator:
    def __init__(self, num_drones: int = 20):
        self.num_drones = num_drones
        self.drones = []
        self.collisions = 0
        self.time = 0.0
        self.setup_environment()

    def setup_environment(self):
        # Створення дронів з більшими початковими інтервалами
        spacing = SIM_HEIGHT / (self.num_drones + 1)
        for i in range(self.num_drones):
            y_pos = 10 + spacing * (i + 1)
            drone = Drone(
                id=i,
                position=np.array([10.0, y_pos]),
                target=np.array([SIM_WIDTH-10, y_pos]),
                velocity=np.array([random.uniform(2, 4), 0])  # Зменшена початкова швидкість
            )
            self.drones.append(drone)

    def check_collisions(self):
        for i, drone1 in enumerate(self.drones):
            for j, drone2 in enumerate(self.drones[i+1:], i+1):
                dist = np.linalg.norm(drone1.position - drone2.position)
                if dist < drone1.specs.safety_radius + drone2.specs.safety_radius:
                    self.collisions += 1

    def apply_cbf(self):
        for i, drone in enumerate(self.drones):
            neighbors = []
            for j, other in enumerate(self.drones):
                if i != j and np.linalg.norm(drone.position - other.position) < drone.specs.comm_range:
                    neighbors.append(other)

            # Сильніше відштовхування
            avoidance_force = np.zeros(2)
            for neighbor in neighbors:
                dist_vec = drone.position - neighbor.position
                dist = np.linalg.norm(dist_vec)
                min_dist = 2.5 * drone.specs.safety_radius

                if dist < min_dist:
                    # Дуже сильне відштовхування на близьких дистанціях
                    avoidance_force += dist_vec / (dist**2 + 0.1) * drone.specs.repulsion_gain

            # Плавне гальмування перед ціллю
            target_dir = drone.target - drone.position
            target_dist = np.linalg.norm(target_dir)
            if target_dist > 0:
                target_force = target_dir / target_dist * min(3.0, target_dist/5)
            else:
                target_force = np.zeros(2)

            # Додаткове обмеження швидкості біля цілі
            if target_dist < 20:
                speed_limit = drone.specs.max_speed * (target_dist / 20)
                speed = np.linalg.norm(drone.velocity)
                if speed > speed_limit:
                    drone.velocity = drone.velocity / speed * speed_limit

            # Комбінування сил з вагою
            total_force = target_force + avoidance_force * 2.0  # Більша вага уникнення
            drone.update(total_force)

    def run_simulation(self, duration: float = 20.0):
        num_steps = int(duration * FPS)

        for _ in range(num_steps):
            self.time += DT
            self.apply_cbf()
            self.check_collisions()

        print("=== Результати симуляції ===")
        print(f"Час симуляції: {self.time:.2f} секунд")
        print(f"Кількість дронів: {self.num_drones}")
        print(f"Загальна кількість зіткнень: {self.collisions}")
        print(f"Середня кількість зіткнень на дрон: {self.collisions/self.num_drones:.2f}")

        avg_speed = np.mean([np.mean([np.linalg.norm(v) for v in np.diff(np.array(drone.trajectory), axis=0)/DT]) for drone in self.drones])
        max_speed = np.max([np.max([np.linalg.norm(v) for v in np.diff(np.array(drone.trajectory), axis=0)/DT]) for drone in self.drones])
        print(f"\nСередня швидкість дронів: {avg_speed:.2f} одиниць/сек")
        print(f"Максимальна швидкість: {max_speed:.2f} одиниць/сек")

        final_distances = [np.linalg.norm(drone.position - drone.target) for drone in self.drones]
        print(f"\nСередня відстань до цілі в кінці: {np.mean(final_distances):.2f} одиниць")
        print(f"Максимальна відстань до цілі в кінці: {np.max(final_distances):.2f} одиниць")
        print(f"Мінімальна відстань до цілі в кінці: {np.min(final_distances):.2f} одиниць")

if __name__ == "__main__":
    simulator = SwarmSimulator(num_drones=20)
    simulator.run_simulation(duration=15)

In [ ]:
import numpy as np
from dataclasses import dataclass, field
from typing import List
import random

# Константи симуляції
SIM_WIDTH = 150
SIM_HEIGHT = 150
FPS = 30
DT = 1/FPS

@dataclass
class DroneSpecs:
    physical_radius: float = 1.5    # Фізичний розмір дрона
    safety_margin: float = 3.0      # Додатковий буфер безпеки
    max_speed: float = 5.0          # Ще більше зменшена швидкість
    max_accel: float = 1.5          # Ще більше зменшене прискорення
    comm_range: float = 70.0        # Збільшена дальність виявлення

    @property
    def safety_radius(self):
        return self.physical_radius + self.safety_margin

@dataclass
class Drone:
    id: int
    position: np.ndarray
    velocity: np.ndarray = field(default_factory=lambda: np.zeros(2))
    acceleration: np.ndarray = field(default_factory=lambda: np.zeros(2))
    target: np.ndarray = field(default_factory=lambda: np.array([SIM_WIDTH/2, SIM_HEIGHT/2]))
    specs: DroneSpecs = field(default_factory=DroneSpecs)
    trajectory: List[np.ndarray] = field(default_factory=list)

    def update(self, acceleration: np.ndarray):
        # Жорстке обмеження прискорення
        accel_norm = np.linalg.norm(acceleration)
        if accel_norm > self.specs.max_accel:
            acceleration = acceleration / accel_norm * self.specs.max_accel

        self.acceleration = acceleration
        self.velocity += self.acceleration * DT

        # Суворе обмеження швидкості
        speed = np.linalg.norm(self.velocity)
        if speed > self.specs.max_speed:
            self.velocity = self.velocity / speed * self.specs.max_speed

        self.position += self.velocity * DT
        self.trajectory.append(self.position.copy())

class SwarmSimulator:
    def __init__(self, num_drones: int = 20):
        self.num_drones = num_drones
        self.drones = []
        self.collisions = 0
        self.time = 0.0
        self.setup_environment()

    def setup_environment(self):
        # Ініціалізація дронів з більшими інтервалами
        spacing = SIM_HEIGHT / (self.num_drones + 1)
        for i in range(self.num_drones):
            y_pos = spacing * (i + 1)
            drone = Drone(
                id=i,
                position=np.array([20.0, y_pos]),  # Початкова позиція далі від краю
                target=np.array([SIM_WIDTH-20, y_pos]),
                velocity=np.array([random.uniform(1.5, 2.5), 0])  # Дуже низька початкова швидкість
            )
            self.drones.append(drone)

    def check_collisions(self):
        for i, drone1 in enumerate(self.drones):
            for j, drone2 in enumerate(self.drones[i+1:], i+1):
                dist = np.linalg.norm(drone1.position - drone2.position)
                if dist < drone1.specs.physical_radius + drone2.specs.physical_radius:
                    self.collisions += 1

    def apply_priority_avoidance(self):
        # Сортуємо дрони за відстанню до цілі (найближчі мають пріоритет)
        sorted_drones = sorted(self.drones, key=lambda d: np.linalg.norm(d.position - d.target))

        for i, drone in enumerate(sorted_drones):
            # Вектор до цілі
            target_dir = drone.target - drone.position
            target_dist = np.linalg.norm(target_dir)
            if target_dist > 0:
                target_force = target_dir / target_dist * min(2.0, target_dist/10)
            else:
                target_force = np.zeros(2)

            # Уникнення зіткнень з вищим пріоритетом (вже обробленими дронами)
            avoidance_force = np.zeros(2)
            for other in sorted_drones[:i]:
                dist_vec = drone.position - other.position
                dist = np.linalg.norm(dist_vec)

                if dist < drone.specs.comm_range:
                    # Сильне відштовхування, якщо занадто близько
                    if dist < 2 * drone.specs.safety_radius:
                        avoidance_force += dist_vec / (dist**3 + 0.01) * 50.0
                    # Помірне відштовхування на середніх дистанціях
                    elif dist < 3 * drone.specs.safety_radius:
                        avoidance_force += dist_vec / (dist**2 + 0.01) * 10.0

            # Комбінуємо сили з пріоритетом уникнення
            total_force = 0.7 * avoidance_force + 0.3 * target_force
            drone.update(total_force)

    def run_simulation(self, duration: float = 20.0):
        num_steps = int(duration * FPS)

        for _ in range(num_steps):
            self.time += DT
            self.apply_priority_avoidance()
            self.check_collisions()

        print("=== Результати симуляції ===")
        print(f"Час симуляції: {self.time:.2f} секунд")
        print(f"Кількість дронів: {self.num_drones}")
        print(f"Загальна кількість зіткнень: {self.collisions}")
        print(f"Середня кількість зіткнень на дрон: {self.collisions/self.num_drones:.2f}")

        # Точніший розрахунок швидкостей через диференціювання траєкторій
        speeds = []
        for drone in self.drones:
            if len(drone.trajectory) > 1:
                trajectory = np.array(drone.trajectory)
                velocities = np.diff(trajectory, axis=0) / DT
                speeds.extend(np.linalg.norm(velocities, axis=1))

        if speeds:
            print(f"\nСередня швидкість дронів: {np.mean(speeds):.2f} одиниць/сек")
            print(f"Максимальна швидкість: {np.max(speeds):.2f} одиниць/сек")
        else:
            print("\nНе вдалося обчислити швидкості")

        final_distances = [np.linalg.norm(drone.position - drone.target) for drone in self.drones]
        print(f"\nСередня відстань до цілі в кінці: {np.mean(final_distances):.2f} одиниць")
        print(f"Максимальна відстань до цілі в кінці: {np.max(final_distances):.2f} одиниць")
        print(f"Мінімальна відстань до цілі в кінці: {np.min(final_distances):.2f} одиниць")

if __name__ == "__main__":
    simulator = SwarmSimulator(num_drones=20)
    simulator.run_simulation(duration=15)

In [ ]:
import numpy as np
from dataclasses import dataclass
from typing import List, Tuple
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import random

@dataclass
class DroneSpecs:
    physical_radius: float
    safety_margin: float
    max_speed: float
    max_acceleration: float
    comm_radius: float

class Drone:
    def __init__(self, drone_id: int, position: np.ndarray, specs: DroneSpecs, route: List[np.ndarray]):
        self.id = drone_id
        self.position = np.array(position, dtype=np.float64)
        self.velocity = np.zeros(2, dtype=np.float64)
        self.specs = specs
        self.route = route
        self.current_waypoint = 0
        self.path_history = [self.position.copy()]

    def update(self, neighbors: List['Drone'], dt: float):
        target_pos = self.route[self.current_waypoint]
        direction = target_pos - self.position
        distance = np.linalg.norm(direction)
        if distance < 0.1:
            self.current_waypoint = (self.current_waypoint + 1) % len(self.route)
            target_pos = self.route[self.current_waypoint]
            direction = target_pos - self.position
            distance = np.linalg.norm(direction)

        if distance > 0:
            desired_velocity = (direction / distance) * self.specs.max_speed
        else:
            desired_velocity = np.zeros(2)
        steering = desired_velocity - self.velocity
        steering = self._limit_vector(steering, self.specs.max_acceleration)

        avoidance = np.zeros(2)
        for neighbor in neighbors:
            if neighbor.id == self.id:
                continue

            delta_pos = self.position - neighbor.position
            dist = np.linalg.norm(delta_pos)
            if dist == 0:
                continue

            combined_radius = (self.specs.physical_radius + neighbor.specs.physical_radius +
                               self.specs.safety_margin + neighbor.specs.safety_margin)

            if self.id > neighbor.id:
                priority_force = self._compute_priority_force(neighbor, combined_radius)
                avoidance += priority_force

            approach_force = self._compute_approach_force(neighbor, dist, delta_pos, combined_radius)
            avoidance += approach_force

            trajectory_force = self._compute_trajectory_force(neighbor, dt, combined_radius)
            avoidance += trajectory_force

            route_force = self._compute_route_conflict_force(neighbor, combined_radius)
            avoidance += route_force

        total_acceleration = steering + avoidance
        total_acceleration = self._limit_vector(total_acceleration, self.specs.max_acceleration)

        self.velocity += total_acceleration * dt
        self.velocity = self._limit_vector(self.velocity, self.specs.max_speed)
        self.position += self.velocity * dt
        self.path_history.append(self.position.copy())

    def _limit_vector(self, vector: np.ndarray, max_magnitude: float) -> np.ndarray:
        magnitude = np.linalg.norm(vector)
        if magnitude > max_magnitude:
            return vector / magnitude * max_magnitude
        return vector

    def _compute_priority_force(self, neighbor: 'Drone', combined_radius: float) -> np.ndarray:
        delta_pos = self.position - neighbor.position
        dist = np.linalg.norm(delta_pos)
        if dist < combined_radius:
            force_mag = (combined_radius - dist) / combined_radius * self.specs.max_acceleration
            return (delta_pos / dist) * force_mag
        return np.zeros(2)

    def _compute_approach_force(self, neighbor: 'Drone', dist: float, delta_pos: np.ndarray, combined_radius: float) -> np.ndarray:
        rel_velocity = self.velocity - neighbor.velocity
        approach_speed = np.dot(rel_velocity, delta_pos) / dist
        if approach_speed <= 0:
            return np.zeros(2)
        ttc = dist / approach_speed
        if ttc < 2.0:
            force_mag = (1.0 / (ttc + 1e-6)) * self.specs.max_acceleration
            return (delta_pos / dist) * force_mag
        return np.zeros(2)

    def _compute_trajectory_force(self, neighbor: 'Drone', dt: float, combined_radius: float) -> np.ndarray:
        horizon = 5
        min_dist = float('inf')
        closest_pos = None
        for t in range(1, horizon + 1):
            future_self = self.position + self.velocity * t * dt
            future_neighbor = neighbor.position + neighbor.velocity * t * dt
            delta = future_self - future_neighbor
            dist = np.linalg.norm(delta)
            if dist < min_dist:
                min_dist = dist
                closest_pos = future_self
        if min_dist < combined_radius:
            direction = self.position - closest_pos
            dist = np.linalg.norm(direction)
            if dist == 0:
                return np.zeros(2)
            force_mag = (combined_radius - min_dist) / combined_radius * self.specs.max_acceleration
            return (direction / dist) * force_mag
        return np.zeros(2)

    def _compute_route_conflict_force(self, neighbor: 'Drone', combined_radius: float) -> np.ndarray:
        look_ahead = 5
        min_dist = float('inf')
        conflict_point = None
        own_route = self.route[self.current_waypoint : self.current_waypoint + look_ahead]
        neighbor_route = neighbor.route[neighbor.current_waypoint : neighbor.current_waypoint + look_ahead]
        for wp_self in own_route:
            for wp_neighbor in neighbor_route:
                dist = np.linalg.norm(wp_self - wp_neighbor)
                if dist < min_dist:
                    min_dist = dist
                    conflict_point = wp_neighbor
        if min_dist < combined_radius:
            direction = self.position - conflict_point
            dist = np.linalg.norm(direction)
            if dist == 0:
                return np.zeros(2)
            force_mag = (combined_radius - min_dist) / combined_radius * self.specs.max_acceleration
            return (direction / dist) * force_mag
        return np.zeros(2)

class SwarmSimulator:
    def __init__(self, area_size: Tuple[float, float], num_drones: int, specs: DroneSpecs, simulation_time: float, dt: float):
        self.area_width, self.area_height = area_size
        self.num_drones = num_drones
        self.specs = specs
        self.dt = dt
        self.simulation_time = simulation_time
        self.drones = []
        self.collisions = 0
        self.positions_history = []
        self._initialize_drones()

    def _initialize_drones(self):
        rows = int(np.ceil(np.sqrt(self.num_drones)))
        cols = int(np.ceil(self.num_drones / rows))
        sector_width = self.area_width / cols
        sector_height = self.area_height / rows

        for i in range(self.num_drones):
            row = i // cols
            col = i % cols
            x_start = col * sector_width
            y_start = row * sector_height

            route = []
            num_rows_in_sector = 5
            num_cols_in_sector = 5
            y_step = sector_height / num_rows_in_sector
            x_step = sector_width / num_cols_in_sector

            for r in range(num_rows_in_sector):
                y = y_start + (r + 0.5) * y_step
                if r % 2 == 0:
                    x_values = [x_start + (c + 0.5) * x_step for c in range(num_cols_in_sector)]
                else:
                    x_values = [x_start + (num_cols_in_sector - c - 0.5) * x_step for c in range(num_cols_in_sector)]
                for x in x_values:
                    route.append(np.array([x, y]))

            start_pos = route[0]
            self.drones.append(Drone(i, start_pos, self.specs, route))

    def run(self, visualize=False):
        self.time = 0.0
        self.collisions = 0
        self.positions_history = []
        num_steps = int(self.simulation_time / self.dt)

        if visualize:
            fig, ax = plt.subplots(figsize=(8, 8))
            ax.set_xlim(0, self.area_width)
            ax.set_ylim(0, self.area_height)
            scat = ax.scatter([], [], c='red', s=50)
            lines = [ax.plot([], [], 'b-', alpha=0.3)[0] for _ in self.drones]
            annotations = [ax.text(0, 0, '', fontsize=8) for _ in self.drones]

            def init():
                scat.set_offsets(np.empty((0, 2)))
                for line in lines:
                    line.set_data([], [])
                for ann in annotations:
                    ann.set_text('')
                return [scat] + lines + annotations

            def update(frame):
                if self.time >= self.simulation_time:
                    return [scat] + lines + annotations

                neighbors = {}
                for drone in self.drones:
                    neighbors[drone.id] = [other for other in self.drones if
                                          np.linalg.norm(drone.position - other.position) <= self.specs.comm_radius and
                                          drone.id != other.id]

                for drone in self.drones:
                    drone.update(neighbors[drone.id], self.dt)

                current_collisions = 0
                for i in range(len(self.drones)):
                    for j in range(i+1, len(self.drones)):
                        d1 = self.drones[i]
                        d2 = self.drones[j]
                        dist = np.linalg.norm(d1.position - d2.position)
                        if dist < (d1.specs.physical_radius + d2.specs.physical_radius):
                            current_collisions += 1
                self.collisions += current_collisions

                self.positions_history.append([drone.position.copy() for drone in self.drones])
                self.time += self.dt

                positions = np.array([drone.position for drone in self.drones])
                scat.set_offsets(positions)

                for idx, drone in enumerate(self.drones):
                    path = np.array(drone.path_history)
                    lines[idx].set_data(path[:,0], path[:,1])
                    annotations[idx].set_position(drone.position + np.array([0.5, 0.5]))
                    annotations[idx].set_text(f'{drone.id}')

                return [scat] + lines + annotations

            ani = FuncAnimation(fig, update, frames=num_steps, init_func=init, blit=False, interval=50)
            plt.close()
            return HTML(ani.to_jshtml())
        else:
            for _ in range(num_steps):
                neighbors = {}
                for drone in self.drones:
                    neighbors[drone.id] = [other for other in self.drones if
                                          np.linalg.norm(drone.position - other.position) <= self.specs.comm_radius and
                                          drone.id != other.id]
                for drone in self.drones:
                    drone.update(neighbors[drone.id], self.dt)
                self.time += self.dt
                current_collisions = 0
                for i in range(len(self.drones)):
                    for j in range(i+1, len(self.drones)):
                        d1 = self.drones[i]
                        d2 = self.drones[j]
                        dist = np.linalg.norm(d1.position - d2.position)
                        if dist < (d1.specs.physical_radius + d2.specs.physical_radius):
                            current_collisions += 1
                self.collisions += current_collisions
            return None

    def print_statistics(self):
        print(f"Simulation Results:")
        print(f"Total simulation time: {self.simulation_time:.1f} s")
        print(f"Number of drones: {self.num_drones}")
        print(f"Total collisions detected: {self.collisions}")
        avg_speed = np.mean([np.linalg.norm(drone.velocity) for drone in self.drones])
        max_speed = np.max([np.linalg.norm(drone.velocity) for drone in self.drones])
        print(f"Average speed: {avg_speed:.2f} m/s")
        print(f"Maximum speed: {max_speed:.2f} m/s")
        total_remaining = 0
        for drone in self.drones:
            remaining = 0
            prev = drone.position
            for wp in drone.route[drone.current_waypoint:]:
                remaining += np.linalg.norm(wp - prev)
                prev = wp
            total_remaining += remaining
        avg_remaining = total_remaining / self.num_drones
        print(f"Average remaining distance: {avg_remaining:.2f} m")

# Example usage
specs = DroneSpecs(
    physical_radius=0.5,
    safety_margin=1.0,
    max_speed=2.0,
    max_acceleration=1.5,
    comm_radius=15.0
)

sim = SwarmSimulator(
    area_size=(50, 50),
    num_drones=4,
    specs=specs,
    simulation_time=20.0,
    dt=0.1
)

anim = sim.run(visualize=True)
sim.print_statistics()
anim

In [ ]:
import numpy as np
from dataclasses import dataclass
from typing import List, Tuple
import random

@dataclass
class DroneSpecs:
    physical_radius: float
    safety_margin: float
    max_speed: float
    max_acceleration: float
    comm_radius: float

class Drone:
    def __init__(self, drone_id: int, position: np.ndarray, specs: DroneSpecs, route: List[np.ndarray]):
        self.id = drone_id
        self.position = np.array(position, dtype=np.float64)
        self.velocity = np.zeros(2, dtype=np.float64)
        self.specs = specs
        self.route = route
        self.current_waypoint = 0
        self.path_history = [self.position.copy()]

    def update(self, neighbors: List['Drone'], dt: float):
        target_pos = self.route[self.current_waypoint]
        direction = target_pos - self.position
        distance = np.linalg.norm(direction)
        if distance < 0.1:
            self.current_waypoint = (self.current_waypoint + 1) % len(self.route)
            target_pos = self.route[self.current_waypoint]
            direction = target_pos - self.position
            distance = np.linalg.norm(direction)

        if distance > 0:
            desired_velocity = (direction / distance) * self.specs.max_speed
        else:
            desired_velocity = np.zeros(2)
        steering = desired_velocity - self.velocity
        steering = self._limit_vector(steering, self.specs.max_acceleration)

        avoidance = np.zeros(2)
        for neighbor in neighbors:
            if neighbor.id == self.id:
                continue

            delta_pos = self.position - neighbor.position
            dist = np.linalg.norm(delta_pos)
            if dist == 0:
                continue

            combined_radius = (self.specs.physical_radius + neighbor.specs.physical_radius +
                               self.specs.safety_margin + neighbor.specs.safety_margin)

            if self.id > neighbor.id:
                priority_force = self._compute_priority_force(neighbor, combined_radius)
                avoidance += priority_force

            approach_force = self._compute_approach_force(neighbor, dist, delta_pos, combined_radius)
            avoidance += approach_force

            trajectory_force = self._compute_trajectory_force(neighbor, dt, combined_radius)
            avoidance += trajectory_force

            route_force = self._compute_route_conflict_force(neighbor, combined_radius)
            avoidance += route_force

        total_acceleration = steering + avoidance
        total_acceleration = self._limit_vector(total_acceleration, self.specs.max_acceleration)

        self.velocity += total_acceleration * dt
        self.velocity = self._limit_vector(self.velocity, self.specs.max_speed)
        self.position += self.velocity * dt
        self.path_history.append(self.position.copy())

    def _limit_vector(self, vector: np.ndarray, max_magnitude: float) -> np.ndarray:
        magnitude = np.linalg.norm(vector)
        if magnitude > max_magnitude:
            return vector / magnitude * max_magnitude
        return vector

    def _compute_priority_force(self, neighbor: 'Drone', combined_radius: float) -> np.ndarray:
        delta_pos = self.position - neighbor.position
        dist = np.linalg.norm(delta_pos)
        if dist < combined_radius:
            force_mag = (combined_radius - dist) / combined_radius * self.specs.max_acceleration
            return (delta_pos / dist) * force_mag
        return np.zeros(2)

    def _compute_approach_force(self, neighbor: 'Drone', dist: float, delta_pos: np.ndarray, combined_radius: float) -> np.ndarray:
        rel_velocity = self.velocity - neighbor.velocity
        approach_speed = np.dot(rel_velocity, delta_pos) / dist
        if approach_speed <= 0:
            return np.zeros(2)
        ttc = dist / approach_speed
        if ttc < 2.0:
            force_mag = (1.0 / (ttc + 1e-6)) * self.specs.max_acceleration
            return (delta_pos / dist) * force_mag
        return np.zeros(2)

    def _compute_trajectory_force(self, neighbor: 'Drone', dt: float, combined_radius: float) -> np.ndarray:
        horizon = 5
        min_dist = float('inf')
        closest_pos = None
        for t in range(1, horizon + 1):
            future_self = self.position + self.velocity * t * dt
            future_neighbor = neighbor.position + neighbor.velocity * t * dt
            delta = future_self - future_neighbor
            dist = np.linalg.norm(delta)
            if dist < min_dist:
                min_dist = dist
                closest_pos = future_self
        if min_dist < combined_radius:
            direction = self.position - closest_pos
            dist = np.linalg.norm(direction)
            if dist == 0:
                return np.zeros(2)
            force_mag = (combined_radius - min_dist) / combined_radius * self.specs.max_acceleration
            return (direction / dist) * force_mag
        return np.zeros(2)

    def _compute_route_conflict_force(self, neighbor: 'Drone', combined_radius: float) -> np.ndarray:
        look_ahead = 5
        min_dist = float('inf')
        conflict_point = None
        own_route = self.route[self.current_waypoint : self.current_waypoint + look_ahead]
        neighbor_route = neighbor.route[neighbor.current_waypoint : neighbor.current_waypoint + look_ahead]
        for wp_self in own_route:
            for wp_neighbor in neighbor_route:
                dist = np.linalg.norm(wp_self - wp_neighbor)
                if dist < min_dist:
                    min_dist = dist
                    conflict_point = wp_neighbor
        if min_dist < combined_radius:
            direction = self.position - conflict_point
            dist = np.linalg.norm(direction)
            if dist == 0:
                return np.zeros(2)
            force_mag = (combined_radius - min_dist) / combined_radius * self.specs.max_acceleration
            return (direction / dist) * force_mag
        return np.zeros(2)

class SwarmSimulator:
    def __init__(self, area_size: Tuple[float, float], num_drones: int, specs: DroneSpecs, simulation_time: float, dt: float):
        self.area_width, self.area_height = area_size
        self.num_drones = num_drones
        self.specs = specs
        self.dt = dt
        self.simulation_time = simulation_time
        self.drones = []
        self.collisions = 0
        self._initialize_drones()

    def _initialize_drones(self):
        rows = int(np.ceil(np.sqrt(self.num_drones)))
        cols = int(np.ceil(self.num_drones / rows))
        sector_width = self.area_width / cols
        sector_height = self.area_height / rows

        for i in range(self.num_drones):
            row = i // cols
            col = i % cols
            x_start = col * sector_width
            y_start = row * sector_height

            route = []
            num_rows_in_sector = 5
            num_cols_in_sector = 5
            y_step = sector_height / num_rows_in_sector
            x_step = sector_width / num_cols_in_sector

            for r in range(num_rows_in_sector):
                y = y_start + (r + 0.5) * y_step
                if r % 2 == 0:
                    x_values = [x_start + (c + 0.5) * x_step for c in range(num_cols_in_sector)]
                else:
                    x_values = [x_start + (num_cols_in_sector - c - 0.5) * x_step for c in range(num_cols_in_sector)]
                for x in x_values:
                    route.append(np.array([x, y]))

            start_pos = route[0]
            self.drones.append(Drone(i, start_pos, self.specs, route))

    def run(self):
        self.time = 0.0
        self.collisions = 0
        num_steps = int(self.simulation_time / self.dt)

        for _ in range(num_steps):
            neighbors = {}
            for drone in self.drones:
                neighbors[drone.id] = [other for other in self.drones if
                                      np.linalg.norm(drone.position - other.position) <= self.specs.comm_radius and
                                      drone.id != other.id]

            for drone in self.drones:
                drone.update(neighbors[drone.id], self.dt)

            current_collisions = 0
            for i in range(len(self.drones)):
                for j in range(i+1, len(self.drones)):
                    d1 = self.drones[i]
                    d2 = self.drones[j]
                    dist = np.linalg.norm(d1.position - d2.position)
                    if dist < (d1.specs.physical_radius + d2.specs.physical_radius):
                        current_collisions += 1
            self.collisions += current_collisions
            self.time += self.dt

    def print_statistics(self):
        print(f"Simulation Results:")
        print(f"Total simulation time: {self.simulation_time:.1f} s")
        print(f"Number of drones: {self.num_drones}")
        print(f"Total collisions detected: {self.collisions}")
        avg_speed = np.mean([np.linalg.norm(drone.velocity) for drone in self.drones])
        max_speed = np.max([np.linalg.norm(drone.velocity) for drone in self.drones])
        print(f"Average speed: {avg_speed:.2f} m/s")
        print(f"Maximum speed: {max_speed:.2f} m/s")
        total_remaining = 0
        for drone in self.drones:
            remaining = 0
            prev = drone.position
            for wp in drone.route[drone.current_waypoint:]:
                remaining += np.linalg.norm(wp - prev)
                prev = wp
            total_remaining += remaining
        avg_remaining = total_remaining / self.num_drones
        print(f"Average remaining distance: {avg_remaining:.2f} m")

# Приклад використання
specs = DroneSpecs(
    physical_radius=0.5,
    safety_margin=1.0,
    max_speed=2.0,
    max_acceleration=1.5,
    comm_radius=15.0
)

sim = SwarmSimulator(
    area_size=(50, 50),
    num_drones=4,
    specs=specs,
    simulation_time=150.0,
    dt=0.1
)

sim.run()
sim.print_statistics()

In [ ]:
import numpy as np
from dataclasses import dataclass
from abc import ABC, abstractmethod
from typing import List, Dict
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist

# ==================== БАЗОВІ КЛАСИ ====================
@dataclass
class AgentSpecs:
    """Специфікації агента з фізичними параметрами"""
    physical_radius: float
    safety_margin: float
    max_speed: float
    max_acceleration: float
    comm_radius: float

class SteeringAlgorithm(ABC):
    """Інтерфейс алгоритму керування"""
    @abstractmethod
    def compute_steering(self, agent, neighbors: List['Agent'], dt: float) -> np.ndarray:
        pass

class Agent:
    """Клас агента з механікою руху"""
    def __init__(self, agent_id: int, position: np.ndarray, specs: AgentSpecs, algorithm: SteeringAlgorithm):
        self.id = agent_id
        self.position = position
        self.velocity = np.zeros(2)
        self.specs = specs
        self.algorithm = algorithm
        self.path_history = [position.copy()]
        self.acceleration_history = []

    def update(self, neighbors: List['Agent'], dt: float):
        steering = self.algorithm.compute_steering(self, neighbors, dt)
        steering = self._limit_vector(steering, self.specs.max_acceleration)
        self.velocity = self._limit_vector(self.velocity + steering * dt, self.specs.max_speed)
        self.position += self.velocity * dt
        self.path_history.append(self.position.copy())
        self.acceleration_history.append(np.linalg.norm(steering))

    def _limit_vector(self, vector: np.ndarray, max_magnitude: float) -> np.ndarray:
        magnitude = np.linalg.norm(vector)
        return vector if magnitude == 0 else vector * min(max_magnitude/magnitude, 1)

# ==================== АЛГОРИТМИ КЕРУВАННЯ ====================
class VicsekAlgorithm(SteeringAlgorithm):
    """Модель Вічека з параметром шуму"""
    def __init__(self, noise: float = 0.1):
        self.noise = noise

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        if not neighbors:
            return np.zeros(2)
        avg_velocity = np.mean([n.velocity for n in neighbors], axis=0)
        noise = np.random.normal(scale=self.noise, size=2)
        return (avg_velocity + noise - agent.velocity) / dt

class BoidsAlgorithm(SteeringAlgorithm):
    """Алгоритм Boids з трьома правилами"""
    def __init__(self, sep_weight: float = 1.5, align_weight: float = 1.0, coh_weight: float = 1.0):
        self.weights = np.array([sep_weight, align_weight, coh_weight])

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        if not neighbors:
            return np.zeros(2)

        positions = np.array([n.position for n in neighbors])
        velocities = np.array([n.velocity for n in neighbors])

        # Правила
        sep = self._separation(agent.position, positions)
        align = self._alignment(agent.velocity, velocities)
        coh = self._cohesion(agent.position, positions)

        return self.weights @ np.array([sep, align, coh])

    def _separation(self, pos: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        diff = pos - neighbors
        distances = np.linalg.norm(diff, axis=1)[:, np.newaxis]
        return np.mean(diff / (distances + 1e-6), axis=0)

    def _alignment(self, vel: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        return np.mean(neighbors, axis=0) - vel

    def _cohesion(self, pos: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        return np.mean(neighbors, axis=0) - pos

class PotentialFieldAlgorithm(SteeringAlgorithm):
    """Метод потенційних полів з ціллю"""
    def __init__(self, goal: np.ndarray = np.array([50,50]), alpha: float = 0.5, beta: float = 1.0):
        self.goal = goal
        self.alpha = alpha  # Коефіцієнт притягання
        self.beta = beta    # Коефіцієнт відштовхування

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        # Притягання до цілі
        to_goal = self.goal - agent.position
        attraction = self.alpha * to_goal

        # Відштовхування від сусідів
        repulsion = np.zeros(2)
        for n in neighbors:
            vec = agent.position - n.position
            dist = np.linalg.norm(vec)
            if dist > 0:
                repulsion += self.beta * vec / dist**2

        return attraction + repulsion - agent.velocity/dt

# ==================== СИСТЕМА СИМУЛЯЦІЇ ====================
class SwarmSimulator:
    def __init__(self, area_size: float, num_agents: int, specs: AgentSpecs,
                 algorithm: SteeringAlgorithm, init_pos: str = 'random'):
        self.agents = self._initialize_agents(num_agents, specs, algorithm, area_size, init_pos)
        self.stats = {
            'collisions': 0,
            'avg_speed': [],
            'max_speed': [],
            'avg_accel': [],
            'neighbor_dist': []
        }

    def _initialize_agents(self, num_agents: int, specs: AgentSpecs,
                          algorithm: SteeringAlgorithm, area_size: float, init_pos: str) -> List[Agent]:
        if init_pos == 'grid':
            grid_size = int(np.ceil(np.sqrt(num_agents)))
            positions = np.meshgrid(np.linspace(0, area_size, grid_size),
                                   np.linspace(0, area_size, grid_size))
            positions = np.vstack(positions).reshape(2,-1).T[:num_agents]
        else:
            positions = np.random.rand(num_agents, 2) * area_size

        return [Agent(i, pos, specs, algorithm) for i, pos in enumerate(positions)]

    def run(self, steps: int, dt: float):
        for _ in range(steps):
            self._update_agents(dt)
            self._collect_statistics()
            self._detect_collisions()

    def _update_agents(self, dt: float):
        for agent in self.agents:
            neighbors = [a for a in self.agents
                        if a != agent and np.linalg.norm(a.position - agent.position) <= agent.specs.comm_radius]
            agent.update(neighbors, dt)

    def _detect_collisions(self):
        positions = np.array([a.position for a in self.agents])
        dist_matrix = cdist(positions, positions)
        np.fill_diagonal(dist_matrix, np.inf)
        collision_pairs = np.argwhere(dist_matrix < self.agents[0].specs.physical_radius * 2)
        self.stats['collisions'] += len(collision_pairs)//2

    def _collect_statistics(self):
        speeds = [np.linalg.norm(a.velocity) for a in self.agents]
        accels = [a.acceleration_history[-1] if a.acceleration_history else 0 for a in self.agents]

        self.stats['avg_speed'].append(np.mean(speeds))
        self.stats['max_speed'].append(np.max(speeds))
        self.stats['avg_accel'].append(np.mean(accels))

        # Обчислення середньої відстані до сусідів
        neighbor_dists = []
        for a in self.agents:
            dists = [np.linalg.norm(n.position - a.position) for n in self.agents if n != a]
            neighbor_dists.extend(dists)
        self.stats['neighbor_dist'].append(np.mean(neighbor_dists) if neighbor_dists else 0)

    def visualize_trajectories(self):
        plt.figure(figsize=(10, 10))
        for agent in self.agents:
            path = np.array(agent.path_history)
            plt.plot(path[:,0], path[:,1], alpha=0.5)
            plt.scatter(path[-1,0], path[-1,1], c='red', s=30)
        plt.title("Траєкторії агентів")
        plt.xlabel("X")
        plt.ylabel("Y")
        plt.grid(True)
        plt.show()

# ==================== ПРИКЛАД ВИКОРИСТАННЯ ====================
if __name__ == "__main__":
    # Конфігурація
    SPECS = AgentSpecs(
        physical_radius=0.5,
        safety_margin=1.0,
        max_speed=2.0,
        max_acceleration=1.5,
        comm_radius=15.0
    )

    ALGORITHMS = {
        'Vicsek': VicsekAlgorithm(noise=0.3),
        'Boids': BoidsAlgorithm(sep_weight=1.5, align_weight=0.8, coh_weight=0.5),
        'Potential Field': PotentialFieldAlgorithm(alpha=0.3, beta=0.8)
    }

    # Запуск симуляцій
    results = {}
    for name, algo in ALGORITHMS.items():
        sim = SwarmSimulator(
            area_size=100,
            num_agents=30,
            specs=SPECS,
            algorithm=algo,
            init_pos='random'
        )
        sim.run(steps=200, dt=0.1)
        results[name] = {
            'collisions': sim.stats['collisions'],
            'avg_speed': np.mean(sim.stats['avg_speed']),
            'neighbor_dist': np.mean(sim.stats['neighbor_dist'])
        }
        sim.visualize_trajectories()

    # Порівняльна візуалізація
    fig, axs = plt.subplots(1, 2, figsize=(15,5))

    # Графік зіткнень
    axs[0].bar(results.keys(), [v['collisions'] for v in results.values()])
    axs[0].set_title('Кількість зіткнень')

    # Графік щільності
    axs[1].bar(results.keys(), [v['neighbor_dist'] for v in results.values()])
    axs[1].set_title('Середня відстань між сусідами')

    plt.tight_layout()
    plt.show()

In [ ]:
import numpy as np
from dataclasses import dataclass
from abc import ABC, abstractmethod
from typing import List, Dict, Tuple
import random

# ==================== БАЗОВІ КЛАСИ ====================
@dataclass
class AgentSpecs:
    physical_radius: float
    safety_margin: float
    max_speed: float
    max_acceleration: float
    comm_radius: float

class SteeringAlgorithm(ABC):
    @abstractmethod
    def compute_steering(self, agent, neighbors: List['Agent'], dt: float) -> np.ndarray:
        pass

class Agent:
    def __init__(self, agent_id: int, position: np.ndarray, specs: AgentSpecs, algorithm: SteeringAlgorithm, route: List[np.ndarray]):
        self.id = agent_id
        self.position = position
        self.velocity = np.zeros(2)
        self.specs = specs
        self.algorithm = algorithm
        self.route = route
        self.current_waypoint = 0
        self.path_history = [position.copy()]

    def update(self, neighbors: List['Agent'], dt: float):
        steering = self.algorithm.compute_steering(self, neighbors, dt)
        steering = self._limit_vector(steering, self.specs.max_acceleration)
        self.velocity = self._limit_vector(self.velocity + steering * dt, self.specs.max_speed)
        self.position += self.velocity * dt
        self.path_history.append(self.position.copy())

    def _limit_vector(self, vector: np.ndarray, max_magnitude: float) -> np.ndarray:
        magnitude = np.linalg.norm(vector)
        return vector if magnitude == 0 else vector * min(max_magnitude/magnitude, 1)

# ==================== РЕАЛІЗАЦІЇ АЛГОРИТМІВ ====================
class VicsekAlgorithm(SteeringAlgorithm):
    def __init__(self, noise: float = 0.1):
        self.noise = noise

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        if not neighbors:
            return np.zeros(2)
        avg_velocity = np.mean([n.velocity for n in neighbors], axis=0)
        noise = np.random.normal(scale=self.noise, size=2)
        return (avg_velocity + noise - agent.velocity) / dt

class BoidsAlgorithm(SteeringAlgorithm):
    def __init__(self, sep_weight: float = 1.5, align_weight: float = 1.0, coh_weight: float = 1.0):
        self.weights = np.array([sep_weight, align_weight, coh_weight])

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        if not neighbors:
            return np.zeros(2)

        positions = np.array([n.position for n in neighbors])
        velocities = np.array([n.velocity for n in neighbors])

        sep = self._separation(agent.position, positions)
        align = self._alignment(agent.velocity, velocities)
        coh = self._cohesion(agent.position, positions)

        return self.weights @ np.array([sep, align, coh])

    def _separation(self, pos: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        diff = pos - neighbors
        distances = np.linalg.norm(diff, axis=1)[:, np.newaxis]
        return np.mean(diff / (distances + 1e-6), axis=0)

    def _alignment(self, vel: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        return np.mean(neighbors, axis=0) - vel

    def _cohesion(self, pos: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        return np.mean(neighbors, axis=0) - pos

class PotentialFieldAlgorithm(SteeringAlgorithm):
    def __init__(self, goal: np.ndarray = np.array([25,25]), alpha: float = 0.5, beta: float = 1.0):
        self.goal = goal
        self.alpha = alpha
        self.beta = beta

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        to_goal = self.goal - agent.position
        attraction = self.alpha * to_goal

        repulsion = np.zeros(2)
        for n in neighbors:
            vec = agent.position - n.position
            dist = np.linalg.norm(vec)
            if dist > 0:
                repulsion += self.beta * vec / dist**2

        return attraction + repulsion - agent.velocity/dt

class DroneNavigationAlgorithm(SteeringAlgorithm):
    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        target_pos = agent.route[agent.current_waypoint]
        direction = target_pos - agent.position
        distance = np.linalg.norm(direction)

        if distance < 0.1:
            agent.current_waypoint = (agent.current_waypoint + 1) % len(agent.route)
            target_pos = agent.route[agent.current_waypoint]
            direction = target_pos - agent.position
            distance = np.linalg.norm(direction)

        desired_velocity = (direction / distance) * agent.specs.max_speed if distance > 0 else np.zeros(2)
        steering = desired_velocity - agent.velocity
        steering = agent._limit_vector(steering, agent.specs.max_acceleration)

        avoidance = np.zeros(2)
        for neighbor in neighbors:
            if neighbor.id == agent.id:
                continue

            delta_pos = agent.position - neighbor.position
            dist = np.linalg.norm(delta_pos)
            if dist == 0:
                continue

            combined_radius = (agent.specs.physical_radius + neighbor.specs.physical_radius +
                             agent.specs.safety_margin + neighbor.specs.safety_margin)

            if agent.id > neighbor.id:
                avoidance += self._priority_force(agent, neighbor, delta_pos, dist, combined_radius)

            avoidance += self._approach_force(agent, neighbor, delta_pos, dist, combined_radius)
            avoidance += self._trajectory_force(agent, neighbor, dt, combined_radius)
            avoidance += self._route_conflict_force(agent, neighbor, combined_radius)

        return steering + avoidance

    def _priority_force(self, agent, neighbor, delta_pos, dist, combined_radius):
        if dist < combined_radius:
            force_mag = (combined_radius - dist)/combined_radius * agent.specs.max_acceleration
            return (delta_pos/dist) * force_mag
        return np.zeros(2)

    def _approach_force(self, agent, neighbor, delta_pos, dist, combined_radius):
        rel_velocity = agent.velocity - neighbor.velocity
        approach_speed = np.dot(rel_velocity, delta_pos)/dist
        if approach_speed <= 0: return np.zeros(2)
        ttc = dist/approach_speed
        return (delta_pos/dist) * (1.0/(ttc + 1e-6)) * agent.specs.max_acceleration if ttc < 2.0 else np.zeros(2)

    def _trajectory_force(self, agent, neighbor, dt, combined_radius):
        horizon = 5
        min_dist = float('inf')
        for t in range(1, horizon+1):
            future_self = agent.position + agent.velocity*t*dt
            future_neighbor = neighbor.position + neighbor.velocity*t*dt
            dist = np.linalg.norm(future_self - future_neighbor)
            if dist < min_dist: min_dist = dist
        if min_dist < combined_radius:
            direction = agent.position - future_neighbor
            return (direction/np.linalg.norm(direction)) * ((combined_radius - min_dist)/combined_radius) * agent.specs.max_acceleration
        return np.zeros(2)

    def _route_conflict_force(self, agent, neighbor, combined_radius):
        look_ahead = 5
        min_dist = float('inf')
        for wp_self in agent.route[agent.current_waypoint : agent.current_waypoint+look_ahead]:
            for wp_neighbor in neighbor.route[neighbor.current_waypoint : neighbor.current_waypoint+look_ahead]:
                dist = np.linalg.norm(wp_self - wp_neighbor)
                if dist < min_dist: min_dist = dist
        if min_dist < combined_radius:
            direction = agent.position - wp_neighbor
            return (direction/np.linalg.norm(direction)) * ((combined_radius - min_dist)/combined_radius) * agent.specs.max_acceleration
        return np.zeros(2)

# ==================== СИСТЕМА СИМУЛЯЦІЇ ====================
class SwarmSimulator:
    def __init__(self, area_size: float, num_agents: int, specs: AgentSpecs, algorithm, init_pos: str = 'grid'):
        self.agents = self._init_agents(num_agents, specs, algorithm, area_size, init_pos)
        self.collisions = 0

    def _init_agents(self, num_agents, specs, algorithm, area_size, init_pos):
        agents = []
        rows = int(np.ceil(np.sqrt(num_agents)))
        cols = int(np.ceil(num_agents/rows))
        sector_size = area_size / max(rows, cols)

        for i in range(num_agents):
            route = self._generate_snake_route(i, sector_size, rows, cols)
            start_pos = route[0]
            agents.append(Agent(i, start_pos, specs, algorithm, route))
        return agents

    def _generate_snake_route(self, drone_id, sector_size, rows, cols):
        row = drone_id // cols
        col = drone_id % cols
        x_start = col * sector_size
        y_start = row * sector_size

        route = []
        steps = 5
        step_size = sector_size/steps
        for i in range(steps):
            y = y_start + (i + 0.5)*step_size
            x_values = [x_start + (j + 0.5)*step_size for j in range(steps)] if i%2==0 else reversed([x_start + (j + 0.5)*step_size for j in range(steps)])
            for x in x_values:
                route.append(np.array([x, y]))
        return route

    def run(self, steps: int, dt: float):
        for _ in range(steps):
            self._update_step(dt)
            self._check_collisions()

    def _update_step(self, dt):
        neighbors = [[a for a in self.agents if a!=agent and np.linalg.norm(a.position-agent.position)<=agent.specs.comm_radius] for agent in self.agents]
        for agent, n in zip(self.agents, neighbors):
            agent.update(n, dt)

    def _check_collisions(self):
        positions = np.array([a.position for a in self.agents])
        dist_matrix = np.linalg.norm(positions[:, None] - positions, axis=2)
        np.fill_diagonal(dist_matrix, np.inf)
        self.collisions += np.sum(dist_matrix < 2*self.agents[0].specs.physical_radius)//2

# ==================== ПОРІВНЯННЯ АЛГОРИТМІВ ====================
specs = AgentSpecs(
    physical_radius=0.5,
    safety_margin=1.0,
    max_speed=2.0,
    max_acceleration=1.5,
    comm_radius=15.0
)

algorithms = {
    "Vicsek": VicsekAlgorithm(noise=0.3),
    "Boids": BoidsAlgorithm(sep_weight=1.5, align_weight=0.8, coh_weight=0.5),
    "Potential Field": PotentialFieldAlgorithm(alpha=0.3, beta=0.8),
    "Drone Navigation": DroneNavigationAlgorithm()
}

print("Порівняння алгоритмів керування:")
print("--------------------------------")
for name, algo in algorithms.items():
    sim = SwarmSimulator(
        area_size=500,
        num_agents=40,
        specs=specs,
        algorithm=algo  # Передаємо екземпляр алгоритму безпосередньо
    )
    sim.run(steps=1500, dt=0.1)
    print(f"{name}: {sim.collisions} зіткнень")

In [ ]:
import numpy as np
from dataclasses import dataclass
from abc import ABC, abstractmethod
from typing import List, Dict, Tuple, Optional
import random

# ==================== БАЗОВІ КЛАСИ ====================
@dataclass
class AgentSpecs:
    physical_radius: float
    safety_margin: float
    max_speed: float
    max_acceleration: float
    comm_radius: float

class SteeringAlgorithm(ABC):
    @abstractmethod
    def compute_steering(self, agent, neighbors: List['Agent'], dt: float) -> np.ndarray:
        pass

class Agent:
    def __init__(self, agent_id: int, position: np.ndarray, specs: AgentSpecs, algorithm: SteeringAlgorithm, route: List[np.ndarray]):
        self.id = agent_id
        self.position = position
        self.velocity = np.zeros(2)
        self.specs = specs
        self.algorithm = algorithm
        self.route = route
        self.current_waypoint = 0
        self.path_history = [position.copy()]

    def update(self, neighbors: List['Agent'], dt: float, area_size: float):
        steering = self.algorithm.compute_steering(self, neighbors, dt)
        steering = self._limit_vector(steering, self.specs.max_acceleration)
        self.velocity = self._limit_vector(self.velocity + steering * dt, self.specs.max_speed)
        self.position += self.velocity * dt
        self._enforce_boundaries(area_size)
        self.path_history.append(self.position.copy())

    def _limit_vector(self, vector: np.ndarray, max_magnitude: float) -> np.ndarray:
        magnitude = np.linalg.norm(vector)
        return vector if magnitude == 0 else vector * min(max_magnitude/magnitude, 1)

    def _enforce_boundaries(self, area_size: float):
        """Зберігає агентів у межах області симуляції"""
        self.position = np.clip(self.position, 0, area_size)

# ==================== РЕАЛІЗАЦІЇ АЛГОРИТМІВ ====================
class VicsekAlgorithm(SteeringAlgorithm):
    def __init__(self, noise: float = 0.1):
        self.noise = noise

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        if not neighbors:
            return np.zeros(2)
        avg_velocity = np.mean([n.velocity for n in neighbors], axis=0)
        noise = np.random.normal(scale=self.noise, size=2)
        return (avg_velocity + noise - agent.velocity) / dt

class BoidsAlgorithm(SteeringAlgorithm):
    def __init__(self, sep_weight: float = 1.5, align_weight: float = 1.0, coh_weight: float = 1.0):
        self.weights = np.array([sep_weight, align_weight, coh_weight])

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        if not neighbors:
            return np.zeros(2)

        positions = np.array([n.position for n in neighbors])
        velocities = np.array([n.velocity for n in neighbors])

        sep = self._separation(agent.position, positions)
        align = self._alignment(agent.velocity, velocities)
        coh = self._cohesion(agent.position, positions)

        return self.weights @ np.array([sep, align, coh])

    def _separation(self, pos: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        diff = pos - neighbors
        distances = np.linalg.norm(diff, axis=1)[:, np.newaxis]
        return np.mean(diff / (distances + 1e-6), axis=0)

    def _alignment(self, vel: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        return np.mean(neighbors, axis=0) - vel

    def _cohesion(self, pos: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        return np.mean(neighbors, axis=0) - pos

class PotentialFieldAlgorithm(SteeringAlgorithm):
    def __init__(self, alpha: float = 0.5, beta: float = 1.0, goal: Optional[np.ndarray] = None):
        self.goal = goal if goal is not None else np.array([25, 25])
        self.alpha = alpha
        self.beta = beta

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        to_goal = self.goal - agent.position
        attraction = self.alpha * to_goal

        repulsion = np.zeros(2)
        for n in neighbors:
            vec = agent.position - n.position
            dist = np.linalg.norm(vec)
            if dist > 0:
                repulsion += self.beta * vec / dist**2

        return attraction + repulsion - agent.velocity/dt

class DroneNavigationAlgorithm(SteeringAlgorithm):
    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        target_pos = agent.route[agent.current_waypoint]
        direction = target_pos - agent.position
        distance = np.linalg.norm(direction)

        if distance < 0.1:
            agent.current_waypoint = (agent.current_waypoint + 1) % len(agent.route)
            target_pos = agent.route[agent.current_waypoint]
            direction = target_pos - agent.position
            distance = np.linalg.norm(direction)

        desired_velocity = (direction / distance) * agent.specs.max_speed if distance > 0 else np.zeros(2)
        steering = desired_velocity - agent.velocity
        steering = agent._limit_vector(steering, agent.specs.max_acceleration)

        avoidance = np.zeros(2)
        for neighbor in neighbors:
            if neighbor.id == agent.id:
                continue

            delta_pos = agent.position - neighbor.position
            dist = np.linalg.norm(delta_pos)
            if dist == 0:
                continue

            combined_radius = (agent.specs.physical_radius + neighbor.specs.physical_radius +
                             agent.specs.safety_margin + neighbor.specs.safety_margin)

            if agent.id > neighbor.id:
                avoidance += self._priority_force(agent, neighbor, delta_pos, dist, combined_radius)

            avoidance += self._approach_force(agent, neighbor, delta_pos, dist, combined_radius)
            avoidance += self._trajectory_force(agent, neighbor, dt, combined_radius)
            avoidance += self._route_conflict_force(agent, neighbor, combined_radius)

        return steering + avoidance

    def _priority_force(self, agent, neighbor, delta_pos, dist, combined_radius):
        if dist < combined_radius:
            force_mag = (combined_radius - dist)/combined_radius * agent.specs.max_acceleration
            return (delta_pos/dist) * force_mag
        return np.zeros(2)

    def _approach_force(self, agent, neighbor, delta_pos, dist, combined_radius):
        rel_velocity = agent.velocity - neighbor.velocity
        approach_speed = np.dot(rel_velocity, delta_pos)/dist
        if approach_speed <= 0: return np.zeros(2)
        ttc = dist/approach_speed
        return (delta_pos/dist) * (1.0/(ttc + 1e-6)) * agent.specs.max_acceleration if ttc < 2.0 else np.zeros(2)

    def _trajectory_force(self, agent, neighbor, dt, combined_radius):
        horizon = 5
        min_dist = float('inf')
        for t in range(1, horizon+1):
            future_self = agent.position + agent.velocity*t*dt
            future_neighbor = neighbor.position + neighbor.velocity*t*dt
            dist = np.linalg.norm(future_self - future_neighbor)
            if dist < min_dist: min_dist = dist
        if min_dist < combined_radius:
            direction = agent.position - future_neighbor
            return (direction/np.linalg.norm(direction)) * ((combined_radius - min_dist)/combined_radius) * agent.specs.max_acceleration
        return np.zeros(2)

    def _route_conflict_force(self, agent, neighbor, combined_radius):
        look_ahead = 5
        min_dist = float('inf')
        for wp_self in agent.route[agent.current_waypoint : agent.current_waypoint+look_ahead]:
            for wp_neighbor in neighbor.route[neighbor.current_waypoint : neighbor.current_waypoint+look_ahead]:
                dist = np.linalg.norm(wp_self - wp_neighbor)
                if dist < min_dist: min_dist = dist
        if min_dist < combined_radius:
            direction = agent.position - wp_neighbor
            return (direction/np.linalg.norm(direction)) * ((combined_radius - min_dist)/combined_radius) * agent.specs.max_acceleration
        return np.zeros(2)

# ==================== СИСТЕМА СИМУЛЯЦІЇ ====================
class SwarmSimulator:
    def __init__(self, area_size: float, num_agents: int, specs: AgentSpecs, algorithm, init_pos: str = 'grid', seed: Optional[int] = None):
        if seed is not None:
            np.random.seed(seed)
            random.seed(seed)

        self.area_size = area_size
        self.agents = self._init_agents(num_agents, specs, algorithm, area_size, init_pos)
        self.collisions = 0

    def _init_agents(self, num_agents, specs, algorithm, area_size, init_pos):
        agents = []
        if init_pos == 'grid':
            rows = int(np.ceil(np.sqrt(num_agents)))
            cols = int(np.ceil(num_agents/rows))
            sector_size = area_size / max(rows, cols)

            for i in range(num_agents):
                route = self._generate_snake_route(i, sector_size, rows, cols, area_size)
                start_pos = route[0]
                agents.append(Agent(i, start_pos, specs, algorithm, route))
        elif init_pos == 'random':
            for i in range(num_agents):
                route = self._generate_random_route(i, area_size)
                start_pos = route[0]
                agents.append(Agent(i, start_pos, specs, algorithm, route))
        return agents

    def _generate_snake_route(self, drone_id, sector_size, rows, cols, area_size):
        row = drone_id // cols
        col = drone_id % cols
        x_start = col * sector_size
        y_start = row * sector_size

        route = []
        steps = 5
        step_size = sector_size/steps
        for i in range(steps):
            y = y_start + (i + 0.5)*step_size
            x_values = [x_start + (j + 0.5)*step_size for j in range(steps)] if i%2==0 else reversed([x_start + (j + 0.5)*step_size for j in range(steps)])
            for x in x_values:
                route.append(np.array([x, y]))
        return route

    def _generate_random_route(self, drone_id, area_size):
        route = []
        steps = 5
        for _ in range(steps):
            x = random.uniform(0.1*area_size, 0.9*area_size)
            y = random.uniform(0.1*area_size, 0.9*area_size)
            route.append(np.array([x, y]))
        return route

    def run(self, steps: int, dt: float):
        for _ in range(steps):
            self._update_step(dt)
            self._check_collisions()

    def _update_step(self, dt):
        neighbors = [[a for a in self.agents if a!=agent and np.linalg.norm(a.position-agent.position)<=agent.specs.comm_radius] for agent in self.agents]
        for agent, n in zip(self.agents, neighbors):
            agent.update(n, dt, self.area_size)

    def _check_collisions(self):
        positions = np.array([a.position for a in self.agents])
        dist_matrix = np.linalg.norm(positions[:, None] - positions, axis=2)
        np.fill_diagonal(dist_matrix, np.inf)
        self.collisions += np.sum(dist_matrix < 2*self.agents[0].specs.physical_radius)//2

# ==================== ПОРІВНЯННЯ АЛГОРИТМІВ ====================
def compare_algorithms(area_size: float = 500, num_agents: int = 400, steps: int = 1500, dt: float = 0.1, seed: Optional[int] = 42):
    specs = AgentSpecs(
        physical_radius=0.5,
        safety_margin=1.0,
        max_speed=2.0,
        max_acceleration=1.5,
        comm_radius=15.0
    )

    algorithms = {
        "Vicsek": VicsekAlgorithm(noise=0.3),
        "Boids": BoidsAlgorithm(sep_weight=1.5, align_weight=0.8, coh_weight=0.5),
        "Potential Field": PotentialFieldAlgorithm(alpha=0.3, beta=0.8, goal=np.array([area_size/2, area_size/2])),
        "Drone Navigation": DroneNavigationAlgorithm()
    }

    print("Порівняння алгоритмів керування:")
    print("--------------------------------")
    print(f"Параметри симуляції: {num_agents} агентів, область {area_size}x{area_size}, {steps} кроків")
    print("--------------------------------")

    results = {}
    for name, algo in algorithms.items():
        sim = SwarmSimulator(
            area_size=area_size,
            num_agents=num_agents,
            specs=specs,
            algorithm=algo,
            seed=seed
        )
        sim.run(steps=steps, dt=dt)
        results[name] = sim.collisions
        print(f"{name}: {sim.collisions} зіткнень")

    return results

if __name__ == "__main__":
    compare_algorithms()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from abc import ABC, abstractmethod
from typing import List, Dict, Tuple, Optional, Any
import random
import json
from pathlib import Path
from datetime import datetime
from google.colab import files

# ==================== БАЗОВІ КЛАСИ ====================
@dataclass
class AgentSpecs:
    physical_radius: float
    safety_margin: float
    max_speed: float
    max_acceleration: float
    comm_radius: float

class SteeringAlgorithm(ABC):
    @abstractmethod
    def compute_steering(self, agent, neighbors: List['Agent'], dt: float) -> np.ndarray:
        pass

@dataclass
class SimulationMetrics:
    collisions: int = 0
    avg_speed: List[float] = None
    max_speed: List[float] = None
    avg_acceleration: List[float] = None
    avg_distance: List[float] = None
    agent_trajectories: Dict[int, List[Tuple[float, float]]] = None

    def __post_init__(self):
        self.avg_speed = []
        self.max_speed = []
        self.avg_acceleration = []
        self.avg_distance = []
        self.agent_trajectories = {}

class Agent:
    def __init__(self, agent_id: int, position: np.ndarray, specs: AgentSpecs,
                 algorithm: SteeringAlgorithm, route: List[np.ndarray]):
        self.id = agent_id
        self.position = position
        self.velocity = np.zeros(2)
        self.acceleration = np.zeros(2)
        self.specs = specs
        self.algorithm = algorithm
        self.route = route
        self.current_waypoint = 0
        self.path_history = [position.copy()]

    def update(self, neighbors: List['Agent'], dt: float, area_size: float):
        steering = self.algorithm.compute_steering(self, neighbors, dt)
        steering = self._limit_vector(steering, self.specs.max_acceleration)
        self.velocity = self._limit_vector(self.velocity + steering * dt, self.specs.max_speed)
        self.position += self.velocity * dt
        self.acceleration = steering
        self._enforce_boundaries(area_size)
        self.path_history.append(self.position.copy())

    def _limit_vector(self, vector: np.ndarray, max_magnitude: float) -> np.ndarray:
        magnitude = np.linalg.norm(vector)
        return vector if magnitude == 0 else vector * min(max_magnitude/magnitude, 1)

    def _enforce_boundaries(self, area_size: float):
        self.position = np.clip(self.position, 0, area_size)

# ==================== АЛГОРИТМИ КЕРУВАННЯ ====================
class VicsekAlgorithm(SteeringAlgorithm):
    def __init__(self, noise: float = 0.1, neighbor_radius: float = 15.0):
        self.noise = noise
        self.neighbor_radius = neighbor_radius

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        neighbors = [n for n in neighbors if np.linalg.norm(n.position - agent.position) <= self.neighbor_radius]
        if not neighbors:
            return np.zeros(2)
        avg_velocity = np.mean([n.velocity for n in neighbors], axis=0)
        noise = np.random.normal(scale=self.noise, size=2)
        return (avg_velocity + noise - agent.velocity) / dt

class BoidsAlgorithm(SteeringAlgorithm):
    def __init__(self, sep_weight: float = 1.5, align_weight: float = 1.0,
                 coh_weight: float = 1.0, visual_range: float = 15.0):
        self.weights = np.array([sep_weight, align_weight, coh_weight])
        self.visual_range = visual_range

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        neighbors = [n for n in neighbors if np.linalg.norm(n.position - agent.position) <= self.visual_range]

        if not neighbors:
            return np.zeros(2)

        positions = np.array([n.position for n in neighbors])
        velocities = np.array([n.velocity for n in neighbors])

        sep = self._separation(agent.position, positions)
        align = self._alignment(agent.velocity, velocities)
        coh = self._cohesion(agent.position, positions)

        return self.weights @ np.array([sep, align, coh])

    def _separation(self, pos: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        diff = pos - neighbors
        distances = np.linalg.norm(diff, axis=1)[:, np.newaxis]
        return np.mean(diff / (distances + 1e-6), axis=0)

    def _alignment(self, vel: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        return np.mean(neighbors, axis=0) - vel

    def _cohesion(self, pos: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        return np.mean(neighbors, axis=0) - pos

class PotentialFieldAlgorithm(SteeringAlgorithm):
    def __init__(self, alpha: float = 0.5, beta: float = 1.0,
                 goal: Optional[np.ndarray] = None, dynamic_goal: bool = False):
        self.goal = goal
        self.alpha = alpha
        self.beta = beta
        self.dynamic_goal = dynamic_goal

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        if self.dynamic_goal and agent.route:
            self.goal = agent.route[-1]

        to_goal = self.goal - agent.position if self.goal is not None else np.zeros(2)
        attraction = self.alpha * to_goal

        repulsion = np.zeros(2)
        for n in neighbors:
            vec = agent.position - n.position
            dist = np.linalg.norm(vec)
            if dist > 0:
                repulsion += self.beta * vec / dist**2

        return attraction + repulsion - agent.velocity/dt

# ==================== СИСТЕМА СИМУЛЯЦІЇ ====================
class SwarmSimulator:
    def __init__(self, config: Dict[str, Any]):
        self.config = config
        if 'seed' in config:
            np.random.seed(config['seed'])
            random.seed(config['seed'])
        self._init_simulation()

    def _init_simulation(self):
        self.agents = []
        self.metrics = SimulationMetrics()

        positions = self._generate_initial_positions()

        for i in range(self.config['num_agents']):
            route = self._generate_route(i, positions[i])
            specs = AgentSpecs(**self.config['agent_specs'])
            algorithm = self._create_algorithm(self.config['algorithm'])
            agent = Agent(i, positions[i], specs, algorithm, route)
            self.agents.append(agent)

    def _generate_initial_positions(self) -> List[np.ndarray]:
        config = self.config['initial_config']
        area_size = self.config['area_size']
        num_agents = self.config['num_agents']

        if config['type'] == 'grid':
            rows = int(np.ceil(np.sqrt(num_agents)))
            cols = int(np.ceil(num_agents/rows))
            sector_size = area_size / max(rows, cols)
            return [np.array([(i%cols)*sector_size + sector_size/2,
                            (i//cols)*sector_size + sector_size/2])
                   for i in range(num_agents)]

        elif config['type'] == 'random':
            return [np.array([random.uniform(0, area_size), random.uniform(0, area_size)])
                   for _ in range(num_agents)]

        elif config['type'] == 'circle':
            radius = config.get('radius', area_size/3)
            center = np.array([area_size/2, area_size/2])
            angles = np.linspace(0, 2*np.pi, num_agents, endpoint=False)
            return [center + radius * np.array([np.cos(a), np.sin(a)]) for a in angles]

        else:
            raise ValueError(f"Unknown initial config type: {config['type']}")

    def _generate_route(self, agent_id: int, start_pos: np.ndarray) -> List[np.ndarray]:
        route_config = self.config['route_config']
        area_size = self.config['area_size']

        if route_config['type'] == 'snake':
            rows = int(np.ceil(np.sqrt(self.config['num_agents'])))
            cols = int(np.ceil(self.config['num_agents']/rows))
            sector_size = area_size / max(rows, cols)
            row = agent_id // cols
            col = agent_id % cols
            x_start = col * sector_size
            y_start = row * sector_size

            route = []
            steps = 5
            step_size = sector_size/steps
            for i in range(steps):
                y = y_start + (i + 0.5)*step_size
                x_values = [x_start + (j + 0.5)*step_size for j in range(steps)] if i%2==0 else reversed([x_start + (j + 0.5)*step_size for j in range(steps)])
                for x in x_values:
                    route.append(np.array([x, y]))
            return route

        elif route_config['type'] == 'random_waypoints':
            steps = 5
            return [np.array([random.uniform(0, area_size), random.uniform(0, area_size)])
                   for _ in range(steps)]

        elif route_config['type'] == 'common_goal':
            goal = np.array(route_config.get('goal', [area_size/2, area_size/2]))
            return [goal.copy()]

        else:
            raise ValueError(f"Unknown route config type: {route_config['type']}")

    def _create_algorithm(self, algo_config: Dict[str, Any]) -> SteeringAlgorithm:
        algo_type = algo_config['type']
        params = algo_config.get('params', {})

        if algo_type == 'VicsekAlgorithm':
            return VicsekAlgorithm(**params)
        elif algo_type == 'BoidsAlgorithm':
            return BoidsAlgorithm(**params)
        elif algo_type == 'PotentialFieldAlgorithm':
            if 'goal' in params:
                params['goal'] = np.array(params['goal'])
            return PotentialFieldAlgorithm(**params)
        else:
            raise ValueError(f"Unknown algorithm type: {algo_type}")

    def run(self):
        for step in range(self.config['steps']):
            self._update_step()
            self._collect_metrics(step)
            self._check_collisions()

    def _update_step(self):
        neighbors = self._find_neighbors()
        for agent, n in zip(self.agents, neighbors):
            agent.update(n, self.config['dt'], self.config['area_size'])

    def _find_neighbors(self) -> List[List[Agent]]:
        positions = np.array([a.position for a in self.agents])
        dist_matrix = np.linalg.norm(positions[:, None] - positions, axis=2)

        neighbors = []
        for i, agent in enumerate(self.agents):
            mask = (dist_matrix[i] <= agent.specs.comm_radius) & (dist_matrix[i] > 0)
            neighbors.append([self.agents[j] for j in np.where(mask)[0]])
        return neighbors

    def _collect_metrics(self, step: int):
        speeds = [np.linalg.norm(a.velocity) for a in self.agents]
        accelerations = [np.linalg.norm(a.acceleration) for a in self.agents]

        self.metrics.avg_speed.append(np.mean(speeds))
        self.metrics.max_speed.append(np.max(speeds))
        self.metrics.avg_acceleration.append(np.mean(accelerations))

        positions = np.array([a.position for a in self.agents])
        dist_matrix = np.linalg.norm(positions[:, None] - positions, axis=2)
        np.fill_diagonal(dist_matrix, np.nan)
        self.metrics.avg_distance.append(np.nanmean(dist_matrix))

        for agent in self.agents:
            if agent.id not in self.metrics.agent_trajectories:
                self.metrics.agent_trajectories[agent.id] = []
            self.metrics.agent_trajectories[agent.id].append(tuple(agent.position))

    def _check_collisions(self):
        positions = np.array([a.position for a in self.agents])
        dist_matrix = np.linalg.norm(positions[:, None] - positions, axis=2)
        np.fill_diagonal(dist_matrix, np.inf)
        self.metrics.collisions += np.sum(dist_matrix < 2*self.agents[0].specs.physical_radius)//2

# ==================== ВІЗУАЛІЗАЦІЯ ====================
class Visualization:
    @staticmethod
    def plot_trajectories(metrics: SimulationMetrics, area_size: float):
        plt.figure(figsize=(10, 10))
        for agent_id, trajectory in metrics.agent_trajectories.items():
            x, y = zip(*trajectory)
            plt.plot(x, y, alpha=0.5, label=f'Agent {agent_id}')
        plt.xlim(0, area_size)
        plt.ylim(0, area_size)
        plt.title("Agent Trajectories")
        plt.legend()
        plt.show()

    @staticmethod
    def plot_metrics(metrics: SimulationMetrics):
        fig, axs = plt.subplots(2, 2, figsize=(15, 10))

        axs[0,0].plot(metrics.avg_speed)
        axs[0,0].set_title('Average Speed')
        axs[0,0].set_xlabel('Step')
        axs[0,0].set_ylabel('Speed')

        axs[0,1].plot(metrics.max_speed)
        axs[0,1].set_title('Maximum Speed')
        axs[0,1].set_xlabel('Step')
        axs[0,1].set_ylabel('Speed')

        axs[1,0].plot(metrics.avg_acceleration)
        axs[1,0].set_title('Average Acceleration')
        axs[1,0].set_xlabel('Step')
        axs[1,0].set_ylabel('Acceleration')

        axs[1,1].plot(metrics.avg_distance)
        axs[1,1].set_title('Average Distance Between Agents')
        axs[1,1].set_xlabel('Step')
        axs[1,1].set_ylabel('Distance')

        plt.tight_layout()
        plt.show()

# ==================== ІНТЕРФЕЙС ДЛЯ COLAB ====================
def run_simulation_in_colab():
    # Стандартна конфігурація
    config = {
        "num_agents": 20,
        "area_size": 100,
        "steps": 500,
        "dt": 0.1,
        "seed": 42,
        "agent_specs": {
            "physical_radius": 0.5,
            "safety_margin": 1.0,
            "max_speed": 2.0,
            "max_acceleration": 1.5,
            "comm_radius": 15.0
        },
        "initial_config": {
            "type": "circle",
            "radius": 30
        },
        "route_config": {
            "type": "common_goal",
            "goal": [50, 50]
        },
        "algorithm": {
            "type": "BoidsAlgorithm",
            "params": {
                "sep_weight": 1.5,
                "align_weight": 0.8,
                "coh_weight": 0.5,
                "visual_range": 20.0
            }
        }
    }

    print("Running simulation with default configuration...")
    simulator = SwarmSimulator(config)
    simulator.run()

    print("\nSimulation results:")
    print(f"Total collisions: {simulator.metrics.collisions}")
    print(f"Final average speed: {simulator.metrics.avg_speed[-1]:.2f}")
    print(f"Final average distance: {simulator.metrics.avg_distance[-1]:.2f}")

    print("\nVisualizing results...")
    Visualization.plot_trajectories(simulator.metrics, config['area_size'])
    Visualization.plot_metrics(simulator.metrics)

# Запуск симуляції в Colab
run_simulation_in_colab()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from abc import ABC, abstractmethod
from typing import List, Dict, Tuple, Optional, Any
import random
import json
from pathlib import Path
from datetime import datetime
from google.colab import files

# ==================== БАЗОВІ КЛАСИ ====================
@dataclass
class AgentSpecs:
    physical_radius: float
    safety_margin: float
    max_speed: float
    max_acceleration: float
    comm_radius: float

class SteeringAlgorithm(ABC):
    @abstractmethod
    def compute_steering(self, agent, neighbors: List['Agent'], dt: float) -> np.ndarray:
        pass

@dataclass
class SimulationMetrics:
    collisions: int = 0
    avg_speed: List[float] = None
    max_speed: List[float] = None
    avg_acceleration: List[float] = None
    avg_distance: List[float] = None
    avg_distance_to_center: List[float] = None
    agent_trajectories: Dict[int, List[Tuple[float, float]]] = None

    def __post_init__(self):
        self.avg_speed = []
        self.max_speed = []
        self.avg_acceleration = []
        self.avg_distance = []
        self.avg_distance_to_center = []
        self.agent_trajectories = {}

class Agent:
    def __init__(self, agent_id: int, position: np.ndarray, specs: AgentSpecs,
                 algorithm: SteeringAlgorithm, route: List[np.ndarray]):
        self.id = agent_id
        self.position = position
        self.velocity = np.zeros(2)
        self.acceleration = np.zeros(2)
        self.specs = specs
        self.algorithm = algorithm
        self.route = route
        self.current_waypoint = 0
        self.path_history = [position.copy()]

    def update(self, neighbors: List['Agent'], dt: float, area_size: float):
        steering = self.algorithm.compute_steering(self, neighbors, dt)
        steering = self._limit_vector(steering, self.specs.max_acceleration)
        self.velocity = self._limit_vector(self.velocity + steering * dt, self.specs.max_speed)
        self.position += self.velocity * dt
        self.acceleration = steering
        self._enforce_boundaries(area_size)
        self.path_history.append(self.position.copy())

    def _limit_vector(self, vector: np.ndarray, max_magnitude: float) -> np.ndarray:
        magnitude = np.linalg.norm(vector)
        return vector if magnitude == 0 else vector * min(max_magnitude/magnitude, 1)

    def _enforce_boundaries(self, area_size: float):
        self.position = np.clip(self.position, 0, area_size)

# ==================== АЛГОРИТМИ КЕРУВАННЯ ====================
class VicsekAlgorithm(SteeringAlgorithm):
    def __init__(self, noise: float = 0.1, neighbor_radius: float = 15.0):
        self.noise = noise
        self.neighbor_radius = neighbor_radius

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        neighbors = [n for n in neighbors if np.linalg.norm(n.position - agent.position) <= self.neighbor_radius]
        if not neighbors:
            return np.zeros(2)
        avg_velocity = np.mean([n.velocity for n in neighbors], axis=0)
        noise = np.random.normal(scale=self.noise, size=2)
        return (avg_velocity + noise - agent.velocity) / dt

class BoidsAlgorithm(SteeringAlgorithm):
    def __init__(self, sep_weight: float = 1.5, align_weight: float = 1.0,
                 coh_weight: float = 1.0, visual_range: float = 15.0):
        self.weights = np.array([sep_weight, align_weight, coh_weight])
        self.visual_range = visual_range

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        neighbors = [n for n in neighbors if np.linalg.norm(n.position - agent.position) <= self.visual_range]

        if not neighbors:
            return np.zeros(2)

        positions = np.array([n.position for n in neighbors])
        velocities = np.array([n.velocity for n in neighbors])

        sep = self._separation(agent.position, positions)
        align = self._alignment(agent.velocity, velocities)
        coh = self._cohesion(agent.position, positions)

        return self.weights @ np.array([sep, align, coh])

    def _separation(self, pos: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        diff = pos - neighbors
        distances = np.linalg.norm(diff, axis=1)[:, np.newaxis]
        return np.mean(diff / (distances + 1e-6), axis=0)

    def _alignment(self, vel: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        return np.mean(neighbors, axis=0) - vel

    def _cohesion(self, pos: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        return np.mean(neighbors, axis=0) - pos

class PotentialFieldAlgorithm(SteeringAlgorithm):
    def __init__(self, alpha: float = 0.5, beta: float = 1.0,
                 goal: Optional[np.ndarray] = None, dynamic_goal: bool = False):
        self.goal = goal
        self.alpha = alpha
        self.beta = beta
        self.dynamic_goal = dynamic_goal

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        if self.dynamic_goal and agent.route:
            self.goal = agent.route[-1]

        to_goal = self.goal - agent.position if self.goal is not None else np.zeros(2)
        attraction = self.alpha * to_goal

        repulsion = np.zeros(2)
        for n in neighbors:
            vec = agent.position - n.position
            dist = np.linalg.norm(vec)
            if dist > 0:
                repulsion += self.beta * vec / dist**2

        return attraction + repulsion - agent.velocity / dt

class LeaderFollowerAlgorithm(SteeringAlgorithm):
    def __init__(self, leader_id: int = 0, follow_distance: float = 5.0):
        self.leader_id = leader_id
        self.follow_distance = follow_distance

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        leader = next((n for n in neighbors if n.id == self.leader_id), None)
        if leader is None:
            return np.zeros(2)

        direction = leader.position - agent.position
        distance = np.linalg.norm(direction)

        if distance > 0:
            desired_velocity = (direction / distance) * agent.specs.max_speed
            return (desired_velocity - agent.velocity) / dt
        else:
            return np.zeros(2)

# ==================== СИСТЕМА СИМУЛЯЦІЇ ====================
class SwarmSimulator:
    def __init__(self, config: Dict[str, Any]):
        self.config = config
        if 'seed' in config:
            np.random.seed(config['seed'])
            random.seed(config['seed'])
        self._init_simulation()

    def _init_simulation(self):
        self.agents = []
        self.metrics = SimulationMetrics()

        positions = self._generate_initial_positions()

        for i in range(self.config['num_agents']):
            route = self._generate_route(i, positions[i])
            specs = AgentSpecs(**self.config['agent_specs'])
            algorithm = self._create_algorithm(self.config['algorithm'])
            agent = Agent(i, positions[i], specs, algorithm, route)
            self.agents.append(agent)

    def _generate_initial_positions(self) -> List[np.ndarray]:
        config = self.config['initial_config']
        area_size = self.config['area_size']
        num_agents = self.config['num_agents']

        if config['type'] == 'grid':
            rows = int(np.ceil(np.sqrt(num_agents)))
            cols = int(np.ceil(num_agents/rows))
            sector_size = area_size / max(rows, cols)
            return [np.array([(i%cols)*sector_size + sector_size/2,
                            (i//cols)*sector_size + sector_size/2])
                   for i in range(num_agents)]

        elif config['type'] == 'random':
            return [np.array([random.uniform(0, area_size), random.uniform(0, area_size)])
                   for _ in range(num_agents)]

        elif config['type'] == 'circle':
            radius = config.get('radius', area_size/3)
            center = np.array([area_size/2, area_size/2])
            angles = np.linspace(0, 2*np.pi, num_agents, endpoint=False)
            return [center + radius * np.array([np.cos(a), np.sin(a)]) for a in angles]

        else:
            raise ValueError(f"Unknown initial config type: {config['type']}")

    def _generate_route(self, agent_id: int, start_pos: np.ndarray) -> List[np.ndarray]:
        route_config = self.config['route_config']
        area_size = self.config['area_size']

        if route_config['type'] == 'snake':
            rows = int(np.ceil(np.sqrt(self.config['num_agents'])))
            cols = int(np.ceil(self.config['num_agents']/rows))
            sector_size = area_size / max(rows, cols)
            row = agent_id // cols
            col = agent_id % cols
            x_start = col * sector_size
            y_start = row * sector_size

            route = []
            steps = 5
            step_size = sector_size/steps
            for i in range(steps):
                y = y_start + (i + 0.5)*step_size
                x_values = [x_start + (j + 0.5)*step_size for j in range(steps)] if i%2==0 else reversed([x_start + (j + 0.5)*step_size for j in range(steps)])
                for x in x_values:
                    route.append(np.array([x, y]))
            return route

        elif route_config['type'] == 'random_waypoints':
            steps = 5
            return [np.array([random.uniform(0, area_size), random.uniform(0, area_size)])
                   for _ in range(steps)]

        elif route_config['type'] == 'common_goal':
            goal = np.array(route_config.get('goal', [area_size/2, area_size/2]))
            return [goal.copy()]

        else:
            raise ValueError(f"Unknown route config type: {route_config['type']}")

    def _create_algorithm(self, algo_config: Dict[str, Any]) -> SteeringAlgorithm:
        algo_type = algo_config['type']
        params = algo_config.get('params', {})

        if algo_type == 'VicsekAlgorithm':
            return VicsekAlgorithm(**params)
        elif algo_type == 'BoidsAlgorithm':
            return BoidsAlgorithm(**params)
        elif algo_type == 'PotentialFieldAlgorithm':
            if 'goal' in params:
                params['goal'] = np.array(params['goal'])
            return PotentialFieldAlgorithm(**params)
        elif algo_type == 'LeaderFollowerAlgorithm':
            return LeaderFollowerAlgorithm(**params)
        else:
            raise ValueError(f"Unknown algorithm type: {algo_type}")

    def run(self):
        for step in range(self.config['steps']):
            self._update_step()
            self._collect_metrics(step)
            self._check_collisions()

    def _update_step(self):
        neighbors = self._find_neighbors()
        for agent, n in zip(self.agents, neighbors):
            agent.update(n, self.config['dt'], self.config['area_size'])

    def _find_neighbors(self) -> List[List[Agent]]:
        positions = np.array([a.position for a in self.agents])
        dist_matrix = np.linalg.norm(positions[:, None] - positions, axis=2)

        neighbors = []
        for i, agent in enumerate(self.agents):
            mask = (dist_matrix[i] <= agent.specs.comm_radius) & (dist_matrix[i] > 0)
            neighbors.append([self.agents[j] for j in np.where(mask)[0]])
        return neighbors

    def _collect_metrics(self, step: int):
        speeds = [np.linalg.norm(a.velocity) for a in self.agents]
        accelerations = [np.linalg.norm(a.acceleration) for a in self.agents]

        self.metrics.avg_speed.append(np.mean(speeds))
        self.metrics.max_speed.append(np.max(speeds))
        self.metrics.avg_acceleration.append(np.mean(accelerations))

        positions = np.array([a.position for a in self.agents])
        dist_matrix = np.linalg.norm(positions[:, None] - positions, axis=2)
        np.fill_diagonal(dist_matrix, np.nan)
        self.metrics.avg_distance.append(np.nanmean(dist_matrix))

        center_of_mass = np.mean(positions, axis=0)
        distances_to_center = np.linalg.norm(positions - center_of_mass, axis=1)
        self.metrics.avg_distance_to_center.append(np.mean(distances_to_center))

        for agent in self.agents:
            if agent.id not in self.metrics.agent_trajectories:
                self.metrics.agent_trajectories[agent.id] = []
            self.metrics.agent_trajectories[agent.id].append(tuple(agent.position))

    def _check_collisions(self):
        positions = np.array([a.position for a in self.agents])
        dist_matrix = np.linalg.norm(positions[:, None] - positions, axis=2)
        np.fill_diagonal(dist_matrix, np.inf)
        self.metrics.collisions += np.sum(dist_matrix < 2*self.agents[0].specs.physical_radius)//2

# ==================== ВІЗУАЛІЗАЦІЯ ====================
class Visualization:
    @staticmethod
    def plot_trajectories(metrics: SimulationMetrics, area_size: float):
        plt.figure(figsize=(10, 10))
        for agent_id, trajectory in metrics.agent_trajectories.items():
            x, y = zip(*trajectory)
            plt.plot(x, y, alpha=0.5, label=f'Agent {agent_id}')
        plt.xlim(0, area_size)
        plt.ylim(0, area_size)
        plt.title("Agent Trajectories")
        plt.legend()
        plt.show()

    @staticmethod
    def plot_velocity_vectors(agents: List[Agent], area_size: float):
        plt.figure(figsize=(10, 10))
        for agent in agents:
            plt.arrow(agent.position[0], agent.position[1], agent.velocity[0], agent.velocity[1],
                      head_width=0.5, head_length=0.5, fc='r', ec='r')
        plt.xlim(0, area_size)
        plt.ylim(0, area_size)
        plt.title("Agent Velocity Vectors")
        plt.show()

    @staticmethod
    def plot_metrics(metrics: SimulationMetrics):
        fig, axs = plt.subplots(2, 2, figsize=(15, 10))

        axs[0,0].plot(metrics.avg_speed)
        axs[0,0].set_title('Average Speed')
        axs[0,0].set_xlabel('Step')
        axs[0,0].set_ylabel('Speed')

        axs[0,1].plot(metrics.max_speed)
        axs[0,1].set_title('Maximum Speed')
        axs[0,1].set_xlabel('Step')
        axs[0,1].set_ylabel('Speed')

        axs[1,0].plot(metrics.avg_acceleration)
        axs[1,0].set_title('Average Acceleration')
        axs[1,0].set_xlabel('Step')
        axs[1,0].set_ylabel('Acceleration')

        axs[1,1].plot(metrics.avg_distance_to_center)
        axs[1,1].set_title('Average Distance to Center')
        axs[1,1].set_xlabel('Step')
        axs[1,1].set_ylabel('Distance')

        plt.tight_layout()
        plt.show()

# ==================== ІНТЕРФЕЙС ДЛЯ COLAB ====================
def run_simulation_in_colab():
    # Стандартна конфігурація
    config = {
        "num_agents": 20,
        "area_size": 100,
        "steps": 500,
        "dt": 0.1,
        "seed": 42,
        "agent_specs": {
            "physical_radius": 0.5,
            "safety_margin": 1.0,
            "max_speed": 2.0,
            "max_acceleration": 1.5,
            "comm_radius": 15.0
        },
        "initial_config": {
            "type": "circle",
            "radius": 30
        },
        "route_config": {
            "type": "common_goal",
            "goal": [50, 50]
        },
        "algorithm": {
            "type": "BoidsAlgorithm",
            "params": {
                "sep_weight": 1.5,
                "align_weight": 0.8,
                "coh_weight": 0.5,
                "visual_range": 20.0
            }
        }
    }

    print("Running simulation with default configuration...")
    simulator = SwarmSimulator(config)
    simulator.run()

    print("\nSimulation results:")
    print(f"Total collisions: {simulator.metrics.collisions}")
    print(f"Final average speed: {simulator.metrics.avg_speed[-1]:.2f}")
    print(f"Final average distance to center: {simulator.metrics.avg_distance_to_center[-1]:.2f}")

    print("\nVisualizing results...")
    Visualization.plot_trajectories(simulator.metrics, config['area_size'])
    Visualization.plot_velocity_vectors(simulator.agents, config['area_size'])
    Visualization.plot_metrics(simulator.metrics)

# Запуск симуляції в Colab
run_simulation_in_colab()

In [ ]:
import numpy as np
from dataclasses import dataclass
from abc import ABC, abstractmethod
from typing import List, Dict, Tuple, Optional, Any
import random

# ==================== БАЗОВІ КЛАСИ ====================
@dataclass
class AgentSpecs:
    physical_radius: float
    safety_margin: float
    max_speed: float
    max_acceleration: float
    comm_radius: float

class SteeringAlgorithm(ABC):
    @abstractmethod
    def compute_steering(self, agent, neighbors: List['Agent'], dt: float) -> np.ndarray:
        pass

@dataclass
class SimulationMetrics:
    collisions: int = 0
    avg_speed: List[float] = None
    max_speed: List[float] = None
    avg_acceleration: List[float] = None
    avg_distance: List[float] = None
    avg_distance_to_center: List[float] = None
    agent_trajectories: Dict[int, List[Tuple[float, float]]] = None

    def __post_init__(self):
        self.avg_speed = []
        self.max_speed = []
        self.avg_acceleration = []
        self.avg_distance = []
        self.avg_distance_to_center = []
        self.agent_trajectories = {}

class Agent:
    def __init__(self, agent_id: int, position: np.ndarray, specs: AgentSpecs,
                 algorithm: SteeringAlgorithm, route: List[np.ndarray]):
        self.id = agent_id
        self.position = position
        self.velocity = np.zeros(2)
        self.acceleration = np.zeros(2)
        self.specs = specs
        self.algorithm = algorithm
        self.route = route
        self.current_waypoint = 0
        self.path_history = [position.copy()]

    def update(self, neighbors: List['Agent'], dt: float, area_size: float):
        steering = self.algorithm.compute_steering(self, neighbors, dt)
        steering = self._limit_vector(steering, self.specs.max_acceleration)
        self.velocity = self._limit_vector(self.velocity + steering * dt, self.specs.max_speed)
        self.position += self.velocity * dt
        self.acceleration = steering
        self._enforce_boundaries(area_size)
        self.path_history.append(self.position.copy())

    def _limit_vector(self, vector: np.ndarray, max_magnitude: float) -> np.ndarray:
        magnitude = np.linalg.norm(vector)
        return vector if magnitude == 0 else vector * min(max_magnitude/magnitude, 1)

    def _enforce_boundaries(self, area_size: float):
        self.position = np.clip(self.position, 0, area_size)

# ==================== АЛГОРИТМИ КЕРУВАННЯ ====================
class VicsekAlgorithm(SteeringAlgorithm):
    def __init__(self, noise: float = 0.1, neighbor_radius: float = 15.0):
        self.noise = noise
        self.neighbor_radius = neighbor_radius

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        neighbors = [n for n in neighbors if np.linalg.norm(n.position - agent.position) <= self.neighbor_radius]
        if not neighbors:
            return np.zeros(2)
        avg_velocity = np.mean([n.velocity for n in neighbors], axis=0)
        noise = np.random.normal(scale=self.noise, size=2)
        return (avg_velocity + noise - agent.velocity) / dt

class BoidsAlgorithm(SteeringAlgorithm):
    def __init__(self, sep_weight: float = 1.5, align_weight: float = 1.0,
                 coh_weight: float = 1.0, visual_range: float = 15.0):
        self.weights = np.array([sep_weight, align_weight, coh_weight])
        self.visual_range = visual_range

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        neighbors = [n for n in neighbors if np.linalg.norm(n.position - agent.position) <= self.visual_range]

        if not neighbors:
            return np.zeros(2)

        positions = np.array([n.position for n in neighbors])
        velocities = np.array([n.velocity for n in neighbors])

        sep = self._separation(agent.position, positions)
        align = self._alignment(agent.velocity, velocities)
        coh = self._cohesion(agent.position, positions)

        return self.weights @ np.array([sep, align, coh])

    def _separation(self, pos: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        diff = pos - neighbors
        distances = np.linalg.norm(diff, axis=1)[:, np.newaxis]
        return np.mean(diff / (distances + 1e-6), axis=0)

    def _alignment(self, vel: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        return np.mean(neighbors, axis=0) - vel

    def _cohesion(self, pos: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        return np.mean(neighbors, axis=0) - pos

class PotentialFieldAlgorithm(SteeringAlgorithm):
    def __init__(self, alpha: float = 0.5, beta: float = 1.0,
                 goal: Optional[np.ndarray] = None, dynamic_goal: bool = False):
        self.goal = goal
        self.alpha = alpha
        self.beta = beta
        self.dynamic_goal = dynamic_goal

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        if self.dynamic_goal and agent.route:
            self.goal = agent.route[-1]

        to_goal = self.goal - agent.position if self.goal is not None else np.zeros(2)
        attraction = self.alpha * to_goal

        repulsion = np.zeros(2)
        for n in neighbors:
            vec = agent.position - n.position
            dist = np.linalg.norm(vec)
            if dist > 0:
                repulsion += self.beta * vec / dist**2

        return attraction + repulsion - agent.velocity / dt

class LeaderFollowerAlgorithm(SteeringAlgorithm):
    def __init__(self, leader_id: int = 0, follow_distance: float = 5.0):
        self.leader_id = leader_id
        self.follow_distance = follow_distance

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        leader = next((n for n in neighbors if n.id == self.leader_id), None)
        if leader is None:
            return np.zeros(2)

        direction = leader.position - agent.position
        distance = np.linalg.norm(direction)

        if distance > 0:
            desired_velocity = (direction / distance) * agent.specs.max_speed
            return (desired_velocity - agent.velocity) / dt
        else:
            return np.zeros(2)

# ==================== СИСТЕМА СИМУЛЯЦІЇ ====================
class SwarmSimulator:
    def __init__(self, config: Dict[str, Any]):
        self.config = config
        if 'seed' in config:
            np.random.seed(config['seed'])
            random.seed(config['seed'])
        self._init_simulation()

    def _init_simulation(self):
        self.agents = []
        self.metrics = SimulationMetrics()

        positions = self._generate_initial_positions()

        for i in range(self.config['num_agents']):
            route = self._generate_route(i, positions[i])
            specs = AgentSpecs(**self.config['agent_specs'])
            algorithm = self._create_algorithm(self.config['algorithm'])
            agent = Agent(i, positions[i], specs, algorithm, route)
            self.agents.append(agent)

    def _generate_initial_positions(self) -> List[np.ndarray]:
        config = self.config['initial_config']
        area_size = self.config['area_size']
        num_agents = self.config['num_agents']

        if config['type'] == 'grid':
            rows = int(np.ceil(np.sqrt(num_agents)))
            cols = int(np.ceil(num_agents/rows))
            sector_size = area_size / max(rows, cols)
            return [np.array([(i%cols)*sector_size + sector_size/2,
                            (i//cols)*sector_size + sector_size/2])
                   for i in range(num_agents)]

        elif config['type'] == 'random':
            return [np.array([random.uniform(0, area_size), random.uniform(0, area_size)])
                   for _ in range(num_agents)]

        elif config['type'] == 'circle':
            radius = config.get('radius', area_size/3)
            center = np.array([area_size/2, area_size/2])
            angles = np.linspace(0, 2*np.pi, num_agents, endpoint=False)
            return [center + radius * np.array([np.cos(a), np.sin(a)]) for a in angles]

        else:
            raise ValueError(f"Unknown initial config type: {config['type']}")

    def _generate_route(self, agent_id: int, start_pos: np.ndarray) -> List[np.ndarray]:
        route_config = self.config['route_config']
        area_size = self.config['area_size']

        if route_config['type'] == 'snake':
            rows = int(np.ceil(np.sqrt(self.config['num_agents'])))
            cols = int(np.ceil(self.config['num_agents']/rows))
            sector_size = area_size / max(rows, cols)
            row = agent_id // cols
            col = agent_id % cols
            x_start = col * sector_size
            y_start = row * sector_size

            route = []
            steps = 5
            step_size = sector_size/steps
            for i in range(steps):
                y = y_start + (i + 0.5)*step_size
                x_values = [x_start + (j + 0.5)*step_size for j in range(steps)] if i%2==0 else reversed([x_start + (j + 0.5)*step_size for j in range(steps)])
                for x in x_values:
                    route.append(np.array([x, y]))
            return route

        elif route_config['type'] == 'random_waypoints':
            steps = 5
            return [np.array([random.uniform(0, area_size), random.uniform(0, area_size)])
                   for _ in range(steps)]

        elif route_config['type'] == 'common_goal':
            goal = np.array(route_config.get('goal', [area_size/2, area_size/2]))
            return [goal.copy()]

        else:
            raise ValueError(f"Unknown route config type: {route_config['type']}")

    def _create_algorithm(self, algo_config: Dict[str, Any]) -> SteeringAlgorithm:
        algo_type = algo_config['type']
        params = algo_config.get('params', {})

        if algo_type == 'VicsekAlgorithm':
            return VicsekAlgorithm(**params)
        elif algo_type == 'BoidsAlgorithm':
            return BoidsAlgorithm(**params)
        elif algo_type == 'PotentialFieldAlgorithm':
            if 'goal' in params:
                params['goal'] = np.array(params['goal'])
            return PotentialFieldAlgorithm(**params)
        elif algo_type == 'LeaderFollowerAlgorithm':
            return LeaderFollowerAlgorithm(**params)
        else:
            raise ValueError(f"Unknown algorithm type: {algo_type}")

    def run(self):
        for step in range(self.config['steps']):
            self._update_step()
            self._collect_metrics(step)
            self._check_collisions()

    def _update_step(self):
        neighbors = self._find_neighbors()
        for agent, n in zip(self.agents, neighbors):
            agent.update(n, self.config['dt'], self.config['area_size'])

    def _find_neighbors(self) -> List[List[Agent]]:
        positions = np.array([a.position for a in self.agents])
        dist_matrix = np.linalg.norm(positions[:, None] - positions, axis=2)

        neighbors = []
        for i, agent in enumerate(self.agents):
            mask = (dist_matrix[i] <= agent.specs.comm_radius) & (dist_matrix[i] > 0)
            neighbors.append([self.agents[j] for j in np.where(mask)[0]])
        return neighbors

    def _collect_metrics(self, step: int):
        speeds = [np.linalg.norm(a.velocity) for a in self.agents]
        accelerations = [np.linalg.norm(a.acceleration) for a in self.agents]

        self.metrics.avg_speed.append(np.mean(speeds))
        self.metrics.max_speed.append(np.max(speeds))
        self.metrics.avg_acceleration.append(np.mean(accelerations))

        positions = np.array([a.position for a in self.agents])
        dist_matrix = np.linalg.norm(positions[:, None] - positions, axis=2)
        np.fill_diagonal(dist_matrix, np.nan)
        self.metrics.avg_distance.append(np.nanmean(dist_matrix))

        center_of_mass = np.mean(positions, axis=0)
        distances_to_center = np.linalg.norm(positions - center_of_mass, axis=1)
        self.metrics.avg_distance_to_center.append(np.mean(distances_to_center))

        for agent in self.agents:
            if agent.id not in self.metrics.agent_trajectories:
                self.metrics.agent_trajectories[agent.id] = []
            self.metrics.agent_trajectories[agent.id].append(tuple(agent.position))

    def _check_collisions(self):
        positions = np.array([a.position for a in self.agents])
        dist_matrix = np.linalg.norm(positions[:, None] - positions, axis=2)
        np.fill_diagonal(dist_matrix, np.inf)
        self.metrics.collisions += np.sum(dist_matrix < 2*self.agents[0].specs.physical_radius)//2

# ==================== ІНТЕРФЕЙС ДЛЯ COLAB ====================
def run_simulations_in_colab():
    algorithms = {
        "Boids": {
            "type": "BoidsAlgorithm",
            "params": {
                "sep_weight": 1.5,
                "align_weight": 0.8,
                "coh_weight": 0.5,
                "visual_range": 20.0
            }
        },
        "Vicsek": {
            "type": "VicsekAlgorithm",
            "params": {
                "noise": 0.2,
                "neighbor_radius": 15.0
            }
        },
        "Potential Field": {
            "type": "PotentialFieldAlgorithm",
            "params": {
                "alpha": 0.3,
                "beta": 0.8,
                "dynamic_goal": True
            }
        },
        "Leader-Follower": {
            "type": "LeaderFollowerAlgorithm",
            "params": {
                "leader_id": 0,
                "follow_distance": 5.0
            }
        }
    }

    results = {}

    for name, algo_config in algorithms.items():
        config = {
            "num_agents": 20,
            "area_size": 100,
            "steps": 500,
            "dt": 0.1,
            "seed": 42,
            "agent_specs": {
                "physical_radius": 0.5,
                "safety_margin": 1.0,
                "max_speed": 2.0,
                "max_acceleration": 1.5,
                "comm_radius": 15.0
            },
            "initial_config": {
                "type": "circle",
                "radius": 30
            },
            "route_config": {
                "type": "common_goal",
                "goal": [50, 50]
            },
            "algorithm": algo_config
        }

        print(f"Running simulation with {name} algorithm...")
        simulator = SwarmSimulator(config)
        simulator.run()
        results[name] = simulator.metrics.collisions

    print("\nSimulation results:")
    for algo_name, collisions in results.items():
        print(f"{algo_name}: {collisions} collisions")

# Запуск симуляцій в Colab
run_simulations_in_colab()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from dataclasses import dataclass
from abc import ABC, abstractmethod
from typing import List, Dict, Tuple, Optional, Any
import random

# ==================== БАЗОВІ КЛАСИ ====================
@dataclass
class AgentSpecs:
    physical_radius: float
    safety_margin: float
    max_speed: float
    max_acceleration: float
    comm_radius: float

class SteeringAlgorithm(ABC):
    @abstractmethod
    def compute_steering(self, agent, neighbors: List['Agent'], dt: float) -> np.ndarray:
        pass

@dataclass
class SimulationMetrics:
    collisions: int = 0
    avg_speed: List[float] = None
    max_speed: List[float] = None
    avg_acceleration: List[float] = None
    avg_distance: List[float] = None
    avg_distance_to_center: List[float] = None
    agent_trajectories: Dict[int, List[Tuple[float, float]]] = None

    def __post_init__(self):
        self.avg_speed = []
        self.max_speed = []
        self.avg_acceleration = []
        self.avg_distance = []
        self.avg_distance_to_center = []
        self.agent_trajectories = {}

class Agent:
    def __init__(self, agent_id: int, position: np.ndarray, specs: AgentSpecs,
                 algorithm: SteeringAlgorithm, route: List[np.ndarray]):
        self.id = agent_id
        self.position = position
        self.velocity = np.zeros(2)
        self.acceleration = np.zeros(2)
        self.specs = specs
        self.algorithm = algorithm
        self.route = route
        self.current_waypoint = 0
        self.path_history = [position.copy()]

    def update(self, neighbors: List['Agent'], dt: float, area_size: float):
        steering = self.algorithm.compute_steering(self, neighbors, dt)
        steering = self._limit_vector(steering, self.specs.max_acceleration)
        self.velocity = self._limit_vector(self.velocity + steering * dt, self.specs.max_speed)
        self.position += self.velocity * dt
        self.acceleration = steering
        self._enforce_boundaries(area_size)
        self.path_history.append(self.position.copy())

    def _limit_vector(self, vector: np.ndarray, max_magnitude: float) -> np.ndarray:
        magnitude = np.linalg.norm(vector)
        return vector if magnitude == 0 else vector * min(max_magnitude/magnitude, 1)

    def _enforce_boundaries(self, area_size: float):
        self.position = np.clip(self.position, 0, area_size)

# ==================== АЛГОРИТМИ КЕРУВАННЯ ====================
class VicsekAlgorithm(SteeringAlgorithm):
    def __init__(self, noise: float = 0.1, neighbor_radius: float = 15.0):
        self.noise = noise
        self.neighbor_radius = neighbor_radius

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        neighbors = [n for n in neighbors if np.linalg.norm(n.position - agent.position) <= self.neighbor_radius]
        if not neighbors:
            return np.zeros(2)
        avg_velocity = np.mean([n.velocity for n in neighbors], axis=0)
        noise = np.random.normal(scale=self.noise, size=2)
        return (avg_velocity + noise - agent.velocity) / dt

class BoidsAlgorithm(SteeringAlgorithm):
    def __init__(self, sep_weight: float = 1.5, align_weight: float = 1.0,
                 coh_weight: float = 1.0, visual_range: float = 15.0):
        self.weights = np.array([sep_weight, align_weight, coh_weight])
        self.visual_range = visual_range

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        neighbors = [n for n in neighbors if np.linalg.norm(n.position - agent.position) <= self.visual_range]

        if not neighbors:
            return np.zeros(2)

        positions = np.array([n.position for n in neighbors])
        velocities = np.array([n.velocity for n in neighbors])

        sep = self._separation(agent.position, positions)
        align = self._alignment(agent.velocity, velocities)
        coh = self._cohesion(agent.position, positions)

        return self.weights @ np.array([sep, align, coh])

    def _separation(self, pos: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        diff = pos - neighbors
        distances = np.linalg.norm(diff, axis=1)[:, np.newaxis]
        return np.mean(diff / (distances + 1e-6), axis=0)

    def _alignment(self, vel: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        return np.mean(neighbors, axis=0) - vel

    def _cohesion(self, pos: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        return np.mean(neighbors, axis=0) - pos

class PotentialFieldAlgorithm(SteeringAlgorithm):
    def __init__(self, alpha: float = 0.5, beta: float = 1.0,
                 goal: Optional[np.ndarray] = None, dynamic_goal: bool = False):
        self.goal = goal
        self.alpha = alpha
        self.beta = beta
        self.dynamic_goal = dynamic_goal

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        if self.dynamic_goal and agent.route:
            self.goal = agent.route[-1]

        to_goal = self.goal - agent.position if self.goal is not None else np.zeros(2)
        attraction = self.alpha * to_goal

        repulsion = np.zeros(2)
        for n in neighbors:
            vec = agent.position - n.position
            dist = np.linalg.norm(vec)
            if dist > 0:
                repulsion += self.beta * vec / dist**2

        return attraction + repulsion - agent.velocity / dt

class LeaderFollowerAlgorithm(SteeringAlgorithm):
    def __init__(self, leader_id: int = 0, follow_distance: float = 5.0):
        self.leader_id = leader_id
        self.follow_distance = follow_distance

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        leader = next((n for n in neighbors if n.id == self.leader_id), None)
        if leader is None:
            return np.zeros(2)

        direction = leader.position - agent.position
        distance = np.linalg.norm(direction)

        if distance > 0:
            desired_velocity = (direction / distance) * agent.specs.max_speed
            return (desired_velocity - agent.velocity) / dt
        else:
            return np.zeros(2)

# ==================== СИСТЕМА СИМУЛЯЦІЇ ====================
class SwarmSimulator:
    def __init__(self, config: Dict[str, Any]):
        self.config = config
        if 'seed' in config:
            np.random.seed(config['seed'])
            random.seed(config['seed'])
        self._init_simulation()

    def _init_simulation(self):
        self.agents = []
        self.metrics = SimulationMetrics()

        positions = self._generate_initial_positions()

        for i in range(self.config['num_agents']):
            route = self._generate_route(i, positions[i])
            specs = AgentSpecs(**self.config['agent_specs'])
            algorithm = self._create_algorithm(self.config['algorithm'])
            agent = Agent(i, positions[i], specs, algorithm, route)
            self.agents.append(agent)

    def _generate_initial_positions(self) -> List[np.ndarray]:
        config = self.config['initial_config']
        area_size = self.config['area_size']
        num_agents = self.config['num_agents']

        if config['type'] == 'grid':
            rows = int(np.ceil(np.sqrt(num_agents)))
            cols = int(np.ceil(num_agents/rows))
            sector_size = area_size / max(rows, cols)
            return [np.array([(i%cols)*sector_size + sector_size/2,
                            (i//cols)*sector_size + sector_size/2])
                   for i in range(num_agents)]

        elif config['type'] == 'random':
            return [np.array([random.uniform(0, area_size), random.uniform(0, area_size)])
                   for _ in range(num_agents)]

        elif config['type'] == 'circle':
            radius = config.get('radius', area_size/3)
            center = np.array([area_size/2, area_size/2])
            angles = np.linspace(0, 2*np.pi, num_agents, endpoint=False)
            return [center + radius * np.array([np.cos(a), np.sin(a)]) for a in angles]

        else:
            raise ValueError(f"Unknown initial config type: {config['type']}")

    def _generate_route(self, agent_id: int, start_pos: np.ndarray) -> List[np.ndarray]:
        route_config = self.config['route_config']
        area_size = self.config['area_size']

        if route_config['type'] == 'snake':
            rows = int(np.ceil(np.sqrt(self.config['num_agents'])))
            cols = int(np.ceil(self.config['num_agents']/rows))
            sector_size = area_size / max(rows, cols)
            row = agent_id // cols
            col = agent_id % cols
            x_start = col * sector_size
            y_start = row * sector_size

            route = []
            steps = 5
            step_size = sector_size/steps
            for i in range(steps):
                y = y_start + (i + 0.5)*step_size
                x_values = [x_start + (j + 0.5)*step_size for j in range(steps)] if i%2==0 else reversed([x_start + (j + 0.5)*step_size for j in range(steps)])
                for x in x_values:
                    route.append(np.array([x, y]))
            return route

        elif route_config['type'] == 'random_waypoints':
            steps = 5
            return [np.array([random.uniform(0, area_size), random.uniform(0, area_size)])
                   for _ in range(steps)]

        elif route_config['type'] == 'common_goal':
            goal = np.array(route_config.get('goal', [area_size/2, area_size/2]))
            return [goal.copy()]

        else:
            raise ValueError(f"Unknown route config type: {route_config['type']}")

    def _create_algorithm(self, algo_config: Dict[str, Any]) -> SteeringAlgorithm:
        algo_type = algo_config['type']
        params = algo_config.get('params', {})

        if algo_type == 'VicsekAlgorithm':
            return VicsekAlgorithm(**params)
        elif algo_type == 'BoidsAlgorithm':
            return BoidsAlgorithm(**params)
        elif algo_type == 'PotentialFieldAlgorithm':
            if 'goal' in params:
                params['goal'] = np.array(params['goal'])
            return PotentialFieldAlgorithm(**params)
        elif algo_type == 'LeaderFollowerAlgorithm':
            return LeaderFollowerAlgorithm(**params)
        else:
            raise ValueError(f"Unknown algorithm type: {algo_type}")

    def run(self):
        for step in range(self.config['steps']):
            self._update_step()
            self._collect_metrics(step)
            self._check_collisions()

    def _update_step(self):
        neighbors = self._find_neighbors()
        for agent, n in zip(self.agents, neighbors):
            agent.update(n, self.config['dt'], self.config['area_size'])

    def _find_neighbors(self) -> List[List[Agent]]:
        positions = np.array([a.position for a in self.agents])
        dist_matrix = np.linalg.norm(positions[:, None] - positions, axis=2)

        neighbors = []
        for i, agent in enumerate(self.agents):
            mask = (dist_matrix[i] <= agent.specs.comm_radius) & (dist_matrix[i] > 0)
            neighbors.append([self.agents[j] for j in np.where(mask)[0]])
        return neighbors

    def _collect_metrics(self, step: int):
        speeds = [np.linalg.norm(a.velocity) for a in self.agents]
        accelerations = [np.linalg.norm(a.acceleration) for a in self.agents]

        self.metrics.avg_speed.append(np.mean(speeds))
        self.metrics.max_speed.append(np.max(speeds))
        self.metrics.avg_acceleration.append(np.mean(accelerations))

        positions = np.array([a.position for a in self.agents])
        dist_matrix = np.linalg.norm(positions[:, None] - positions, axis=2)
        np.fill_diagonal(dist_matrix, np.nan)
        self.metrics.avg_distance.append(np.nanmean(dist_matrix))

        center_of_mass = np.mean(positions, axis=0)
        distances_to_center = np.linalg.norm(positions - center_of_mass, axis=1)
        self.metrics.avg_distance_to_center.append(np.mean(distances_to_center))

        for agent in self.agents:
            if agent.id not in self.metrics.agent_trajectories:
                self.metrics.agent_trajectories[agent.id] = []
            self.metrics.agent_trajectories[agent.id].append(tuple(agent.position))

    def _check_collisions(self):
        positions = np.array([a.position for a in self.agents])
        dist_matrix = np.linalg.norm(positions[:, None] - positions, axis=2)
        np.fill_diagonal(dist_matrix, np.inf)
        self.metrics.collisions += np.sum(dist_matrix < 2*self.agents[0].specs.physical_radius)//2

# ==================== ВІЗУАЛІЗАЦІЯ ====================
class Visualization:
    @staticmethod
    def animate_simulation(simulator: SwarmSimulator, area_size: float):
        fig, ax = plt.subplots(figsize=(10, 10))
        ax.set_xlim(0, area_size)
        ax.set_ylim(0, area_size)
        ax.set_title("Agent Movement Simulation")

        scat = ax.scatter([], [], c='blue', s=10)
        lines = [ax.plot([], [], 'b-', alpha=0.5)[0] for _ in range(simulator.config['num_agents'])]

        def init():
            ax.set_xlim(0, area_size)
            ax.set_ylim(0, area_size)
            return scat, *lines

        def update(frame):
            simulator._update_step()
            positions = np.array([agent.position for agent in simulator.agents])

            scat.set_offsets(positions)

            for idx, agent in enumerate(simulator.agents):
                trajectory = np.array(agent.path_history)
                lines[idx].set_data(trajectory[:, 0], trajectory[:, 1])

            return scat, *lines

        ani = FuncAnimation(fig, update, frames=simulator.config['steps'], init_func=init, blit=True, interval=50, repeat=False)
        plt.show()

# ==================== ІНТЕРФЕЙС ДЛЯ COLAB ====================
def run_simulation_in_colab():
    # Стандартна конфігурація
    config = {
        "num_agents": 20,
        "area_size": 100,
        "steps": 500,
        "dt": 0.1,
        "seed": 42,
        "agent_specs": {
            "physical_radius": 0.5,
            "safety_margin": 1.0,
            "max_speed": 2.0,
            "max_acceleration": 1.5,
            "comm_radius": 15.0
        },
        "initial_config": {
            "type": "circle",
            "radius": 30
        },
        "route_config": {
            "type": "common_goal",
            "goal": [50, 50]
        },
        "algorithm": {
            "type": "BoidsAlgorithm",
            "params": {
                "sep_weight": 1.5,
                "align_weight": 0.8,
                "coh_weight": 0.5,
                "visual_range": 20.0
            }
        }
    }

    print("Running simulation with default configuration...")
    simulator = SwarmSimulator(config)

    print("\nVisualizing results...")
    Visualization.animate_simulation(simulator, config['area_size'])

# Запуск симуляції в Colab
run_simulation_in_colab()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from dataclasses import dataclass
from abc import ABC, abstractmethod
from typing import List, Dict, Tuple, Optional, Any
import random
from IPython.display import HTML

# ==================== БАЗОВІ КЛАСИ ====================
@dataclass
class AgentSpecs:
    physical_radius: float
    safety_margin: float
    max_speed: float
    max_acceleration: float
    comm_radius: float

class SteeringAlgorithm(ABC):
    @abstractmethod
    def compute_steering(self, agent, neighbors: List['Agent'], dt: float) -> np.ndarray:
        pass

@dataclass
class SimulationMetrics:
    collisions: int = 0
    avg_speed: List[float] = None
    max_speed: List[float] = None
    avg_acceleration: List[float] = None
    avg_distance: List[float] = None
    avg_distance_to_center: List[float] = None
    agent_trajectories: Dict[int, List[Tuple[float, float]]] = None

    def __post_init__(self):
        self.avg_speed = []
        self.max_speed = []
        self.avg_acceleration = []
        self.avg_distance = []
        self.avg_distance_to_center = []
        self.agent_trajectories = {}

class Agent:
    def __init__(self, agent_id: int, position: np.ndarray, specs: AgentSpecs,
                 algorithm: SteeringAlgorithm, route: List[np.ndarray]):
        self.id = agent_id
        self.position = position
        self.velocity = np.zeros(2)
        self.acceleration = np.zeros(2)
        self.specs = specs
        self.algorithm = algorithm
        self.route = route
        self.current_waypoint = 0
        self.path_history = [position.copy()]

    def update(self, neighbors: List['Agent'], dt: float, area_size: float):
        steering = self.algorithm.compute_steering(self, neighbors, dt)
        steering = self._limit_vector(steering, self.specs.max_acceleration)
        self.velocity = self._limit_vector(self.velocity + steering * dt, self.specs.max_speed)
        self.position += self.velocity * dt
        self.acceleration = steering
        self._enforce_boundaries(area_size)
        self.path_history.append(self.position.copy())

    def _limit_vector(self, vector: np.ndarray, max_magnitude: float) -> np.ndarray:
        magnitude = np.linalg.norm(vector)
        return vector if magnitude == 0 else vector * min(max_magnitude/magnitude, 1)

    def _enforce_boundaries(self, area_size: float):
        self.position = np.clip(self.position, 0, area_size)

# ==================== АЛГОРИТМИ КЕРУВАННЯ ====================
class VicsekAlgorithm(SteeringAlgorithm):
    def __init__(self, noise: float = 0.1, neighbor_radius: float = 15.0):
        self.noise = noise
        self.neighbor_radius = neighbor_radius

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        neighbors = [n for n in neighbors if np.linalg.norm(n.position - agent.position) <= self.neighbor_radius]
        if not neighbors:
            return np.zeros(2)
        avg_velocity = np.mean([n.velocity for n in neighbors], axis=0)
        noise = np.random.normal(scale=self.noise, size=2)
        return (avg_velocity + noise - agent.velocity) / dt

class BoidsAlgorithm(SteeringAlgorithm):
    def __init__(self, sep_weight: float = 1.5, align_weight: float = 1.0,
                 coh_weight: float = 1.0, visual_range: float = 15.0):
        self.weights = np.array([sep_weight, align_weight, coh_weight])
        self.visual_range = visual_range

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        neighbors = [n for n in neighbors if np.linalg.norm(n.position - agent.position) <= self.visual_range]
        if not neighbors:
            return np.zeros(2)
        positions = np.array([n.position for n in neighbors])
        velocities = np.array([n.velocity for n in neighbors])
        sep = self._separation(agent.position, positions)
        align = self._alignment(agent.velocity, velocities)
        coh = self._cohesion(agent.position, positions)
        return self.weights @ np.array([sep, align, coh])

    def _separation(self, pos: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        diff = pos - neighbors
        distances = np.linalg.norm(diff, axis=1)[:, np.newaxis]
        return np.mean(diff / (distances + 1e-6), axis=0)

    def _alignment(self, vel: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        return np.mean(neighbors, axis=0) - vel

    def _cohesion(self, pos: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        return np.mean(neighbors, axis=0) - pos

class PotentialFieldAlgorithm(SteeringAlgorithm):
    def __init__(self, alpha: float = 0.5, beta: float = 1.0,
                 goal: Optional[np.ndarray] = None, dynamic_goal: bool = False):
        self.goal = goal
        self.alpha = alpha
        self.beta = beta
        self.dynamic_goal = dynamic_goal

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        if self.dynamic_goal and agent.route:
            self.goal = agent.route[-1]
        to_goal = self.goal - agent.position if self.goal is not None else np.zeros(2)
        attraction = self.alpha * to_goal
        repulsion = np.zeros(2)
        for n in neighbors:
            vec = agent.position - n.position
            dist = np.linalg.norm(vec)
            if dist > 0:
                repulsion += self.beta * vec / dist**2
        return attraction + repulsion - agent.velocity / dt

class LeaderFollowerAlgorithm(SteeringAlgorithm):
    def __init__(self, leader_id: int = 0, follow_distance: float = 5.0):
        self.leader_id = leader_id
        self.follow_distance = follow_distance

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        leader = next((n for n in neighbors if n.id == self.leader_id), None)
        if leader is None:
            return np.zeros(2)
        direction = leader.position - agent.position
        distance = np.linalg.norm(direction)
        if distance > 0:
            desired_velocity = (direction / distance) * agent.specs.max_speed
            return (desired_velocity - agent.velocity) / dt
        else:
            return np.zeros(2)

# ==================== СИСТЕМА СИМУЛЯЦІЇ ====================
class SwarmSimulator:
    def __init__(self, config: Dict[str, Any]):
        self.config = config
        if 'seed' in config:
            np.random.seed(config['seed'])
            random.seed(config['seed'])
        self._init_simulation()

    def _init_simulation(self):
        self.agents = []
        self.metrics = SimulationMetrics()
        positions = self._generate_initial_positions()
        for i in range(self.config['num_agents']):
            route = self._generate_route(i, positions[i])
            specs = AgentSpecs(**self.config['agent_specs'])
            algorithm = self._create_algorithm(self.config['algorithm'])
            agent = Agent(i, positions[i], specs, algorithm, route)
            self.agents.append(agent)

    def _generate_initial_positions(self) -> List[np.ndarray]:
        config = self.config['initial_config']
        area_size = self.config['area_size']
        num_agents = self.config['num_agents']
        if config['type'] == 'grid':
            rows = int(np.ceil(np.sqrt(num_agents)))
            cols = int(np.ceil(num_agents/rows))
            sector_size = area_size / max(rows, cols)
            return [np.array([(i%cols)*sector_size + sector_size/2,
                            (i//cols)*sector_size + sector_size/2])
                   for i in range(num_agents)]
        elif config['type'] == 'random':
            return [np.array([random.uniform(0, area_size), random.uniform(0, area_size)])
                   for _ in range(num_agents)]
        elif config['type'] == 'circle':
            radius = config.get('radius', area_size/3)
            center = np.array([area_size/2, area_size/2])
            angles = np.linspace(0, 2*np.pi, num_agents, endpoint=False)
            return [center + radius * np.array([np.cos(a), np.sin(a)]) for a in angles]
        else:
            raise ValueError(f"Unknown initial config type: {config['type']}")

    def _generate_route(self, agent_id: int, start_pos: np.ndarray) -> List[np.ndarray]:
        route_config = self.config['route_config']
        area_size = self.config['area_size']
        if route_config['type'] == 'snake':
            rows = int(np.ceil(np.sqrt(self.config['num_agents'])))
            cols = int(np.ceil(self.config['num_agents']/rows))
            sector_size = area_size / max(rows, cols)
            row = agent_id // cols
            col = agent_id % cols
            x_start = col * sector_size
            y_start = row * sector_size
            route = []
            steps = 5
            step_size = sector_size/steps
            for i in range(steps):
                y = y_start + (i + 0.5)*step_size
                x_values = [x_start + (j + 0.5)*step_size for j in range(steps)] if i%2==0 else reversed([x_start + (j + 0.5)*step_size for j in range(steps)])
                for x in x_values:
                    route.append(np.array([x, y]))
            return route
        elif route_config['type'] == 'random_waypoints':
            steps = 5
            return [np.array([random.uniform(0, area_size), random.uniform(0, area_size)])
                   for _ in range(steps)]
        elif route_config['type'] == 'common_goal':
            goal = np.array(route_config.get('goal', [area_size/2, area_size/2]))
            return [goal.copy()]
        else:
            raise ValueError(f"Unknown route config type: {route_config['type']}")

    def _create_algorithm(self, algo_config: Dict[str, Any]) -> SteeringAlgorithm:
        algo_type = algo_config['type']
        params = algo_config.get('params', {})
        if algo_type == 'VicsekAlgorithm':
            return VicsekAlgorithm(**params)
        elif algo_type == 'BoidsAlgorithm':
            return BoidsAlgorithm(**params)
        elif algo_type == 'PotentialFieldAlgorithm':
            if 'goal' in params:
                params['goal'] = np.array(params['goal'])
            return PotentialFieldAlgorithm(**params)
        elif algo_type == 'LeaderFollowerAlgorithm':
            return LeaderFollowerAlgorithm(**params)
        else:
            raise ValueError(f"Unknown algorithm type: {algo_type}")

    def update_step(self):  # Змінили назву на публічний метод
        neighbors = self._find_neighbors()
        for agent, n in zip(self.agents, neighbors):
            agent.update(n, self.config['dt'], self.config['area_size'])

    def _find_neighbors(self) -> List[List[Agent]]:
        positions = np.array([a.position for a in self.agents])
        dist_matrix = np.linalg.norm(positions[:, None] - positions, axis=2)
        neighbors = []
        for i, agent in enumerate(self.agents):
            mask = (dist_matrix[i] <= agent.specs.comm_radius) & (dist_matrix[i] > 0)
            neighbors.append([self.agents[j] for j in np.where(mask)[0]])
        return neighbors

# ==================== ВІЗУАЛІЗАЦІЯ ====================
class Visualization:
    @staticmethod
    def animate_simulation(simulator: SwarmSimulator, area_size: float):
        fig, ax = plt.subplots(figsize=(10, 10))
        ax.set_xlim(0, area_size)
        ax.set_ylim(0, area_size)
        ax.set_title("Agent Movement Simulation")

        scat = ax.scatter([], [], c='blue', s=10)
        lines = [ax.plot([], [], 'b-', alpha=0.5)[0] for _ in range(simulator.config['num_agents'])]

        def init():
            ax.set_xlim(0, area_size)
            ax.set_ylim(0, area_size)
            return scat, *lines

        def update(frame):
            simulator.update_step()
            positions = np.array([agent.position for agent in simulator.agents])
            scat.set_offsets(positions)
            for idx, agent in enumerate(simulator.agents):
                trajectory = np.array(agent.path_history)
                lines[idx].set_data(trajectory[:, 0], trajectory[:, 1])
            return scat, *lines

        ani = FuncAnimation(fig, update, frames=simulator.config['steps'], init_func=init, blit=True, interval=50)
        return ani  # Повертаємо об'єкт анімації замість plt.show()

# ==================== ІНТЕРФЕЙС ДЛЯ COLAB ====================
def run_simulation_in_colab():
    # Стандартна конфігурація
    config = {
        "num_agents": 20,
        "area_size": 100,
        "steps": 500,
        "dt": 0.1,
        "seed": 42,
        "agent_specs": {
            "physical_radius": 0.5,
            "safety_margin": 1.0,
            "max_speed": 2.0,
            "max_acceleration": 1.5,
            "comm_radius": 15.0
        },
        "initial_config": {
            "type": "circle",
            "radius": 30
        },
        "route_config": {
            "type": "common_goal",
            "goal": [50, 50]
        },
        "algorithm": {
            "type": "BoidsAlgorithm",
            "params": {
                "sep_weight": 1.5,
                "align_weight": 0.8,
                "coh_weight": 0.5,
                "visual_range": 20.0
            }
        }
    }

    print("Running simulation with default configuration...")
    simulator = SwarmSimulator(config)

    print("\nVisualizing results...")
    ani = Visualization.animate_simulation(simulator, config['area_size'])
    return HTML(ani.to_html5_video())  # Відображаємо анімацію як HTML5-відео

# Запуск симуляції в Colab
run_simulation_in_colab()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from dataclasses import dataclass
from abc import ABC, abstractmethod
from typing import List, Dict, Tuple, Optional, Any
import random

# ==================== БАЗОВІ КЛАСИ ====================
@dataclass
class AgentSpecs:
    physical_radius: float
    safety_margin: float
    max_speed: float
    max_acceleration: float
    comm_radius: float

class SteeringAlgorithm(ABC):
    @abstractmethod
    def compute_steering(self, agent, neighbors: List['Agent'], dt: float) -> np.ndarray:
        pass

@dataclass
class SimulationMetrics:
    collisions: int = 0
    avg_speed: List[float] = None
    max_speed: List[float] = None
    avg_acceleration: List[float] = None
    avg_distance: List[float] = None
    avg_distance_to_center: List[float] = None
    agent_trajectories: Dict[int, List[Tuple[float, float]]] = None

    def __post_init__(self):
        self.avg_speed = []
        self.max_speed = []
        self.avg_acceleration = []
        self.avg_distance = []
        self.avg_distance_to_center = []
        self.agent_trajectories = {}

class Agent:
    def __init__(self, agent_id: int, position: np.ndarray, specs: AgentSpecs,
                 algorithm: SteeringAlgorithm, route: List[np.ndarray]):
        self.id = agent_id
        self.position = position
        self.velocity = np.zeros(2)
        self.acceleration = np.zeros(2)
        self.specs = specs
        self.algorithm = algorithm
        self.route = route
        self.current_waypoint = 0
        self.path_history = [position.copy()]

    def update(self, neighbors: List['Agent'], dt: float, area_size: float):
        steering = self.algorithm.compute_steering(self, neighbors, dt)
        steering = self._limit_vector(steering, self.specs.max_acceleration)
        self.velocity = self._limit_vector(self.velocity + steering * dt, self.specs.max_speed)
        self.position += self.velocity * dt
        self.acceleration = steering
        self._enforce_boundaries(area_size)
        self.path_history.append(self.position.copy())

    def _limit_vector(self, vector: np.ndarray, max_magnitude: float) -> np.ndarray:
        magnitude = np.linalg.norm(vector)
        return vector if magnitude == 0 else vector * min(max_magnitude/magnitude, 1)

    def _enforce_boundaries(self, area_size: float):
        self.position = np.clip(self.position, 0, area_size)

# ==================== АЛГОРИТМИ КЕРУВАННЯ ====================
class VicsekAlgorithm(SteeringAlgorithm):
    def __init__(self, noise: float = 0.1, neighbor_radius: float = 15.0):
        self.noise = noise
        self.neighbor_radius = neighbor_radius

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        neighbors = [n for n in neighbors if np.linalg.norm(n.position - agent.position) <= self.neighbor_radius]
        if not neighbors:
            return np.zeros(2)
        avg_velocity = np.mean([n.velocity for n in neighbors], axis=0)
        noise = np.random.normal(scale=self.noise, size=2)
        return (avg_velocity + noise - agent.velocity) / dt

class BoidsAlgorithm(SteeringAlgorithm):
    def __init__(self, sep_weight: float = 1.5, align_weight: float = 1.0,
                 coh_weight: float = 1.0, visual_range: float = 15.0):
        self.weights = np.array([sep_weight, align_weight, coh_weight])
        self.visual_range = visual_range

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        neighbors = [n for n in neighbors if np.linalg.norm(n.position - agent.position) <= self.visual_range]
        if not neighbors:
            return np.zeros(2)
        positions = np.array([n.position for n in neighbors])
        velocities = np.array([n.velocity for n in neighbors])
        sep = self._separation(agent.position, positions)
        align = self._alignment(agent.velocity, velocities)
        coh = self._cohesion(agent.position, positions)
        return self.weights @ np.array([sep, align, coh])

    def _separation(self, pos: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        diff = pos - neighbors
        distances = np.linalg.norm(diff, axis=1)[:, np.newaxis]
        return np.mean(diff / (distances + 1e-6), axis=0)

    def _alignment(self, vel: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        return np.mean(neighbors, axis=0) - vel

    def _cohesion(self, pos: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        return np.mean(neighbors, axis=0) - pos

class PotentialFieldAlgorithm(SteeringAlgorithm):
    def __init__(self, alpha: float = 0.5, beta: float = 1.0,
                 goal: Optional[np.ndarray] = None, dynamic_goal: bool = False):
        self.goal = goal
        self.alpha = alpha
        self.beta = beta
        self.dynamic_goal = dynamic_goal

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        if self.dynamic_goal and agent.route:
            self.goal = agent.route[-1]
        to_goal = self.goal - agent.position if self.goal is not None else np.zeros(2)
        attraction = self.alpha * to_goal
        repulsion = np.zeros(2)
        for n in neighbors:
            vec = agent.position - n.position
            dist = np.linalg.norm(vec)
            if dist > 0:
                repulsion += self.beta * vec / dist**2
        return attraction + repulsion - agent.velocity / dt

class LeaderFollowerAlgorithm(SteeringAlgorithm):
    def __init__(self, leader_id: int = 0, follow_distance: float = 5.0):
        self.leader_id = leader_id
        self.follow_distance = follow_distance

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        leader = next((n for n in neighbors if n.id == self.leader_id), None)
        if leader is None:
            return np.zeros(2)
        direction = leader.position - agent.position
        distance = np.linalg.norm(direction)
        if distance > 0:
            desired_velocity = (direction / distance) * agent.specs.max_speed
            return (desired_velocity - agent.velocity) / dt
        else:
            return np.zeros(2)

class RandomWalkAlgorithm(SteeringAlgorithm):
    def __init__(self, max_random_acceleration: float = 0.1):
        self.max_random_acceleration = max_random_acceleration

    def compute_steering(self, agent: Agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        random_acceleration = np.random.uniform(
            low=-self.max_random_acceleration,
            high=self.max_random_acceleration,
            size=(2,)
        )
        return random_acceleration

# ==================== СИСТЕМА СИМУЛЯЦІЇ ====================
class SwarmSimulator:
    def __init__(self, config: Dict[str, Any]):
        self.config = config
        if 'seed' in config:
            np.random.seed(config['seed'])
            random.seed(config['seed'])
        self._init_simulation()

    def _init_simulation(self):
        self.agents = []
        self.metrics = SimulationMetrics()
        positions = self._generate_initial_positions()
        for i in range(self.config['num_agents']):
            route = self._generate_route(i, positions[i])
            specs = AgentSpecs(**self.config['agent_specs'])
            algorithm = self._create_algorithm(self.config['algorithm'])
            agent = Agent(i, positions[i], specs, algorithm, route)
            self.agents.append(agent)

    def _generate_initial_positions(self) -> List[np.ndarray]:
        config = self.config['initial_config']
        area_size = self.config['area_size']
        num_agents = self.config['num_agents']
        if config['type'] == 'grid':
            rows = int(np.ceil(np.sqrt(num_agents)))
            cols = int(np.ceil(num_agents/rows))
            sector_size = area_size / max(rows, cols)
            return [np.array([(i%cols)*sector_size + sector_size/2,
                            (i//cols)*sector_size + sector_size/2])
                    for i in range(num_agents)]
        elif config['type'] == 'random':
            return [np.array([random.uniform(0, area_size), random.uniform(0, area_size)])
                    for _ in range(num_agents)]
        elif config['type'] == 'circle':
            radius = config.get('radius', area_size/3)
            center = np.array([area_size/2, area_size/2])
            angles = np.linspace(0, 2*np.pi, num_agents, endpoint=False)
            return [center + radius * np.array([np.cos(a), np.sin(a)]) for a in angles]
        else:
            raise ValueError(f"Unknown initial config type: {config['type']}")

    def _generate_route(self, agent_id: int, start_pos: np.ndarray) -> List[np.ndarray]:
        route_config = self.config['route_config']
        area_size = self.config['area_size']
        if route_config['type'] == 'snake':
            rows = int(np.ceil(np.sqrt(self.config['num_agents'])))
            cols = int(np.ceil(self.config['num_agents']/rows))
            sector_size = area_size / max(rows, cols)
            row = agent_id // cols
            col = agent_id % cols
            x_start = col * sector_size
            y_start = row * sector_size
            route = []
            steps = 5
            step_size = sector_size/steps
            for i in range(steps):
                y = y_start + (i + 0.5)*step_size
                x_values = [x_start + (j + 0.5)*step_size for j in range(steps)] if i%2==0 else reversed([x_start + (j + 0.5)*step_size for j in range(steps)])
                for x in x_values:
                    route.append(np.array([x, y]))
            return route
        elif route_config['type'] == 'random_waypoints':
            steps = 5
            return [np.array([random.uniform(0, area_size), random.uniform(0, area_size)])
                    for _ in range(steps)]
        elif route_config['type'] == 'common_goal':
            goal = np.array(route_config.get('goal', [area_size/2, area_size/2]))
            return [goal.copy()]
        else:
            raise ValueError(f"Unknown route config type: {route_config['type']}")

    def _create_algorithm(self, algo_config: Dict[str, Any]) -> SteeringAlgorithm:
        algo_type = algo_config['type']
        params = algo_config.get('params', {})
        if algo_type == 'VicsekAlgorithm':
            return VicsekAlgorithm(**params)
        elif algo_type == 'BoidsAlgorithm':
            return BoidsAlgorithm(**params)
        elif algo_type == 'PotentialFieldAlgorithm':
            if 'goal' in params:
                params['goal'] = np.array(params['goal'])
            return PotentialFieldAlgorithm(**params)
        elif algo_type == 'LeaderFollowerAlgorithm':
            return LeaderFollowerAlgorithm(**params)
        elif algo_type == 'RandomWalkAlgorithm':
            return RandomWalkAlgorithm(**params)  # Додано обробку RandomWalkAlgorithm
        else:
            raise ValueError(f"Unknown algorithm type: {algo_type}")

    def update_step(self):  # Змінили назву на публічний метод
        neighbors = self._find_neighbors()
        for agent, n in zip(self.agents, neighbors):
            agent.update(n, self.config['dt'], self.config['area_size'])

    def _find_neighbors(self) -> List[List[Agent]]:
        positions = np.array([a.position for a in self.agents])
        dist_matrix = np.linalg.norm(positions[:, None] - positions, axis=2)
        neighbors = []
        for i, agent in enumerate(self.agents):
            mask = (dist_matrix[i] <= agent.specs.comm_radius) & (dist_matrix[i] > 0)
            neighbors.append([self.agents[j] for j in np.where(mask)[0]])
        return neighbors

# ==================== ВІЗУАЛІЗАЦІЯ ====================
class Visualization:
    @staticmethod
    def animate_simulation(simulator: SwarmSimulator, area_size: float):
        fig, ax = plt.subplots(figsize=(10, 10))
        ax.set_xlim(0, area_size)
        ax.set_ylim(0, area_size)
        ax.set_title("Agent Movement Simulation")

        scat = ax.scatter([], [], c='blue', s=10)
        lines = [ax.plot([], [], 'b-', alpha=0.5)[0] for _ in range(simulator.config['num_agents'])]

        def init():
            ax.set_xlim(0, area_size)
            ax.set_ylim(0, area_size)
            return scat, *lines

        def update(frame):
            simulator.update_step()
            positions = np.array([agent.position for agent in simulator.agents])
            scat.set_offsets(positions)
            for idx, agent in enumerate(simulator.agents):
                trajectory = np.array(agent.path_history)
                lines[idx].set_data(trajectory[:, 0], trajectory[:, 1])
            return scat, *lines

        ani = FuncAnimation(fig, update, frames=simulator.config['steps'], init_func=init, blit=True, interval=50)
        plt.show() # blocking, хороше рішення для локального середовища.
        return ani

# ==================== ІНТЕРФЕЙС ДЛЯ COLAB ====================
def run_simulation_in_colab():
    # Стандартна конфігурація
    config = {
        "num_agents": 20,
        "area_size": 100,
        "steps": 500,
        "dt": 0.1,
        "seed": 42,
        "agent_specs": {
            "physical_radius": 0.5,
            "safety_margin": 1.0,
            "max_speed": 2.0,
            "max_acceleration": 1.5,
            "comm_radius": 15.0
        },
        "initial_config": {
            "type": "circle",
            "radius": 30
        },
        "route_config": {
            "type": "common_goal",
            "goal": [50, 50]
        },
        "algorithm": {
            "type": "RandomWalkAlgorithm", # Змінено на RandomWalkAlgorithm
            "params": {
                "max_random_acceleration": 0.5, # Параметр для RandomWalkAlgorithm
            }
        }
    }

    print("Running simulation with RandomWalkAlgorithm...")
    simulator = SwarmSimulator(config)

    print("\nVisualizing results...")
    Visualization.animate_simulation(simulator, config['area_size']) # Блокуючий виклик, Colab відобразить анімацію.


# Запуск симуляції в Colab
run_simulation_in_colab()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from dataclasses import dataclass
from abc import ABC, abstractmethod
from typing import List, Dict, Tuple, Optional, Any
import random
import base64
from io import BytesIO
import matplotlib.animation as animation
import tempfile
import os

# ==================== БАЗОВІ КЛАСИ ====================
@dataclass
class AgentSpecs:
    physical_radius: float
    safety_margin: float
    max_speed: float
    max_acceleration: float
    comm_radius: float

class SteeringAlgorithm(ABC):
    @abstractmethod
    def compute_steering(self, agent, neighbors: List['Agent'], dt: float) -> np.ndarray:
        pass

@dataclass
class SimulationMetrics:
    collisions: int = 0
    avg_speed: List[float] = None
    max_speed: List[float] = None
    avg_acceleration: List[float] = None
    avg_distance: List[float] = None
    avg_distance_to_center: List[float] = None
    agent_trajectories: Dict[int, List[Tuple[float, float]]] = None

    def __post_init__(self):
        self.avg_speed = []
        self.max_speed = []
        self.avg_acceleration = []
        self.avg_distance = []
        self.avg_distance_to_center = []
        self.agent_trajectories = {}

class Agent:
    def __init__(self, agent_id: int, position: np.ndarray, specs: AgentSpecs,
                 algorithm: SteeringAlgorithm, route: List[np.ndarray]):
        self.id = agent_id
        self.position = position
        self.velocity = np.zeros(2)
        self.acceleration = np.zeros(2)
        self.specs = specs
        self.algorithm = algorithm
        self.route = route
        self.current_waypoint = 0
        self.path_history = [position.copy()]

    def update(self, neighbors: List['Agent'], dt: float, area_size: float):
        steering = self.algorithm.compute_steering(self, neighbors, dt)
        steering = self._limit_vector(steering, self.specs.max_acceleration)
        self.velocity = self._limit_vector(self.velocity + steering * dt, self.specs.max_speed)
        self.position += self.velocity * dt
        self.acceleration = steering
        self._enforce_boundaries(area_size)
        self.path_history.append(self.position.copy())

    def _limit_vector(self, vector: np.ndarray, max_magnitude: float) -> np.ndarray:
        magnitude = np.linalg.norm(vector)
        return vector if magnitude == 0 else vector * min(max_magnitude/magnitude, 1)

    def _enforce_boundaries(self, area_size: float):
        self.position = np.clip(self.position, 0, area_size)

# ==================== АЛГОРИТМИ КЕРУВАННЯ ====================
class VicsekAlgorithm(SteeringAlgorithm):
    def __init__(self, noise: float = 0.1, neighbor_radius: float = 15.0):
        self.noise = noise
        self.neighbor_radius = neighbor_radius

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        neighbors = [n for n in neighbors if np.linalg.norm(n.position - agent.position) <= self.neighbor_radius]
        if not neighbors:
            return np.zeros(2)
        avg_velocity = np.mean([n.velocity for n in neighbors], axis=0)
        noise = np.random.normal(scale=self.noise, size=2)
        return (avg_velocity + noise - agent.velocity) / dt

class BoidsAlgorithm(SteeringAlgorithm):
    def __init__(self, sep_weight: float = 1.5, align_weight: float = 1.0,
                 coh_weight: float = 1.0, visual_range: float = 15.0):
        self.weights = np.array([sep_weight, align_weight, coh_weight])
        self.visual_range = visual_range

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        neighbors = [n for n in neighbors if np.linalg.norm(n.position - agent.position) <= self.visual_range]
        if not neighbors:
            return np.zeros(2)
        positions = np.array([n.position for n in neighbors])
        velocities = np.array([n.velocity for n in neighbors])
        sep = self._separation(agent.position, positions)
        align = self._alignment(agent.velocity, velocities)
        coh = self._cohesion(agent.position, positions)
        return self.weights @ np.array([sep, align, coh])

    def _separation(self, pos: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        diff = pos - neighbors
        distances = np.linalg.norm(diff, axis=1)[:, np.newaxis]
        return np.mean(diff / (distances + 1e-6), axis=0)

    def _alignment(self, vel: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        return np.mean(neighbors, axis=0) - vel

    def _cohesion(self, pos: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        return np.mean(neighbors, axis=0) - pos

class PotentialFieldAlgorithm(SteeringAlgorithm):
    def __init__(self, alpha: float = 0.5, beta: float = 1.0,
                 goal: Optional[np.ndarray] = None, dynamic_goal: bool = False):
        self.goal = goal
        self.alpha = alpha
        self.beta = beta
        self.dynamic_goal = dynamic_goal

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        if self.dynamic_goal and agent.route:
            self.goal = agent.route[-1]
        to_goal = self.goal - agent.position if self.goal is not None else np.zeros(2)
        attraction = self.alpha * to_goal
        repulsion = np.zeros(2)
        for n in neighbors:
            vec = agent.position - n.position
            dist = np.linalg.norm(vec)
            if dist > 0:
                repulsion += self.beta * vec / dist**2
        return attraction + repulsion - agent.velocity / dt

class LeaderFollowerAlgorithm(SteeringAlgorithm):
    def __init__(self, leader_id: int = 0, follow_distance: float = 5.0):
        self.leader_id = leader_id
        self.follow_distance = follow_distance

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        leader = next((n for n in neighbors if n.id == self.leader_id), None)
        if leader is None:
            return np.zeros(2)
        direction = leader.position - agent.position
        distance = np.linalg.norm(direction)
        if distance > 0:
            desired_velocity = (direction / distance) * agent.specs.max_speed
            return (desired_velocity - agent.velocity) / dt
        else:
            return np.zeros(2)

class RandomWalkAlgorithm(SteeringAlgorithm):
    def __init__(self, max_random_acceleration: float = 0.1):
        self.max_random_acceleration = max_random_acceleration

    def compute_steering(self, agent: Agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        random_acceleration = np.random.uniform(
            low=-self.max_random_acceleration,
            high=self.max_random_acceleration,
            size=(2,)
        )
        return random_acceleration

# ==================== СИСТЕМА СИМУЛЯЦІЇ ====================
class SwarmSimulator:
    def __init__(self, config: Dict[str, Any]):
        self.config = config
        if 'seed' in config:
            np.random.seed(config['seed'])
            random.seed(config['seed'])
        self._init_simulation()

    def _init_simulation(self):
        self.agents = []
        self.metrics = SimulationMetrics()
        positions = self._generate_initial_positions()
        for i in range(self.config['num_agents']):
            route = self._generate_route(i, positions[i])
            specs = AgentSpecs(**self.config['agent_specs'])
            algorithm = self._create_algorithm(self.config['algorithm'])
            agent = Agent(i, positions[i], specs, algorithm, route)
            self.agents.append(agent)

    def _generate_initial_positions(self) -> List[np.ndarray]:
        config = self.config['initial_config']
        area_size = self.config['area_size']
        num_agents = self.config['num_agents']
        if config['type'] == 'grid':
            rows = int(np.ceil(np.sqrt(num_agents)))
            cols = int(np.ceil(num_agents/rows))
            sector_size = area_size / max(rows, cols)
            return [np.array([(i%cols)*sector_size + sector_size/2,
                            (i//cols)*sector_size + sector_size/2])
                    for i in range(num_agents)]
        elif config['type'] == 'random':
            return [np.array([random.uniform(0, area_size), random.uniform(0, area_size)])
                    for _ in range(num_agents)]
        elif config['type'] == 'circle':
            radius = config.get('radius', area_size/3)
            center = np.array([area_size/2, area_size/2])
            angles = np.linspace(0, 2*np.pi, num_agents, endpoint=False)
            return [center + radius * np.array([np.cos(a), np.sin(a)]) for a in angles]
        else:
            raise ValueError(f"Unknown initial config type: {config['type']}")

    def _generate_route(self, agent_id: int, start_pos: np.ndarray) -> List[np.ndarray]:
        route_config = self.config['route_config']
        area_size = self.config['area_size']
        if route_config['type'] == 'snake':
            rows = int(np.ceil(np.sqrt(self.config['num_agents'])))
            cols = int(np.ceil(self.config['num_agents']/rows))
            sector_size = area_size / max(rows, cols)
            row = agent_id // cols
            col = agent_id % cols
            x_start = col * sector_size
            y_start = row * sector_size
            route = []
            steps = 5
            step_size = sector_size/steps
            for i in range(steps):
                y = y_start + (i + 0.5)*step_size
                x_values = [x_start + (j + 0.5)*step_size for j in range(steps)] if i%2==0 else reversed([x_start + (j + 0.5)*step_size for j in range(steps)])
                for x in x_values:
                    route.append(np.array([x, y]))
            return route
        elif route_config['type'] == 'random_waypoints':
            steps = 5
            return [np.array([random.uniform(0, area_size), random.uniform(0, area_size)])
                    for _ in range(steps)]
        elif route_config['type'] == 'common_goal':
            goal = np.array(route_config.get('goal', [area_size/2, area_size/2]))
            return [goal.copy()]
        else:
            raise ValueError(f"Unknown route config type: {route_config['type']}")

    def _create_algorithm(self, algo_config: Dict[str, Any]) -> SteeringAlgorithm:
        algo_type = algo_config['type']
        params = algo_config.get('params', {})
        if algo_type == 'VicsekAlgorithm':
            return VicsekAlgorithm(**params)
        elif algo_type == 'BoidsAlgorithm':
            return BoidsAlgorithm(**params)
        elif algo_type == 'PotentialFieldAlgorithm':
            if 'goal' in params:
                params['goal'] = np.array(params['goal'])
            return PotentialFieldAlgorithm(**params)
        elif algo_type == 'LeaderFollowerAlgorithm':
            return LeaderFollowerAlgorithm(**params)
        elif algo_type == 'RandomWalkAlgorithm':
            return RandomWalkAlgorithm(**params)  # Додано обробку RandomWalkAlgorithm
        else:
            raise ValueError(f"Unknown algorithm type: {algo_type}")

    def update_step(self):  # Змінили назву на публічний метод
        neighbors = self._find_neighbors()
        for agent, n in zip(self.agents, neighbors):
            agent.update(n, self.config['dt'], self.config['area_size'])

    def _find_neighbors(self) -> List[List[Agent]]:
        positions = np.array([a.position for a in self.agents])
        dist_matrix = np.linalg.norm(positions[:, None] - positions, axis=2)
        neighbors = []
        for i, agent in enumerate(self.agents):
            mask = (dist_matrix[i] <= agent.specs.comm_radius) & (dist_matrix[i] > 0)
            neighbors.append([self.agents[j] for j in np.where(mask)[0]])
        return neighbors

# ==================== ВІЗУАЛІЗАЦІЯ ====================
class Visualization:
    @staticmethod
    def animate_simulation(simulator: SwarmSimulator, area_size: float):
        fig, ax = plt.subplots(figsize=(10, 10))
        ax.set_xlim(0, area_size)
        ax.set_ylim(0, area_size)
        ax.set_title("Agent Movement Simulation")

        scat = ax.scatter([], [], c='blue', s=10)
        lines = [ax.plot([], [], 'b-', alpha=0.5)[0] for _ in range(simulator.config['num_agents'])]

        def init():
            ax.set_xlim(0, area_size)
            ax.set_ylim(0, area_size)
            return scat, *lines

        def update(frame):
            simulator.update_step()
            positions = np.array([agent.position for agent in simulator.agents])
            scat.set_offsets(positions)
            for idx, agent in enumerate(simulator.agents):
                trajectory = np.array(agent.path_history)
                lines[idx].set_data(trajectory[:, 0], trajectory[:, 1])
            return scat, *lines

        ani = FuncAnimation(fig, update, frames=simulator.config['steps'], init_func=init, blit=True, interval=50)

        # Use a writer that is more likely to be available in a Colab environment
        Writer = animation.writers['html']  # Use the 'html' writer
        writer = Writer(fps=15, metadata=dict(artist='Swarm Simulator'))

        # Save the animation to a temporary file
        with tempfile.NamedTemporaryFile(suffix='.html', delete=False) as tmp_file:
            ani.save(tmp_file.name, writer=writer)
            html_output = open(tmp_file.name, 'r').read() # Read the file

        plt.close(fig)  # Close the plot to prevent it from displaying
        os.remove(tmp_file.name) # Remove the temp file

        return html_output

# ==================== ІНТЕРФЕЙС ДЛЯ COLAB ====================
def run_simulation_in_colab():
    # Стандартна конфігурація
    config = {
        "num_agents": 20,
        "area_size": 100,
        "steps": 500,
        "dt": 0.1,
        "seed": 42,
        "agent_specs": {
            "physical_radius": 0.5,
            "safety_margin": 1.0,
            "max_speed": 2.0,
            "max_acceleration": 1.5,
            "comm_radius": 15.0
        },
        "initial_config": {
            "type": "circle",
            "radius": 30
        },
        "route_config": {
            "type": "common_goal",
            "goal": [50, 50]
        },
        "algorithm": {
            "type": "RandomWalkAlgorithm", # Змінено на RandomWalkAlgorithm
            "params": {
                "max_random_acceleration": 0.5, # Параметр для RandomWalkAlgorithm
            }
        }
    }

    print("Running simulation with RandomWalkAlgorithm...")
    simulator = SwarmSimulator(config)

    print("\nVisualizing results...")
    html_output = Visualization.animate_simulation(simulator, config['area_size']) # Отримуємо HTML

    return html_output # Повертаємо HTML

# Запуск симуляції в Colab
html_content = run_simulation_in_colab()

# Виводимо HTML у Colab, щоб відобразити анімацію
from IPython.display import HTML
display(HTML(html_content))


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from dataclasses import dataclass
from abc import ABC, abstractmethod
from typing import List, Dict, Tuple, Optional, Any
import random
import base64
from io import BytesIO
import matplotlib.animation as animation
from IPython.display import HTML
import matplotlib

# Налаштовуємо бекенд для Matplotlib у Colab
matplotlib.use('agg')

# ==================== БАЗОВІ КЛАСИ ====================
@dataclass
class AgentSpecs:
    physical_radius: float
    safety_margin: float
    max_speed: float
    max_acceleration: float
    comm_radius: float

class SteeringAlgorithm(ABC):
    @abstractmethod
    def compute_steering(self, agent, neighbors: List['Agent'], dt: float) -> np.ndarray:
        pass

@dataclass
class SimulationMetrics:
    collisions: int = 0
    avg_speed: List[float] = None
    max_speed: List[float] = None
    avg_acceleration: List[float] = None
    avg_distance: List[float] = None
    avg_distance_to_center: List[float] = None
    agent_trajectories: Dict[int, List[Tuple[float, float]]] = None

    def __post_init__(self):
        self.avg_speed = []
        self.max_speed = []
        self.avg_acceleration = []
        self.avg_distance = []
        self.avg_distance_to_center = []
        self.agent_trajectories = {}

class Agent:
    def __init__(self, agent_id: int, position: np.ndarray, specs: AgentSpecs,
                 algorithm: SteeringAlgorithm, route: List[np.ndarray]):
        self.id = agent_id
        self.position = position
        self.velocity = np.zeros(2)
        self.acceleration = np.zeros(2)
        self.specs = specs
        self.algorithm = algorithm
        self.route = route
        self.current_waypoint = 0
        self.path_history = [position.copy()]

    def update(self, neighbors: List['Agent'], dt: float, area_size: float):
        steering = self.algorithm.compute_steering(self, neighbors, dt)
        steering = self._limit_vector(steering, self.specs.max_acceleration)
        self.velocity = self._limit_vector(self.velocity + steering * dt, self.specs.max_speed)
        self.position += self.velocity * dt
        self.acceleration = steering
        self._enforce_boundaries(area_size)
        self.path_history.append(self.position.copy())

    def _limit_vector(self, vector: np.ndarray, max_magnitude: float) -> np.ndarray:
        magnitude = np.linalg.norm(vector)
        return vector if magnitude == 0 else vector * min(max_magnitude/magnitude, 1)

    def _enforce_boundaries(self, area_size: float):
        self.position = np.clip(self.position, 0, area_size)

# ==================== АЛГОРИТМИ КЕРУВАННЯ ====================
class VicsekAlgorithm(SteeringAlgorithm):
    def __init__(self, noise: float = 0.1, neighbor_radius: float = 15.0):
        self.noise = noise
        self.neighbor_radius = neighbor_radius

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        neighbors = [n for n in neighbors if np.linalg.norm(n.position - agent.position) <= self.neighbor_radius]
        if not neighbors:
            return np.zeros(2)
        avg_velocity = np.mean([n.velocity for n in neighbors], axis=0)
        noise = np.random.normal(scale=self.noise, size=2)
        return (avg_velocity + noise - agent.velocity) / dt

class BoidsAlgorithm(SteeringAlgorithm):
    def __init__(self, sep_weight: float = 1.5, align_weight: float = 1.0,
                 coh_weight: float = 1.0, visual_range: float = 15.0):
        self.weights = np.array([sep_weight, align_weight, coh_weight])
        self.visual_range = visual_range

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        neighbors = [n for n in neighbors if np.linalg.norm(n.position - agent.position) <= self.visual_range]
        if not neighbors:
            return np.zeros(2)
        positions = np.array([n.position for n in neighbors])
        velocities = np.array([n.velocity for n in neighbors])
        sep = self._separation(agent.position, positions)
        align = self._alignment(agent.velocity, velocities)
        coh = self._cohesion(agent.position, positions)
        return self.weights @ np.array([sep, align, coh])

    def _separation(self, pos: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        diff = pos - neighbors
        distances = np.linalg.norm(diff, axis=1)[:, np.newaxis]
        return np.mean(diff / (distances + 1e-6), axis=0)

    def _alignment(self, vel: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        return np.mean(neighbors, axis=0) - vel

    def _cohesion(self, pos: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        return np.mean(neighbors, axis=0) - pos

class PotentialFieldAlgorithm(SteeringAlgorithm):
    def __init__(self, alpha: float = 0.5, beta: float = 1.0,
                 goal: Optional[np.ndarray] = None, dynamic_goal: bool = False):
        self.goal = goal
        self.alpha = alpha
        self.beta = beta
        self.dynamic_goal = dynamic_goal

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        if self.dynamic_goal and agent.route:
            self.goal = agent.route[-1]
        to_goal = self.goal - agent.position if self.goal is not None else np.zeros(2)
        attraction = self.alpha * to_goal
        repulsion = np.zeros(2)
        for n in neighbors:
            vec = agent.position - n.position
            dist = np.linalg.norm(vec)
            if dist > 0:
                repulsion += self.beta * vec / dist**2
        return attraction + repulsion - agent.velocity / dt

class LeaderFollowerAlgorithm(SteeringAlgorithm):
    def __init__(self, leader_id: int = 0, follow_distance: float = 5.0):
        self.leader_id = leader_id
        self.follow_distance = follow_distance

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        leader = next((n for n in neighbors if n.id == self.leader_id), None)
        if leader is None:
            return np.zeros(2)
        direction = leader.position - agent.position
        distance = np.linalg.norm(direction)
        if distance > 0:
            desired_velocity = (direction / distance) * agent.specs.max_speed
            return (desired_velocity - agent.velocity) / dt
        else:
            return np.zeros(2)

class RandomWalkAlgorithm(SteeringAlgorithm):
    def __init__(self, max_random_acceleration: float = 0.1):
        self.max_random_acceleration = max_random_acceleration

    def compute_steering(self, agent: Agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        random_acceleration = np.random.uniform(
            low=-self.max_random_acceleration,
            high=self.max_random_acceleration,
            size=(2,)
        )
        return random_acceleration

# ==================== СИСТЕМА СИМУЛЯЦІЇ ====================
class SwarmSimulator:
    def __init__(self, config: Dict[str, Any]):
        self.config = config
        if 'seed' in config:
            np.random.seed(config['seed'])
            random.seed(config['seed'])
        self._init_simulation()

    def _init_simulation(self):
        self.agents = []
        self.metrics = SimulationMetrics()
        positions = self._generate_initial_positions()
        for i in range(self.config['num_agents']):
            route = self._generate_route(i, positions[i])
            specs = AgentSpecs(**self.config['agent_specs'])
            algorithm = self._create_algorithm(self.config['algorithm'])
            agent = Agent(i, positions[i], specs, algorithm, route)
            self.agents.append(agent)

    def _generate_initial_positions(self) -> List[np.ndarray]:
        config = self.config['initial_config']
        area_size = self.config['area_size']
        num_agents = self.config['num_agents']
        if config['type'] == 'grid':
            rows = int(np.ceil(np.sqrt(num_agents)))
            cols = int(np.ceil(num_agents/rows))
            sector_size = area_size / max(rows, cols)
            return [np.array([(i%cols)*sector_size + sector_size/2,
                            (i//cols)*sector_size + sector_size/2])
                    for i in range(num_agents)]
        elif config['type'] == 'random':
            return [np.array([random.uniform(0, area_size), random.uniform(0, area_size)])
                    for _ in range(num_agents)]
        elif config['type'] == 'circle':
            radius = config.get('radius', area_size/3)
            center = np.array([area_size/2, area_size/2])
            angles = np.linspace(0, 2*np.pi, num_agents, endpoint=False)
            return [center + radius * np.array([np.cos(a), np.sin(a)]) for a in angles]
        else:
            raise ValueError(f"Unknown initial config type: {config['type']}")

    def _generate_route(self, agent_id: int, start_pos: np.ndarray) -> List[np.ndarray]:
        route_config = self.config['route_config']
        area_size = self.config['area_size']
        if route_config['type'] == 'snake':
            rows = int(np.ceil(np.sqrt(self.config['num_agents'])))
            cols = int(np.ceil(self.config['num_agents']/rows))
            sector_size = area_size / max(rows, cols)
            row = agent_id // cols
            col = agent_id % cols
            x_start = col * sector_size
            y_start = row * sector_size
            route = []
            steps = 5
            step_size = sector_size/steps
            for i in range(steps):
                y = y_start + (i + 0.5)*step_size
                x_values = [x_start + (j + 0.5)*step_size for j in range(steps)] if i%2==0 else reversed([x_start + (j + 0.5)*step_size for j in range(steps)])
                for x in x_values:
                    route.append(np.array([x, y]))
            return route
        elif route_config['type'] == 'random_waypoints':
            steps = 5
            return [np.array([random.uniform(0, area_size), random.uniform(0, area_size)])
                    for _ in range(steps)]
        elif route_config['type'] == 'common_goal':
            goal = np.array(route_config.get('goal', [area_size/2, area_size/2]))
            return [goal.copy()]
        else:
            raise ValueError(f"Unknown route config type: {route_config['type']}")

    def _create_algorithm(self, algo_config: Dict[str, Any]) -> SteeringAlgorithm:
        algo_type = algo_config['type']
        params = algo_config.get('params', {})
        if algo_type == 'VicsekAlgorithm':
            return VicsekAlgorithm(**params)
        elif algo_type == 'BoidsAlgorithm':
            return BoidsAlgorithm(**params)
        elif algo_type == 'PotentialFieldAlgorithm':
            if 'goal' in params:
                params['goal'] = np.array(params['goal'])
            return PotentialFieldAlgorithm(**params)
        elif algo_type == 'LeaderFollowerAlgorithm':
            return LeaderFollowerAlgorithm(**params)
        elif algo_type == 'RandomWalkAlgorithm':
            return RandomWalkAlgorithm(**params)
        else:
            raise ValueError(f"Unknown algorithm type: {algo_type}")

    def update_step(self):
        neighbors = self._find_neighbors()
        for agent, n in zip(self.agents, neighbors):
            agent.update(n, self.config['dt'], self.config['area_size'])

    def _find_neighbors(self) -> List[List[Agent]]:
        positions = np.array([a.position for a in self.agents])
        dist_matrix = np.linalg.norm(positions[:, None] - positions, axis=2)
        neighbors = []
        for i, agent in enumerate(self.agents):
            mask = (dist_matrix[i] <= agent.specs.comm_radius) & (dist_matrix[i] > 0)
            neighbors.append([self.agents[j] for j in np.where(mask)[0]])
        return neighbors

# ==================== ВІЗУАЛІЗАЦІЯ ====================
class Visualization:
    @staticmethod
    def animate_simulation(simulator: SwarmSimulator, area_size: float):
        fig, ax = plt.subplots(figsize=(10, 10))
        ax.set_xlim(0, area_size)
        ax.set_ylim(0, area_size)
        ax.set_title("Agent Movement Simulation")

        # Ініціалізація точок для агентів
        scat = ax.scatter([], [], c='blue', s=50)

        # Ініціалізація ліній для траєкторій
        lines = [ax.plot([], [], 'b-', alpha=0.3, linewidth=1)[0] for _ in range(simulator.config['num_agents'])]

        def init():
            scat.set_offsets(np.empty((0, 2)))
            for line in lines:
                line.set_data([], [])
            return [scat] + lines

        def update(frame):
            simulator.update_step()
            positions = np.array([agent.position for agent in simulator.agents])
            scat.set_offsets(positions)

            # Оновлення траєкторій
            for idx, agent in enumerate(simulator.agents):
                trajectory = np.array(agent.path_history)
                lines[idx].set_data(trajectory[:, 0], trajectory[:, 1])

            return [scat] + lines

        # Створюємо анімацію
        ani = FuncAnimation(
            fig,
            update,
            frames=simulator.config['steps'],
            init_func=init,
            blit=True,
            interval=50,
            repeat=False
        )

        # Конвертуємо анімацію в HTML
        plt.close(fig)  # Закриваємо фігуру, щоб уникнути подвійного відображення
        return HTML(ani.to_jshtml())

# ==================== ІНТЕРФЕЙС ДЛЯ COLAB ====================
def run_simulation_in_colab():
    # Стандартна конфігурація
    config = {
        "num_agents": 15,  # Зменшено кількість агентів для кращої візуалізації
        "area_size": 100,
        "steps": 100,  # Зменшено кількість кроків для швидшого тестування
        "dt": 0.1,
        "seed": 42,
        "agent_specs": {
            "physical_radius": 0.5,
            "safety_margin": 1.0,
            "max_speed": 2.0,
            "max_acceleration": 1.5,
            "comm_radius": 15.0
        },
        "initial_config": {
            "type": "circle",
            "radius": 30
        },
        "route_config": {
            "type": "common_goal",
            "goal": [50, 50]
        },
        "algorithm": {
            "type": "BoidsAlgorithm",  # Використовуємо Boids для більш цікавої поведінки
            "params": {
                "sep_weight": 1.5,
                "align_weight": 0.8,
                "coh_weight": 0.5,
                "visual_range": 20.0
            }
        }
    }

    print("Running simulation with Boids algorithm...")
    simulator = SwarmSimulator(config)

    print("\nGenerating animation...")
    return Visualization.animate_simulation(simulator, config['area_size'])

# Запуск симуляції в Colab
animation_html = run_simulation_in_colab()
animation_html

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from dataclasses import dataclass
from abc import ABC, abstractmethod
from typing import List, Dict, Tuple, Optional, Any
import random
import base64
from io import BytesIO
import matplotlib.animation as animation
import matplotlib
import tempfile
import os
from IPython.display import HTML  # Import HTML from IPython.display

# Налаштовуємо бекенд для Matplotlib у Colab
matplotlib.use('agg')

# ==================== БАЗОВІ КЛАСИ ====================
@dataclass
class AgentSpecs:
    physical_radius: float
    safety_margin: float
    max_speed: float
    max_acceleration: float
    comm_radius: float

class SteeringAlgorithm(ABC):
    @abstractmethod
    def compute_steering(self, agent, neighbors: List['Agent'], dt: float) -> np.ndarray:
        pass

@dataclass
class SimulationMetrics:
    collisions: int = 0
    avg_speed: List[float] = None
    max_speed: List[float] = None
    avg_acceleration: List[float] = None
    avg_distance: List[float] = None
    avg_distance_to_center: List[float] = None
    agent_trajectories: Dict[int, List[Tuple[float, float]]] = None

    def __post_init__(self):
        self.avg_speed = []
        self.max_speed = []
        self.avg_acceleration = []
        self.avg_distance = []
        self.avg_distance_to_center = []
        self.agent_trajectories = {}

class Agent:
    def __init__(self, agent_id: int, position: np.ndarray, specs: AgentSpecs,
                 algorithm: SteeringAlgorithm, route: List[np.ndarray]):
        self.id = agent_id
        self.position = position
        self.velocity = np.zeros(2)
        self.acceleration = np.zeros(2)
        self.specs = specs
        self.algorithm = algorithm
        self.route = route
        self.current_waypoint = 0
        self.path_history = [position.copy()]
        self.color = 'blue'  # Додаємо колір агенту для відображення зіткнень

    def update(self, neighbors: List['Agent'], dt: float, area_size: float):
        steering = self.algorithm.compute_steering(self, neighbors, dt)
        steering = self._limit_vector(steering, self.specs.max_acceleration)
        self.velocity = self._limit_vector(self.velocity + steering * dt, self.specs.max_speed)
        self.position += self.velocity * dt
        self.acceleration = steering
        self._enforce_boundaries(area_size)
        self.path_history.append(self.position.copy())

    def _limit_vector(self, vector: np.ndarray, max_magnitude: float) -> np.ndarray:
        magnitude = np.linalg.norm(vector)
        return vector if magnitude == 0 else vector * min(max_magnitude/magnitude, 1)

    def _enforce_boundaries(self, area_size: float):
        self.position = np.clip(self.position, 0, area_size)

# ==================== АЛГОРИТМИ КЕРУВАННЯ ====================
class VicsekAlgorithm(SteeringAlgorithm):
    def __init__(self, noise: float = 0.1, neighbor_radius: float = 15.0):
        self.noise = noise
        self.neighbor_radius = neighbor_radius

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        neighbors = [n for n in neighbors if np.linalg.norm(n.position - agent.position) <= self.neighbor_radius]
        if not neighbors:
            return np.zeros(2)
        avg_velocity = np.mean([n.velocity for n in neighbors], axis=0)
        noise = np.random.normal(scale=self.noise, size=2)
        return (avg_velocity + noise - agent.velocity) / dt

class BoidsAlgorithm(SteeringAlgorithm):
    def __init__(self, sep_weight: float = 1.5, align_weight: float = 1.0,
                 coh_weight: float = 1.0, visual_range: float = 15.0):
        self.weights = np.array([sep_weight, align_weight, coh_weight])
        self.visual_range = visual_range

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        neighbors = [n for n in neighbors if np.linalg.norm(n.position - agent.position) <= self.visual_range]
        if not neighbors:
            return np.zeros(2)
        positions = np.array([n.position for n in neighbors])
        velocities = np.array([n.velocity for n in neighbors])
        sep = self._separation(agent.position, positions)
        align = self._alignment(agent.velocity, velocities)
        coh = self._cohesion(agent.position, positions)
        return self.weights @ np.array([sep, align, coh])

    def _separation(self, pos: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        diff = pos - neighbors
        distances = np.linalg.norm(diff, axis=1)[:, np.newaxis]
        return np.mean(diff / (distances + 1e-6), axis=0)

    def _alignment(self, vel: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        return np.mean(neighbors, axis=0) - vel

    def _cohesion(self, pos: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        return np.mean(neighbors, axis=0) - pos

class PotentialFieldAlgorithm(SteeringAlgorithm):
    def __init__(self, alpha: float = 0.5, beta: float = 1.0,
                 goal: Optional[np.ndarray] = None, dynamic_goal: bool = False):
        self.goal = goal
        self.alpha = alpha
        self.beta = beta
        self.dynamic_goal = dynamic_goal

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        if self.dynamic_goal and agent.route:
            self.goal = agent.route[-1]
        to_goal = self.goal - agent.position if self.goal is not None else np.zeros(2)
        attraction = self.alpha * to_goal
        repulsion = np.zeros(2)
        for n in neighbors:
            vec = agent.position - n.position
            dist = np.linalg.norm(vec)
            if dist > 0:
                repulsion += self.beta * vec / dist**2
        return attraction + repulsion - agent.velocity / dt

class LeaderFollowerAlgorithm(SteeringAlgorithm):
    def __init__(self, leader_id: int = 0, follow_distance: float = 5.0):
        self.leader_id = leader_id
        self.follow_distance = follow_distance

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        leader = next((n for n in neighbors if n.id == self.leader_id), None)
        if leader is None:
            return np.zeros(2)
        direction = leader.position - agent.position
        distance = np.linalg.norm(direction)
        if distance > 0:
            desired_velocity = (direction / distance) * agent.specs.max_speed
            return (desired_velocity - agent.velocity) / dt
        else:
            return np.zeros(2)

class RandomWalkAlgorithm(SteeringAlgorithm):
    def __init__(self, max_random_acceleration: float = 0.1):
        self.max_random_acceleration = max_random_acceleration

    def compute_steering(self, agent: Agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        random_acceleration = np.random.uniform(
            low=-self.max_random_acceleration,
            high=self.max_random_acceleration,
            size=(2,)
        )
        return random_acceleration

# ==================== СИСТЕМА СИМУЛЯЦІЇ ====================
class SwarmSimulator:
    def __init__(self, config: Dict[str, Any]):
        self.config = config
        if 'seed' in config:
            np.random.seed(config['seed'])
            random.seed(config['seed'])
        self._init_simulation()

    def _init_simulation(self):
        self.agents = []
        self.metrics = SimulationMetrics()
        positions = self._generate_initial_positions()
        for i in range(self.config['num_agents']):
            route = self._generate_route(i, positions[i])
            specs = AgentSpecs(**self.config['agent_specs'])
            algorithm = self._create_algorithm(self.config['algorithm'])
            agent = Agent(i, positions[i], specs, algorithm, route)
            self.agents.append(agent)

    def _generate_initial_positions(self) -> List[np.ndarray]:
        config = self.config['initial_config']
        area_size = self.config['area_size']
        num_agents = self.config['num_agents']
        if config['type'] == 'grid':
            rows = int(np.ceil(np.sqrt(num_agents)))
            cols = int(np.ceil(num_agents/rows))
            sector_size = area_size / max(rows, cols)
            return [np.array([(i%cols)*sector_size + sector_size/2,
                            (i//cols)*sector_size + sector_size/2])
                    for i in range(num_agents)]
        elif config['type'] == 'random':
            return [np.array([random.uniform(0, area_size), random.uniform(0, area_size)])
                    for _ in range(num_agents)]
        elif config['type'] == 'circle':
            radius = config.get('radius', area_size/3)
            center = np.array([area_size/2, area_size/2])
            angles = np.linspace(0, 2*np.pi, num_agents, endpoint=False)
            return [center + radius * np.array([np.cos(a), np.sin(a)]) for a in angles]
        else:
            raise ValueError(f"Unknown initial config type: {config['type']}")

    def _generate_route(self, agent_id: int, start_pos: np.ndarray) -> List[np.ndarray]:
        route_config = self.config['route_config']
        area_size = self.config['area_size']
        if route_config['type'] == 'snake':
            rows = int(np.ceil(np.sqrt(self.config['num_agents'])))
            cols = int(np.ceil(self.config['num_agents']/rows))
            sector_size = area_size / max(rows, cols)
            row = agent_id // cols
            col = agent_id % cols
            x_start = col * sector_size
            y_start = row * sector_size
            route = []
            steps = 5
            step_size = sector_size/steps
            for i in range(steps):
                y = y_start + (i + 0.5)*step_size
                x_values = [x_start + (j + 0.5)*step_size for j in range(steps)] if i%2==0 else reversed([x_start + (j + 0.5)*step_size for j in range(steps)])
                for x in x_values:
                    route.append(np.array([x, y]))
            return route
        elif route_config['type'] == 'random_waypoints':
            steps = 5
            return [np.array([random.uniform(0, area_size), random.uniform(0, area_size)])
                    for _ in range(steps)]
        elif route_config['type'] == 'common_goal':
            goal = np.array(route_config.get('goal', [area_size/2, area_size/2]))
            return [goal.copy()]
        else:
            raise ValueError(f"Unknown route config type: {route_config['type']}")

    def _create_algorithm(self, algo_config: Dict[str, Any]) -> SteeringAlgorithm:
        algo_type = algo_config['type']
        params = algo_config.get('params', {})
        if algo_type == 'VicsekAlgorithm':
            return VicsekAlgorithm(**params)
        elif algo_type == 'BoidsAlgorithm':
            return BoidsAlgorithm(**params)
        elif algo_type == 'PotentialFieldAlgorithm':
            if 'goal' in params:
                params['goal'] = np.array(params['goal'])
            return PotentialFieldAlgorithm(**params)
        elif algo_type == 'LeaderFollowerAlgorithm':
            return LeaderFollowerAlgorithm(**params)
        elif algo_type == 'RandomWalkAlgorithm':
            return RandomWalkAlgorithm(**params)
        else:
            raise ValueError(f"Unknown algorithm type: {algo_type}")

    def update_step(self):
        neighbors = self._find_neighbors()
        collisions = self._handle_collisions() # Обробка зіткнень
        for agent, n in zip(self.agents, neighbors):
            agent.update(n, self.config['dt'], self.config['area_size'])
        return collisions

    def _find_neighbors(self) -> List[List[Agent]]:
        positions = np.array([a.position for a in self.agents])
        dist_matrix = np.linalg.norm(positions[:, None] - positions, axis=2)
        neighbors = []
        for i, agent in enumerate(self.agents):
            mask = (dist_matrix[i] <= agent.specs.comm_radius) & (dist_matrix[i] > 0)
            neighbors.append([self.agents[j] for j in np.where(mask)[0]])
        return neighbors

    def _handle_collisions(self) -> int:
        """
        Визначає та обробляє зіткнення між агентами.

        Returns:
            int: Кількість зіткнень, що відбулися на цьому кроці.
        """
        collisions = 0
        agent_positions = np.array([agent.position for agent in self.agents])
        for i in range(self.config['num_agents']):
            for j in range(i + 1, self.config['num_agents']):
                distance = np.linalg.norm(agent_positions[i] - agent_positions[j])
                if distance < self.agents[i].specs.physical_radius + self.agents[j].specs.physical_radius:
                    collisions += 1
                    self.agents[i].color = 'red'  # Змінюємо колір на червоний при зіткненні
                    self.agents[j].color = 'red'
        return collisions

# ==================== ВІЗУАЛІЗАЦІЯ ====================
class Visualization:
    @staticmethod
    def animate_simulation(simulator: SwarmSimulator, area_size: float):
        fig, ax = plt.subplots(figsize=(10, 10))
        ax.set_xlim(0, area_size)
        ax.set_ylim(0, area_size)
        ax.set_title("Agent Movement Simulation")

        # Ініціалізація точок для агентів
        scat = ax.scatter([], [], c='blue', s=50)

        # Ініціалізація ліній для траєкторій
        lines = [ax.plot([], [], 'b-', alpha=0.3, linewidth=1)[0] for _ in range(simulator.config['num_agents'])]

        # Додавання текстової мітки для відображення кількості зіткнень
        collision_text = ax.text(0.02, 0.95, "Collisions: 0", transform=ax.transAxes)

        def init():
            scat.set_offsets(np.empty((0, 2)))
            for line in lines:
                line.set_data([], [])
            return [scat] + lines + [collision_text]

        def update(frame):
            collisions = simulator.update_step() # Отримуємо кількість зіткнень з симулятора
            positions = np.array([agent.position for agent in simulator.agents])
            colors = [agent.color for agent in simulator.agents] # Отримуємо кольори агентів
            scat.set_offsets(positions)
            scat.set_color(colors) # Встановлюємо кольори точок

            # Оновлюємо траєкторії
            for idx, agent in enumerate(simulator.agents):
                trajectory = np.array(agent.path_history)
                lines[idx].set_data(trajectory[:, 0], trajectory[:, 1])

            # Оновлюємо текст лічильника зіткнень
            collision_text.set_text(f"Collisions: {collisions}")

            return [scat] + lines + [collision_text]

        # Створюємо анімацію
        ani = FuncAnimation(
            fig,
            update,
            frames=simulator.config['steps'],
            init_func=init,
            blit=True,
            interval=50,
            repeat=False
        )

        # Конвертуємо анімацію в HTML
        plt.close(fig)  # Закриваємо фігуру, щоб уникнути подвійного відображення
        return ani.to_jshtml()  # Return the HTML string

# ==================== ІНТЕРФЕЙС ДЛЯ COLAB ====================
def run_simulation_in_colab():
    # Стандартна конфігурація
    config = {
        "num_agents": 15,
        "area_size": 100,
        "steps": 100,
        "dt": 0.1,
        "seed": 42,
        "agent_specs": {
            "physical_radius": 0.5,
            "safety_margin": 1.0,
            "max_speed": 2.0,
            "max_acceleration": 1.5,
            "comm_radius": 15.0
        },
        "initial_config": {
            "type": "circle",
            "radius": 30
        },
        "route_config": {
            "type": "common_goal",
            "goal": [50, 50]
        },
        "algorithm": {
            "type": "BoidsAlgorithm",
            "params": {
                "sep_weight": 1.5,
                "align_weight": 0.8,
                "coh_weight": 0.5,
                "visual_range": 20.0
            }
        }
    }

    print("Running simulation with Boids algorithm...")
    simulator = SwarmSimulator(config)

    print("\nGenerating animation...")
    animation_html = Visualization.animate_simulation(simulator, config['area_size'])
    return animation_html

# Запуск симуляції в Colab
animation_html = run_simulation_in_colab()
HTML(animation_html)  # Display the animation


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from dataclasses import dataclass
from abc import ABC, abstractmethod
from typing import List, Dict, Tuple, Optional, Any
import random
from IPython.display import HTML
import matplotlib

# Налаштування бекенду для Matplotlib у Colab
matplotlib.use('agg')

# ==================== БАЗОВІ КЛАСИ ====================
@dataclass
class AgentSpecs:
    physical_radius: float
    safety_margin: float
    max_speed: float
    max_acceleration: float
    comm_radius: float

class SteeringAlgorithm(ABC):
    @abstractmethod
    def compute_steering(self, agent, neighbors: List['Agent'], dt: float) -> np.ndarray:
        pass

@dataclass
class SimulationMetrics:
    collisions: int = 0
    avg_speed: List[float] = None
    max_speed: List[float] = None
    avg_acceleration: List[float] = None
    avg_distance: List[float] = None
    agent_trajectories: Dict[int, List[Tuple[float, float]]] = None

    def __post_init__(self):
        self.avg_speed = []
        self.max_speed = []
        self.avg_acceleration = []
        self.avg_distance = []
        self.agent_trajectories = {}

class Agent:
    def __init__(self, agent_id: int, position: np.ndarray, specs: AgentSpecs,
                 algorithm: SteeringAlgorithm, route: List[np.ndarray]):
        self.id = agent_id
        self.position = position
        self.velocity = np.zeros(2)
        self.acceleration = np.zeros(2)
        self.specs = specs
        self.algorithm = algorithm
        self.route = route
        self.current_waypoint = 0
        self.path_history = [position.copy()]

    def update(self, neighbors: List['Agent'], dt: float, area_size: float):
        steering = self.algorithm.compute_steering(self, neighbors, dt)
        steering = self._limit_vector(steering, self.specs.max_acceleration)
        self.velocity = self._limit_vector(self.velocity + steering * dt, self.specs.max_speed)
        self.position += self.velocity * dt
        self.acceleration = steering
        self._enforce_boundaries(area_size)
        self.path_history.append(self.position.copy())

    def _limit_vector(self, vector: np.ndarray, max_magnitude: float) -> np.ndarray:
        magnitude = np.linalg.norm(vector)
        return vector if magnitude == 0 else vector * min(max_magnitude/magnitude, 1)

    def _enforce_boundaries(self, area_size: float):
        self.position = np.clip(self.position, 0, area_size)

# ==================== АЛГОРИТМИ КЕРУВАННЯ ====================
class VicsekAlgorithm(SteeringAlgorithm):
    def __init__(self, noise: float = 0.1, neighbor_radius: float = 15.0):
        self.noise = noise
        self.neighbor_radius = neighbor_radius

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        neighbors = [n for n in neighbors if np.linalg.norm(n.position - agent.position) <= self.neighbor_radius]
        if not neighbors:
            return np.zeros(2)
        avg_velocity = np.mean([n.velocity for n in neighbors], axis=0)
        noise = np.random.normal(scale=self.noise, size=2)
        return (avg_velocity + noise - agent.velocity) / dt

class BoidsAlgorithm(SteeringAlgorithm):
    def __init__(self, sep_weight: float = 1.5, align_weight: float = 1.0,
                 coh_weight: float = 1.0, visual_range: float = 15.0):
        self.weights = np.array([sep_weight, align_weight, coh_weight])
        self.visual_range = visual_range

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        neighbors = [n for n in neighbors if np.linalg.norm(n.position - agent.position) <= self.visual_range]
        if not neighbors:
            return np.zeros(2)
        positions = np.array([n.position for n in neighbors])
        velocities = np.array([n.velocity for n in neighbors])
        sep = self._separation(agent.position, positions)
        align = self._alignment(agent.velocity, velocities)
        coh = self._cohesion(agent.position, positions)
        return self.weights @ np.array([sep, align, coh])

    def _separation(self, pos: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        diff = pos - neighbors
        distances = np.linalg.norm(diff, axis=1)[:, np.newaxis]
        return np.mean(diff / (distances + 1e-6), axis=0)

    def _alignment(self, vel: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        return np.mean(neighbors, axis=0) - vel

    def _cohesion(self, pos: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        return np.mean(neighbors, axis=0) - pos

class PotentialFieldAlgorithm(SteeringAlgorithm):
    def __init__(self, alpha: float = 0.5, beta: float = 1.0,
                 goal: Optional[np.ndarray] = None, dynamic_goal: bool = False):
        self.goal = goal
        self.alpha = alpha
        self.beta = beta
        self.dynamic_goal = dynamic_goal

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        if self.dynamic_goal and agent.route:
            self.goal = agent.route[-1]
        to_goal = self.goal - agent.position if self.goal is not None else np.zeros(2)
        attraction = self.alpha * to_goal
        repulsion = np.zeros(2)
        for n in neighbors:
            vec = agent.position - n.position
            dist = np.linalg.norm(vec)
            if dist > 0:
                repulsion += self.beta * vec / dist**2
        return attraction + repulsion - agent.velocity / dt

class LeaderFollowerAlgorithm(SteeringAlgorithm):
    def __init__(self, leader_id: int = 0, follow_distance: float = 5.0):
        self.leader_id = leader_id
        self.follow_distance = follow_distance

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        leader = next((n for n in neighbors if n.id == self.leader_id), None)
        if leader is None or agent.id == self.leader_id:
            return np.zeros(2)
        direction = leader.position - agent.position
        distance = np.linalg.norm(direction)
        desired_velocity = (direction / distance) * agent.specs.max_speed if distance > 0 else np.zeros(2)
        return (desired_velocity - agent.velocity) / dt

class RandomWalkAlgorithm(SteeringAlgorithm):
    def __init__(self, max_random_acceleration: float = 0.1):
        self.max_random_acceleration = max_random_acceleration

    def compute_steering(self, agent: Agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        return np.random.uniform(-self.max_random_acceleration, self.max_random_acceleration, size=2)

# ==================== СИСТЕМА СИМУЛЯЦІЇ ====================
class SwarmSimulator:
    def __init__(self, config: Dict[str, Any]):
        self.config = config
        np.random.seed(config.get('seed', 42))
        random.seed(config.get('seed', 42))
        self._init_simulation()

    def _init_simulation(self):
        self.agents = []
        self.metrics = SimulationMetrics()
        positions = self._generate_initial_positions()
        for i in range(self.config['num_agents']):
            route = self._generate_route(i, positions[i])
            specs = AgentSpecs(**self.config['agent_specs'])
            algorithm = self._create_algorithm(self.config['algorithm'])
            agent = Agent(i, positions[i], specs, algorithm, route)
            self.agents.append(agent)

    def _generate_initial_positions(self) -> List[np.ndarray]:
        config = self.config['initial_config']
        area_size = self.config['area_size']
        num_agents = self.config['num_agents']
        if config['type'] == 'grid':
            rows = int(np.ceil(np.sqrt(num_agents)))
            cols = int(np.ceil(num_agents/rows))
            sector_size = area_size / max(rows, cols)
            return [np.array([(i%cols)*sector_size + sector_size/2,
                            (i//cols)*sector_size + sector_size/2])
                   for i in range(num_agents)]
        elif config['type'] == 'random':
            return [np.array([random.uniform(0, area_size), random.uniform(0, area_size)])
                   for _ in range(num_agents)]
        elif config['type'] == 'circle':
            radius = config.get('radius', area_size/3)
            center = np.array([area_size/2, area_size/2])
            angles = np.linspace(0, 2*np.pi, num_agents, endpoint=False)
            return [center + radius * np.array([np.cos(a), np.sin(a)]) for a in angles]
        else:
            raise ValueError(f"Unknown initial config type: {config['type']}")

    def _generate_route(self, agent_id: int, start_pos: np.ndarray) -> List[np.ndarray]:
        route_config = self.config['route_config']
        area_size = self.config['area_size']
        if route_config['type'] == 'snake':
            rows = int(np.ceil(np.sqrt(self.config['num_agents'])))
            cols = int(np.ceil(self.config['num_agents']/rows))
            sector_size = area_size / max(rows, cols)
            row = agent_id // cols
            col = agent_id % cols
            x_start = col * sector_size
            y_start = row * sector_size
            route = []
            steps = 5
            step_size = sector_size/steps
            for i in range(steps):
                y = y_start + (i + 0.5)*step_size
                x_values = [x_start + (j + 0.5)*step_size for j in range(steps)] if i%2==0 else reversed([x_start + (j + 0.5)*step_size for j in range(steps)])
                for x in x_values:
                    route.append(np.array([x, y]))
            return route
        elif route_config['type'] == 'common_goal':
            goal = np.array(route_config.get('goal', [area_size/2, area_size/2]))
            return [goal.copy()]
        else:
            raise ValueError(f"Unknown route config type: {route_config['type']}")

    def _create_algorithm(self, algo_config: Dict[str, Any]) -> SteeringAlgorithm:
        algo_type = algo_config['type']
        params = algo_config.get('params', {})
        if algo_type == 'VicsekAlgorithm':
            return VicsekAlgorithm(**params)
        elif algo_type == 'BoidsAlgorithm':
            return BoidsAlgorithm(**params)
        elif algo_type == 'PotentialFieldAlgorithm':
            if 'goal' in params:
                params['goal'] = np.array(params['goal'])
            return PotentialFieldAlgorithm(**params)
        elif algo_type == 'LeaderFollowerAlgorithm':
            return LeaderFollowerAlgorithm(**params)
        elif algo_type == 'RandomWalkAlgorithm':
            return RandomWalkAlgorithm(**params)
        else:
            raise ValueError(f"Unknown algorithm type: {algo_type}")

    def update_step(self):
        neighbors = self._find_neighbors()
        for agent, n in zip(self.agents, neighbors):
            agent.update(n, self.config['dt'], self.config['area_size'])
        self._collect_metrics()

    def _find_neighbors(self) -> List[List[Agent]]:
        positions = np.array([a.position for a in self.agents])
        dist_matrix = np.linalg.norm(positions[:, None] - positions, axis=2)
        neighbors = []
        for i, agent in enumerate(self.agents):
            mask = (dist_matrix[i] <= agent.specs.comm_radius) & (dist_matrix[i] > 0)
            neighbors.append([self.agents[j] for j in np.where(mask)[0]])
        return neighbors

    def _collect_metrics(self):
        positions = np.array([a.position for a in self.agents])
        velocities = np.array([a.velocity for a in self.agents])
        accelerations = np.array([a.acceleration for a in self.agents])

        # Збір метрик
        self.metrics.avg_speed.append(np.mean(np.linalg.norm(velocities, axis=1)))
        self.metrics.max_speed.append(np.max(np.linalg.norm(velocities, axis=1)))
        self.metrics.avg_acceleration.append(np.mean(np.linalg.norm(accelerations, axis=1)))

        # Розрахунок середньої відстані
        dist_matrix = np.linalg.norm(positions[:, None] - positions, axis=2)
        np.fill_diagonal(dist_matrix, np.nan)
        self.metrics.avg_distance.append(np.nanmean(dist_matrix))

        # Запис траєкторій
        for agent in self.agents:
            if agent.id not in self.metrics.agent_trajectories:
                self.metrics.agent_trajectories[agent.id] = []
            self.metrics.agent_trajectories[agent.id].append(tuple(agent.position))

        # Перевірка зіткнень
        collision_dist = 2 * self.agents[0].specs.physical_radius
        self.metrics.collisions += np.sum(dist_matrix < collision_dist) // 2

# ==================== ВІЗУАЛІЗАЦІЯ ====================
class Visualization:
    @staticmethod
    def animate_simulation(simulator: SwarmSimulator):
        fig, ax = plt.subplots(figsize=(10, 10))
        ax.set_xlim(0, simulator.config['area_size'])
        ax.set_ylim(0, simulator.config['area_size'])
        ax.set_title("Swarm Simulation")

        # Ініціалізація елементів візуалізації
        scat = ax.scatter([], [], c='blue', s=50, alpha=0.6)
        lines = [ax.plot([], [], 'b-', alpha=0.3, linewidth=1)[0] for _ in range(simulator.config['num_agents'])]

        def init():
            scat.set_offsets(np.empty((0, 2)))
            for line in lines:
                line.set_data([], [])
            return [scat] + lines

        def update(frame):
            simulator.update_step()
            positions = np.array([agent.position for agent in simulator.agents])

            # Оновлення точок
            scat.set_offsets(positions)

            # Оновлення траєкторій
            for idx, agent in enumerate(simulator.agents):
                trajectory = np.array(agent.path_history)
                lines[idx].set_data(trajectory[:, 0], trajectory[:, 1])

            return [scat] + lines

        ani = FuncAnimation(
            fig,
            update,
            frames=simulator.config['steps'],
            init_func=init,
            blit=True,
            interval=50,
            repeat=False
        )
        plt.close(fig)
        return HTML(ani.to_jshtml())

# ==================== ІНТЕРФЕЙС ДЛЯ COLAB ====================
def run_simulation():
    # Конфігурація за замовчуванням
    config = {
        "num_agents": 15,  # Зменшено для кращої продуктивності
        "area_size": 100,
        "steps": 50,  # Зменшено для швидшого тестування
        "dt": 0.1,
        "seed": 42,
        "agent_specs": {
            "physical_radius": 0.5,
            "safety_margin": 1.0,
            "max_speed": 2.0,
            "max_acceleration": 1.5,
            "comm_radius": 15.0
        },
        "initial_config": {
            "type": "circle",
            "radius": 30
        },
        "route_config": {
            "type": "common_goal",
            "goal": [50, 50]
        },
        "algorithm": {
            "type": "BoidsAlgorithm",
            "params": {
                "sep_weight": 1.5,
                "align_weight": 0.8,
                "coh_weight": 0.5,
                "visual_range": 20.0
            }
        }
    }

    # Ініціалізація та запуск симуляції
    simulator = SwarmSimulator(config)
    print("Starting simulation...")

    # Генерація анімації
    anim_html = Visualization.animate_simulation(simulator)

    # Вивід результатів
    print("\nSimulation Results:")
    print(f"Total Collisions: {simulator.metrics.collisions}")
    print(f"Average Speed: {np.mean(simulator.metrics.avg_speed):.2f}")
    print(f"Max Speed: {np.max(simulator.metrics.max_speed):.2f}")
    print(f"Average Distance: {np.mean(simulator.metrics.avg_distance):.2f}")

    return anim_html

# Запуск симуляції
simulation_animation = run_simulation()
simulation_animation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from dataclasses import dataclass
from abc import ABC, abstractmethod
from typing import List, Dict, Tuple, Optional, Any
import random
from IPython.display import HTML, display
import matplotlib

# Налаштування бекенду для Matplotlib у Colab
matplotlib.use('agg')

# ==================== БАЗОВІ КЛАСИ ====================
@dataclass
class AgentSpecs:
    physical_radius: float
    safety_margin: float
    max_speed: float
    max_acceleration: float
    comm_radius: float

class SteeringAlgorithm(ABC):
    @abstractmethod
    def compute_steering(self, agent, neighbors: List['Agent'], dt: float) -> np.ndarray:
        pass

@dataclass
class SimulationMetrics:
    collisions: int = 0
    avg_speed: List[float] = None
    max_speed: List[float] = None
    avg_acceleration: List[float] = None
    avg_distance: List[float] = None
    agent_trajectories: Dict[int, List[Tuple[float, float]]] = None

    def __post_init__(self):
        self.avg_speed = []
        self.max_speed = []
        self.avg_acceleration = []
        self.avg_distance = []
        self.agent_trajectories = {}

class Agent:
    def __init__(self, agent_id: int, position: np.ndarray, specs: AgentSpecs,
                 algorithm: SteeringAlgorithm, route: List[np.ndarray]):
        self.id = agent_id
        self.position = position
        self.velocity = np.zeros(2)
        self.acceleration = np.zeros(2)
        self.specs = specs
        self.algorithm = algorithm
        self.route = route
        self.current_waypoint = 0
        self.path_history = [position.copy()]

    def update(self, neighbors: List['Agent'], dt: float, area_size: float):
        steering = self.algorithm.compute_steering(self, neighbors, dt)
        steering = self._limit_vector(steering, self.specs.max_acceleration)
        self.velocity = self._limit_vector(self.velocity + steering * dt, self.specs.max_speed)
        self.position += self.velocity * dt
        self.acceleration = steering
        self._enforce_boundaries(area_size)
        self.path_history.append(self.position.copy())

    def _limit_vector(self, vector: np.ndarray, max_magnitude: float) -> np.ndarray:
        magnitude = np.linalg.norm(vector)
        return vector if magnitude == 0 else vector * min(max_magnitude/magnitude, 1)

    def _enforce_boundaries(self, area_size: float):
        self.position = np.clip(self.position, 0, area_size)

# ==================== АЛГОРИТМИ КЕРУВАННЯ ====================
class VicsekAlgorithm(SteeringAlgorithm):
    def __init__(self, noise: float = 0.1, neighbor_radius: float = 15.0):
        self.noise = noise
        self.neighbor_radius = neighbor_radius

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        neighbors = [n for n in neighbors if np.linalg.norm(n.position - agent.position) <= self.neighbor_radius]
        if not neighbors:
            return np.zeros(2)
        avg_velocity = np.mean([n.velocity for n in neighbors], axis=0)
        noise = np.random.normal(scale=self.noise, size=2)
        return (avg_velocity + noise - agent.velocity) / dt

class BoidsAlgorithm(SteeringAlgorithm):
    def __init__(self, sep_weight: float = 1.5, align_weight: float = 1.0,
                 coh_weight: float = 1.0, visual_range: float = 15.0):
        self.weights = np.array([sep_weight, align_weight, coh_weight])
        self.visual_range = visual_range

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        neighbors = [n for n in neighbors if np.linalg.norm(n.position - agent.position) <= self.visual_range]
        if not neighbors:
            return np.zeros(2)
        positions = np.array([n.position for n in neighbors])
        velocities = np.array([n.velocity for n in neighbors])
        sep = self._separation(agent.position, positions)
        align = self._alignment(agent.velocity, velocities)
        coh = self._cohesion(agent.position, positions)
        return self.weights @ np.array([sep, align, coh])

    def _separation(self, pos: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        diff = pos - neighbors
        distances = np.linalg.norm(diff, axis=1)[:, np.newaxis]
        return np.mean(diff / (distances + 1e-6), axis=0)

    def _alignment(self, vel: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        return np.mean(neighbors, axis=0) - vel

    def _cohesion(self, pos: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        return np.mean(neighbors, axis=0) - pos

class PotentialFieldAlgorithm(SteeringAlgorithm):
    def __init__(self, alpha: float = 0.5, beta: float = 1.0,
                 goal: Optional[np.ndarray] = None, dynamic_goal: bool = False):
        self.goal = goal
        self.alpha = alpha
        self.beta = beta
        self.dynamic_goal = dynamic_goal

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        if self.dynamic_goal and agent.route:
            self.goal = agent.route[-1]
        to_goal = self.goal - agent.position if self.goal is not None else np.zeros(2)
        attraction = self.alpha * to_goal
        repulsion = np.zeros(2)
        for n in neighbors:
            vec = agent.position - n.position
            dist = np.linalg.norm(vec)
            if dist > 0:
                repulsion += self.beta * vec / dist**2
        return attraction + repulsion - agent.velocity / dt

class LeaderFollowerAlgorithm(SteeringAlgorithm):
    def __init__(self, leader_id: int = 0, follow_distance: float = 5.0):
        self.leader_id = leader_id
        self.follow_distance = follow_distance

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        leader = next((n for n in neighbors if n.id == self.leader_id), None)
        if leader is None or agent.id == self.leader_id:
            return np.zeros(2)
        direction = leader.position - agent.position
        distance = np.linalg.norm(direction)
        desired_velocity = (direction / distance) * agent.specs.max_speed if distance > 0 else np.zeros(2)
        return (desired_velocity - agent.velocity) / dt

class RandomWalkAlgorithm(SteeringAlgorithm):
    def __init__(self, max_random_acceleration: float = 0.1):
        self.max_random_acceleration = max_random_acceleration

    def compute_steering(self, agent: Agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        return np.random.uniform(-self.max_random_acceleration, self.max_random_acceleration, size=2)

# ==================== СИСТЕМА СИМУЛЯЦІЇ ====================
class SwarmSimulator:
    def __init__(self, config: Dict[str, Any]):
        self.config = config
        np.random.seed(config.get('seed', 42))
        random.seed(config.get('seed', 42))
        self._init_simulation()

    def _init_simulation(self):
        self.agents = []
        self.metrics = SimulationMetrics()
        positions = self._generate_initial_positions()
        for i in range(self.config['num_agents']):
            route = self._generate_route(i, positions[i])
            specs = AgentSpecs(**self.config['agent_specs'])
            algorithm = self._create_algorithm(self.config['algorithm'])
            agent = Agent(i, positions[i], specs, algorithm, route)
            self.agents.append(agent)

    def _generate_initial_positions(self) -> List[np.ndarray]:
        config = self.config['initial_config']
        area_size = self.config['area_size']
        num_agents = self.config['num_agents']

        if config['type'] == 'grid':
            rows = int(np.ceil(np.sqrt(num_agents)))
            cols = int(np.ceil(num_agents/rows))
            sector_size = area_size / max(rows, cols)
            return [np.array([(i%cols)*sector_size + sector_size/2,
                            (i//cols)*sector_size + sector_size/2])
                    for i in range(num_agents)]
        elif config['type'] == 'random':
            return [np.array([random.uniform(0, area_size), random.uniform(0, area_size)])
                    for _ in range(num_agents)]
        elif config['type'] == 'circle':
            radius = config.get('radius', area_size/3)
            center = np.array([area_size/2, area_size/2])
            angles = np.linspace(0, 2*np.pi, num_agents, endpoint=False)
            return [center + radius * np.array([np.cos(a), np.sin(a)]) for a in angles]
        elif config['type'] == 'line':
            spacing = area_size / (num_agents + 1)
            return [np.array([spacing*(i+1), area_size/2]) for i in range(num_agents)]
        else:
            raise ValueError(f"Unknown initial config type: {config['type']}")

    def _generate_route(self, agent_id: int, start_pos: np.ndarray) -> List[np.ndarray]:
        route_config = self.config['route_config']
        area_size = self.config['area_size']

        if route_config['type'] == 'circle':
            radius = area_size/3
            center = np.array([area_size/2, area_size/2])
            angles = np.linspace(0, 2*np.pi, 20, endpoint=False)
            return [center + radius * np.array([np.cos(a), np.sin(a)]) for a in angles]
        elif route_config['type'] == 'straight':
            if agent_id % 2 == 0:
                return [np.array([area_size*0.2, area_size*0.2]),
                        np.array([area_size*0.8, area_size*0.8])]
            else:
                return [np.array([area_size*0.8, area_size*0.2]),
                        np.array([area_size*0.2, area_size*0.8])]
        elif route_config['type'] == 'common_goal':
            goal = np.array(route_config.get('goal', [area_size/2, area_size/2]))
            return [goal.copy()]
        else:
            raise ValueError(f"Unknown route config type: {route_config['type']}")

    def _create_algorithm(self, algo_config: Dict[str, Any]) -> SteeringAlgorithm:
        algo_type = algo_config['type']
        params = algo_config.get('params', {})
        if algo_type == 'VicsekAlgorithm':
            return VicsekAlgorithm(**params)
        elif algo_type == 'BoidsAlgorithm':
            return BoidsAlgorithm(**params)
        elif algo_type == 'PotentialFieldAlgorithm':
            if 'goal' in params:
                params['goal'] = np.array(params['goal'])
            return PotentialFieldAlgorithm(**params)
        elif algo_type == 'LeaderFollowerAlgorithm':
            return LeaderFollowerAlgorithm(**params)
        elif algo_type == 'RandomWalkAlgorithm':
            return RandomWalkAlgorithm(**params)
        else:
            raise ValueError(f"Unknown algorithm type: {algo_type}")

    def update_step(self):
        neighbors = self._find_neighbors()
        for agent, n in zip(self.agents, neighbors):
            agent.update(n, self.config['dt'], self.config['area_size'])
        self._collect_metrics()

    def _find_neighbors(self) -> List[List[Agent]]:
        positions = np.array([a.position for a in self.agents])
        dist_matrix = np.linalg.norm(positions[:, None] - positions, axis=2)
        neighbors = []
        for i, agent in enumerate(self.agents):
            mask = (dist_matrix[i] <= agent.specs.comm_radius) & (dist_matrix[i] > 0)
            neighbors.append([self.agents[j] for j in np.where(mask)[0]])
        return neighbors

    def _collect_metrics(self):
        positions = np.array([a.position for a in self.agents])
        velocities = np.array([a.velocity for a in self.agents])
        accelerations = np.array([a.acceleration for a in self.agents])

        self.metrics.avg_speed.append(np.mean(np.linalg.norm(velocities, axis=1)))
        self.metrics.max_speed.append(np.max(np.linalg.norm(velocities, axis=1)))
        self.metrics.avg_acceleration.append(np.mean(np.linalg.norm(accelerations, axis=1)))

        dist_matrix = np.linalg.norm(positions[:, None] - positions, axis=2)
        np.fill_diagonal(dist_matrix, np.nan)
        self.metrics.avg_distance.append(np.nanmean(dist_matrix))

        for agent in self.agents:
            if agent.id not in self.metrics.agent_trajectories:
                self.metrics.agent_trajectories[agent.id] = []
            self.metrics.agent_trajectories[agent.id].append(tuple(agent.position))

        collision_dist = 2 * self.agents[0].specs.physical_radius
        self.metrics.collisions += np.sum(dist_matrix < collision_dist) // 2

# ==================== ВІЗУАЛІЗАЦІЯ ====================
class Visualization:
    @staticmethod
    def animate_simulation(simulator: SwarmSimulator):
        fig, ax = plt.subplots(figsize=(10, 10))
        ax.set_xlim(0, simulator.config['area_size'])
        ax.set_ylim(0, simulator.config['area_size'])
        ax.set_title(f"Swarm Simulation - {simulator.config['algorithm']['type']}")

        scat = ax.scatter([], [], c='blue', s=50, alpha=0.6)
        lines = [ax.plot([], [], 'b-', alpha=0.3, linewidth=1)[0] for _ in range(simulator.config['num_agents'])]

        def init():
            scat.set_offsets(np.empty((0, 2)))
            for line in lines:
                line.set_data([], [])
            return [scat] + lines

        def update(frame):
            simulator.update_step()
            positions = np.array([agent.position for agent in simulator.agents])

            scat.set_offsets(positions)

            for idx, agent in enumerate(simulator.agents):
                trajectory = np.array(agent.path_history)
                lines[idx].set_data(trajectory[:, 0], trajectory[:, 1])

            return [scat] + lines

        ani = FuncAnimation(
            fig,
            update,
            frames=simulator.config['steps'],
            init_func=init,
            blit=True,
            interval=50,
            repeat=False
        )
        plt.close(fig)
        return ani

    @staticmethod
    def plot_metrics(metrics: SimulationMetrics, title: str):
        fig, axs = plt.subplots(2, 2, figsize=(15, 10))

        axs[0,0].plot(metrics.avg_speed)
        axs[0,0].set_title('Average Speed')
        axs[0,0].set_xlabel('Step')
        axs[0,0].set_ylabel('Speed')

        axs[0,1].plot(metrics.max_speed)
        axs[0,1].set_title('Maximum Speed')
        axs[0,1].set_xlabel('Step')
        axs[0,1].set_ylabel('Speed')

        axs[1,0].plot(metrics.avg_acceleration)
        axs[1,0].set_title('Average Acceleration')
        axs[1,0].set_xlabel('Step')
        axs[1,0].set_ylabel('Acceleration')

        axs[1,1].plot(metrics.avg_distance)
        axs[1,1].set_title('Average Distance Between Agents')
        axs[1,1].set_xlabel('Step')
        axs[1,1].set_ylabel('Distance')

        fig.suptitle(title)
        plt.tight_layout()
        plt.show()

# ==================== ТЕСТУВАННЯ АЛГОРИТМІВ ====================
def test_algorithms():
    # Спільна конфігурація для всіх алгоритмів
    base_config = {
        "num_agents": 15,
        "area_size": 100,
        "steps": 100,
        "dt": 0.1,
        "seed": 42,
        "agent_specs": {
            "physical_radius": 0.5,
            "safety_margin": 1.0,
            "max_speed": 2.0,
            "max_acceleration": 1.5,
            "comm_radius": 15.0
        },
        "initial_config": {
            "type": "circle",
            "radius": 30
        }
    }

    algorithms = [
        {
            "name": "Vicsek",
            "config": {
                "type": "VicsekAlgorithm",
                "params": {
                    "noise": 0.3,
                    "neighbor_radius": 20.0
                }
            },
            "route_config": {
                "type": "circle"
            }
        },
        {
            "name": "Boids",
            "config": {
                "type": "BoidsAlgorithm",
                "params": {
                    "sep_weight": 1.5,
                    "align_weight": 0.8,
                    "coh_weight": 0.5,
                    "visual_range": 20.0
                }
            },
            "route_config": {
                "type": "circle"
            }
        },
        {
            "name": "Potential Field",
            "config": {
                "type": "PotentialFieldAlgorithm",
                "params": {
                    "alpha": 0.5,
                    "beta": 1.0,
                    "dynamic_goal": True
                }
            },
            "route_config": {
                "type": "straight"
            }
        },
        {
            "name": "Leader-Follower",
            "config": {
                "type": "LeaderFollowerAlgorithm",
                "params": {
                    "leader_id": 0,
                    "follow_distance": 5.0
                }
            },
            "route_config": {
                "type": "straight"
            }
        },
        {
            "name": "Random Walk",
            "config": {
                "type": "RandomWalkAlgorithm",
                "params": {
                    "max_random_acceleration": 0.5
                }
            },
            "route_config": {
                "type": "common_goal",
                "goal": [50, 50]
            }
        }
    ]

    results = []

    print("=== Testing Algorithms with Same Initial Conditions ===")

    for algo in algorithms:
        # Створення конфігурації для поточного алгоритму
        config = base_config.copy()
        config["algorithm"] = algo["config"]
        config["route_config"] = algo["route_config"]

        print(f"\nRunning {algo['name']} algorithm...")
        simulator = SwarmSimulator(config)

        # Запуск симуляції
        for _ in range(config["steps"]):
            simulator.update_step()

        # Збереження результатів
        results.append({
            "name": algo["name"],
            "metrics": simulator.metrics,
            "config": config
        })

        # Візуалізація
        ani = Visualization.animate_simulation(simulator)
        display(HTML(ani.to_jshtml()))
        Visualization.plot_metrics(simulator.metrics, f"Metrics - {algo['name']}")

        print(f"Collisions: {simulator.metrics.collisions}")
        print(f"Avg Speed: {np.mean(simulator.metrics.avg_speed):.2f}")
        print(f"Avg Distance: {np.mean(simulator.metrics.avg_distance):.2f}")

    return results

# Запуск тестування
algorithm_results = test_algorithms()

In [ ]:
import numpy as np
from dataclasses import dataclass
from abc import ABC, abstractmethod
from typing import List, Dict, Tuple, Optional, Any
import random

# ==================== БАЗОВІ КЛАСИ ====================
@dataclass
class AgentSpecs:
    physical_radius: float
    safety_margin: float
    max_speed: float
    max_acceleration: float
    comm_radius: float

class SteeringAlgorithm(ABC):
    @abstractmethod
    def compute_steering(self, agent, neighbors: List['Agent'], dt: float) -> np.ndarray:
        pass

@dataclass
class SimulationMetrics:
    collisions: int = 0
    avg_speed: List[float] = None
    max_speed: List[float] = None
    avg_acceleration: List[float] = None
    avg_distance: List[float] = None
    agent_trajectories: Dict[int, List[Tuple[float, float]]] = None

    def __post_init__(self):
        self.avg_speed = []
        self.max_speed = []
        self.avg_acceleration = []
        self.avg_distance = []
        self.agent_trajectories = {}

class Agent:
    def __init__(self, agent_id: int, position: np.ndarray, specs: AgentSpecs,
                 algorithm: SteeringAlgorithm, route: List[np.ndarray]):
        self.id = agent_id
        self.position = position
        self.velocity = np.zeros(2)
        self.acceleration = np.zeros(2)
        self.specs = specs
        self.algorithm = algorithm
        self.route = route
        self.current_waypoint = 0
        self.path_history = [position.copy()]

    def update(self, neighbors: List['Agent'], dt: float, area_size: float):
        steering = self.algorithm.compute_steering(self, neighbors, dt)
        steering = self._limit_vector(steering, self.specs.max_acceleration)
        self.velocity = self._limit_vector(self.velocity + steering * dt, self.specs.max_speed)
        self.position += self.velocity * dt
        self.acceleration = steering
        self._enforce_boundaries(area_size)
        self.path_history.append(self.position.copy())

    def _limit_vector(self, vector: np.ndarray, max_magnitude: float) -> np.ndarray:
        magnitude = np.linalg.norm(vector)
        return vector if magnitude == 0 else vector * min(max_magnitude/magnitude, 1)

    def _enforce_boundaries(self, area_size: float):
        self.position = np.clip(self.position, 0, area_size)

# ==================== АЛГОРИТМИ КЕРУВАННЯ ====================
class VicsekAlgorithm(SteeringAlgorithm):
    def __init__(self, noise: float = 0.1, neighbor_radius: float = 15.0):
        self.noise = noise
        self.neighbor_radius = neighbor_radius

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        neighbors = [n for n in neighbors if np.linalg.norm(n.position - agent.position) <= self.neighbor_radius]
        if not neighbors:
            return np.zeros(2)
        avg_velocity = np.mean([n.velocity for n in neighbors], axis=0)
        noise = np.random.normal(scale=self.noise, size=2)
        return (avg_velocity + noise - agent.velocity) / dt

class BoidsAlgorithm(SteeringAlgorithm):
    def __init__(self, sep_weight: float = 1.5, align_weight: float = 1.0,
                 coh_weight: float = 1.0, visual_range: float = 15.0):
        self.weights = np.array([sep_weight, align_weight, coh_weight])
        self.visual_range = visual_range

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        neighbors = [n for n in neighbors if np.linalg.norm(n.position - agent.position) <= self.visual_range]
        if not neighbors:
            return np.zeros(2)
        positions = np.array([n.position for n in neighbors])
        velocities = np.array([n.velocity for n in neighbors])
        sep = self._separation(agent.position, positions)
        align = self._alignment(agent.velocity, velocities)
        coh = self._cohesion(agent.position, positions)
        return self.weights @ np.array([sep, align, coh])

    def _separation(self, pos: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        diff = pos - neighbors
        distances = np.linalg.norm(diff, axis=1)[:, np.newaxis]
        return np.mean(diff / (distances + 1e-6), axis=0)

    def _alignment(self, vel: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        return np.mean(neighbors, axis=0) - vel

    def _cohesion(self, pos: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        return np.mean(neighbors, axis=0) - pos

class PotentialFieldAlgorithm(SteeringAlgorithm):
    def __init__(self, alpha: float = 0.5, beta: float = 1.0,
                 goal: Optional[np.ndarray] = None, dynamic_goal: bool = False):
        self.goal = goal
        self.alpha = alpha
        self.beta = beta
        self.dynamic_goal = dynamic_goal

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        if self.dynamic_goal and agent.route:
            self.goal = agent.route[-1]
        to_goal = self.goal - agent.position if self.goal is not None else np.zeros(2)
        attraction = self.alpha * to_goal

        repulsion = np.zeros(2)
        for n in neighbors:
            vec = agent.position - n.position
            dist = np.linalg.norm(vec)
            if dist > 0:
                repulsion += self.beta * vec / dist**2

        return attraction + repulsion - agent.velocity / dt

class LeaderFollowerAlgorithm(SteeringAlgorithm):
    def __init__(self, leader_id: int = 0, follow_distance: float = 5.0):
        self.leader_id = leader_id
        self.follow_distance = follow_distance

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        leader = next((n for n in neighbors if n.id == self.leader_id), None)
        if leader is None or agent.id == self.leader_id:
            return np.zeros(2)
        direction = leader.position - agent.position
        distance = np.linalg.norm(direction)
        desired_velocity = (direction / distance) * agent.specs.max_speed if distance > 0 else np.zeros(2)
        return (desired_velocity - agent.velocity) / dt

class RandomWalkAlgorithm(SteeringAlgorithm):
    def __init__(self, max_random_acceleration: float = 0.1):
        self.max_random_acceleration = max_random_acceleration

    def compute_steering(self, agent: Agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        return np.random.uniform(-self.max_random_acceleration, self.max_random_acceleration, size=2)

# ==================== СИСТЕМА СИМУЛЯЦІЇ ====================
class SwarmSimulator:
    def __init__(self, config: Dict[str, Any]):
        self.config = config
        np.random.seed(config.get('seed', 42))
        random.seed(config.get('seed', 42))
        self._init_simulation()

    def _init_simulation(self):
        self.agents = []
        self.metrics = SimulationMetrics()
        positions = self._generate_initial_positions()
        for i in range(self.config['num_agents']):
            route = self._generate_route(i, positions[i])
            specs = AgentSpecs(**self.config['agent_specs'])
            algorithm = self._create_algorithm(self.config['algorithm'])
            agent = Agent(i, positions[i], specs, algorithm, route)
            self.agents.append(agent)

    def _generate_initial_positions(self) -> List[np.ndarray]:
        config = self.config['initial_config']
        area_size = self.config['area_size']
        num_agents = self.config['num_agents']

        if config['type'] == 'grid':
            rows = int(np.ceil(np.sqrt(num_agents)))
            cols = int(np.ceil(num_agents/rows))
            sector_size = area_size / max(rows, cols)
            return [np.array([(i%cols)*sector_size + sector_size/2,
                            (i//cols)*sector_size + sector_size/2])
                    for i in range(num_agents)]
        elif config['type'] == 'random':
            return [np.array([random.uniform(0, area_size), random.uniform(0, area_size)])
                    for _ in range(num_agents)]
        elif config['type'] == 'circle':
            radius = config.get('radius', area_size/3)
            center = np.array([area_size/2, area_size/2])
            angles = np.linspace(0, 2*np.pi, num_agents, endpoint=False)
            return [center + radius * np.array([np.cos(a), np.sin(a)]) for a in angles]
        elif config['type'] == 'line':
            spacing = area_size / (num_agents + 1)
            return [np.array([spacing*(i+1), area_size/2]) for i in range(num_agents)]
        else:
            raise ValueError(f"Unknown initial config type: {config['type']}")

    def _generate_route(self, agent_id: int, start_pos: np.ndarray) -> List[np.ndarray]:
        route_config = self.config['route_config']
        area_size = self.config['area_size']

        if route_config['type'] == 'circle':
            radius = area_size/3
            center = np.array([area_size/2, area_size/2])
            angles = np.linspace(0, 2*np.pi, 20, endpoint=False)
            return [center + radius * np.array([np.cos(a), np.sin(a)]) for a in angles]
        elif route_config['type'] == 'straight':
            if agent_id % 2 == 0:
                return [np.array([area_size*0.2, area_size*0.2]),
                        np.array([area_size*0.8, area_size*0.8])]
            else:
                return [np.array([area_size*0.8, area_size*0.2]),
                        np.array([area_size*0.2, area_size*0.8])]
        elif route_config['type'] == 'common_goal':
            goal = np.array(route_config.get('goal', [area_size/2, area_size/2]))
            return [goal.copy()]
        else:
            raise ValueError(f"Unknown route config type: {route_config['type']}")

    def _create_algorithm(self, algo_config: Dict[str, Any]) -> SteeringAlgorithm:
        algo_type = algo_config['type']
        params = algo_config.get('params', {})
        if algo_type == 'VicsekAlgorithm':
            return VicsekAlgorithm(**params)
        elif algo_type == 'BoidsAlgorithm':
            return BoidsAlgorithm(**params)
        elif algo_type == 'PotentialFieldAlgorithm':
            if 'goal' in params:
                params['goal'] = np.array(params['goal'])
            return PotentialFieldAlgorithm(**params)
        elif algo_type == 'LeaderFollowerAlgorithm':
            return LeaderFollowerAlgorithm(**params)
        elif algo_type == 'RandomWalkAlgorithm':
            return RandomWalkAlgorithm(**params)
        else:
            raise ValueError(f"Unknown algorithm type: {algo_type}")

    def update_step(self):
        neighbors = self._find_neighbors()
        for agent, n in zip(self.agents, neighbors):
            agent.update(n, self.config['dt'], self.config['area_size'])
        self._collect_metrics()

    def _find_neighbors(self) -> List[List[Agent]]:
        positions = np.array([a.position for a in self.agents])
        dist_matrix = np.linalg.norm(positions[:, None] - positions, axis=2)
        neighbors = []
        for i, agent in enumerate(self.agents):
            mask = (dist_matrix[i] <= agent.specs.comm_radius) & (dist_matrix[i] > 0)
            neighbors.append([self.agents[j] for j in np.where(mask)[0]])
        return neighbors

    def _collect_metrics(self):
        positions = np.array([a.position for a in self.agents])
        velocities = np.array([a.velocity for a in self.agents])
        accelerations = np.array([a.acceleration for a in self.agents])

        self.metrics.avg_speed.append(np.mean(np.linalg.norm(velocities, axis=1)))
        self.metrics.max_speed.append(np.max(np.linalg.norm(velocities, axis=1)))
        self.metrics.avg_acceleration.append(np.mean(np.linalg.norm(accelerations, axis=1)))

        dist_matrix = np.linalg.norm(positions[:, None] - positions, axis=2)
        np.fill_diagonal(dist_matrix, np.nan)
        self.metrics.avg_distance.append(np.nanmean(dist_matrix))

        for agent in self.agents:
            if agent.id not in self.metrics.agent_trajectories:
                self.metrics.agent_trajectories[agent.id] = []
            self.metrics.agent_trajectories[agent.id].append(tuple(agent.position))

        collision_dist = 2 * self.agents[0].specs.physical_radius
        self.metrics.collisions += np.sum(dist_matrix < collision_dist) // 2

# ==================== ТЕСТУВАННЯ АЛГОРИТМІВ ====================
def test_algorithms():
    # Спільна конфігурація для всіх алгоритмів
    base_config = {
        "num_agents": 15,
        "area_size": 100,
        "steps": 10000,  # Час симуляції
        "dt": 0.1,
        "seed": 42,
        "agent_specs": {
            "physical_radius": 0.5,
            "safety_margin": 1.0,
            "max_speed": 2.0,
            "max_acceleration": 1.5,
            "comm_radius": 15.0
        },
        "initial_config": {
            "type": "circle",
            "radius": 30
        }
    }

    algorithms = [
        {
            "name": "Vicsek",
            "config": {
                "type": "VicsekAlgorithm",
                "params": {
                    "noise": 0.3,
                    "neighbor_radius": 20.0
                }
            }
        },
        {
            "name": "Boids",
            "config": {
                "type": "BoidsAlgorithm",
                "params": {
                    "sep_weight": 1.5,
                    "align_weight": 0.8,
                    "coh_weight": 0.5,
                    "visual_range": 20.0
                }
            }
        },
        {
            "name": "Potential Field",
            "config": {
                "type": "PotentialFieldAlgorithm",
                "params": {
                    "alpha": 0.5,
                    "beta": 1.0,
                    "dynamic_goal": True
                }
            }
        },
        {
            "name": "Leader-Follower",
            "config": {
                "type": "LeaderFollowerAlgorithm",
                "params": {
                    "leader_id": 0,
                    "follow_distance": 5.0
                }
            }
        },
        {
            "name": "Random Walk",
            "config": {
                "type": "RandomWalkAlgorithm",
                "params": {
                    "max_random_acceleration": 0.5
                }
            }
        }
    ]

    route_configs = [
        {"type": "circle"},
        {"type": "straight"},
        {"type": "common_goal", "goal": [50, 50]}
    ]

    results = {algo["name"]: {"collisions": [], "avg_speed": [], "avg_distance": []} for algo in algorithms}

    print("=== Testing Algorithms with Different Route Configurations ===")

    for route_config in route_configs:
        print(f"\nRoute Configuration: {route_config['type']}")

        for algo in algorithms:
            for _ in range(1):  # Кількість запусків для кожного алгоритму
                # Створення конфігурації для поточного алгоритму
                config = base_config.copy()
                config["algorithm"] = algo["config"]
                config["route_config"] = route_config
                config["initial_config"] = {"type": "circle", "radius": 30}

                simulator = SwarmSimulator(config)

                # Запуск симуляції
                for _ in range(config["steps"]):
                    simulator.update_step()

                # Збереження результатів
                results[algo["name"]]["collisions"].append(simulator.metrics.collisions)
                results[algo["name"]]["avg_speed"].append(np.mean(simulator.metrics.avg_speed))
                results[algo["name"]]["avg_distance"].append(np.mean(simulator.metrics.avg_distance))

    # Вивід усереднених даних
    print("\n=== Averaged Results ===")
    for algo_name, metrics in results.items():
        print(f"\nAlgorithm: {algo_name}")
        print(f"Average Collisions: {np.mean(metrics['collisions']):.2f}")
        print(f"Average Speed: {np.mean(metrics['avg_speed']):.2f}")
        print(f"Average Distance: {np.mean(metrics['avg_distance']):.2f}")

# Запуск тестування
test_algorithms()

In [ ]:
import numpy as np
from dataclasses import dataclass
from abc import ABC, abstractmethod
from typing import List, Dict, Tuple, Optional, Any
import random

# ==================== БАЗОВІ КЛАСИ ====================
@dataclass
class AgentSpecs:
    physical_radius: float
    safety_margin: float
    max_speed: float
    max_acceleration: float
    comm_radius: float

class SteeringAlgorithm(ABC):
    @abstractmethod
    def compute_steering(self, agent, neighbors: List['Agent'], dt: float) -> np.ndarray:
        pass

@dataclass
class SimulationMetrics:
    collisions: int = 0
    avg_speed: List[float] = None
    max_speed: List[float] = None
    avg_acceleration: List[float] = None
    avg_distance: List[float] = None
    agent_trajectories: Dict[int, List[Tuple[float, float]]] = None

    def __post_init__(self):
        self.avg_speed = []
        self.max_speed = []
        self.avg_acceleration = []
        self.avg_distance = []
        self.agent_trajectories = {}

class Agent:
    def __init__(self, agent_id: int, position: np.ndarray, specs: AgentSpecs,
                 algorithm: SteeringAlgorithm, route: List[np.ndarray]):
        self.id = agent_id
        self.position = position
        self.velocity = np.zeros(2)
        self.acceleration = np.zeros(2)
        self.specs = specs
        self.algorithm = algorithm
        self.route = route
        self.current_waypoint = 0
        self.path_history = [position.copy()]

    def update(self, neighbors: List['Agent'], dt: float, area_size: float):
        steering = self.algorithm.compute_steering(self, neighbors, dt)
        steering = self._limit_vector(steering, self.specs.max_acceleration)
        self.velocity = self._limit_vector(self.velocity + steering * dt, self.specs.max_speed)
        self.position += self.velocity * dt
        self.acceleration = steering
        self._enforce_boundaries(area_size)
        self.path_history.append(self.position.copy())

    def _limit_vector(self, vector: np.ndarray, max_magnitude: float) -> np.ndarray:
        magnitude = np.linalg.norm(vector)
        return vector if magnitude == 0 else vector * min(max_magnitude/magnitude, 1)

    def _enforce_boundaries(self, area_size: float):
        self.position = np.clip(self.position, 0, area_size)

# ==================== АЛГОРИТМИ КЕРУВАННЯ ====================
class VicsekAlgorithm(SteeringAlgorithm):
    def __init__(self, noise: float = 0.1, neighbor_radius: float = 15.0):
        self.noise = noise
        self.neighbor_radius = neighbor_radius

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        neighbors = [n for n in neighbors if np.linalg.norm(n.position - agent.position) <= self.neighbor_radius]
        if not neighbors:
            return np.zeros(2)
        avg_velocity = np.mean([n.velocity for n in neighbors], axis=0)
        noise = np.random.normal(scale=self.noise, size=2)
        return (avg_velocity + noise - agent.velocity) / dt

class BoidsAlgorithm(SteeringAlgorithm):
    def __init__(self, sep_weight: float = 1.5, align_weight: float = 1.0,
                 coh_weight: float = 1.0, visual_range: float = 15.0):
        self.weights = np.array([sep_weight, align_weight, coh_weight])
        self.visual_range = visual_range

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        neighbors = [n for n in neighbors if np.linalg.norm(n.position - agent.position) <= self.visual_range]
        if not neighbors:
            return np.zeros(2)
        positions = np.array([n.position for n in neighbors])
        velocities = np.array([n.velocity for n in neighbors])
        sep = self._separation(agent.position, positions)
        align = self._alignment(agent.velocity, velocities)
        coh = self._cohesion(agent.position, positions)
        return self.weights @ np.array([sep, align, coh])

    def _separation(self, pos: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        diff = pos - neighbors
        distances = np.linalg.norm(diff, axis=1)[:, np.newaxis]
        return np.mean(diff / (distances + 1e-6), axis=0)

    def _alignment(self, vel: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        return np.mean(neighbors, axis=0) - vel

    def _cohesion(self, pos: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        return np.mean(neighbors, axis=0) - pos

class PotentialFieldAlgorithm(SteeringAlgorithm):
    def __init__(self, alpha: float = 0.5, beta: float = 1.0,
                 goal: Optional[np.ndarray] = None, dynamic_goal: bool = False):
        self.goal = goal
        self.alpha = alpha
        self.beta = beta
        self.dynamic_goal = dynamic_goal

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        if self.dynamic_goal and agent.route:
            self.goal = agent.route[-1]
        to_goal = self.goal - agent.position if self.goal is not None else np.zeros(2)
        attraction = self.alpha * to_goal
        repulsion = np.zeros(2)
        for n in neighbors:
            vec = agent.position - n.position
            dist = np.linalg.norm(vec)
            if dist > 0:
                repulsion += self.beta * vec / dist**2
        return attraction + repulsion - agent.velocity / dt

class LeaderFollowerAlgorithm(SteeringAlgorithm):
    def __init__(self, leader_id: int = 0, follow_distance: float = 5.0):
        self.leader_id = leader_id
        self.follow_distance = follow_distance

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        leader = next((n for n in neighbors if n.id == self.leader_id), None)
        if leader is None or agent.id == self.leader_id:
            return np.zeros(2)
        direction = leader.position - agent.position
        distance = np.linalg.norm(direction)
        desired_velocity = (direction / distance) * agent.specs.max_speed if distance > 0 else np.zeros(2)
        return (desired_velocity - agent.velocity) / dt

class RandomWalkAlgorithm(SteeringAlgorithm):
    def __init__(self, max_random_acceleration: float = 0.1):
        self.max_random_acceleration = max_random_acceleration

    def compute_steering(self, agent: Agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        return np.random.uniform(-self.max_random_acceleration, self.max_random_acceleration, size=2)

# ==================== СИСТЕМА СИМУЛЯЦІЇ ====================
class SwarmSimulator:
    def __init__(self, config: Dict[str, Any]):
        self.config = config
        np.random.seed(config.get('seed', 42))
        random.seed(config.get('seed', 42))
        self._init_simulation()

    def _init_simulation(self):
        self.agents = []
        self.metrics = SimulationMetrics()
        positions = self._generate_initial_positions()
        for i in range(self.config['num_agents']):
            route = self._generate_route(i, positions[i])
            specs = AgentSpecs(**self.config['agent_specs'])
            algorithm = self._create_algorithm(self.config['algorithm'])
            agent = Agent(i, positions[i], specs, algorithm, route)
            self.agents.append(agent)

    def _generate_initial_positions(self) -> List[np.ndarray]:
        config = self.config['initial_config']
        area_size = self.config['area_size']
        num_agents = self.config['num_agents']

        if config['type'] == 'grid':
            rows = int(np.ceil(np.sqrt(num_agents)))
            cols = int(np.ceil(num_agents/rows))
            sector_size = area_size / max(rows, cols)
            return [np.array([(i%cols)*sector_size + sector_size/2,
                            (i//cols)*sector_size + sector_size/2])
                    for i in range(num_agents)]
        elif config['type'] == 'random':
            return [np.array([random.uniform(0, area_size), random.uniform(0, area_size)])
                    for _ in range(num_agents)]
        elif config['type'] == 'circle':
            radius = config.get('radius', area_size/3)
            center = np.array([area_size/2, area_size/2])
            angles = np.linspace(0, 2*np.pi, num_agents, endpoint=False)
            return [center + radius * np.array([np.cos(a), np.sin(a)]) for a in angles]
        elif config['type'] == 'line':
            spacing = area_size / (num_agents + 1)
            return [np.array([spacing*(i+1), area_size/2]) for i in range(num_agents)]
        else:
            raise ValueError(f"Unknown initial config type: {config['type']}")

    def _generate_route(self, agent_id: int, start_pos: np.ndarray) -> List[np.ndarray]:
        route_config = self.config['route_config']
        area_size = self.config['area_size']

        if route_config['type'] == 'circle':
            radius = area_size/3
            center = np.array([area_size/2, area_size/2])
            angles = np.linspace(0, 2*np.pi, 20, endpoint=False)
            return [center + radius * np.array([np.cos(a), np.sin(a)]) for a in angles]
        elif route_config['type'] == 'straight':
            if agent_id % 2 == 0:
                return [np.array([area_size*0.2, area_size*0.2]),
                        np.array([area_size*0.8, area_size*0.8])]
            else:
                return [np.array([area_size*0.8, area_size*0.2]),
                        np.array([area_size*0.2, area_size*0.8])]
        elif route_config['type'] == 'common_goal':
            goal = np.array(route_config.get('goal', [area_size/2, area_size/2]))
            return [goal.copy()]
        else:
            raise ValueError(f"Unknown route config type: {route_config['type']}")

    def _create_algorithm(self, algo_config: Dict[str, Any]) -> SteeringAlgorithm:
        algo_type = algo_config['type']
        params = algo_config.get('params', {})
        if algo_type == 'VicsekAlgorithm':
            return VicsekAlgorithm(**params)
        elif algo_type == 'BoidsAlgorithm':
            return BoidsAlgorithm(**params)
        elif algo_type == 'PotentialFieldAlgorithm':
            if 'goal' in params:
                params['goal'] = np.array(params['goal'])
            return PotentialFieldAlgorithm(**params)
        elif algo_type == 'LeaderFollowerAlgorithm':
            return LeaderFollowerAlgorithm(**params)
        elif algo_type == 'RandomWalkAlgorithm':
            return RandomWalkAlgorithm(**params)
        else:
            raise ValueError(f"Unknown algorithm type: {algo_type}")

    def update_step(self):
        neighbors = self._find_neighbors()
        for agent, n in zip(self.agents, neighbors):
            agent.update(n, self.config['dt'], self.config['area_size'])
        self._collect_metrics()

    def _find_neighbors(self) -> List[List[Agent]]:
        positions = np.array([a.position for a in self.agents])
        dist_matrix = np.linalg.norm(positions[:, None] - positions, axis=2)
        neighbors = []
        for i, agent in enumerate(self.agents):
            mask = (dist_matrix[i] <= agent.specs.comm_radius) & (dist_matrix[i] > 0)
            neighbors.append([self.agents[j] for j in np.where(mask)[0]])
        return neighbors

    def _collect_metrics(self):
        positions = np.array([a.position for a in self.agents])
        velocities = np.array([a.velocity for a in self.agents])
        accelerations = np.array([a.acceleration for a in self.agents])

        self.metrics.avg_speed.append(np.mean(np.linalg.norm(velocities, axis=1)))
        self.metrics.max_speed.append(np.max(np.linalg.norm(velocities, axis=1)))
        self.metrics.avg_acceleration.append(np.mean(np.linalg.norm(accelerations, axis=1)))

        dist_matrix = np.linalg.norm(positions[:, None] - positions, axis=2)
        np.fill_diagonal(dist_matrix, np.nan)
        self.metrics.avg_distance.append(np.nanmean(dist_matrix))

        for agent in self.agents:
            if agent.id not in self.metrics.agent_trajectories:
                self.metrics.agent_trajectories[agent.id] = []
            self.metrics.agent_trajectories[agent.id].append(tuple(agent.position))

        collision_dist = 2 * self.agents[0].specs.physical_radius
        self.metrics.collisions += np.sum(dist_matrix < collision_dist) // 2

# ==================== ТЕСТУВАННЯ АЛГОРИТМІВ ====================
def test_algorithms():
    # Спільна конфігурація для всіх алгоритмів
    base_config = {
        "num_agents": 150,
        "area_size": 100,
        "steps": 1000,
        "dt": 0.1,
        "seed": 42,
        "agent_specs": {
            "physical_radius": 0.5,
            "safety_margin": 1.0,
            "max_speed": 2.0,
            "max_acceleration": 1.5,
            "comm_radius": 15.0
        }
    }

    algorithms = [
        {
            "name": "Vicsek",
            "config": {
                "type": "VicsekAlgorithm",
                "params": {
                    "noise": 0.3,
                    "neighbor_radius": 20.0
                }
            }
        },
        {
            "name": "Boids",
            "config": {
                "type": "BoidsAlgorithm",
                "params": {
                    "sep_weight": 1.5,
                    "align_weight": 0.8,
                    "coh_weight": 0.5,
                    "visual_range": 20.0
                }
            }
        },
        {
            "name": "Potential Field",
            "config": {
                "type": "PotentialFieldAlgorithm",
                "params": {
                    "alpha": 0.5,
                    "beta": 1.0,
                    "dynamic_goal": True
                }
            }
        },
        {
            "name": "Leader-Follower",
            "config": {
                "type": "LeaderFollowerAlgorithm",
                "params": {
                    "leader_id": 0,
                    "follow_distance": 5.0
                }
            }
        },
        {
            "name": "Random Walk",
            "config": {
                "type": "RandomWalkAlgorithm",
                "params": {
                    "max_random_acceleration": 0.5
                }
            }
        }
    ]

    route_configs = [
        {"type": "circle"},
        {"type": "straight"},
        {"type": "common_goal", "goal": [50, 50]}
    ]

    results = []

    print("=== Testing Algorithms with Different Route Configurations ===")

    for route_config in route_configs:
        print(f"\nRoute Configuration: {route_config['type']}")

        for algo in algorithms:
            # Створення конфігурації для поточного алгоритму
            config = base_config.copy()
            config["algorithm"] = algo["config"]
            config["route_config"] = route_config
            config["initial_config"] = {"type": "circle", "radius": 30}

            print(f"\nRunning {algo['name']} algorithm...")
            simulator = SwarmSimulator(config)

            # Запуск симуляції
            for _ in range(config["steps"]):
                simulator.update_step()

            # Збереження результатів
            results.append({
                "algorithm": algo["name"],
                "route_config": route_config["type"],
                "metrics": simulator.metrics,
                "config": config
            })

            # Вивід текстових даних
            print(f"Collisions: {simulator.metrics.collisions}")
            print(f"Avg Speed: {np.mean(simulator.metrics.avg_speed):.2f}")
            print(f"Avg Distance: {np.mean(simulator.metrics.avg_distance):.2f}")

    return results

# Запуск тестування
algorithm_results = test_algorithms()

In [ ]:
import numpy as np
from dataclasses import dataclass
from abc import ABC, abstractmethod
from typing import List, Dict, Tuple, Optional, Any
import random
import time

# ==================== БАЗОВІ КЛАСИ ====================
@dataclass
class AgentSpecs:
    physical_radius: float
    safety_margin: float
    max_speed: float
    max_acceleration: float
    comm_radius: float

class SteeringAlgorithm(ABC):
    @abstractmethod
    def compute_steering(self, agent, neighbors: List['Agent'], dt: float) -> np.ndarray:
        pass

@dataclass
class SimulationMetrics:
    collisions: int = 0
    avg_speed: List[float] = None
    max_speed: List[float] = None
    avg_acceleration: List[float] = None
    avg_distance: List[float] = None
    agent_trajectories: Dict[int, List[Tuple[float, float]]] = None

    def __post_init__(self):
        self.avg_speed = []
        self.max_speed = []
        self.avg_acceleration = []
        self.avg_distance = []
        self.agent_trajectories = {}

class Agent:
    def __init__(self, agent_id: int, position: np.ndarray, specs: AgentSpecs,
                 algorithm: SteeringAlgorithm, route: List[np.ndarray]):
        self.id = agent_id
        self.position = position
        self.velocity = np.zeros(2)
        self.acceleration = np.zeros(2)
        self.specs = specs
        self.algorithm = algorithm
        self.route = route
        self.current_waypoint = 0
        self.path_history = [position.copy()]

    def update(self, neighbors: List['Agent'], dt: float, area_size: float):
        steering = self.algorithm.compute_steering(self, neighbors, dt)
        steering = self._limit_vector(steering, self.specs.max_acceleration)
        self.velocity = self._limit_vector(self.velocity + steering * dt, self.specs.max_speed)
        self.position += self.velocity * dt
        self.acceleration = steering
        self._enforce_boundaries(area_size)
        self.path_history.append(self.position.copy())

    def _limit_vector(self, vector: np.ndarray, max_magnitude: float) -> np.ndarray:
        magnitude = np.linalg.norm(vector)
        return vector if magnitude == 0 else vector * min(max_magnitude/magnitude, 1)

    def _enforce_boundaries(self, area_size: float):
        self.position = np.clip(self.position, 0, area_size)

# ==================== АЛГОРИТМИ КЕРУВАННЯ ====================
class VicsekAlgorithm(SteeringAlgorithm):
    def __init__(self, noise: float = 0.1, neighbor_radius: float = 15.0):
        self.noise = noise
        self.neighbor_radius = neighbor_radius

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        neighbors = [n for n in neighbors if np.linalg.norm(n.position - agent.position) <= self.neighbor_radius]
        if not neighbors:
            return np.zeros(2)
        avg_velocity = np.mean([n.velocity for n in neighbors], axis=0)
        noise = np.random.normal(scale=self.noise, size=2)
        return (avg_velocity + noise - agent.velocity) / dt

class BoidsAlgorithm(SteeringAlgorithm):
    def __init__(self, sep_weight: float = 1.5, align_weight: float = 1.0,
                 coh_weight: float = 1.0, visual_range: float = 15.0):
        self.weights = np.array([sep_weight, align_weight, coh_weight])
        self.visual_range = visual_range

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        neighbors = [n for n in neighbors if np.linalg.norm(n.position - agent.position) <= self.visual_range]
        if not neighbors:
            return np.zeros(2)
        positions = np.array([n.position for n in neighbors])
        velocities = np.array([n.velocity for n in neighbors])
        sep = self._separation(agent.position, positions)
        align = self._alignment(agent.velocity, velocities)
        coh = self._cohesion(agent.position, positions)
        return self.weights @ np.array([sep, align, coh])

    def _separation(self, pos: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        diff = pos - neighbors
        distances = np.linalg.norm(diff, axis=1)[:, np.newaxis]
        return np.mean(diff / (distances + 1e-6), axis=0)

    def _alignment(self, vel: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        return np.mean(neighbors, axis=0) - vel

    def _cohesion(self, pos: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        return np.mean(neighbors, axis=0) - pos

class PotentialFieldAlgorithm(SteeringAlgorithm):
    def __init__(self, alpha: float = 0.5, beta: float = 1.0,
                 goal: Optional[np.ndarray] = None, dynamic_goal: bool = False):
        self.goal = goal
        self.alpha = alpha
        self.beta = beta
        self.dynamic_goal = dynamic_goal

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        if self.dynamic_goal and agent.route:
            self.goal = agent.route[-1]
        to_goal = self.goal - agent.position if self.goal is not None else np.zeros(2)
        attraction = self.alpha * to_goal
        repulsion = np.zeros(2)
        for n in neighbors:
            vec = agent.position - n.position
            dist = np.linalg.norm(vec)
            if dist > 0:
                repulsion += self.beta * vec / dist**2
        return attraction + repulsion - agent.velocity / dt

class LeaderFollowerAlgorithm(SteeringAlgorithm):
    def __init__(self, leader_id: int = 0, follow_distance: float = 5.0):
        self.leader_id = leader_id
        self.follow_distance = follow_distance

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        leader = next((n for n in neighbors if n.id == self.leader_id), None)
        if leader is None or agent.id == self.leader_id:
            return np.zeros(2)
        direction = leader.position - agent.position
        distance = np.linalg.norm(direction)
        desired_velocity = (direction / distance) * agent.specs.max_speed if distance > 0 else np.zeros(2)
        return (desired_velocity - agent.velocity) / dt

class RandomWalkAlgorithm(SteeringAlgorithm):
    def __init__(self, max_random_acceleration: float = 0.1):
        self.max_random_acceleration = max_random_acceleration

    def compute_steering(self, agent: Agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        return np.random.uniform(-self.max_random_acceleration, self.max_random_acceleration, size=2)

# ==================== СИСТЕМА СИМУЛЯЦІЇ ====================
class SwarmSimulator:
    def __init__(self, config: Dict[str, Any]):
        self.config = config
        np.random.seed(config.get('seed', 42))
        random.seed(config.get('seed', 42))
        self._init_simulation()

    def _init_simulation(self):
        self.agents = []
        self.metrics = SimulationMetrics()
        positions = self._generate_initial_positions()
        for i in range(self.config['num_agents']):
            route = self._generate_route(i, positions[i])
            specs = AgentSpecs(**self.config['agent_specs'])
            algorithm = self._create_algorithm(self.config['algorithm'])
            agent = Agent(i, positions[i], specs, algorithm, route)
            self.agents.append(agent)

    def _generate_initial_positions(self) -> List[np.ndarray]:
        config = self.config['initial_config']
        area_size = self.config['area_size']
        num_agents = self.config['num_agents']

        if config['type'] == 'grid':
            rows = int(np.ceil(np.sqrt(num_agents)))
            cols = int(np.ceil(num_agents/rows))
            sector_size = area_size / max(rows, cols)
            return [np.array([(i%cols)*sector_size + sector_size/2,
                            (i//cols)*sector_size + sector_size/2])
                    for i in range(num_agents)]
        elif config['type'] == 'random':
            return [np.array([random.uniform(0, area_size), random.uniform(0, area_size)])
                    for _ in range(num_agents)]
        elif config['type'] == 'circle':
            radius = config.get('radius', area_size/3)
            center = np.array([area_size/2, area_size/2])
            angles = np.linspace(0, 2*np.pi, num_agents, endpoint=False)
            return [center + radius * np.array([np.cos(a), np.sin(a)]) for a in angles]
        elif config['type'] == 'line':
            spacing = area_size / (num_agents + 1)
            return [np.array([spacing*(i+1), area_size/2]) for i in range(num_agents)]
        else:
            raise ValueError(f"Unknown initial config type: {config['type']}")

    def _generate_route(self, agent_id: int, start_pos: np.ndarray) -> List[np.ndarray]:
        route_config = self.config['route_config']
        area_size = self.config['area_size']

        if route_config['type'] == 'circle':
            radius = area_size/3
            center = np.array([area_size/2, area_size/2])
            angles = np.linspace(0, 2*np.pi, 20, endpoint=False)
            return [center + radius * np.array([np.cos(a), np.sin(a)]) for a in angles]
        elif route_config['type'] == 'straight':
            if agent_id % 2 == 0:
                return [np.array([area_size*0.2, area_size*0.2]),
                        np.array([area_size*0.8, area_size*0.8])]
            else:
                return [np.array([area_size*0.8, area_size*0.2]),
                        np.array([area_size*0.2, area_size*0.8])]
        elif route_config['type'] == 'common_goal':
            goal = np.array(route_config.get('goal', [area_size/2, area_size/2]))
            return [goal.copy()]
        else:
            raise ValueError(f"Unknown route config type: {route_config['type']}")

    def _create_algorithm(self, algo_config: Dict[str, Any]) -> SteeringAlgorithm:
        algo_type = algo_config['type']
        params = algo_config.get('params', {})
        if algo_type == 'VicsekAlgorithm':
            return VicsekAlgorithm(**params)
        elif algo_type == 'BoidsAlgorithm':
            return BoidsAlgorithm(**params)
        elif algo_type == 'PotentialFieldAlgorithm':
            if 'goal' in params:
                params['goal'] = np.array(params['goal'])
            return PotentialFieldAlgorithm(**params)
        elif algo_type == 'LeaderFollowerAlgorithm':
            return LeaderFollowerAlgorithm(**params)
        elif algo_type == 'RandomWalkAlgorithm':
            return RandomWalkAlgorithm(**params)
        else:
            raise ValueError(f"Unknown algorithm type: {algo_type}")

    def update_step(self):
        neighbors = self._find_neighbors()
        for agent, n in zip(self.agents, neighbors):
            agent.update(n, self.config['dt'], self.config['area_size'])
        self._collect_metrics()

    def _find_neighbors(self) -> List[List[Agent]]:
        positions = np.array([a.position for a in self.agents])
        dist_matrix = np.linalg.norm(positions[:, None] - positions, axis=2)
        neighbors = []
        for i, agent in enumerate(self.agents):
            mask = (dist_matrix[i] <= agent.specs.comm_radius) & (dist_matrix[i] > 0)
            neighbors.append([self.agents[j] for j in np.where(mask)[0]])
        return neighbors

    def _collect_metrics(self):
        positions = np.array([a.position for a in self.agents])
        velocities = np.array([a.velocity for a in self.agents])
        accelerations = np.array([a.acceleration for a in self.agents])

        self.metrics.avg_speed.append(np.mean(np.linalg.norm(velocities, axis=1)))
        self.metrics.max_speed.append(np.max(np.linalg.norm(velocities, axis=1)))
        self.metrics.avg_acceleration.append(np.mean(np.linalg.norm(accelerations, axis=1)))

        dist_matrix = np.linalg.norm(positions[:, None] - positions, axis=2)
        np.fill_diagonal(dist_matrix, np.nan)
        self.metrics.avg_distance.append(np.nanmean(dist_matrix))

        for agent in self.agents:
            if agent.id not in self.metrics.agent_trajectories:
                self.metrics.agent_trajectories[agent.id] = []
            self.metrics.agent_trajectories[agent.id].append(tuple(agent.position))

        collision_dist = 2 * self.agents[0].specs.physical_radius
        self.metrics.collisions += np.sum(dist_matrix < collision_dist) // 2

# ==================== ТЕСТУВАННЯ АЛГОРИТМІВ ====================
def test_algorithms():
    # Спільна конфігурація для всіх алгоритмів
    base_config = {
        "num_agents": 15,
        "area_size": 100,
        "steps": 1000,  # Збільшено час симуляції в 10 разів
        "dt": 0.1,
        "seed": 42,
        "agent_specs": {
            "physical_radius": 0.5,
            "safety_margin": 1.0,
            "max_speed": 2.0,
            "max_acceleration": 1.5,
            "comm_radius": 15.0
        }
    }

    algorithms = [
        {
            "name": "Vicsek",
            "config": {
                "type": "VicsekAlgorithm",
                "params": {
                    "noise": 0.3,
                    "neighbor_radius": 20.0
                }
            }
        },
        {
            "name": "Boids",
            "config": {
                "type": "BoidsAlgorithm",
                "params": {
                    "sep_weight": 1.5,
                    "align_weight": 0.8,
                    "coh_weight": 0.5,
                    "visual_range": 20.0
                }
            }
        },
        {
            "name": "Potential Field",
            "config": {
                "type": "PotentialFieldAlgorithm",
                "params": {
                    "alpha": 0.5,
                    "beta": 1.0,
                    "dynamic_goal": True
                }
            }
        },
        {
            "name": "Leader-Follower",
            "config": {
                "type": "LeaderFollowerAlgorithm",
                "params": {
                    "leader_id": 0,
                    "follow_distance": 5.0
                }
            }
        },
        {
            "name": "Random Walk",
            "config": {
                "type": "RandomWalkAlgorithm",
                "params": {
                    "max_random_acceleration": 0.5
                }
            }
        }
    ]

    route_configs = [
        {"type": "circle"},
        {"type": "straight"},
        {"type": "common_goal", "goal": [50, 50]}
    ]

    N = 10  # Кількість запусків для кожного алгоритму та траєкторії
    results = {algo["name"]: {route["type"]: {"collisions": [], "avg_speed": [], "avg_distance": [], "execution_time": []}
                              for route in route_configs} for algo in algorithms}

    print("=== Testing Algorithms with Different Route Configurations ===")

    for route_config in route_configs:
        print(f"\nRoute Configuration: {route_config['type']}")

        for algo in algorithms:
            print(f"\nRunning {algo['name']} algorithm...")

            for i in range(N):
                start_time = time.time()

                # Створення конфігурації для поточного алгоритму
                config = base_config.copy()
                config["algorithm"] = algo["config"]
                config["route_config"] = route_config
                config["initial_config"] = {"type": "circle", "radius": 30}

                simulator = SwarmSimulator(config)

                # Запуск симуляції
                for _ in range(config["steps"]):
                    simulator.update_step()

                # Збереження результатів
                results[algo["name"]][route_config["type"]]["collisions"].append(simulator.metrics.collisions)
                results[algo["name"]][route_config["type"]]["avg_speed"].append(np.mean(simulator.metrics.avg_speed))
                results[algo["name"]][route_config["type"]]["avg_distance"].append(np.mean(simulator.metrics.avg_distance))

                end_time = time.time()
                results[algo["name"]][route_config["type"]]["execution_time"].append(end_time - start_time)

                # Вивід текстових даних
                print(f"Run {i+1}:")
                print(f"  Collisions: {simulator.metrics.collisions}")
                print(f"  Avg Speed: {np.mean(simulator.metrics.avg_speed):.2f}")
                print(f"  Avg Distance: {np.mean(simulator.metrics.avg_distance):.2f}")
                print(f"  Execution Time: {end_time - start_time:.2f} seconds")

    # Вивід усереднених даних
    print("\n=== Averaged Results ===")
    for algo_name, route_results in results.items():
        print(f"\nAlgorithm: {algo_name}")
        for route_type, metrics in route_results.items():
            print(f"\nRoute Configuration: {route_type}")
            print(f"  Average Collisions: {np.mean(metrics['collisions']):.2f}")
            print(f"  Average Speed: {np.mean(metrics['avg_speed']):.2f}")
            print(f"  Average Distance: {np.mean(metrics['avg_distance']):.2f}")
            print(f"  Average Execution Time: {np.mean(metrics['execution_time']):.2f} seconds")

# Запуск тестування
test_algorithms()

In [ ]:
import numpy as np
from dataclasses import dataclass
from abc import ABC, abstractmethod
from typing import List, Dict, Tuple, Optional, Any
import random
import time

# ==================== БАЗОВІ КЛАСИ ====================
@dataclass
class AgentSpecs:
    physical_radius: float
    safety_margin: float
    max_speed: float
    max_acceleration: float
    comm_radius: float

class SteeringAlgorithm(ABC):
    @abstractmethod
    def compute_steering(self, agent, neighbors: List['Agent'], dt: float) -> np.ndarray:
        pass

@dataclass
class SimulationMetrics:
    collisions: int = 0
    avg_speed: List[float] = None
    max_speed: List[float] = None
    avg_acceleration: List[float] = None
    avg_distance: List[float] = None
    agent_trajectories: Dict[int, List[Tuple[float, float]]] = None

    def __post_init__(self):
        self.avg_speed = []
        self.max_speed = []
        self.avg_acceleration = []
        self.avg_distance = []
        self.agent_trajectories = {}

class Agent:
    def __init__(self, agent_id: int, position: np.ndarray, specs: AgentSpecs,
                 algorithm: SteeringAlgorithm, route: List[np.ndarray]):
        self.id = agent_id
        self.position = position
        self.velocity = np.zeros(2)
        self.acceleration = np.zeros(2)
        self.specs = specs
        self.algorithm = algorithm
        self.route = route
        self.current_waypoint = 0
        self.path_history = [position.copy()]

    def update(self, neighbors: List['Agent'], dt: float, area_size: float):
        steering = self.algorithm.compute_steering(self, neighbors, dt)
        steering = self._limit_vector(steering, self.specs.max_acceleration)
        self.velocity = self._limit_vector(self.velocity + steering * dt, self.specs.max_speed)
        self.position += self.velocity * dt
        self.acceleration = steering
        self._enforce_boundaries(area_size)
        self.path_history.append(self.position.copy())

    def _limit_vector(self, vector: np.ndarray, max_magnitude: float) -> np.ndarray:
        magnitude = np.linalg.norm(vector)
        return vector if magnitude == 0 else vector * min(max_magnitude/magnitude, 1)

    def _enforce_boundaries(self, area_size: float):
        self.position = np.clip(self.position, 0, area_size)

# ==================== АЛГОРИТМИ КЕРУВАННЯ ====================
class VicsekAlgorithm(SteeringAlgorithm):
    def __init__(self, noise: float = 0.1, neighbor_radius: float = 15.0):
        self.noise = noise
        self.neighbor_radius = neighbor_radius

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        neighbors = [n for n in neighbors if np.linalg.norm(n.position - agent.position) <= self.neighbor_radius]
        if not neighbors:
            return np.zeros(2)
        avg_velocity = np.mean([n.velocity for n in neighbors], axis=0)
        noise = np.random.normal(scale=self.noise, size=2)
        return (avg_velocity + noise - agent.velocity) / dt

class BoidsAlgorithm(SteeringAlgorithm):
    def __init__(self, sep_weight: float = 1.5, align_weight: float = 1.0,
                 coh_weight: float = 1.0, visual_range: float = 15.0):
        self.weights = np.array([sep_weight, align_weight, coh_weight])
        self.visual_range = visual_range

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        neighbors = [n for n in neighbors if np.linalg.norm(n.position - agent.position) <= self.visual_range]
        if not neighbors:
            return np.zeros(2)
        positions = np.array([n.position for n in neighbors])
        velocities = np.array([n.velocity for n in neighbors])
        sep = self._separation(agent.position, positions)
        align = self._alignment(agent.velocity, velocities)
        coh = self._cohesion(agent.position, positions)
        return self.weights @ np.array([sep, align, coh])

    def _separation(self, pos: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        diff = pos - neighbors
        distances = np.linalg.norm(diff, axis=1)[:, np.newaxis]
        return np.mean(diff / (distances + 1e-6), axis=0)

    def _alignment(self, vel: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        return np.mean(neighbors, axis=0) - vel

    def _cohesion(self, pos: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        return np.mean(neighbors, axis=0) - pos

class PotentialFieldAlgorithm(SteeringAlgorithm):
    def __init__(self, alpha: float = 0.5, beta: float = 1.0,
                 goal: Optional[np.ndarray] = None, dynamic_goal: bool = False):
        self.goal = goal
        self.alpha = alpha
        self.beta = beta
        self.dynamic_goal = dynamic_goal

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        if self.dynamic_goal and agent.route:
            self.goal = agent.route[-1]
        to_goal = self.goal - agent.position if self.goal is not None else np.zeros(2)
        attraction = self.alpha * to_goal
        repulsion = np.zeros(2)
        for n in neighbors:
            vec = agent.position - n.position
            dist = np.linalg.norm(vec)
            if dist > 0:
                repulsion += self.beta * vec / dist**2
        return attraction + repulsion - agent.velocity / dt

class LeaderFollowerAlgorithm(SteeringAlgorithm):
    def __init__(self, leader_id: int = 0, follow_distance: float = 5.0):
        self.leader_id = leader_id
        self.follow_distance = follow_distance

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        leader = next((n for n in neighbors if n.id == self.leader_id), None)
        if leader is None or agent.id == self.leader_id:
            return np.zeros(2)
        direction = leader.position - agent.position
        distance = np.linalg.norm(direction)
        desired_velocity = (direction / distance) * agent.specs.max_speed if distance > 0 else np.zeros(2)
        return (desired_velocity - agent.velocity) / dt

class RandomWalkAlgorithm(SteeringAlgorithm):
    def __init__(self, max_random_acceleration: float = 0.1):
        self.max_random_acceleration = max_random_acceleration

    def compute_steering(self, agent: Agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        return np.random.uniform(-self.max_random_acceleration, self.max_random_acceleration, size=2)

# ==================== СИСТЕМА СИМУЛЯЦІЇ ====================
class SwarmSimulator:
    def __init__(self, config: Dict[str, Any]):
        self.config = config
        np.random.seed(config.get('seed', 42))
        random.seed(config.get('seed', 42))
        self._init_simulation()

    def _init_simulation(self):
        self.agents = []
        self.metrics = SimulationMetrics()
        positions = self._generate_initial_positions()
        for i in range(self.config['num_agents']):
            route = self._generate_route(i, positions[i])
            specs = AgentSpecs(**self.config['agent_specs'])
            algorithm = self._create_algorithm(self.config['algorithm'])
            agent = Agent(i, positions[i], specs, algorithm, route)
            self.agents.append(agent)

    def _generate_initial_positions(self) -> List[np.ndarray]:
        config = self.config['initial_config']
        area_size = self.config['area_size']
        num_agents = self.config['num_agents']

        if config['type'] == 'grid':
            rows = int(np.ceil(np.sqrt(num_agents)))
            cols = int(np.ceil(num_agents/rows))
            sector_size = area_size / max(rows, cols)
            return [np.array([(i%cols)*sector_size + sector_size/2,
                            (i//cols)*sector_size + sector_size/2])
                    for i in range(num_agents)]
        elif config['type'] == 'random':
            return [np.array([random.uniform(0, area_size), random.uniform(0, area_size)])
                    for _ in range(num_agents)]
        elif config['type'] == 'circle':
            radius = config.get('radius', area_size/3)
            center = np.array([area_size/2, area_size/2])
            angles = np.linspace(0, 2*np.pi, num_agents, endpoint=False)
            return [center + radius * np.array([np.cos(a), np.sin(a)]) for a in angles]
        elif config['type'] == 'line':
            spacing = area_size / (num_agents + 1)
            return [np.array([spacing*(i+1), area_size/2]) for i in range(num_agents)]
        else:
            raise ValueError(f"Unknown initial config type: {config['type']}")

    def _generate_route(self, agent_id: int, start_pos: np.ndarray) -> List[np.ndarray]:
        route_config = self.config['route_config']
        area_size = self.config['area_size']

        if route_config['type'] == 'circle':
            radius = area_size/3
            center = np.array([area_size/2, area_size/2])
            angles = np.linspace(0, 2*np.pi, 20, endpoint=False)
            return [center + radius * np.array([np.cos(a), np.sin(a)]) for a in angles]
        elif route_config['type'] == 'straight':
            if agent_id % 2 == 0:
                return [np.array([area_size*0.2, area_size*0.2]),
                        np.array([area_size*0.8, area_size*0.8])]
            else:
                return [np.array([area_size*0.8, area_size*0.2]),
                        np.array([area_size*0.2, area_size*0.8])]
        elif route_config['type'] == 'common_goal':
            goal = np.array(route_config.get('goal', [area_size/2, area_size/2]))
            return [goal.copy()]
        else:
            raise ValueError(f"Unknown route config type: {route_config['type']}")

    def _create_algorithm(self, algo_config: Dict[str, Any]) -> SteeringAlgorithm:
        algo_type = algo_config['type']
        params = algo_config.get('params', {})
        if algo_type == 'VicsekAlgorithm':
            return VicsekAlgorithm(**params)
        elif algo_type == 'BoidsAlgorithm':
            return BoidsAlgorithm(**params)
        elif algo_type == 'PotentialFieldAlgorithm':
            if 'goal' in params:
                params['goal'] = np.array(params['goal'])
            return PotentialFieldAlgorithm(**params)
        elif algo_type == 'LeaderFollowerAlgorithm':
            return LeaderFollowerAlgorithm(**params)
        elif algo_type == 'RandomWalkAlgorithm':
            return RandomWalkAlgorithm(**params)
        else:
            raise ValueError(f"Unknown algorithm type: {algo_type}")

    def update_step(self):
        neighbors = self._find_neighbors()
        for agent, n in zip(self.agents, neighbors):
            agent.update(n, self.config['dt'], self.config['area_size'])
        self._collect_metrics()

    def _find_neighbors(self) -> List[List[Agent]]:
        positions = np.array([a.position for a in self.agents])
        dist_matrix = np.linalg.norm(positions[:, None] - positions, axis=2)
        neighbors = []
        for i, agent in enumerate(self.agents):
            mask = (dist_matrix[i] <= agent.specs.comm_radius) & (dist_matrix[i] > 0)
            neighbors.append([self.agents[j] for j in np.where(mask)[0]])
        return neighbors

    def _collect_metrics(self):
        positions = np.array([a.position for a in self.agents])
        velocities = np.array([a.velocity for a in self.agents])
        accelerations = np.array([a.acceleration for a in self.agents])

        self.metrics.avg_speed.append(np.mean(np.linalg.norm(velocities, axis=1)))
        self.metrics.max_speed.append(np.max(np.linalg.norm(velocities, axis=1)))
        self.metrics.avg_acceleration.append(np.mean(np.linalg.norm(accelerations, axis=1)))

        dist_matrix = np.linalg.norm(positions[:, None] - positions, axis=2)
        np.fill_diagonal(dist_matrix, np.nan)
        self.metrics.avg_distance.append(np.nanmean(dist_matrix))

        for agent in self.agents:
            if agent.id not in self.metrics.agent_trajectories:
                self.metrics.agent_trajectories[agent.id] = []
            self.metrics.agent_trajectories[agent.id].append(tuple(agent.position))

        collision_dist = 2 * self.agents[0].specs.physical_radius
        self.metrics.collisions += np.sum(dist_matrix < collision_dist) // 2

# ==================== ТЕСТУВАННЯ АЛГОРИТМІВ ====================
def test_algorithms():
    # Спільна конфігурація для всіх алгоритмів
    base_config = {
        "num_agents": 15,
        "area_size": 100,
        "steps": 1000,  # Збільшено час симуляції в 10 разів
        "dt": 0.1,
        "seed": 42,
        "agent_specs": {
            "physical_radius": 0.5,
            "safety_margin": 1.0,
            "max_speed": 2.0,
            "max_acceleration": 1.5,
            "comm_radius": 15.0
        }
    }

    algorithms = [
        {
            "name": "Vicsek",
            "config": {
                "type": "VicsekAlgorithm",
                "params": {
                    "noise": 0.3,
                    "neighbor_radius": 20.0
                }
            }
        },
        {
            "name": "Boids",
            "config": {
                "type": "BoidsAlgorithm",
                "params": {
                    "sep_weight": 1.5,
                    "align_weight": 0.8,
                    "coh_weight": 0.5,
                    "visual_range": 20.0
                }
            }
        },
        {
            "name": "Potential Field",
            "config": {
                "type": "PotentialFieldAlgorithm",
                "params": {
                    "alpha": 0.5,
                    "beta": 1.0,
                    "dynamic_goal": True
                }
            }
        },
        {
            "name": "Leader-Follower",
            "config": {
                "type": "LeaderFollowerAlgorithm",
                "params": {
                    "leader_id": 0,
                    "follow_distance": 5.0
                }
            }
        },
        {
            "name": "Random Walk",
            "config": {
                "type": "RandomWalkAlgorithm",
                "params": {
                    "max_random_acceleration": 0.5
                }
            }
        }
    ]

    route_configs = [
        {"type": "circle"},
        {"type": "straight"},
        {"type": "common_goal", "goal": [50, 50]}
    ]

    N = 10  # Кількість запусків для кожного алгоритму та траєкторії
    results = {algo["name"]: {route["type"]: {"collisions": [], "avg_speed": [], "avg_distance": [], "execution_time": []}
                              for route in route_configs} for algo in algorithms}

    print("=== Testing Algorithms with Different Route Configurations ===")

    for route_config in route_configs:
        print(f"\nRoute Configuration: {route_config['type']}")

        for algo in algorithms:
            print(f"\nRunning {algo['name']} algorithm...")

            for _ in range(N):
                start_time = time.time()

                # Створення конфігурації для поточного алгоритму
                config = base_config.copy()
                config["algorithm"] = algo["config"]
                config["route_config"] = route_config
                config["initial_config"] = {"type": "circle", "radius": 30}

                simulator = SwarmSimulator(config)

                # Запуск симуляції
                for _ in range(config["steps"]):
                    simulator.update_step()

                # Збереження результатів
                results[algo["name"]][route_config["type"]]["collisions"].append(simulator.metrics.collisions)
                results[algo["name"]][route_config["type"]]["avg_speed"].append(np.mean(simulator.metrics.avg_speed))
                results[algo["name"]][route_config["type"]]["avg_distance"].append(np.mean(simulator.metrics.avg_distance))

                end_time = time.time()
                results[algo["name"]][route_config["type"]]["execution_time"].append(end_time - start_time)

    # Вивід усереднених даних
    print("\n=== Averaged Results ===")
    for algo_name, route_results in results.items():
        print(f"\nAlgorithm: {algo_name}")
        for route_type, metrics in route_results.items():
            print(f"\nRoute Configuration: {route_type}")
            print(f"  Average Collisions: {np.mean(metrics['collisions']):.2f}")
            print(f"  Average Speed: {np.mean(metrics['avg_speed']):.2f}")
            print(f"  Average Distance: {np.mean(metrics['avg_distance']):.2f}")
            print(f"  Average Execution Time: {np.mean(metrics['execution_time']):.2f} seconds")

# Запуск тестування
test_algorithms()

In [ ]:
import numpy as np
from dataclasses import dataclass
from abc import ABC, abstractmethod
from typing import List, Dict, Tuple, Optional, Any
import random
import time

# ==================== БАЗОВІ КЛАСИ ====================
@dataclass
class AgentSpecs:
    physical_radius: float
    safety_margin: float
    max_speed: float
    max_acceleration: float
    comm_radius: float

class SteeringAlgorithm(ABC):
    @abstractmethod
    def compute_steering(self, agent, neighbors: List['Agent'], dt: float) -> np.ndarray:
        pass

@dataclass
class SimulationMetrics:
    collisions: int = 0
    avg_speed: List[float] = None
    max_speed: List[float] = None
    avg_acceleration: List[float] = None
    avg_distance: List[float] = None
    agent_trajectories: Dict[int, List[Tuple[float, float]]] = None

    def __post_init__(self):
        self.avg_speed = []
        self.max_speed = []
        self.avg_acceleration = []
        self.avg_distance = []
        self.agent_trajectories = {}

class Agent:
    def __init__(self, agent_id: int, position: np.ndarray, specs: AgentSpecs,
                 algorithm: SteeringAlgorithm, route: List[np.ndarray]):
        self.id = agent_id
        self.position = position
        self.velocity = np.zeros(2)
        self.acceleration = np.zeros(2)
        self.specs = specs
        self.algorithm = algorithm
        self.route = route
        self.current_waypoint = 0
        self.path_history = [position.copy()]

    def update(self, neighbors: List['Agent'], dt: float, area_size: float):
        steering = self.algorithm.compute_steering(self, neighbors, dt)
        steering = self._limit_vector(steering, self.specs.max_acceleration)
        self.velocity = self._limit_vector(self.velocity + steering * dt, self.specs.max_speed)
        self.position += self.velocity * dt
        self.acceleration = steering
        self._enforce_boundaries(area_size)
        self.path_history.append(self.position.copy())

    def _limit_vector(self, vector: np.ndarray, max_magnitude: float) -> np.ndarray:
        magnitude = np.linalg.norm(vector)
        return vector if magnitude == 0 else vector * min(max_magnitude/magnitude, 1)

    def _enforce_boundaries(self, area_size: float):
        self.position = np.clip(self.position, 0, area_size)

# ==================== АЛГОРИТМИ КЕРУВАННЯ ====================
class VicsekAlgorithm(SteeringAlgorithm):
    def __init__(self, noise: float = 0.1, neighbor_radius: float = 15.0):
        self.noise = noise
        self.neighbor_radius = neighbor_radius

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        neighbors = [n for n in neighbors if np.linalg.norm(n.position - agent.position) <= self.neighbor_radius]
        if not neighbors:
            return np.zeros(2)
        avg_velocity = np.mean([n.velocity for n in neighbors], axis=0)
        noise = np.random.normal(scale=self.noise, size=2)
        return (avg_velocity + noise - agent.velocity) / dt

class BoidsAlgorithm(SteeringAlgorithm):
    def __init__(self, sep_weight: float = 1.5, align_weight: float = 1.0,
                 coh_weight: float = 1.0, visual_range: float = 15.0):
        self.weights = np.array([sep_weight, align_weight, coh_weight])
        self.visual_range = visual_range

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        neighbors = [n for n in neighbors if np.linalg.norm(n.position - agent.position) <= self.visual_range]
        if not neighbors:
            return np.zeros(2)
        positions = np.array([n.position for n in neighbors])
        velocities = np.array([n.velocity for n in neighbors])
        sep = self._separation(agent.position, positions)
        align = self._alignment(agent.velocity, velocities)
        coh = self._cohesion(agent.position, positions)
        return self.weights @ np.array([sep, align, coh])

    def _separation(self, pos: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        diff = pos - neighbors
        distances = np.linalg.norm(diff, axis=1)[:, np.newaxis]
        return np.mean(diff / (distances + 1e-6), axis=0)

    def _alignment(self, vel: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        return np.mean(neighbors, axis=0) - vel

    def _cohesion(self, pos: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        return np.mean(neighbors, axis=0) - pos

class PotentialFieldAlgorithm(SteeringAlgorithm):
    def __init__(self, alpha: float = 0.5, beta: float = 1.0,
                 goal: Optional[np.ndarray] = None, dynamic_goal: bool = False):
        self.goal = goal
        self.alpha = alpha
        self.beta = beta
        self.dynamic_goal = dynamic_goal

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        if self.dynamic_goal and agent.route:
            self.goal = agent.route[-1]
        to_goal = self.goal - agent.position if self.goal is not None else np.zeros(2)
        attraction = self.alpha * to_goal
        repulsion = np.zeros(2)
        for n in neighbors:
            vec = agent.position - n.position
            dist = np.linalg.norm(vec)
            if dist > 0:
                repulsion += self.beta * vec / dist**2
        return attraction + repulsion - agent.velocity / dt

class LeaderFollowerAlgorithm(SteeringAlgorithm):
    def __init__(self, leader_id: int = 0, follow_distance: float = 5.0):
        self.leader_id = leader_id
        self.follow_distance = follow_distance

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        leader = next((n for n in neighbors if n.id == self.leader_id), None)
        if leader is None or agent.id == self.leader_id:
            return np.zeros(2)
        direction = leader.position - agent.position
        distance = np.linalg.norm(direction)
        desired_velocity = (direction / distance) * agent.specs.max_speed if distance > 0 else np.zeros(2)
        return (desired_velocity - agent.velocity) / dt

class RandomWalkAlgorithm(SteeringAlgorithm):
    def __init__(self, max_random_acceleration: float = 0.1):
        self.max_random_acceleration = max_random_acceleration

    def compute_steering(self, agent: Agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        return np.random.uniform(-self.max_random_acceleration, self.max_random_acceleration, size=2)

class CBRAlgorithm(SteeringAlgorithm):
    def __init__(self, prediction_horizon: int = 15, max_speed_xy: float = 10.0, max_speed_z: float = 5.0, max_accel: float = 3.0):
        self.prediction_horizon = prediction_horizon
        self.max_speed_xy = max_speed_xy
        self.max_speed_z = max_speed_z
        self.max_accel = max_accel

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        # Implement CBR-based steering logic here
        # For simplicity, we'll use a basic implementation
        desired_velocity = np.zeros(2)
        for n in neighbors:
            delta_pos = agent.position - n.position
            dist = np.linalg.norm(delta_pos)
            if dist < agent.specs.comm_radius:
                desired_velocity += delta_pos / dist
        return desired_velocity

# ==================== СИСТЕМА СИМУЛЯЦІЇ ====================
class SwarmSimulator:
    def __init__(self, config: Dict[str, Any]):
        self.config = config
        np.random.seed(config.get('seed', 42))
        random.seed(config.get('seed', 42))
        self._init_simulation()

    def _init_simulation(self):
        self.agents = []
        self.metrics = SimulationMetrics()
        positions = self._generate_initial_positions()
        for i in range(self.config['num_agents']):
            route = self._generate_route(i, positions[i])
            specs = AgentSpecs(**self.config['agent_specs'])
            algorithm = self._create_algorithm(self.config['algorithm'])
            agent = Agent(i, positions[i], specs, algorithm, route)
            self.agents.append(agent)

    def _generate_initial_positions(self) -> List[np.ndarray]:
        config = self.config['initial_config']
        area_size = self.config['area_size']
        num_agents = self.config['num_agents']

        if config['type'] == 'grid':
            rows = int(np.ceil(np.sqrt(num_agents)))
            cols = int(np.ceil(num_agents/rows))
            sector_size = area_size / max(rows, cols)
            return [np.array([(i%cols)*sector_size + sector_size/2,
                            (i//cols)*sector_size + sector_size/2])
                    for i in range(num_agents)]
        elif config['type'] == 'random':
            return [np.array([random.uniform(0, area_size), random.uniform(0, area_size)])
                    for _ in range(num_agents)]
        elif config['type'] == 'circle':
            radius = config.get('radius', area_size/3)
            center = np.array([area_size/2, area_size/2])
            angles = np.linspace(0, 2*np.pi, num_agents, endpoint=False)
            return [center + radius * np.array([np.cos(a), np.sin(a)]) for a in angles]
        elif config['type'] == 'line':
            spacing = area_size / (num_agents + 1)
            return [np.array([spacing*(i+1), area_size/2]) for i in range(num_agents)]
        else:
            raise ValueError(f"Unknown initial config type: {config['type']}")

    def _generate_route(self, agent_id: int, start_pos: np.ndarray) -> List[np.ndarray]:
        route_config = self.config['route_config']
        area_size = self.config['area_size']

        if route_config['type'] == 'circle':
            radius = area_size/3
            center = np.array([area_size/2, area_size/2])
            angles = np.linspace(0, 2*np.pi, 20, endpoint=False)
            return [center + radius * np.array([np.cos(a), np.sin(a)]) for a in angles]
        elif route_config['type'] == 'straight':
            if agent_id % 2 == 0:
                return [np.array([area_size*0.2, area_size*0.2]),
                        np.array([area_size*0.8, area_size*0.8])]
            else:
                return [np.array([area_size*0.8, area_size*0.2]),
                        np.array([area_size*0.2, area_size*0.8])]
        elif route_config['type'] == 'common_goal':
            goal = np.array(route_config.get('goal', [area_size/2, area_size/2]))
            return [goal.copy()]
        else:
            raise ValueError(f"Unknown route config type: {route_config['type']}")

    def _create_algorithm(self, algo_config: Dict[str, Any]) -> SteeringAlgorithm:
        algo_type = algo_config['type']
        params = algo_config.get('params', {})
        if algo_type == 'VicsekAlgorithm':
            return VicsekAlgorithm(**params)
        elif algo_type == 'BoidsAlgorithm':
            return BoidsAlgorithm(**params)
        elif algo_type == 'PotentialFieldAlgorithm':
            if 'goal' in params:
                params['goal'] = np.array(params['goal'])
            return PotentialFieldAlgorithm(**params)
        elif algo_type == 'LeaderFollowerAlgorithm':
            return LeaderFollowerAlgorithm(**params)
        elif algo_type == 'RandomWalkAlgorithm':
            return RandomWalkAlgorithm(**params)
        elif algo_type == 'CBRAlgorithm':
            return CBRAlgorithm(**params)
        else:
            raise ValueError(f"Unknown algorithm type: {algo_type}")

    def update_step(self):
        neighbors = self._find_neighbors()
        for agent, n in zip(self.agents, neighbors):
            agent.update(n, self.config['dt'], self.config['area_size'])
        self._collect_metrics()

    def _find_neighbors(self) -> List[List[Agent]]:
        positions = np.array([a.position for a in self.agents])
        dist_matrix = np.linalg.norm(positions[:, None] - positions, axis=2)
        neighbors = []
        for i, agent in enumerate(self.agents):
            mask = (dist_matrix[i] <= agent.specs.comm_radius) & (dist_matrix[i] > 0)
            neighbors.append([self.agents[j] for j in np.where(mask)[0]])
        return neighbors

    def _collect_metrics(self):
        positions = np.array([a.position for a in self.agents])
        velocities = np.array([a.velocity for a in self.agents])
        accelerations = np.array([a.acceleration for a in self.agents])

        self.metrics.avg_speed.append(np.mean(np.linalg.norm(velocities, axis=1)))
        self.metrics.max_speed.append(np.max(np.linalg.norm(velocities, axis=1)))
        self.metrics.avg_acceleration.append(np.mean(np.linalg.norm(accelerations, axis=1)))

        dist_matrix = np.linalg.norm(positions[:, None] - positions, axis=2)
        np.fill_diagonal(dist_matrix, np.nan)
        self.metrics.avg_distance.append(np.nanmean(dist_matrix))

        for agent in self.agents:
            if agent.id not in self.metrics.agent_trajectories:
                self.metrics.agent_trajectories[agent.id] = []
            self.metrics.agent_trajectories[agent.id].append(tuple(agent.position))

        collision_dist = 2 * self.agents[0].specs.physical_radius
        self.metrics.collisions += np.sum(dist_matrix < collision_dist) // 2

# ==================== ТЕСТУВАННЯ АЛГОРИТМІВ ====================
def test_algorithms():
    # Спільна конфігурація для всіх алгоритмів
    base_config = {
        "num_agents": 150,
        "area_size": 100,
        "steps": 100,  # Збільшено час симуляції в 10 разів
        "dt": 0.1,
        "seed": 42,
        "agent_specs": {
            "physical_radius": 0.5,
            "safety_margin": 1.0,
            "max_speed": 2.0,
            "max_acceleration": 1.5,
            "comm_radius": 15.0
        }
    }

    algorithms = [
        {
            "name": "Vicsek",
            "config": {
                "type": "VicsekAlgorithm",
                "params": {
                    "noise": 0.3,
                    "neighbor_radius": 20.0
                }
            }
        },
        {
            "name": "Boids",
            "config": {
                "type": "BoidsAlgorithm",
                "params": {
                    "sep_weight": 1.5,
                    "align_weight": 0.8,
                    "coh_weight": 0.5,
                    "visual_range": 20.0
                }
            }
        },
        {
            "name": "Potential Field",
            "config": {
                "type": "PotentialFieldAlgorithm",
                "params": {
                    "alpha": 0.5,
                    "beta": 1.0,
                    "dynamic_goal": True
                }
            }
        },
        {
            "name": "Leader-Follower",
            "config": {
                "type": "LeaderFollowerAlgorithm",
                "params": {
                    "leader_id": 0,
                    "follow_distance": 5.0
                }
            }
        },
        {
            "name": "Random Walk",
            "config": {
                "type": "RandomWalkAlgorithm",
                "params": {
                    "max_random_acceleration": 0.5
                }
            }
        },
        {
            "name": "CBR",
            "config": {
                "type": "CBRAlgorithm",
                "params": {
                    "prediction_horizon": 15,
                    "max_speed_xy": 10.0,
                    "max_speed_z": 5.0,
                    "max_accel": 3.0
                }
            }
        }
    ]

    route_configs = [
        {"type": "circle"},
        {"type": "straight"},
        {"type": "common_goal", "goal": [50, 50]}
    ]

    N = 10  # Кількість запусків для кожного алгоритму та траєкторії
    results = {algo["name"]: {route["type"]: {"collisions": [], "avg_speed": [], "avg_distance": [], "execution_time": []}
                              for route in route_configs} for algo in algorithms}

    print("=== Testing Algorithms with Different Route Configurations ===")

    for route_config in route_configs:
        print(f"\nRoute Configuration: {route_config['type']}")

        for algo in algorithms:
            print(f"\nRunning {algo['name']} algorithm...")

            for _ in range(N):
                start_time = time.time()

                # Створення конфігурації для поточного алгоритму
                config = base_config.copy()
                config["algorithm"] = algo["config"]
                config["route_config"] = route_config
                config["initial_config"] = {"type": "circle", "radius": 30}

                simulator = SwarmSimulator(config)

                # Запуск симуляції
                for _ in range(config["steps"]):
                    simulator.update_step()

                # Збереження результатів
                results[algo["name"]][route_config["type"]]["collisions"].append(simulator.metrics.collisions)
                results[algo["name"]][route_config["type"]]["avg_speed"].append(np.mean(simulator.metrics.avg_speed))
                results[algo["name"]][route_config["type"]]["avg_distance"].append(np.mean(simulator.metrics.avg_distance))

                end_time = time.time()
                results[algo["name"]][route_config["type"]]["execution_time"].append(end_time - start_time)

    # Вивід усереднених даних
    print("\n=== Averaged Results ===")
    for algo_name, route_results in results.items():
        print(f"\nAlgorithm: {algo_name}")
        for route_type, metrics in route_results.items():
            print(f"\nRoute Configuration: {route_type}")
            print(f"  Average Collisions: {np.mean(metrics['collisions']):.2f}")
            print(f"  Average Speed: {np.mean(metrics['avg_speed']):.2f}")
            print(f"  Average Distance: {np.mean(metrics['avg_distance']):.2f}")
            print(f"  Average Execution Time: {np.mean(metrics['execution_time']):.2f} seconds")

# Запуск тестування
test_algorithms()

In [ ]:
import numpy as np
from dataclasses import dataclass
from abc import ABC, abstractmethod
from typing import List, Dict, Tuple, Optional, Any
import random
import time

# ==================== БАЗОВІ КЛАСИ ====================
@dataclass
class AgentSpecs:
    physical_radius: float
    safety_margin: float
    max_speed: float
    max_acceleration: float
    comm_radius: float

class SteeringAlgorithm(ABC):
    @abstractmethod
    def compute_steering(self, agent, neighbors: List['Agent'], dt: float) -> np.ndarray:
        pass

@dataclass
class SimulationMetrics:
    collisions: int = 0
    avg_speed: List[float] = None
    max_speed: List[float] = None
    avg_acceleration: List[float] = None
    avg_distance: List[float] = None
    agent_trajectories: Dict[int, List[Tuple[float, float]]] = None

    def __post_init__(self):
        self.avg_speed = []
        self.max_speed = []
        self.avg_acceleration = []
        self.avg_distance = []
        self.agent_trajectories = {}

class Agent:
    def __init__(self, agent_id: int, position: np.ndarray, specs: AgentSpecs,
                 algorithm: SteeringAlgorithm, route: List[np.ndarray]):
        self.id = agent_id
        self.position = position
        self.velocity = np.zeros(2)
        self.acceleration = np.zeros(2)
        self.specs = specs
        self.algorithm = algorithm
        self.route = route
        self.current_waypoint = 0
        self.path_history = [position.copy()]

    def update(self, neighbors: List['Agent'], dt: float, area_size: float):
        steering = self.algorithm.compute_steering(self, neighbors, dt)
        steering = self._limit_vector(steering, self.specs.max_acceleration)
        self.velocity = self._limit_vector(self.velocity + steering * dt, self.specs.max_speed)
        self.position += self.velocity * dt
        self.acceleration = steering
        self._enforce_boundaries(area_size)
        self.path_history.append(self.position.copy())

    def _limit_vector(self, vector: np.ndarray, max_magnitude: float) -> np.ndarray:
        magnitude = np.linalg.norm(vector)
        return vector if magnitude == 0 else vector * min(max_magnitude/magnitude, 1)

    def _enforce_boundaries(self, area_size: float):
        self.position = np.clip(self.position, 0, area_size)

# ==================== АЛГОРИТМИ КЕРУВАННЯ ====================
class VicsekAlgorithm(SteeringAlgorithm):
    def __init__(self, noise: float = 0.1, neighbor_radius: float = 15.0):
        self.noise = noise
        self.neighbor_radius = neighbor_radius

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        neighbors = [n for n in neighbors if np.linalg.norm(n.position - agent.position) <= self.neighbor_radius]
        if not neighbors:
            return np.zeros(2)
        avg_velocity = np.mean([n.velocity for n in neighbors], axis=0)
        noise = np.random.normal(scale=self.noise, size=2)
        return (avg_velocity + noise - agent.velocity) / dt

class BoidsAlgorithm(SteeringAlgorithm):
    def __init__(self, sep_weight: float = 1.5, align_weight: float = 1.0,
                 coh_weight: float = 1.0, visual_range: float = 15.0):
        self.weights = np.array([sep_weight, align_weight, coh_weight])
        self.visual_range = visual_range

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        neighbors = [n for n in neighbors if np.linalg.norm(n.position - agent.position) <= self.visual_range]
        if not neighbors:
            return np.zeros(2)
        positions = np.array([n.position for n in neighbors])
        velocities = np.array([n.velocity for n in neighbors])
        sep = self._separation(agent.position, positions)
        align = self._alignment(agent.velocity, velocities)
        coh = self._cohesion(agent.position, positions)
        return self.weights @ np.array([sep, align, coh])

    def _separation(self, pos: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        diff = pos - neighbors
        distances = np.linalg.norm(diff, axis=1)[:, np.newaxis]
        return np.mean(diff / (distances + 1e-6), axis=0)

    def _alignment(self, vel: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        return np.mean(neighbors, axis=0) - vel

    def _cohesion(self, pos: np.ndarray, neighbors: np.ndarray) -> np.ndarray:
        return np.mean(neighbors, axis=0) - pos

class PotentialFieldAlgorithm(SteeringAlgorithm):
    def __init__(self, alpha: float = 0.5, beta: float = 1.0,
                 goal: Optional[np.ndarray] = None, dynamic_goal: bool = False):
        self.goal = goal
        self.alpha = alpha
        self.beta = beta
        self.dynamic_goal = dynamic_goal

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        if self.dynamic_goal and agent.route:
            self.goal = agent.route[-1]
        to_goal = self.goal - agent.position if self.goal is not None else np.zeros(2)
        attraction = self.alpha * to_goal
        repulsion = np.zeros(2)
        for n in neighbors:
            vec = agent.position - n.position
            dist = np.linalg.norm(vec)
            if dist > 0:
                repulsion += self.beta * vec / dist**2
        return attraction + repulsion - agent.velocity / dt

class LeaderFollowerAlgorithm(SteeringAlgorithm):
    def __init__(self, leader_id: int = 0, follow_distance: float = 5.0):
        self.leader_id = leader_id
        self.follow_distance = follow_distance

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        leader = next((n for n in neighbors if n.id == self.leader_id), None)
        if leader is None or agent.id == self.leader_id:
            return np.zeros(2)
        direction = leader.position - agent.position
        distance = np.linalg.norm(direction)
        desired_velocity = (direction / distance) * agent.specs.max_speed if distance > 0 else np.zeros(2)
        return (desired_velocity - agent.velocity) / dt

class RandomWalkAlgorithm(SteeringAlgorithm):
    def __init__(self, max_random_acceleration: float = 0.1):
        self.max_random_acceleration = max_random_acceleration

    def compute_steering(self, agent: Agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        return np.random.uniform(-self.max_random_acceleration, self.max_random_acceleration, size=2)

class CBRAlgorithm(SteeringAlgorithm):
    def __init__(self, prediction_horizon: int = 15, max_speed_xy: float = 10.0, max_speed_z: float = 5.0, max_accel: float = 3.0):
        self.prediction_horizon = prediction_horizon
        self.max_speed_xy = max_speed_xy
        self.max_speed_z = max_speed_z
        self.max_accel = max_accel

    def compute_steering(self, agent, neighbors: List[Agent], dt: float) -> np.ndarray:
        # Implement CBR-based steering logic here
        # For simplicity, we'll use a basic implementation
        desired_velocity = np.zeros(2)
        for n in neighbors:
            delta_pos = agent.position - n.position
            dist = np.linalg.norm(delta_pos)
            if dist < agent.specs.comm_radius:
                desired_velocity += delta_pos / dist
        return desired_velocity

# ==================== СИСТЕМА СИМУЛЯЦІЇ ====================
class SwarmSimulator:
    def __init__(self, config: Dict[str, Any]):
        self.config = config
        np.random.seed(config.get('seed', 42))
        random.seed(config.get('seed', 42))
        self._init_simulation()

    def _init_simulation(self):
        self.agents = []
        self.metrics = SimulationMetrics()
        positions = self._generate_initial_positions()
        for i in range(self.config['num_agents']):
            route = self._generate_route(i, positions[i])
            specs = AgentSpecs(**self.config['agent_specs'])
            algorithm = self._create_algorithm(self.config['algorithm'])
            agent = Agent(i, positions[i], specs, algorithm, route)
            self.agents.append(agent)

    def _generate_initial_positions(self) -> List[np.ndarray]:
        config = self.config['initial_config']
        area_size = self.config['area_size']
        num_agents = self.config['num_agents']

        if config['type'] == 'grid':
            rows = int(np.ceil(np.sqrt(num_agents)))
            cols = int(np.ceil(num_agents/rows))
            sector_size = area_size / max(rows, cols)
            return [np.array([(i%cols)*sector_size + sector_size/2,
                            (i//cols)*sector_size + sector_size/2])
                    for i in range(num_agents)]
        elif config['type'] == 'random':
            return [np.array([random.uniform(0, area_size), random.uniform(0, area_size)])
                    for _ in range(num_agents)]
        elif config['type'] == 'circle':
            radius = config.get('radius', area_size/3)
            center = np.array([area_size/2, area_size/2])
            angles = np.linspace(0, 2*np.pi, num_agents, endpoint=False)
            return [center + radius * np.array([np.cos(a), np.sin(a)]) for a in angles]
        elif config['type'] == 'line':
            spacing = area_size / (num_agents + 1)
            return [np.array([spacing*(i+1), area_size/2]) for i in range(num_agents)]
        else:
            raise ValueError(f"Unknown initial config type: {config['type']}")

    def _generate_route(self, agent_id: int, start_pos: np.ndarray) -> List[np.ndarray]:
        route_config = self.config['route_config']
        area_size = self.config['area_size']

        if route_config['type'] == 'circle':
            radius = area_size/3
            center = np.array([area_size/2, area_size/2])
            angles = np.linspace(0, 2*np.pi, 20, endpoint=False)
            return [center + radius * np.array([np.cos(a), np.sin(a)]) for a in angles]
        elif route_config['type'] == 'straight':
            if agent_id % 2 == 0:
                return [np.array([area_size*0.2, area_size*0.2]),
                        np.array([area_size*0.8, area_size*0.8])]
            else:
                return [np.array([area_size*0.8, area_size*0.2]),
                        np.array([area_size*0.2, area_size*0.8])]
        elif route_config['type'] == 'common_goal':
            goal = np.array(route_config.get('goal', [area_size/2, area_size/2]))
            return [goal.copy()]
        else:
            raise ValueError(f"Unknown route config type: {route_config['type']}")

    def _create_algorithm(self, algo_config: Dict[str, Any]) -> SteeringAlgorithm:
        algo_type = algo_config['type']
        params = algo_config.get('params', {})
        if algo_type == 'VicsekAlgorithm':
            return VicsekAlgorithm(**params)
        elif algo_type == 'BoidsAlgorithm':
            return BoidsAlgorithm(**params)
        elif algo_type == 'PotentialFieldAlgorithm':
            if 'goal' in params:
                params['goal'] = np.array(params['goal'])
            return PotentialFieldAlgorithm(**params)
        elif algo_type == 'LeaderFollowerAlgorithm':
            return LeaderFollowerAlgorithm(**params)
        elif algo_type == 'RandomWalkAlgorithm':
            return RandomWalkAlgorithm(**params)
        elif algo_type == 'CBRAlgorithm':
            return CBRAlgorithm(**params)
        else:
            raise ValueError(f"Unknown algorithm type: {algo_type}")

    def update_step(self):
        neighbors = self._find_neighbors()
        for agent, n in zip(self.agents, neighbors):
            agent.update(n, self.config['dt'], self.config['area_size'])
        self._collect_metrics()

    def _find_neighbors(self) -> List[List[Agent]]:
        positions = np.array([a.position for a in self.agents])
        dist_matrix = np.linalg.norm(positions[:, None] - positions, axis=2)
        neighbors = []
        for i, agent in enumerate(self.agents):
            mask = (dist_matrix[i] <= agent.specs.comm_radius) & (dist_matrix[i] > 0)
            neighbors.append([self.agents[j] for j in np.where(mask)[0]])
        return neighbors

    def _collect_metrics(self):
        positions = np.array([a.position for a in self.agents])
        velocities = np.array([a.velocity for a in self.agents])
        accelerations = np.array([a.acceleration for a in self.agents])

        self.metrics.avg_speed.append(np.mean(np.linalg.norm(velocities, axis=1)))
        self.metrics.max_speed.append(np.max(np.linalg.norm(velocities, axis=1)))
        self.metrics.avg_acceleration.append(np.mean(np.linalg.norm(accelerations, axis=1)))

        dist_matrix = np.linalg.norm(positions[:, None] - positions, axis=2)
        np.fill_diagonal(dist_matrix, np.nan)
        self.metrics.avg_distance.append(np.nanmean(dist_matrix))

        for agent in self.agents:
            if agent.id not in self.metrics.agent_trajectories:
                self.metrics.agent_trajectories[agent.id] = []
            self.metrics.agent_trajectories[agent.id].append(tuple(agent.position))

        collision_dist = 2 * self.agents[0].specs.physical_radius
        self.metrics.collisions += np.sum(dist_matrix < collision_dist) // 2

# ==================== ТЕСТУВАННЯ АЛГОРИТМІВ ====================
def test_algorithms():
    base_config = {
        "num_agents": 150,
        "area_size": 100,
        "steps": 100,
        "dt": 0.1,
        "seed": 42,
        "agent_specs": {
            "physical_radius": 0.5,
            "safety_margin": 1.0,
            "max_speed": 2.0,
            "max_acceleration": 1.5,
            "comm_radius": 15.0
        }
    }

    algorithms = [
        {
            "name": "Vicsek",
            "config": {
                "type": "VicsekAlgorithm",
                "params": {
                    "noise": 0.1,          # Зменшено шум
                    "neighbor_radius": 25.0 # Збільшено радіус
                }
            }
        },
        {
            "name": "Boids",
            "config": {
                "type": "BoidsAlgorithm",
                "params": {
                    "sep_weight": 2.0,     # Сильніше відштовхування
                    "align_weight": 1.0,
                    "coh_weight": 0.3,    # Зменшена когезія
                    "visual_range": 15.0   # Зменшена дистанція
                }
            }
        },
        {
            "name": "Potential Field",
            "config": {
                "type": "PotentialFieldAlgorithm",
                "params": {
                    "alpha": 0.7,          # Сильніше притягання до цілі
                    "beta": 1.5,           # Сильніше відштовхування
                    "dynamic_goal": True
                }
            }
        },
        {
            "name": "Leader-Follower",
            "config": {
                "type": "LeaderFollowerAlgorithm",
                "params": {
                    "leader_id": 0,
                    "follow_distance": 8.0 # Збільшена дистанція
                }
            }
        },
        {
            "name": "Random Walk",
            "config": {
                "type": "RandomWalkAlgorithm",
                "params": {
                    "max_random_acceleration": 0.3 # Плавніший рух
                }
            }
        },
        {
            "name": "CBR",
            "config": {                     # Без змін (вже оптимально)
                "type": "CBRAlgorithm",
                "params": {
                    "prediction_horizon": 15,
                    "max_speed_xy": 10.0,
                    "max_speed_z": 5.0,
                    "max_accel": 3.0
                }
            }
        }
    ]

    route_configs = [
        {"type": "circle"},
        {"type": "straight"},
        {"type": "common_goal", "goal": [50, 50]}
    ]

    N = 10  # Кількість запусків для кожного алгоритму та траєкторії
    results = {algo["name"]: {route["type"]: {"collisions": [], "avg_speed": [], "avg_distance": [], "execution_time": []}
                              for route in route_configs} for algo in algorithms}

    print("=== Testing Algorithms with Different Route Configurations ===")

    for route_config in route_configs:
        print(f"\nRoute Configuration: {route_config['type']}")

        for algo in algorithms:
            print(f"\nRunning {algo['name']} algorithm...")

            for _ in range(N):
                start_time = time.time()

                # Створення конфігурації для поточного алгоритму
                config = base_config.copy()
                config["algorithm"] = algo["config"]
                config["route_config"] = route_config
                config["initial_config"] = {"type": "circle", "radius": 30}

                simulator = SwarmSimulator(config)

                # Запуск симуляції
                for _ in range(config["steps"]):
                    simulator.update_step()

                # Збереження результатів
                results[algo["name"]][route_config["type"]]["collisions"].append(simulator.metrics.collisions)
                results[algo["name"]][route_config["type"]]["avg_speed"].append(np.mean(simulator.metrics.avg_speed))
                results[algo["name"]][route_config["type"]]["avg_distance"].append(np.mean(simulator.metrics.avg_distance))

                end_time = time.time()
                results[algo["name"]][route_config["type"]]["execution_time"].append(end_time - start_time)

    # Вивід усереднених даних
    print("\n=== Averaged Results ===")
    for algo_name, route_results in results.items():
        print(f"\nAlgorithm: {algo_name}")
        for route_type, metrics in route_results.items():
            print(f"\nRoute Configuration: {route_type}")
            print(f"  Average Collisions: {np.mean(metrics['collisions']):.2f}")
            print(f"  Average Speed: {np.mean(metrics['avg_speed']):.2f}")
            print(f"  Average Distance: {np.mean(metrics['avg_distance']):.2f}")
            print(f"  Average Execution Time: {np.mean(metrics['execution_time']):.2f} seconds")

# Запуск тестування
test_algorithms()